___
## Introduction

This section of the project was initially devised before discovering that the London School of Economics (LSE) had a dedicated webpage listing all faculty members. The code presented here reflects an earlier approach, wherein we sought to gather information on individuals associated with each department directly from departmental pages. This broader-scope code aimed to compile a comprehensive list not only of faculty but also encompassed PHD students, research students, and visitors linked to each department.
<br>
It's important to note that this exploratory analysis serves as a precursor to the main analysis, as it was developed prior to our realization of the existence of a centralized page containing all staff members. Consequently, this segment should be regarded as supplementary, offering insights beyond the primary focus of the project. Additionally, as this phase was primarily investigative, it does not include any visualization components.
<br>
The subsequent code provides an overview of our initial approach, shedding light on the breadth of data exploration undertaken before refining our methodology to focus on the centralized staff list provided by LSE.

___
## Data Acquisition

In [1]:
import requests
from bs4 import BeautifulSoup
import re

### 1. Extract the homepage for each department

In [2]:
base_url= 'https://info.lse.ac.uk/Staff/Departments-and-Institutes'

In [3]:
departments_dict={}
response = requests.get(base_url)
soup = BeautifulSoup(response.content, 'html.parser')
departments=staff= soup.find_all('a', class_='sys_16')
for department in departments:
    href = department.get('href')
    dep_name = department.text
    departments_dict[dep_name]={'Link': href}
departments_dict

{'Department of Accounting': {'Link': 'http://www.lse.ac.uk/accounting/Home.aspx'},
 'Department of Anthropology': {'Link': 'http://www.lse.ac.uk/anthropology/home.aspx'},
 'Data Science Institute': {'Link': 'https://www.lse.ac.uk/dsi'},
 'Department of Economics': {'Link': 'http://www.lse.ac.uk/economics/home.aspx'},
 'Department of Economic History': {'Link': 'http://www.lse.ac.uk/Economic-History'},
 'European Institute': {'Link': 'http://www.lse.ac.uk/european-institute'},
 'Department of Finance': {'Link': 'http://www.lse.ac.uk/Finance'},
 'Firoz Lalji Institute for Africa': {'Link': 'https://www.lse.ac.uk/africa'},
 'Department of Gender Studies': {'Link': 'http://www.lse.ac.uk/Gender'},
 'Department of Geography and Environment': {'Link': 'http://www.lse.ac.uk/Geography-and-Environment'},
 'Department of Government': {'Link': 'http://www.lse.ac.uk/government/home.aspx'},
 'Department of Health Policy': {'Link': 'http://www.lse.ac.uk/health-policy'},
 'Department of International

### 2. Extract the people link for each department

In [4]:
for department, value in departments_dict.items():
  response = requests.get(value['Link'])
  soup = BeautifulSoup(response.content, 'html.parser')
  nav_links= soup.find_all('nav', class_='departmentNav')
  for nav_link in nav_links:
      a_tags = nav_link.find_all('a')
      for a_tag in a_tags:
          if a_tag.text == 'People':
              value['People_link']=f"https://www.lse.ac.uk{a_tag['href']}"
departments_dict

{'Department of Accounting': {'Link': 'http://www.lse.ac.uk/accounting/Home.aspx',
  'People_link': 'https://www.lse.ac.uk/accounting/people'},
 'Department of Anthropology': {'Link': 'http://www.lse.ac.uk/anthropology/home.aspx',
  'People_link': 'https://www.lse.ac.uk/anthropology/people'},
 'Data Science Institute': {'Link': 'https://www.lse.ac.uk/dsi'},
 'Department of Economics': {'Link': 'http://www.lse.ac.uk/economics/home.aspx',
  'People_link': 'https://www.lse.ac.uk/economics/people'},
 'Department of Economic History': {'Link': 'http://www.lse.ac.uk/Economic-History',
  'People_link': 'https://www.lse.ac.uk/Economic-History/People'},
 'European Institute': {'Link': 'http://www.lse.ac.uk/european-institute'},
 'Department of Finance': {'Link': 'http://www.lse.ac.uk/Finance',
  'People_link': 'https://www.lse.ac.uk/finance/people'},
 'Firoz Lalji Institute for Africa': {'Link': 'https://www.lse.ac.uk/africa'},
 'Department of Gender Studies': {'Link': 'http://www.lse.ac.uk/Gen

After inspecting if we successfully got the people link for all the departments, we see that there are some missing values, see code below:

In [5]:
for key, value in departments_dict.items():
  if len(value)!=2:
    print(f'Missing people link for the {key}.')

Missing people link for the Data Science Institute.
Missing people link for the European Institute.
Missing people link for the Firoz Lalji Institute for Africa.
Missing people link for the International Inequalities Institute.
Missing people link for the Language Centre.
Missing people link for the Marshall Institute.
Missing people link for the Department of Philosophy, Logic and Scientific Method.
Missing people link for the School of Public Policy.


We figured out that the links are always of the form: http://www.lse.ac.uk/**department_name**/people. So we implemented a code to get the people link for all the departments who didn't have one through our webscraping:

In [6]:
for department, value in departments_dict.items():
  if len(value)!=2:
    link=value['Link']
    split_link=link.split('/')
    value['People_link']=f"{split_link[0]}/{split_link[1]}/{split_link[2]}/{split_link[3]}/people"
departments_dict

{'Department of Accounting': {'Link': 'http://www.lse.ac.uk/accounting/Home.aspx',
  'People_link': 'https://www.lse.ac.uk/accounting/people'},
 'Department of Anthropology': {'Link': 'http://www.lse.ac.uk/anthropology/home.aspx',
  'People_link': 'https://www.lse.ac.uk/anthropology/people'},
 'Data Science Institute': {'Link': 'https://www.lse.ac.uk/dsi',
  'People_link': 'https://www.lse.ac.uk/dsi/people'},
 'Department of Economics': {'Link': 'http://www.lse.ac.uk/economics/home.aspx',
  'People_link': 'https://www.lse.ac.uk/economics/people'},
 'Department of Economic History': {'Link': 'http://www.lse.ac.uk/Economic-History',
  'People_link': 'https://www.lse.ac.uk/Economic-History/People'},
 'European Institute': {'Link': 'http://www.lse.ac.uk/european-institute',
  'People_link': 'http://www.lse.ac.uk/european-institute/people'},
 'Department of Finance': {'Link': 'http://www.lse.ac.uk/Finance',
  'People_link': 'https://www.lse.ac.uk/finance/people'},
 'Firoz Lalji Institute fo

Now we check if there is no more missing data:

In [7]:
for key, value in departments_dict.items():
  assert len(value)==2, 'Missing People link!'
print('Test passed, no missing data')

Test passed, no missing data


### 3. Extract the people, their link and role for each department

Below are some functions we used to help clean the names

In [8]:
def clean_name(name):
  clean=name.split(' (')[0].split('(')[0].replace('\xa0', ' ').split('MSc')[0].replace("WT and ST", "").replace("(", "").replace(")", "").replace("WT", "").replace("AT", "").replace("ST", "").replace("2021", "").replace("2022", "").replace("2023", "").replace("2024", "").replace("  ", " ")
  if name=='Andrew Murray':
    return'Professor Veerle Heyvaert'
  if clean[-1]==' ':
    clean=clean[:-1]
  if len(clean)==0:
    return clean
  if clean[0]==' ':
    return clean[1:]
  return clean
def rem_dup_dr(name):
  clean=name
  if 'Dr ' in name:
    clean = name.split('Dr ')[1]
  if 'W. ' in name:
    clean= clean.replace('W. ', '')
  return clean
def b_is_true(list_, role, key):
  return len(list_)==0 and role not in ['Academic visitors', 'Department committees ', 'Key Contacts', 'Research Students'] and key!='Language Centre'
def strong_is_true(list_, role):
  return len(list_)==0 and role not in ['Afilliated Research Fellows', 'Academic visitors', 'Emeritus', 'Department committees ', 'Guest teachers', 'Graduate teaching assistants (GTAs)', 'Professional services team ', 'Research Students', 'Emeritus, Affiliated & Visiting Academic Staff', 'Guest Lecturers']
def bad_first_link(name, list_):
  return name and name['href'] in ['/government', '/tii', '/statistics/people/xinghao-qiao'] and len(list_)>1
def title_is_a_name(name):
  return 'title' in name.attrs and name['title'] not in ['Hakan Sandal-Wilson', 'Christoph Hertrich', 'Home', 'home', 'View profile', 'European'] and '200' not in name['title'] and 'Department' not in name['title'] and 'European' not in name['title'] and 'LSE' not in name['title']
def b_name_is_true(name, person):
  return ('profile' in name or 'European' in name or 'Centre' in name or 'LSE' in name or 'Website' in name) and person
def span(name, person):
  return name and person.find('span') and 'lse.ac.uk' not in person.find('span').text and 'Email:' not in person.find('span').text and 'Visiting' not in person.find('span').text
def correct_link(name):
  return name and '@' not in name['href'] and ('/person' in name['href'] or 'author' in name['href'] or 'experts' in name['href'] or 'researcher' in name['href'] or 'amraj' in name['href'] or 'escp' in name['href'] or 'expert' in name['href'] or 'institute' in name['href'] or 'lucat/user' in name['href'] or 'rebeccanewton' in name['href'] or 'sites' in name['href'] or 'linkedin' in name['href'] or 'github' in name['href'] or 'Pages' in name['href'] or 'sociology' in name['href'] or '/People' in name['href'] or '/people' in name['href'] or 'about' in name['href'] or 'se/~tpers/' in name['href'] or 'personal' in name['href'] or 'spotlight' in name['href'] or 'profile' in name['href'])
def correct_name(name):
  return len(name)<50 and len(name.replace("xao", ''))>1 and '@' not in name and 'Visiting' not in name and 'Glucksmann, Miriam' not in name
def correct_link_a(person, role, a_person):
  return len(person)>0 and role not in ['PhD students and recent graduates','Key Contacts'] and 'Contact' not in a_person.text and 'href' in a_person.attrs and len(a_person.text)>2 and ('#' not in a_person['href'] or 'staffmember' in a_person['href'])
def correct_link_strong(person, dict_list):
  return person and len(person.text)>2 and ':' not in person.text and 'Ask me' not in person.text and 'Room' not in person.text and clean_name(person.text) not in dict_list and 'Guest' not in person.text and 'Graduate' not in person.text
def alt_link(person):
  return ':' not in person.text and not person.find('a') and '.' not in person.text and not person.find('strong') and 'Emeritus' not in person.text and 'Visiting' not in person.text and 'Affiliated' not in person.text
def deleted(dict_):
  for useless_el in ['Research students', 'CASE', 'Education Research and Policy Hub', 'Global Sexual & Gender Identities Policy Lab', 'International Inequalities Institute', 'Mannheim Centre for Criminology', 'Women in Social and Public Policy Research Hub']:
    del people_dict[useless_el]
  return dict_

Here is the actual webscraping code, it goes to each department page, and then selects the list of people, and their links if they have one per department. Some errors are of course to be expected as each department has a different people page layout.

In [9]:
all_dep_people_dict={}
my_deps=list(departments_dict.keys())
for key, value in departments_dict.items():
  if key in my_deps:
    response = requests.get(value['People_link'])
    soup = BeautifulSoup(response.content, 'html.parser')
    titles= soup.find_all('h2', class_='accordion__title')
    people_dict={}
    for el in range(len(titles)):
      if titles[el].text in people_dict.keys():
        accordion_section = soup.find_all('section', class_='accordion')[el]
        titles[el]=accordion_section.find_previous_sibling('h2')
      people_dict[titles[el].text] = {}
    roles= soup.find_all('div', class_='accordion__content')
    if len(roles)==0:
      div_content = soup.find('div', class_='pageContent pageContent--std')
      try:
          a_elements = div_content.find_all('a')
          hrefs = [a['href'] for a in a_elements[:-1]]
          titles = [a for a in a_elements[:-1]]
      except AttributeError as e:
          div_content=soup.find('div', class_='connections-list cn-list-body cn-clear')
          people_content=div_content.find_all('div', class_='cn-entry')
          people_dict['head']={}
          for person in people_content:
            href=None
            name=person.find('img')['alt'].split('Photo of ')[1]
            if person.find('a', class_='url'):
              href=person.find('a', class_='url')['href']
              people_dict['head'][rem_dup_dr(name)]=href
            if rem_dup_dr(name) not in people_dict['head'].keys():
              people_dict['head'][name]=href
            all_dep_people_dict[key]=people_dict
          continue
      roles=[]
      for href in range(len(hrefs)):
        if 'http://www.lse.ac.uk/' not in hrefs[href]:
          hrefs[href]=f"https://www.lse.ac.uk/{hrefs[href]}"
        people_dict[titles[href].text]={}
      for url in hrefs:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        roles.append(soup)
    for role in range(len(roles)):
      people= roles[role].find_all('div', class_="accordion__txt")
      if b_is_true(people, titles[role].text, key):
        people= roles[role].find_all('b')
      if strong_is_true(people, titles[role].text):
        people=roles[role].find_all('strong')[2:]
      if len(people)==0 and titles[role].text=='Academic visitors':
        people=roles[role].find_all('p')+roles[role].find_all('strong')
      for person in people:
        name=person.find('a', class_="sys_0 sys_t0")
        if not name and key=='LSE Law School':
          name=person.find('a', class_='sys_0 sys_t2644')
        if bad_first_link(name, person.find_all('a', class_="sys_0 sys_t0")):
          name=person.find_all('a', class_="sys_0 sys_t0")[1]
        if name is not None and 'href' in name.attrs and name['href']!='/government':
          if title_is_a_name(name):
            name, href= name['title'], f"https://www.lse.ac.uk{name['href']}"
            if name=='Andrew Murray':
              name=clean_name(name)
          else:
            name, href=name.text.split(' (')[0], f"https://www.lse.ac.uk{name['href']}"
            if name in ['More information', 'Profile']:
              name=person.find('strong').text
            if person.find('p') and b_name_is_true(name, person.find('p').find('b')):
              name=person.find('p').find('b').text
            elif 'profile' in name or 'LSE' in name:
              name=person.find('p').find('strong').text
          for link in person.find_all('a', class_="sys_0 sys_t0"):
            if link.text=='Profile' or 'More information' in link.text:
              href=f"https://www.lse.ac.uk{link['href']}"
          people_dict[titles[role].text][name]=href
          continue
        name=person.find('a', class_="sys_16")
        if span(name, person):
          name, href= person.find('span').text, f"{name['href']}"
          if 'mailto' in href:
            href=None
          if href and 'wiki/Geopolitical' in href:
            href=person.find_all('p')[-3].find('a')['href']
          people_dict[titles[role].text][clean_name(name)]=href
          continue
        if name and ('Website' in name.text or 'More information' in name.text or 'Find out' in name.text) and person.find('strong'):
            name, href= person.find('strong').text, f"{name['href']}"
            if name=='More information':
              name=person.find('strong').text
            if len(href)>=96:
              href=None
            people_dict[titles[role].text][name]=href
            continue
        if correct_link(name):
          name, href=name.text, name['href']
          if ('More information' in name or 'profile' in name) and person.find('p').find('b'):
            name=person.find('p').find('b').text
          elif 'rofile' in name:
            name=person.find('p').find('strong').text
          if 'Dr' in name and len(name)<=5:
            name=person.find('a')['title']
          people_dict[titles[role].text][name.split(' (')[0]]=href
          continue
        else:
          name, href=person.find('strong'), None
          if name:
            name, href=clean_name(name.text), None
            temp_name=name.split('-')
            if ('Research interests' in name or 'Email' in name) and (person.find('h1') or person.find('h3')):
              if person.find('h1'):
                name=person.find('h1').text.split(':')[0].split('|')[0]
                if person.find('h1').find('a') and 'png' not in person.find('h1').find('a')['href']:
                  href=person.find('h1').find('a')['href']
              else:
                name=person.find('h3').text.split(':')[0].split('|')[0]
            if len(temp_name)>1:
              try:
                if len(temp_name[1]) == 0 or 'E' in temp_name[1]:
                  name = name.split('-')[0]
              except IndexError:
                name = name.split('-')[0]
            for link in person.find_all('a', class_="sys_16"):
              if 'title' in link.attrs.keys():
                if name in link['title'] and '@' not in link['href']:
                  href=link['href']
              if 'people/graduate' not in link['href'] and 'people/department' not in link['href'] and 'eople' in link['href']:
                href=link['href']
            if 'Departmental' in name:
              name=person.find_all('strong')[1].text
            if ':' in name and person.find('b'):
              name=person.find('b').text
            if 'Law School' in name or 'Department' in name:
              name=person.find('a').text
            if 'Vacancy' not in name and 'Ask me' not in name:
              people_dict[titles[role].text][name.split(' -')[0]]=href
            continue
          if person.find('h2'):
            name, href=person.find('h2'), None
            if name:
              name, href=name.find('b').text.replace("-", "").replace("  ", ""), None
              people_dict[titles[role].text][name]=href
              continue
          if person.find('p') and '<br/>' in [str(item) for item in person.find('p').contents] and not person.find('b'):
            name, href=person.find('p').contents[0].text, None
            if '<b>' in name:
              name.replace("<b>", "").replace("</b>", "")
            people_dict[titles[role].text][name]=href
            continue
          if person.find('b'):
            name, href=person.find('b').text.split('(')[0], None
            if ':' in name:
              continue
            for link in person.find_all('a'):
              if 'sites' in link['href'] or 'james' in link['href'] or '/people' in link['href']:
                href=link['href']
            people_dict[titles[role].text][clean_name(name)]=href
            continue
          else:
            name, href=person.text, None
            name=name.split('\nV')[0].replace('\n', '').split(' (')[0].split('(')[0]
            if correct_name(name):
              people_dict[titles[role].text][clean_name(name)]=href
              continue
      if len(people)==0 and key!='Department of Methodology':
        people=roles[role].find_all('p')
        if len(people)==7 and titles[role].text!='Spanish & Italian':
          for paragraph in people[:3]:
            text_nodes = paragraph.find_all(string=True)
            if len(text_nodes) > 1:
                name = text_nodes[1].strip()
                people_dict[titles[role].text][name]=None
        else:
          for person in people:
            href=None
            if correct_link_a(person.find_all('a'), titles[role].text, person.find('a')):
              for diff_person in person.find_all('a'):
                href=diff_person['href']
                if '//' not in href:
                  href='https://www.lse.ac.uk/'+href
                name=diff_person.text
                if name!='Read' and '@' not in name and len(name)>0:
                  people_dict[titles[role].text][clean_name(name)]=href
            if correct_link_strong(person.find('strong'), people_dict[titles[role].text].keys()):
              people_dict[titles[role].text][clean_name(person.find('strong').text)]=None
            elif person.find('b') and clean_name(person.find('b').text) not in people_dict[titles[role].text].keys():
              people_dict[titles[role].text][clean_name(person.find('b').text)]=None
            elif alt_link(person):
              name=clean_name(person.text.split('|')[0])
              if len(name)>2 and len(name)<30 and name not in people_dict[titles[role].text].keys():
                people_dict[titles[role].text][name]=href
      if len(people)==0:
          people=roles[role].find_all('li')
          for person in people:
            if person.find('a') and 'title' in person.find('a').attrs:
              people_dict[titles[role].text][person.find('a')['title']]=f"https://www.lse.ac.uk/{person.find('a')['href']}"
            elif person.find('a') and '@' not in person.find('a')['href']:
              people_dict[titles[role].text][person.find('a').text]=person.find('a')['href']
            elif person.find('strong'):
              people_dict[titles[role].text][person.find('strong').text]=None
            else:
              people_dict[titles[role].text][person.text.split(' (')[0]]=None
      if soup.find('strong').text=='Senior Management Team' or soup.find('strong').text=='Leadership':
        people_dict['Head']={}
        for person in soup.find_all('div', class_='accordion__txt')[:4]:
          name=person.find('strong').text.split('-')[0]
          href=None
          if person.find('a', class_="sys_0 sys_t0"):
            name, href=person.find('strong').text.split('-')[0], f"https://www.lse.ac.uk{person.find('a', class_='sys_0 sys_t0')['href']}"
          people_dict['Head'][name]=href
      if soup.find('h3') and 'Professional services staff' in soup.find('h3').text:
        people_dict['Head']={}
        for person in soup.find_all('div', class_='accordion__txt')[:10]:
          name, href=person.find('a').text, person.find('a')['href']
          if 'www.lse.ac.uk' not in href:
            href=f"http://www.lse.ac.uk{href}"
          people_dict['Head'][name]=href
      if soup.find('h3') and 'Management team' in soup.find('h3').text:
        people_dict['Head']={}
        for person in soup.find_all('div', class_='accordion__txt')[:3]:
          name, href=person.find('a').text, person.find('a')['href']
          if 'www.lse.ac.uk' not in href:
            href=f"http://www.lse.ac.uk{href}"
          if '@' in name:
            name, href=person.find('strong').text, None
          people_dict['Head'][name]=href

    if 'CASE' in people_dict.keys():
      people_dict=deleted(people_dict)
    els_to_del=[]
    for role in people_dict.keys():
      if len(people_dict[role])==0:
        els_to_del.append(role)
    for el in els_to_del:
      del people_dict[el]


    all_dep_people_dict[key]=people_dict
all_dep_people_dict

{'Department of Accounting': {'Academic Faculty': {'Mr Per Ahblom': 'https://www.lse.ac.uk/accounting/people/per-ahblom',
   'Professor Alnoor Bhimani': 'https://www.lse.ac.uk/accounting/people/al-bhimani',
   'Dr Jose Carabias Palmeiro': 'https://www.lse.ac.uk/accounting/people/jose-carabias-palmeiro',
   'Dr Stefano Cascino': 'https://www.lse.ac.uk/accounting/people/stefano-cascino',
   'Furkan Cetin': 'https://www.lse.ac.uk/accounting/people/Furkin-Cetin/Furkan-Cetin',
   'Dr Maria Correia': 'https://www.lse.ac.uk/accounting/people/maria-correia',
   'Pascal Frantz': 'https://www.lse.ac.uk/accounting/people/pascal-frantz',
   'Saipriya Kamath': 'https://www.lse.ac.uk/accounting/people/saipriya-kamath',
   'Gulnaz Khamidullina': 'https://www.lse.ac.uk/accounting/people/Gulnaz-Khamidullina/Gulnaz-Khamidullina',
   'Jeroen Koenraadt': 'https://www.lse.ac.uk/accounting/people/Jeroen-Koenraadt/Jeroen-Koenraadt',
   'Dr Xi Li': 'https://www.lse.ac.uk/accounting/people/xi-li',
   'Dr Nadia

### 4. TESTING, please read instructions below

Below is the result the above gave us on the 29th of April. Due to the webpages getting updated, only if the above code does not work, please run the cell below and use the dictionnary we got on the 29th of april: 

In [10]:
all_dep_people_dict={'Department of Accounting': {'Academic Faculty': {'Mr Per Ahblom': 'https://www.lse.ac.uk/accounting/people/per-ahblom',
   'Professor Alnoor Bhimani': 'https://www.lse.ac.uk/accounting/people/al-bhimani',
   'Dr Jose Carabias Palmeiro': 'https://www.lse.ac.uk/accounting/people/jose-carabias-palmeiro',
   'Dr Stefano Cascino': 'https://www.lse.ac.uk/accounting/people/stefano-cascino',
   'Furkan Cetin': 'https://www.lse.ac.uk/accounting/people/Furkin-Cetin/Furkan-Cetin',
   'Dr Maria Correia': 'https://www.lse.ac.uk/accounting/people/maria-correia',
   'Pascal Frantz': 'https://www.lse.ac.uk/accounting/people/pascal-frantz',
   'Saipriya Kamath': 'https://www.lse.ac.uk/accounting/people/saipriya-kamath',
   'Gulnaz Khamidullina': 'https://www.lse.ac.uk/accounting/people/Gulnaz-Khamidullina/Gulnaz-Khamidullina',
   'Jeroen Koenraadt': 'https://www.lse.ac.uk/accounting/people/Jeroen-Koenraadt/Jeroen-Koenraadt',
   'Dr Xi Li': 'https://www.lse.ac.uk/accounting/people/xi-li',
   'Dr Nadia Matringe': 'https://www.lse.ac.uk/accounting/people/nadia-matringe',
   'Andrea Mennicken': 'https://www.lse.ac.uk/accounting/people/andrea-mennicken',
   'Dr Julia Morley': 'https://www.lse.ac.uk/accounting/people/julia-morley',
   'Jacob Ott': 'https://www.lse.ac.uk/accounting/people/jacon-ott/Jacob-Ott',
   'Tommaso Palermo': 'https://www.lse.ac.uk/accounting/people/tommaso-palermo',
   'Michael Power': 'https://www.lse.ac.uk/accounting/people/michael-power',
   'Dr Aneesh Raghunandan': 'https://www.lse.ac.uk/accounting/people/aneesh-raghunandan',
   'Alexa Scherf': 'https://www.lse.ac.uk/accounting/people/Alexandra-Scherf/Alexandra-Scherf',
   'Ane Tamayo': 'https://www.lse.ac.uk/accounting/people/ane-tamayo',
   'Oscar Timmermans': 'https://www.lse.ac.uk/accounting/people/Oscar-Timmermans/Oscar-Timmermans',
   'Professor Wim A Van der Stede': 'https://www.lse.ac.uk/accounting/people/academic-faculty/wim-a-van-der-stede'},
  'Emeritus Faculty\xa0': {'Professor Richard Macve': 'https://www.lse.ac.uk/accounting/people/richard-macve',
   'Emeritus Professor Michael Bromwich': 'https://www.lse.ac.uk/accounting/people/visiting-faculty/michael-bromwich',
   'Professor Peter Pope': 'https://www.lse.ac.uk/accounting/people/peter-pope',
   'Peter Miller': 'https://www.lse.ac.uk/accounting/people/peter-miller'},
  'BSc year 1 and 2 Learning Support and Mentors': {'Chris Constantinou': None,
   'David Trodden': None,
   'Ebrahim Mohamed': None},
  'Guest Teachers': {'Khamid Irgashev': None,
   'Dr Farooq Mahmood': None,
   'Imran Malik': None,
   'Thomas McDermott': None,
   'Philip McDonald': None},
  'Professional Services Staff': {'Yvonne Guthrie\xa0 ': None,
   'Gary Crow': None,
   'Jack Heaney': None,
   'Sandra Ma': None,
   'Rebecca Baker\xa0\xa0': None,
   'Justin Adams\xa0': None,
   'Dorothy': None,
   'Yuhua ChenHudson ': None},
  'Research Students': {'Alistair Marsden': 'https://www.lse.ac.uk/accounting/people/phd-students/alistair-marsden',
   'Yinan Li': 'https://www.lse.ac.uk/accounting/people/phd-students/Yinan-Li',
   'Kathyayini Madduri': 'https://www.lse.ac.uk/accounting/people/phd-students/kathyayini-madduri',
   'Anna Romanova': 'https://www.lse.ac.uk/accounting/people/phd-students/Anna-Romanova',
   'Wan Chu Cheong': 'https://www.lse.ac.uk/accounting/people/phd-students/Wan-Chu-Cheong',
   'Xinyun Liu': 'https://www.lse.ac.uk/accounting/people/phd-students/Xinyun-Liu',
   'Parth Shah': 'https://www.lse.ac.uk/accounting/people/phd-students/Parth-Shah',
   'Jiachen Gao': 'https://www.lse.ac.uk/accounting/people/phd-students/Jiachen-Gao',
   'Lorenzo Pirozzi': 'https://www.lse.ac.uk/accounting/people/phd-students/Lorenzo-Pirozzi'}},
 'Department of Anthropology': {'Academic staff': {'Dr Catherine Allerton': 'https://www.lse.ac.uk/anthropology/people/catherine-allerton',
   'Professor Rita Astuti': 'https://www.lse.ac.uk/anthropology/people/rita-astuti',
   'Dr Mukulika Banerjee': 'https://www.lse.ac.uk/anthropology/people/mukulika-banerjee',
   'Professor Laura Bear': 'https://www.lse.ac.uk/anthropology/people/laura-bear',
   'Dr Fenella Cannell': 'https://www.lse.ac.uk/anthropology/people/fenella-cannell',
   'Dr Clara Devlieger': 'https://www.lse.ac.uk/anthropology/people/clara-devlieger',
   'Dr Yazan Doughan': 'https://www.lse.ac.uk/anthropology/people/yazan-doughan/Dr-Yazan-Doughan',
   'Professor Katy Gardner': 'https://www.lse.ac.uk/anthropology/people/katy-gardner',
   'Professor Deborah James': 'https://www.lse.ac.uk/anthropology/people/deborah-james',
   'Dr Nicholas Long': 'https://www.lse.ac.uk/anthropology/people/nicholas-long',
   'Dr Mathijs Pelkmans': 'https://www.lse.ac.uk/anthropology/people/mathijs-pelkmans',
   'Andrea Pia': 'https://www.lse.ac.uk/anthropology/people/andrea-pia',
   'Dr Michael W. Scott': 'https://www.lse.ac.uk/anthropology/people/michael-scott',
   'Dr Alpa Shah': 'https://www.lse.ac.uk/anthropology/people/alpa-shah',
   'Professor Charles Stafford': 'https://www.lse.ac.uk/anthropology/people/charles-stafford',
   'Dr Hans Steinmüller': 'https://www.lse.ac.uk/anthropology/people/hans-steinmueller',
   'Dr Harry Walker': 'https://www.lse.ac.uk/anthropology/people/harry-walker',
   'Dr Gisa Weszkalnys': 'https://www.lse.ac.uk/anthropology/people/gisa-weszkalnys',
   'Dr Mareike Winchell': 'https://www.lse.ac.uk/anthropology/people/Mareike-Winchell/Dr-Mareike-Winchell'},
  'Administrative staff': {'Ms Yanina Hinrichsen': None,
   'Ms Chloe Davies': None,
   'Ms Megan Garry-Evans': None,
   'Mr Tomas Hinrichsen': None,
   'Mr James Johnston': None,
   'Ms Renata Todd': None},
  'Affiliated Anthropologists\xa0': {'Dr Noah Walker-Crawford': 'https://www.lse.ac.uk/anthropology/people/noah-crawford-walker/Dr-Noah-Walker-Crawford'},
  'Research and LSE Fellows': {'Dr Anjana Bala': 'https://www.lse.ac.uk/anthropology/people/anjana-bala/Dr-Anjana-Bala',
   'Dr Gareth Breen': 'https://www.lse.ac.uk/anthropology/people/gareth-breen/Dr-Gareth-Breen',
   'Dr Agathe Faure': 'https://www.lse.ac.uk/anthropology/people/PhD-profiles/Agathe-Faure/Agathe-Faure',
   'jan david hauck': 'https://www.lse.ac.uk/anthropology/people/jan-david-hauck/jan-david-hauck',
   'Dr Mayanka Mukherji': None,
   'Dr Stephanie Postar': 'https://www.lse.ac.uk/anthropology/people/stephanie-postar/Dr-Stephanie-Postar',
   'Dr Sofía Ugarte': 'https://www.lse.ac.uk/anthropology/people/sofia-ugarte/sofia-ugarte',
   'Dr Frederick Wojnarowski': 'https://www.lse.ac.uk/anthropology/people/Frederick-Wojnarowski/Dr-Frederick-Wojnarowski',
   'Dr Teodor Zidaru': 'https://www.lse.ac.uk/anthropology/people/Teodor-Zidaru/Dr-Teodor-Zidaru'},
  'Visiting staff': {'Professor Karin Barber': 'https://www.lse.ac.uk/anthropology/people/karin-barber/karin-barber',
   'Eona Bell': 'https://www.lse.ac.uk/anthropology/people/Eona-Bell/Eona-Bell',
   'Hélène Bloch': 'https://www.lse.ac.uk/anthropology/people/helene-bloch/Hélène-Bloch',
   'Dr Rebecca Bowers': 'https://www.lse.ac.uk/anthropology/people/becky-bowers/Dr-Rebecca-Bowers',
   'Sanda Caracentev': 'https://www.lse.ac.uk/anthropology/people/sanda-caracentev/Sanda-Caracentev',
   'Dr Chris Chaplin': 'https://www.lse.ac.uk/anthropology/people/chris-chaplin/Dr-Chris-Chaplin',
   'Professor Harriet Evans': 'https://www.lse.ac.uk/anthropology/people/harriet-evans/Professor-Harriet-Evans',
   'Dr Luis Garcia Briceno': 'https://www.lse.ac.uk/anthropology/people/luis-garcia-briceno/Dr-Luis-Garcia-Briceno',
   'Dr Charlotte Hawkins': 'https://www.lse.ac.uk/anthropology/people/charlotte-hawkins/Dr-Charlotte-Hawkins',
   'Dr Carrie Heitmeyer': 'https://www.lse.ac.uk/anthropology/people/Carrie-Heitmeyer/Dr-Carrie-Heitmeyer',
   'Dr Mathilde Heslon': 'https://www.lse.ac.uk/anthropology/people/mathilde-heslon/Dr-Mathilde-Heslon',
   'Insa Koch': 'https://www.lse.ac.uk/anthropology/people/insa-koch/Insa-Koch',
   'Professor Adam Kuper': 'https://www.lse.ac.uk/anthropology/people/adam-kuper',
   'Dr Megan Laws ': 'https://www.lse.ac.uk/anthropology/people/megan-laws',
   'Dr Loretta Lou': 'https://www.lse.ac.uk/anthropology/people/loretta-lou',
   'Dr Arthur Mason': 'https://www.lse.ac.uk/anthropology/people/arthur-mason/Dr-Arthur-Mason',
   'Dr William Matthews': 'https://www.lse.ac.uk/anthropology/people/william-matthews',
   'Claire Moll Namas': 'https://www.lse.ac.uk/anthropology/people/claire-moll-namas/Claire-Moll-Namas',
   'Dr Fuad Musallam': 'https://www.lse.ac.uk/anthropology/people/fuad-musallam',
   'Dr Giulio Ongaro': 'https://www.lse.ac.uk/anthropology/people/Giulio-Ongaro/Dr-Giulio-Ongaro',
   'Dr Ottavia Paterno': 'https://www.lse.ac.uk/anthropology/people/ottavia-paterno/Dr-Ottavia-Paterno',
   'Dr Dagna Rams': 'https://www.lse.ac.uk/anthropology/people/dagna-rams/Dr-Dagna-Rams',
   'Zimran Samuel': 'https://www.lse.ac.uk/anthropology/people/zimran-samuel/Zimran-Samuel',
   'Dr Marina Sapritsky-Nahum': 'https://www.lse.ac.uk/anthropology/people/marina-sapritsky',
   'Dr Mitchell Sedgwick': 'https://www.lse.ac.uk/anthropology/people/mitchell-sedgwick',
   'Dr Imani Strong': 'https://www.lse.ac.uk/anthropology/people/imani-strong/Dr-Imani-Strong',
   'Kelzang Tashi': 'https://www.lse.ac.uk/anthropology/people/kelzang-tashi/Dr-Kelzang-Tashi',
   'Professor Sylvia Yanagisako': 'https://www.lse.ac.uk/anthropology/people/sylvia-yanagisako/Professor-Sylvia-Yanagisako'},
  'Retired academic staff': {'Professor Maurice Bloch': 'https://www.lse.ac.uk/anthropology/people/maurice-bloch',
   'Professor Stephan Feuchtwang': 'https://www.lse.ac.uk/anthropology/people/stephan-feuchtwang',
   'Professor Jean La Fontaine': None,
   'Professor Chris Fuller': 'https://www.lse.ac.uk/anthropology/people/chris-fuller',
   'Professor Martha Mundy': 'https://www.lse.ac.uk/anthropology/people/martha-mundy',
   'Professor Jonathan Parry': 'https://www.lse.ac.uk/anthropology/people/jonathan-parry'}},
 'Data Science Institute': {'Academic Staff': {'Kenneth Benoit': 'https://www.lse.ac.uk/Methodology/People/Academic-Staff/Kenneth-Benoit/Kenneth-Benoit',
   'Jonathan Cardoso-Silva': 'https://www.lse.ac.uk/DSI/People/Jonathan-Cardoso-Silva',
   'Dr Ghita Berrada': 'https://www.lse.ac.uk/DSI/People/Ghita-Berrada',
   'Jinshuai Ma': 'https://www.lse.ac.uk/DSI/People/Jinshua-Ma',
   'Professor Zoltán Szabó': 'https://www.lse.ac.uk/statistics/people/zoltan-szabo'},
  'Professional Services Staff': {'Jill Beattie': None,
   'Ellie Finlay': None,
   'Lou Flandrin': None,
   'Kevin Kittoe': None,
   'Johanna Tate': None,
   'Ali Eggleton': None},
  'Guest Teachers': {'Dr Stuart Bramwell': None,
   'Tabtim Duenger': None,
   'Riya Chhikara': None,
   'Sara Luxmoore': None,
   'Mustafa Can Ozkan': None,
   'Alexander Soldatkin': None,
   'Andreas': None},
  'PhD Students': {'Vincent Chung': 'https://www.lse.ac.uk/geography-and-environment/people/phd-students/vincent-chung-jie-lun',
   'Wendy Wang': 'https://www.lse.ac.uk/Methodology/People/Research-Students/Wanting-Wang/Wanting-Wang'},
  'DSI Affiliates': {'Dr\xa0Miqdad Asaria': 'https://www.lse.ac.uk/health-policy/people/miqdad-asaria',
   'Dr Mona Azadkia': 'https://sites.google.com/view/monaazadkia/',
   'Dr\xa0Marcos Barreto': 'https://www.lse.ac.uk/Statistics/People/Dr-Marcos-Barreto',
   'Dr Tuğkan Batu': 'http://www.maths.lse.ac.uk/Personal/batu/',
   'Dr Liam Beiser-McGrath': 'https://www.liambeisermcgrath.com',
   'Dr Michael Blackwell': 'https://personal.lse.ac.uk/blackwel/',
   'Dr Chris Blunt': 'http://cjblunt.com/',
   'Dr Siân Brooke': 'https://www.sianbrooke.co.uk/',
   'Professor Umut Cetin': 'https://www.lse.ac.uk/Statistics/People/Professor-Umut-Cetin',
   'Dr Yining Chen': 'https://www.lse.ac.uk/statistics/people/yining-chen',
   'Dr\xa0Yunxiao Chen': 'https://stats.lse.ac.uk/cheny185/',
   'Dr Neil Cummins': 'http://neilcummins.com/',
   'Dr Daniel De Kadt': 'https://www.ddekadt.com',
   'Zachary Dickson': 'https://www.lse.ac.uk/government/people/research-staff/zachary-dickson',
   'Professor Alexander Evans': 'https://www.lse.ac.uk/school-of-public-policy/people/alexander-evans',
   'Dr Giulia Ferrari': 'https://www.lse.ac.uk/women-peace-security/people/Giulia-Ferrari',
   'Dr Florian Foos': 'http://www.florianfoos.net/',
   'Dr Michael Ganslmeier': 'https://www.michael-ganslmeier.com/',
   'Dr Alexandra Gomes': 'https://www.lse.ac.uk/Cities/staff-profiles/Alexandra-Gomes',
   'Dr Patrick Gildersleve': 'https://www.lse.ac.uk/Methodology/People/Academic-Staff/Patrick-Gildersleve/Patrick-Gildersleve',
   'Dr Anushri Gupta': 'https://www.lse.ac.uk/management/people/research-staff/agupta',
   'Dr Kostas Kalogeropoulos': 'https://www.lse.ac.uk/Statistics/People/Dr-Kostas-Kalogeropoulos',
   'Dr Casey Kearney': 'https://sites.google.com/g.harvard.edu/caseykearney/home',
   'Dr Marion Lieutaud': 'https://www.lse.ac.uk/Methodology/People/Academic-Staff/Marion-Lieutaud/Marion-Lieutaud',
   'Dr Joshua Loftus': 'https://joshualoftus.com/',
   'Dr Jens Koed Madsen': 'https://www.lse.ac.uk/PBS/People/Dr-Jens-Koed-Madsen',
   'Dr Blake Miller': 'http://blakeapm.com/',
   'Andrew Moles': 'https://info.lse.ac.uk/current-students/digital-skills-lab',
   'Dr Carl Müller-Crepon': 'http://carlmueller-crepon.org/',
   'Dr Michael Muthukrishna': 'http://michael.muthukrishna.com/',
   'Dr Marie Oldfield': 'http://www.oldfieldconsultancy.co.uk',
   'Dr Katerina Papadaki': 'https://www.lse.ac.uk/Mathematics/people/Katerina-Papadaki',
   'Dr Francesca Panero': 'https://www.lse.ac.uk/statistics/people/francesca-panero',
   'Dr Jean-Christophe Plantin': 'https://www.lse.ac.uk/media-and-communications/people/academic-staff/jean-christophe-plantin',
   'Mathias Poertner': 'http://www.mathiaspoertner.com/',
   'Dr\xa0Alison Powell': 'https://www.lse.ac.uk/media-and-communications/people/academic-staff/alison-powell',
   'Dr\xa0Eleanor Power': 'https://www.lse.ac.uk/Methodology/People/Academic-Staff/Eleanor-Power/Eleanor-Power',
   'Dr\xa0Xinghao Qiao': 'https://personal.lse.ac.uk/qiaox/',
   'Professor Johannes Ruf': 'http://www.maths.lse.ac.uk/Personal/jruf/',
   'Dr Thomas Robinson': 'https://ts-robinson.com/',
   'Dr Dorottya Sallai': 'https://www.lse.ac.uk/management/people/academic-staff/dsallai',
   'Dr Melissa Sands': 'https://www.lse.ac.uk/government/people/academic-staff/melissa-sands',
   'Dr\xa0Chengchun Shi': 'https://callmespring.github.io/',
   'Dr Rachel Spicer': 'https://raspicer.github.io/',
   'Dr Divya Srivastrava': 'https://www.lse.ac.uk/health-policy/people/dr-divya-srivastava',
   'Professor\xa0Zoltán Szabó': 'https://zoltansz.github.io/',
   'Dr Milena Tsvetkova': 'http://tsvetkova.me/',
   'Professor László Végh': 'https://personal.lse.ac.uk/veghl/',
   'Dr Will Venters': 'https://www.willventers.com/',
   'Dr Kate Vredenburgh': 'https://katevredenburgh.com/',
   'Dr Tengyao Wang': 'https://personal.lse.ac.uk/wangt60/',
   'Dr Yan Wang': 'https://ywangsoc.com/',
   'Professor Henry Wynn': 'https://www.lse.ac.uk/statistics/people/henry-wynn'},
  'Visiting Appointments': {'Dr Nicole Baerg': 'http://www.nicolebaerg.com',
   'John Burn-Murdoch': 'https://www.ft.com/stream/e191658e-c66a-45bc-9bad-343bdc4210b3',
   'Dr Gökhan Çiflikli': 'https://www.gokhan.io/',
   'Dr James Hamp\xa0': 'https://jameshamp.github.io/',
   'Ajay Kumar ': 'https://em-lyon.com/en/ajay-kumar',
   'Dr Tom Paskhalis': 'https://tom.paskhal.is/',
   'Professor Leonard Smith': 'https://ece.vt.edu/people/profile/lennys',
   'Dr Miriam Sorace': 'https://miriamsorace.github.io/',
   'Dr Erica Thompson': 'https://www.ericathompson.co.uk/',
   'Dr Joshua Townsley': 'https://www.linkedin.com/in/joshua-townsley-phd-72268b77/'},
  'Visiting Research Students': {'Philipp Darius': None,
   'Fintan Oeri': None,
   'Clint Claessen': None,
   'Riccardo Ricciardi': None,
   'Paride Carrara': None,
   'Nader Hotait': None,
   'Anastasiia Menshikova': None},
  'DSI Management Committee': {'Professor Bernhard Von Stengel': 'https://www.lse.ac.uk/Mathematics/people/Bernhard-Von-Stengel',
   'Professor Patrick Sturgis': 'https://www.lse.ac.uk//Methodology/People/Academic-Staff/Patrick-Sturgis/Patrick-Sturgis',
   'Professor Zoltán Szabó': 'https://www.lse.ac.uk/Statistics/People/Professor-Zoltan-Szabo',
   'Dr Kate Vredenburgh': 'https://katevredenburgh.com/',
   'Professor Olmo Silva': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/olmo-silva',
   'Professor Michael Laver': 'https://scholar.google.com/citations?user=xIEarMwAAAAJ&hl=en',
   'Professor Susana Mourato': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/susana-mourato'},
  'Data Science Education Forum': {'Professor Irini Moustaki': 'https://www.lse.ac.uk/Statistics/People/Professor-Irini-Moustaki',
   'Professor Peter Allen': 'https://www.lse.ac.uk/Mathematics/people/Peter-Allen',
   'Dr Milena Tsvetkova': 'https://www.lse.ac.uk/Methodology/People/Academic-Staff/Milena-Tsvetkova/Milena-Tsvetkova',
   'Dr Melissa Sands': 'https://www.lse.ac.uk/government/people/academic-staff/melissa-sands',
   'Dr Thomas Smith': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/thomas-smith',
   'Jeni Brown': 'https://www.linkedin.com/in/jeni-brown-65ba8aa',
   'Dr Yang Yang': None,
   'Professor Emma McCoy': 'https://info.lse.ac.uk/staff/divisions/executive-office/people/emma-mccoy/Emma-McCoy'}},
 'Department of Economics': {'Faculty A-Z': {'Philippe Aghion': 'https://www.lse.ac.uk/economics/people/faculty/philippe-aghion',
   'Nava Ashraf': 'https://www.lse.ac.uk/economics/people/faculty/nava-ashraf',
   'Oriana Bandiera': 'https://www.lse.ac.uk/economics/people/faculty/oriana-bandiera',
   'Clare Balboni': 'https://www.lse.ac.uk/economics/people/faculty/clare-balboni',
   'Charles Bean': 'https://www.lse.ac.uk/economics/people/faculty/charles-bean',
   'Tim Besley': 'https://www.lse.ac.uk/economics/people/faculty/tim-besley',
   'Mohan Bijapur': 'https://www.lse.ac.uk/economics/people/faculty/mohan-bijapur',
   'Juraj Briskar': 'https://www.lse.ac.uk/economics/people/faculty/juraj-briskar',
   'Gharad Bryan': 'https://www.lse.ac.uk/economics/people/faculty/gharad-bryan',
   'Robin Burgess': 'https://www.lse.ac.uk/economics/people/faculty/robin-burgess',
   'Michael Callen': 'https://www.lse.ac.uk/economics/people/faculty/michael-callen',
   'Francesco Caselli': 'https://www.lse.ac.uk/economics/people/faculty/francesco-caselli',
   'Frank Cowell': 'https://www.lse.ac.uk/economics/people/faculty/frank-cowell',
   'Wouter Den Haan': 'https://www.lse.ac.uk/economics/people/faculty/wouter-den-haan',
   'Maarten De-Ridder': 'https://www.lse.ac.uk/economics/people/faculty/maarten-de-ridder',
   'Swati Dhingra': 'https://www.lse.ac.uk/economics/people/faculty/swati-dhingra',
   'Jeremiah Dittmar': 'https://www.lse.ac.uk/economics/people/faculty/jeremiah-dittmar',
   'Matthias Doepke': 'https://www.lse.ac.uk/economics/people/faculty/matthias-doepke',
   'Andrew Ellis': 'https://www.lse.ac.uk/economics/people/faculty/andrew-ellis',
   'Alessandro Gavazza': 'https://www.lse.ac.uk/economics/people/faculty/alessandro-gavazza',
   'Maitreesh Ghatak': 'https://www.lse.ac.uk/economics/people/faculty/maitreesh-ghatak',
   'Michael Gmeiner': 'https://www.lse.ac.uk/economics/people/faculty/michael-gmeiner',
   'Vassilis Hajivassiliou': 'https://www.lse.ac.uk/economics/people/faculty/vassilis-hajivassiliou',
   'Jonathon Hazell': 'https://www.lse.ac.uk/economics/people/faculty/jonathon-hazell',
   'Javier Hidalgo': 'https://www.lse.ac.uk/economics/people/faculty/havier-hidalgo',
   'Ethan Ilzetzki': 'https://www.lse.ac.uk/economics/people/faculty/ethan-ilzetzki',
   'Xavier Jaravel': 'https://www.lse.ac.uk/economics/people/faculty/xavier-jaravel',
   'Keyu Jin': 'https://www.lse.ac.uk/economics/people/faculty/keyu-jin',
   'Katarzyna Krajniewska': 'https://www.lse.ac.uk/economics/people/faculty/katarzyna-krajniewska',
   'Camille Landais': 'https://www.lse.ac.uk/economics/people/faculty/camille-landais',
   'Jonathan Leape': 'https://www.lse.ac.uk/economics/people/faculty/jonathan-leape',
   'Gilat Levy': 'https://www.lse.ac.uk/economics/people/faculty/gilat-levy',
   'Matthew Levy': 'https://www.lse.ac.uk/economics/people/faculty/matthew-levy',
   'Stephen Machin': 'https://www.lse.ac.uk/economics/people/faculty/stephen-machin',
   'Isabela Manelici': 'https://www.lse.ac.uk/economics/people/faculty/isabela-manelici',
   'Alan Manning': 'https://www.lse.ac.uk/economics/people/faculty/alan-manning',
   'Antonio Mele': 'https://www.lse.ac.uk/economics/people/faculty/antonio-mele',
   'Guy Michaels': 'https://www.lse.ac.uk/economics/people/faculty/guy-michaels',
   'Benjamin Moll': 'https://www.lse.ac.uk/economics/people/faculty/benjamin-moll',
   'John Moore': 'https://www.lse.ac.uk/economics/people/faculty/john-moore',
   'Dmitry Mukhin': 'https://www.lse.ac.uk/economics/people/faculty/dmitry-mukhin',
   'Francesco Nava': 'https://www.lse.ac.uk/economics/people/faculty/francesco-nava',
   'Rachel Ngai': 'https://www.lse.ac.uk/economics/people/faculty/rachel-ngai',
   'Junius Olivier': 'https://www.lse.ac.uk/economics/people/faculty/junius-olivier',
   'Taisuke Otsu': 'https://www.lse.ac.uk/economics/people/faculty/taisuke-otsu',
   'Martin Pesendorfer': 'https://www.lse.ac.uk/economics/people/faculty/martin-pesendorfer',
   'Dimitra Petropoulou': 'https://www.lse.ac.uk/economics/people/faculty/dimitra-petropoulou',
   'Michele Piccione': 'https://www.lse.ac.uk/economics/people/faculty/michele-piccione',
   'Steve Pischke': 'https://www.lse.ac.uk/economics/people/faculty/steve-pischke',
   'Christopher Pissarides': 'https://www.lse.ac.uk/economics/people/faculty/christopher-pissarides',
   'Ronny Razin': 'https://www.lse.ac.uk/economics/people/faculty/ronny-razin',
   'Ricardo Reis': 'https://www.lse.ac.uk/economics/people/faculty/ricardo-reis',
   'David Ren': 'https://www.lse.ac.uk/economics/people/faculty/david-ren',
   'Nico Rosetti': 'https://www.lse.ac.uk/economics/people/faculty/nicolo-rosetti',
   'Jane Olmstead-Rumsey': 'http://www.lse.ac.uk/economics/people/faculty/jane-olmstead-rumsey',
   'Ragvir Sabharwal': 'https://www.lse.ac.uk/economics/people/faculty/ragvir-sabharwal',
   'Thomas Sampson': 'https://www.lse.ac.uk/economics/people/faculty/thomas-sampson',
   'Christopher Sandmann': 'https://www.lse.ac.uk/economics/people/faculty/christopher-sandmann',
   'Marcia Schafgans': 'https://www.lse.ac.uk/economics/people/faculty/marcia-schafgans',
   'Mark Schankerman': 'https://www.lse.ac.uk/economics/people/faculty/mark-schankerman',
   'Pasquale Schiraldi': 'https://www.lse.ac.uk/economics/people/faculty/pasquale-schiraldi',
   'Judith Shapiro': 'https://www.lse.ac.uk/economics/people/faculty/judith-shapiro',
   'Kevin Sheedy': 'https://www.lse.ac.uk/economics/people/faculty/kevin-sheedy',
   'Kate Smith': 'http://www.lse.ac.uk/economics/people/faculty/kate-smith',
   'Chiara Sotis': 'https://www.lse.ac.uk/economics/people/faculty/chiara-sotis',
   'Johannes Spinnewijn': 'https://www.lse.ac.uk/economics/people/faculty/johannes-spinnewijn',
   'Nicholas Stern': 'https://www.lse.ac.uk/economics/people/faculty/nicholas-stern',
   'Daniel Sturm': 'https://www.lse.ac.uk/economics/people/faculty/daniel-sturm',
   'Balazs Szentes': 'https://www.lse.ac.uk/economics/people/faculty/balazs-szentes',
   'Silvana Tenreyro': 'https://www.lse.ac.uk/economics/people/faculty/silvana-tenreyro',
   'John Van Reenen': 'https://www.lse.ac.uk/economics/people/faculty/john-van-reenen',
   'Yike Wang': 'https://www.lse.ac.uk/economics/people/faculty/yike-wang',
   'Alwyn Young': 'https://www.lse.ac.uk/economics/people/faculty/alwyn-young',
   'Shengxing Zhang': 'https://www.lse.ac.uk/economics/people/faculty/shengxing-zhang'},
  'Faculty activity in Research Centres': {'Philippe Aghion': 'https://www.lse.ac.uk/economics/people/faculty/philippe-aghion',
   'Tim Besley': 'https://www.lse.ac.uk/economics/people/faculty/tim-besley',
   'Swati Dhingra': 'https://www.lse.ac.uk/economics/people/faculty/swati-dhingra',
   'Jeremiah Dittmar': 'https://www.lse.ac.uk/economics/people/faculty/jeremiah-dittmar',
   'Stephen Machin': 'https://www.lse.ac.uk/economics/people/faculty/stephen-machin',
   'Alan Manning': 'https://www.lse.ac.uk/economics/people/faculty/alan-manning',
   'Guy Michaels': 'https://www.lse.ac.uk/economics/people/faculty/guy-michaels',
   'Steve Pischke': 'https://www.lse.ac.uk/economics/people/faculty/steve-pischke',
   'Thomas Sampson': 'https://www.lse.ac.uk/economics/people/faculty/thomas-sampson',
   'Johannes Spinnewijn': 'https://www.lse.ac.uk/economics/people/faculty/johannes-spinnewijn',
   'Daniel Sturm': 'https://www.lse.ac.uk/economics/people/faculty/daniel-sturm',
   'Charles Bean': 'https://www.lse.ac.uk/economics/people/faculty/charles-bean',
   'Francesco Caselli': 'https://www.lse.ac.uk/economics/people/faculty/francesco-caselli',
   'Wouter Den Haan': 'https://www.lse.ac.uk/economics/people/faculty/wouter-den-haan',
   'Ethan Ilzetzki': 'https://www.lse.ac.uk/economics/people/faculty/ethan-ilzetzki',
   'Keyu Jin': 'https://www.lse.ac.uk/economics/people/faculty/keyu-jin',
   'Mervyn King': 'https://www.lse.ac.uk/economics/people/faculty/mervyn-king',
   'Rachel Ngai': None,
   'Christopher Pissarides': 'https://www.lse.ac.uk/economics/people/faculty/christopher-pissarides',
   'Ricardo Reis': 'https://www.lse.ac.uk/economics/people/faculty/ricardo-reis',
   'Kevin Sheedy': 'https://www.lse.ac.uk/economics/people/faculty/kevin-sheedy',
   'Silvana Tenreyro': 'https://www.lse.ac.uk/economics/people/faculty/silvana-tenreyro',
   'Shengxing Zhang': 'https://www.lse.ac.uk/economics/people/faculty/shengxing-zhang',
   'Nava Ashraf': 'https://www.lse.ac.uk/economics/people/faculty/nava-ashraf',
   'Oriana Bandiera': 'https://www.lse.ac.uk/economics/people/faculty/oriana-bandiera',
   'Gharad Bryan': 'https://www.lse.ac.uk/economics/people/faculty/gharad-bryan',
   'Robin Burgess': 'https://www.lse.ac.uk/economics/people/faculty/robin-burgess',
   'Maitreesh Ghatak': 'https://www.lse.ac.uk/economics/people/faculty/maitreesh-ghatak',
   'Jonathan Leape': 'https://www.lse.ac.uk/economics/people/faculty/jonathan-leape',
   'Nicholas Stern': 'https://www.lse.ac.uk/economics/people/faculty/nicholas-stern',
   'John Sutton': 'https://www.lse.ac.uk/economics/people/faculty/john-sutton',
   'Frank Cowell': 'https://www.lse.ac.uk/economics/people/faculty/frank-cowell',
   'Andrew Ellis': 'https://www.lse.ac.uk/economics/people/faculty/andrew-ellis',
   'Alessandro Gavazza': 'https://www.lse.ac.uk/economics/people/faculty/alessandro-gavazza',
   'Vassilis Hajivassiliou': 'https://www.lse.ac.uk/economics/people/faculty/vassilis-hajivassiliou',
   'Javier Hidalgo': 'https://www.lse.ac.uk/economics/people/faculty/havier-hidalgo',
   'Xavier Jaravel': 'https://www.lse.ac.uk/economics/people/faculty/xavier-jaravel',
   'Tatiana Komarova': None,
   'Camille Landais': 'https://www.lse.ac.uk/economics/people/faculty/camille-landais',
   'Gilat Levy': 'https://www.lse.ac.uk/economics/people/faculty/gilat-levy',
   'Matthew Levy': 'https://www.lse.ac.uk/economics/people/faculty/matthew-levy',
   'Rachael Meager': None,
   'John Moore': 'https://www.lse.ac.uk/economics/people/faculty/john-moore',
   'Francesco Nava': 'https://www.lse.ac.uk/economics/people/faculty/francesco-nava',
   'Taisuke Otsu': 'https://www.lse.ac.uk/economics/people/faculty/taisuke-otsu',
   'Martin Pesendorfer': 'https://www.lse.ac.uk/economics/people/faculty/martin-pesendorfer',
   'Michele Piccione': 'https://www.lse.ac.uk/economics/people/faculty/michele-piccione',
   'Ronny Razin': 'https://www.lse.ac.uk/economics/people/faculty/ronny-razin',
   'Daniel Reck': None,
   'Marcia Schafgans': 'https://www.lse.ac.uk/economics/people/faculty/marcia-schafgans',
   'Mark Schankerman': 'https://www.lse.ac.uk/economics/people/faculty/mark-schankerman',
   'Pasquale Schiraldi': 'https://www.lse.ac.uk/economics/people/faculty/pasquale-schiraldi',
   'Balazs Szentes': 'https://www.lse.ac.uk/economics/people/faculty/balazs-szentes'},
  'Professional Services Staff': {'Nallini Samuel': None,
   'Andy Wilson': None,
   'Sharon Peate': None,
   'Dária': None,
   'Deborah Adams': None,
   'Irina Zaraisky': None,
   'Oceana Taylor': None,
   'Lakmini': None,
   'Sarah Burton': None,
   "Alice O'Donkor": None,
   'John Canfield': None,
   'Kelly Lewis': None,
   'Rudi Clayton': None,
   'Narmin Crorie': None,
   'Ben Westall': None,
   'Emma Taverner': None,
   'Shareen Suleman': None,
   'Shubhra Mitra': None,
   'Lorna Severn': None,
   'Mohsin Ahmed': None,
   'Lakmini Staskus': None},
  'Department Leadership': {'Francesco Caselli': 'https://www.lse.ac.uk/economics/people/faculty/francesco-caselli',
   'Dimitra Petropoulou': 'https://www.lse.ac.uk/economics/people/faculty/dimitra-petropoulou'},
  'Programme Directors': {'Dimitra Petropoulou': 'https://www.lse.ac.uk/economics/people/faculty/dimitra-petropoulou',
   'Antonio Mele': 'https://www.lse.ac.uk/economics/people/faculty/antonio-mele',
   'Michael Gmeiner': 'https://www.lse.ac.uk/economics/people/faculty/michael-gmeiner',
   'Frank Cowell': 'https://www.lse.ac.uk/economics/people/faculty/frank-cowell',
   'Vassilis Hajivassiliou': 'https://www.lse.ac.uk/economics/people/faculty/vassilis-hajivassiliou',
   'Maitreesh Ghatak': 'https://www.lse.ac.uk/economics/people/faculty/maitreesh-ghatak'},
  'Visiting and Emeritus Professors and Academics': {'Dave Donaldson': None,
   'Anne Karing': None,
   'Per Krusell': 'http://people.su.se/%7Epkrus/',
   'Torsten Persson': 'http://perseus.iies.su.se/~tpers/',
   'Raul Sanchez de la Sierra': None,
   'Michael Whinston': None,
   'Margaret Bray': 'https://www.lse.ac.uk/economics/people/faculty/margaret-bray',
   'Lord Meghnad Desai': None,
   'Christopher Dougherty': 'https://www.lse.ac.uk/economics/people/faculty/christopher-dougherty',
   'Richard Jackman': 'https://www.lse.ac.uk/economics/people/faculty/richard-jackman',
   'Mervyn King': 'https://www.lse.ac.uk/economics/people/faculty/mervyn-king',
   'Lord Richard Layard': 'http://cep.lse.ac.uk/_new/staff/person.asp?id=970',
   'Alan Marin': None,
   'Peter Robinson': 'https://www.lse.ac.uk/economics/people/faculty/peter-robinson',
   'John Sutton': 'https://www.lse.ac.uk/economics/people/faculty/john-sutton',
   'Jim Thomas': None,
   'Christine Whitehead': 'https://www.lse.ac.uk/economics/people/faculty/christine-whitehead'},
  'Guest teachers': {'Michele Fioretti': None,
   'Michael McMahon': None,
   'Clement Minaudier': None,
   'David Seim': None,
   'Tim Willems': None},
  'LSE Fellows': {'Ragvir Sabharwal': 'https://personal.lse.ac.uk/sabharw5/index.html',
   'Chiara Sotis': None,
   'Juraj Briskar': 'https://www.econbriskar.com'},
  'Department Teaching\xa0Fellows': {'Mr Rasif Alakbarov': None,
   'Mr Daniel Albuquerque': None,
   'Mr Philipp Barteska': None,
   'Mr Marco Bellifemine': None,
   'Mr Adam Brzezinski': None,
   'Mr Arnaud Dyevre': None,
   'Ms Tiernan Evans': None,
   'Mr Miguel Fajardo-Steinhauser': None,
   'Ms Shadi Farahzadi': None,
   'Mr Jack Fisher': None,
   'Ms Salome Fofana': None,
   'Mr Maximilian Guennewig': None,
   'Evie Jamieson': None,
   'Ms Evgeniya Kudinova': None,
   'Mr Peter Lambert': None,
   'Mr Edoardo Leonardi': None,
   'Mr Mars Leung': None,
   'Mr Issac Zenon Martinez Centeno': None,
   'Mr Roberto Maura-Rivero': None,
   'Mr Thomas Monk': None,
   'Ms Sidharth Moktan': None,
   'Ms Kamila Nowakowicz': None,
   "Mr Loughlan O'Doherty": None,
   'Ms Lisa Pacheco': None,
   'Mr Hugo Reichardt': None,
   'Mr Soroush Sabet': None,
   'Mr Jack Shannon': None,
   'Ms Lachi Singh': None,
   'Mr Aditya Soenarjo': None,
   'Mr Rui Sun': None,
   'Mr Kazunari Tanabe': None,
   'Ms Maria Ventura': None,
   'Dr Rui Miguel Vieira Marques da Costa': None,
   'Mr Wei Wang': None,
   'Ms Cecilia Wood': None,
   'Mr Linchuan Xu': None,
   'Mr Chia-Hung Yeh': None},
  'Research students': {'Sierene Abdelsayed': 'https://www.lse.ac.uk/economics/people/research-students/sierene-abdelsayed',
   'Ola Aboukhsaiwan': 'http://www.lse.ac.uk/economics/people/research-students/Ola-Aboukhsaiwan',
   'Rasif Alakbarov': 'https://www.lse.ac.uk/economics/people/research-students/rasif-alakbarov',
   'Pinjas Albagli': 'https://www.lse.ac.uk/economics/people/research-students/pinjas-albagli',
   'Laure Anique': 'https://www.lse.ac.uk/economics/people/research-students/laure-anique',
   'Dimitris Athanasiou': 'https://www.lse.ac.uk/economics/people/research-students/dimitris-athanasiou',
   'Sreevidya Ayyar': 'http://www.lse.ac.uk/economics/people/research-students/sreevidya-ayyar',
   'Ignacio Banares Sanchez': 'https://www.lse.ac.uk/economics/people/research-students/ignacio-banares-sanchez',
   'Philipp Barteska': 'https://www.lse.ac.uk/economics/phd-job-market/job-market-candidates-2023-24/philipp-barteska/philipp-barteska',
   'jasper-bechtold': 'https://www.lse.ac.uk/economics/people/research-students/jasper-bechtold',
   'Marco Bellifemine': 'http://www.lse.ac.uk/economics/people/research-students/marco-bellifemine',
   'Alex Ben Hassine': 'https://www.lse.ac.uk/economics/people/research-students/alex-ben-hassine',
   'Shania Bhalotia': 'https://www.lse.ac.uk/economics/people/research-students/shania-bhalotia',
   'raffaele-blasone': 'https://www.lse.ac.uk/economics/people/research-students/raffaele-blasone',
   'Alix Bonargent': 'https://www.lse.ac.uk/economics/phd-job-market/job-market-candidates-2023-24/alix-bonargent/alix-bonargent',
   'Isadora Bousquat Arabe': 'https://www.lse.ac.uk/economics/people/research-students/isadora-bousquat-arabe',
   'Julio Brandao Roll': 'https://www.lse.ac.uk/economics/people/research-students/julio-brandao-roll',
   'Luzia Bruckamp': 'http://www.lse.ac.uk/economics/people/research-students/luzia-bruckamp',
   'Daniel Chandler': 'https://www.lse.ac.uk/economics/people/research-students/daniel-chandler',
   'Adrien Couturier': 'http://www.lse.ac.uk/economics/people/research-students/adrien-couturier',
   'ben-dahmen': 'https://www.lse.ac.uk/economics/people/research-students/ben-dahmen',
   'wallace-de-jesus-inocencio': 'https://www.lse.ac.uk/economics/people/research-students/wallace-de-jesus-inocencio',
   'Chloe De Meulenaer': 'https://www.lse.ac.uk/economics/people/research-students/chloé-de-meulenaer',
   'Zhiyao Deng': 'https://www.lse.ac.uk/economics/people/research-students/zhiyao-deng',
   'Gabriela Deschamps Ochoa': 'https://www.lse.ac.uk/economics/people/research-students/gabriela-deschamps-ochoa',
   'Sitong Ding': 'https://www.lse.ac.uk/economics/people/research-students/sitong-ding',
   'Tim Dobermann': 'https://www.lse.ac.uk/economics/people/research-students/tim-dobermann',
   'Gaia Dossi': 'https://www.lse.ac.uk/economics/phd-job-market/job-market-candidates-2023-24/gaia-dossi/gaia-dossi',
   'Arnaud Dyevre': 'https://www.lse.ac.uk/economics/phd-job-market/job-market-candidates-2023-24/arnaud-dyevre/arnaud-dyevre',
   'Sebastian Ernst': 'http://www.lse.ac.uk/economics/people/research-students/sebastian-ernst',
   'Tiernan Evans': 'https://www.lse.ac.uk/economics/people/research-students/tiernan-evans',
   'Andres Fajardo-Ramirez': 'https://www.lse.ac.uk/economics/people/research-students/andres-fajardo-ramirez',
   'Miguel Fajardo Steinhäuser': 'https://www.lse.ac.uk/economics/people/research-students/miguel-fajardo-steinhäuser',
   'Lin Fan': 'https://www.lse.ac.uk/economics/people/research-students/lin-fan',
   'Tuo Fan': 'https://www.lse.ac.uk/economics/people/research-students/tuo-fan',
   'Shadi Farahzadi': 'https://www.lse.ac.uk/economics/phd-job-market/job-market-candidates-2023-24/shadi-farahzadi/shadi-farahzadi',
   'Bennet Luca Feld': 'https://www.lse.ac.uk/economics/people/research-students/bennet-lucca-feld',
   'Diego Ferreras Garrucho': 'https://www.lse.ac.uk/economics/people/research-students/diego-ferreras-garrucho',
   'salome-fofana': 'https://www.lse.ac.uk/economics/people/research-students/salome-fofana',
   'Marie Fuchs': 'https://www.lse.ac.uk/economics/people/research-students/marie-fuchs',
   'betsenat-gebrewold': 'https://www.lse.ac.uk/economics/people/research-students/betsenat-gebrewold',
   'Thomas Glinnan': 'https://www.lse.ac.uk/economics/people/research-students/thomas-glinnan',
   'Marta Guasch Rusiñol': 'https://www.lse.ac.uk/economics/people/research-students/marta-guasch-rusiñol',
   'Michelle Harnisch': 'https://www.lse.ac.uk/economics/people/research-students/michelle-harnisch',
   'anton-heil': 'https://www.lse.ac.uk/economics/people/research-students/anton-heil',
   'Stephan Hobler': 'https://www.lse.ac.uk/economics/people/research-students/stephan-hobler',
   'Yuxiao Hu': 'https://www.lse.ac.uk/economics/people/research-students/yuxiao-hu',
   'Shaohua Huang': 'https://www.lse.ac.uk/economics/people/research-students/Shaohua-Huang',
   'Xitong Hui': 'https://www.lse.ac.uk/economics/people/research-students/xitong-hui',
   'yu-hui': 'https://www.lse.ac.uk/economics/people/research-students/yu-hui',
   'Azhar Hussain': 'https://www.lse.ac.uk/economics/people/research-students/azhar-hussain',
   'Antonio Armindo Iavarone': 'https://www.lse.ac.uk/economics/people/research-students/antonio-armindo-iavarone',
   'Felix Iglhaut': 'http://www.lse.ac.uk/economics/people/research-students/felix-iglhaut',
   'Muskan Jain': 'http://www.lse.ac.uk/economics/people/research-students/muskan-jain',
   'Amen Jalal': 'https://www.lse.ac.uk/economics/people/research-students/amen-jalal',
   'evie-jamieson': 'https://www.lse.ac.uk/economics/people/research-students/evie-jamieson',
   'Animesh Jayant': 'https://www.lse.ac.uk/economics/people/research-students/animesh-jayant',
   'Ningyuan Jia': 'https://www.lse.ac.uk/economics/phd-job-market/job-market-candidates-2023-24/ningyuan-jia/ningyuan-jia',
   'zihan-jia': 'https://www.lse.ac.uk/economics/people/research-students/zihan-jia',
   'Ananya Kotia': 'https://www.lse.ac.uk/economics/people/research-students/ananya-kotia',
   'Evgeniya Kudinova': 'https://www.lse.ac.uk/economics/phd-job-market/job-market-candidates-2023-24/evgeniya-kudinova/evgeniya-kudinova',
   'Luiza Kunz Aires': 'https://www.lse.ac.uk/economics/people/research-students/luiza-kunz-aires',
   'Peter Lambert': 'https://www.lse.ac.uk/economics/people/research-students/peter-lambert',
   'Gabriel Leite Mariante': 'https://www.lse.ac.uk/economics/people/research-students/gabriel-leite-mariante',
   'hongting-leng': 'https://www.lse.ac.uk/economics/people/research-students/hongting-leng',
   'mars-leung': 'https://www.lse.ac.uk/economics/people/research-students/mars-leung',
   'Runpu Li': 'https://www.lse.ac.uk/economics/people/research-students/runpu-li',
   'Francisco Libano-Monteiro': 'https://www.lse.ac.uk/economics/people/research-students/francesco-libano-monteiro',
   'javier-lopez': 'https://www.lse.ac.uk/economics/people/research-students/javier-lopez',
   'Runhong Ma': 'https://www.lse.ac.uk/economics/people/research-students/runhong-ma',
   'delia-macaluso': 'https://www.lse.ac.uk/economics/people/research-students/delia-macaluso',
   'Beatriz Machado Ribeiro': 'http://www.lse.ac.uk/economics/people/research-students/beatriz-machado-ribeiro',
   'Covadonga Machicado Alvarez': 'https://www.lse.ac.uk/economics/people/research-students/Covadonga-Machicado-Alvarez',
   'Edward Manuel': 'https://www.lse.ac.uk/economics/people/research-students/edward-manuel',
   'Isaac Martinez-Centeno': 'https://www.lse.ac.uk/economics/people/research-students/isaac-martinez-centeno',
   'Johannes Matt': 'https://www.lse.ac.uk/economics/people/research-students/johannes-matt',
   'Roberto Maura-Rivero': 'https://www.lse.ac.uk/economics/people/research-students/roberto-maura-rivero',
   'Giovanni Minolfi': 'https://www.lse.ac.uk/economics/people/research-students/giovanni-minolfi',
   'Negar Mohammadi Jazi': 'http://www.lse.ac.uk/economics/people/research-students/negar-mohammadi-jazi',
   'Sidharth Moktan': 'https://www.lse.ac.uk/economics/people/research-students/sidharth-moktan',
   'Thomas Monk': 'https://www.lse.ac.uk/economics/people/research-students/thomas-monk',
   'Mateus Morais': 'http://www.lse.ac.uk/economics/people/research-students/Mateus-Morais',
   'Marta Morando': 'https://www.lse.ac.uk/economics/people/research-students/marta-morando',
   'Bernardo Mottironi': 'https://www.lse.ac.uk/economics/people/research-students/bernardo-mottironi',
   'Davi Moura': 'https://www.lse.ac.uk/economics/people/research-students/davi-moura',
   'ellen-munroe': 'https://www.lse.ac.uk/economics/people/research-students/ellen-munroe',
   'Canishk Naik': 'https://www.lse.ac.uk/economics/people/research-students/canishk-naik',
   'Alexander Newton': 'https://www.lse.ac.uk/economics/people/research-students/alexander-newton',
   'charles-nourse': 'https://www.lse.ac.uk/economics/people/research-students/charles-nourse',
   'Kamila Nowakowicz': 'https://www.lse.ac.uk/economics/people/research-students/kamila-nowakowicz',
   "Matthew O'Brien": 'https://www.lse.ac.uk/economics/people/research-students/matthew-obrien',
   "Loughlan O'Doherty": 'https://www.lse.ac.uk/economics/people/research-students/loughlan-odoherty',
   'Shinji Okazaki': 'https://www.lse.ac.uk/economics/people/research-students/shinji-okazaki',
   'Lisa Beihy Pacheco': 'http://www.lse.ac.uk/economics/people/research-students/lisa-beihy-pacheco',
   'William Parker': 'https://www.lse.ac.uk/economics/people/research-students/william-parker',
   'Paula Patzelt': 'http://www.lse.ac.uk/economics/people/research-students/paula-patzelt',
   'Tatiana Pazem': 'https://www.lse.ac.uk/economics/people/research-students/tatiana-pazem',
   'Carlos Pérez-Cavero': 'http://www.lse.ac.uk/economics/people/research-students/Carlos-Pérez-Cavero',
   'gailius-praninskas': 'https://www.lse.ac.uk/economics/people/research-students/gailius-praninskas',
   'Ludovica Priviero': 'https://www.lse.ac.uk/economics/people/research-students/Ludovica-Priviero',
   'Yuhao Qian': 'https://www.lse.ac.uk/economics/people/research-students/yuhao-qian',
   'Michelle Rao': 'https://www.lse.ac.uk/economics/people/research-students/michelle-rao',
   'Khawaja Arsalan Rasheed': 'https://www.lse.ac.uk/economics/people/research-students/khawaja-arsalan-rasheed',
   'Giorgio Ravalli': 'https://www.lse.ac.uk/economics/people/research-students/giorgio-ravalli',
   'Hugo Reichardt': 'https://www.lse.ac.uk/economics/phd-job-market/job-market-candidates-2023-24/hugo-reichardt/hugo-reichardt',
   'Emiliano Rinaldi': 'http://www.lse.ac.uk/economics/people/research-students/emiliano-rinaldi',
   'Rebecca Rose': 'https://www.lse.ac.uk/economics/people/research-students/rebecca-rose',
   'Soroush Sabet': 'https://www.lse.ac.uk/economics/people/research-students/soroush-sabet',
   'Yasaman Saeidi': 'https://www.lse.ac.uk/economics/people/research-students/yasaman-saeidi',
   'Veronica Salazar Restrepo': 'https://www.lse.ac.uk/economics/phd-job-market/job-market-candidates-2023-24/veronica-salazar-restrepo/veronica-salazar-restrepo',
   'Maria Santos Feliciano': 'https://www.lse.ac.uk/economics/people/research-students/maria-santos-feliciano',
   'Yannick Schindler': 'https://www.lse.ac.uk/economics/people/research-students/yannick-schindler',
   'Patrick Schneider': 'https://www.lse.ac.uk/economics/people/research-students/patrick-schneider',
   'mitch-scott': 'https://www.lse.ac.uk/economics/people/research-students/mitch-scott',
   'brayan-segura-solano': 'https://www.lse.ac.uk/economics/people/research-students/brayan-segura-solano',
   'meet-shah': 'https://www.lse.ac.uk/economics/people/research-students/meet-shah',
   'Javad Shamsi': 'https://www.lse.ac.uk/economics/people/research-students/javad-shamsi',
   'John Shannon': 'https://www.lse.ac.uk/economics/people/research-students/john-shannon',
   'Callum Shaw': 'https://www.lse.ac.uk/economics/people/research-students/callum-shaw',
   'Pol Simpson': 'https://www.lse.ac.uk/economics/people/research-students/pol-simpson',
   'Ameek Singh': 'https://www.lse.ac.uk/economics/people/research-students/ameek-singh',
   'Lachi Singh': 'https://www.lse.ac.uk/economics/people/research-students/lachi-singh',
   'Aditya Soenarjo': 'https://www.lse.ac.uk/economics/phd-job-market/job-market-candidates-2023-24/aditya-soenarjo/aditya-soenarjo',
   'Caterina Soto-Vieira': 'https://www.lse.ac.uk/economics/people/research-students/caterina-soto-vieira',
   'Meng Su': 'https://www.lse.ac.uk/economics/people/research-students/meng-su',
   'Hamza Syed': 'https://www.lse.ac.uk/economics/people/research-students/hamza-syed',
   'Abhijit Tagade': 'http://www.lse.ac.uk/economics/people/research-students/abhijit-tagade',
   'Kazunari Tanabe': 'http://www.lse.ac.uk/economics/people/research-students/kazunari-tanabe',
   'andreas-teichgraeber': 'https://www.lse.ac.uk/economics/people/research-students/Andreas-Teichgraeber',
   'Jack Thiemel': 'https://www.lse.ac.uk/economics/people/research-students/jack-thiemel',
   'Likun Tian': 'https://www.lse.ac.uk/economics/people/research-students/Likun-Tian',
   'Xavier Tierney': 'http://www.lse.ac.uk/economics/people/research-students/xavier-tierney',
   'Enrico Duilio Turri': 'http://www.lse.ac.uk/economics/people/research-students/enrico-duilio-turri',
   'Maria Ventura': 'https://www.lse.ac.uk/economics/phd-job-market/job-market-candidates-2023-24/maria-ventura/maria-ventura',
   'wei-wang': 'https://www.lse.ac.uk/economics/people/research-students/wei-wang',
   'Zixin Wang': 'https://www.lse.ac.uk/economics/people/research-students/zixin-wang',
   'Peter Ward-Griffin': 'https://www.lse.ac.uk/economics/people/research-students/peter-ward-griffin',
   'Lukas Wiedemann': 'https://www.lse.ac.uk/economics/people/research-students/lukas-wiedemann',
   'Sarah Winton': 'https://www.lse.ac.uk/economics/people/research-students/sarah-winton',
   'Yoshiki Wiskamp': 'https://www.lse.ac.uk/economics/people/research-students/yoshiki-wiskamp',
   'Jeongkyung Won': 'https://www.lse.ac.uk/economics/people/research-students/jeongkyung-won',
   'Cecilia Wood': 'https://www.lse.ac.uk/economics/people/research-students/cecilia-wood',
   'Linchuan Xu': 'https://www.lse.ac.uk/economics/people/research-students/linchuan-xu',
   'Chia-Hung Yeh': 'https://www.lse.ac.uk/economics/people/research-students/chia-hung-yeh',
   'Tianhao Yin': 'https://www.lse.ac.uk/economics/people/research-students/tianhao-yin',
   'Omry Yoresh': 'https://www.lse.ac.uk/economics/people/research-students/omry-yoresh',
   'Yuanhang Yu': 'https://www.lse.ac.uk/economics/people/research-students/yuanhang-yu',
   'alfred-zhang': 'https://www.lse.ac.uk/economics/people/research-students/alfred-zhang',
   'Jingfeng Zhang': 'https://www.lse.ac.uk/economics/people/research-students/Jingfeng-Zhang',
   'Weijie Zhang': 'https://www.lse.ac.uk/economics/people/research-students/weijie-zhang',
   'Eddy (Weijian) Zou': 'https://www.lse.ac.uk/economics/people/research-students/eddy-weijian-zou'},
  'Graduate teaching assistants': {'Luiza Kunz-Aires': None,
   'Pinjas Albagli': None,
   'Luzia Bruckamp': None,
   'Giacomo Carlini': None,
   'Fraser Clark': None,
   'Adrian Couturier-Roguet': None,
   'Gabriela Deschamps Ochoa': None,
   'Sacha Dray': None,
   'Sitong Ding': None,
   'Andres Felipe Fajardo Ramirez': None,
   'Johannes Anton Heil': None,
   'Yuxiao Hu': None,
   'Azhar Hussain': None,
   'Antonio Iavarone': None,
   'Ananya Kotia': None,
   'Filip Klimenka': None,
   'Antonio Lavarone': None,
   'Gabriel Leite-Mariante': None,
   'Runpu Li': None,
   'Manuel Francisco Libano Monteiro': None,
   'Johannes Matt': None,
   'Bernardo Mottironi': None,
   'Yuhao Qian': None,
   'Michelle Rao': None,
   'Emiliano Rinaldi': None,
   'Rebecca Rose': None,
   'Yasaman Saeidi': None,
   'Ameek Singh': None,
   'Guido Spano': None,
   'Hamza Mohammad Syed': None,
   'Jack Thiemel': None,
   'Anshuman Tiwari': None,
   'Hugo Vilares': None,
   'Peter Ward-Griffin': None,
   'Lukas Weidemann': None,
   'Sarah Winton': None,
   'Mengshan Xu': None,
   'Yu Yi': None}},
 'Department of Economic History': {'Faculty, Fellows and Teachers': {'Dr Olivier Accominotti': 'https://www.lse.ac.uk/Economic-History/People/Faculty-and-teachers/Accominotti/Dr-Olivier-Accominotti',
   'Dudley Baines': 'https://www.lse.ac.uk/Economic-History/People/Faculty-and-teachers/Dudley-Baines',
   'Dr Gerben Bakker': 'https://www.lse.ac.uk/Economic-History/People/Faculty-and-teachers/Bakker/Dr-Gerben-Bakker',
   'Dr Jordan Claridge': 'https://www.lse.ac.uk/Economic-History/People/Faculty-and-teachers/Dr-Jordan-Claridge',
   'Professor Neil Cummins': 'https://www.lse.ac.uk/Economic-History/People/Faculty-and-teachers/Professor-Neil-Cummins',
   'Professor Kent Deng': 'https://www.lse.ac.uk/Economic-History/People/Faculty-and-teachers/Professor-Kent-Deng',
   'Dr Leigh Gardner': 'https://www.lse.ac.uk/Economic-History/People/Faculty-and-teachers/Professor-Leigh-Gardner',
   'Andrés Guiot Isaac': 'https://www.lse.ac.uk/Economic-History/People/Faculty-and-teachers/Dr-Andrés-Guiot-Isaac',
   'Professor Sara Horrell': 'https://www.lse.ac.uk/Economic-History/People/Faculty-and-teachers/Professor-Sara-Horrell',
   'Professor Jane Humphries': 'https://www.lse.ac.uk/Economic-History/People/Faculty-and-teachers/Humphries/Professor-Jane-Humphries',
   'Tehreem Husain': 'https://www.lse.ac.uk/Economic-History/People/Faculty-and-teachers/Dr-Tehreem-Husain',
   'Professor Janet Hunter': 'https://www.lse.ac.uk/Economic-History/People/Faculty-and-teachers/Hunter/Professor-Janet-Hunter',
   'Dr Alejandra Irigoin': 'https://www.lse.ac.uk/Economic-History/People/Faculty-and-teachers/Irigoin/Dr-Alejandra-Irigoin',
   'Dr Mina Ishizu': None,
   'Jason Lennard': 'https://www.lse.ac.uk/Economic-History/People/Faculty-and-teachers/Dr-Jason-Lennard',
   'Professor Colin Lewis': 'https://www.lse.ac.uk/Economic-History/People/Faculty-and-teachers/Professor-Colin-Lewis',
   'Dr Chris Minns': 'https://www.lse.ac.uk/Economic-History/People/Faculty-and-teachers/Minns/Professor-Chris-Minns',
   'Safya Morshed': 'https://www.lse.ac.uk/Economic-History/People/Faculty-and-teachers/Safya-Morshed',
   'Professor Mary Morgan': 'https://www.lse.ac.uk/Economic-History/People/Faculty-and-teachers/Morgan/Professor-Mary-Morgan',
   'Dr Natacha Postel-Vinay': 'https://www.lse.ac.uk/Economic-History/People/Faculty-and-teachers/Dr-Natacha-Postel-Vinay',
   'Professor Albrecht Ritschl': 'https://www.lse.ac.uk/Economic-History/People/Faculty-and-teachers/Professor-Albrecht-Ritschl',
   'Professor Joan Roses': 'https://www.lse.ac.uk/Economic-History/People/Faculty-and-teachers/Roses/Professor-Joan-Roses',
   'Professor Tirthankar Roy': 'https://www.lse.ac.uk/Economic-History/People/Faculty-and-teachers/Roy/Professor-Tirthankar-Roy',
   'Dr Anne Ruderman': 'https://www.lse.ac.uk/Economic-History/People/Faculty-and-teachers/Ruderman/Dr-Anne-Ruderman',
   'Dr Mohamed Saleh': 'https://www.lse.ac.uk/Economic-History/People/Faculty-and-teachers/Dr-Mohamed-Saleh',
   'Professor Max Schulze': 'https://www.lse.ac.uk/Economic-History/People/Faculty-and-teachers/Schulze/Professor-Max-Schulze',
   'Dr Eric Schneider': 'https://www.lse.ac.uk/Economic-History/People/Faculty-and-teachers/Schneider/Professor-Eric-Schneider',
   'Dr Sabine Schneider': 'https://www.lse.ac.uk/Economic-History/People/Faculty-and-teachers/Dr-Sabine-Schneider',
   'Dr Rebecca Simson': None,
   'Professor Oliver Volckart': 'https://www.lse.ac.uk/Economic-History/People/Faculty-and-teachers/Professor-Oliver-Volckart',
   'Professor Patrick Wallis': 'https://www.lse.ac.uk/Economic-History/People/Faculty-and-teachers/Wallis/Professor-Patrick-Wallis',
   'Dr Melanie Meng Xue': 'https://www.lse.ac.uk/Economic-History/People/Faculty-and-teachers/Dr-Melanie-Meng-Xue'},
  'Public Engagement and Impact': {'Christopher Kissane': 'https://www.lse.ac.uk/Economic-History/People/Public-Engagement-and-Impact/Christopher-Kissane'},
  'Affiliated Faculty': {'Dr Jeremiah Dittmar': 'https://www.lse.ac.uk/economics/people/faculty/jeremiah-dittmar',
   'Dr Nadia Matringe': 'https://www.lse.ac.uk/accounting/people/nadia-matringe'},
  'Graduate Teaching Assistants 2023-24': {'Luisa Bicalho Ritzkat ': None,
   'Mario Cuenda Garcia': 'https://www.lse.ac.uk/Economic-History/People/PhDs/Mario-Cuenda-Garcia',
   'Nick Fitzhenry ': None,
   'Andres Irarrazaval Garcia Huidobro ': None,
   'Yangyang Liu ': None,
   'Zane Jennings ': None,
   'Viren Mahurkar ': None,
   'William Mitchell ': None,
   'Andy Park ': None,
   'Nick Peyton ': None,
   'Sheila Pugh ': None,
   'Greg Salter ': None,
   'Noah Sutter ': None,
   'Tianning Zhu ': None},
  'Visiting Academics': {'Dr Ahmet Akarli': None,
   'Professor Robert Bennett': 'https://www.lse.ac.uk/Economic-History/People/Visiting-Academics/Robert-Bennett/Professor-Robert-Bennett',
   'Professor Youssef Cassis': 'https://www.lse.ac.uk/Economic-History/People/Visiting-Academics/Professor-Youssef-Cassis',
   'Professor David Chambers': None,
   'Jordi Domenich Feliu': None,
   'Terry Gourvish': None,
   'Leslie Hannah': 'https://www.lse.ac.uk/Economic-History/People/Visiting-Academics/Leslie-Hannah/Leslie-Hannah',
   'Dr Karolina Hutkova': 'https://www.lse.ac.uk/Economic-History/People/Faculty-and-teachers/Dr-Karolina-Hutkova',
   'Professor Maarten Prak': None,
   'Professor Andrew Seltzer': 'https://www.lse.ac.uk/Economic-History/People/Visiting-Academics/Professor-Andrew-Seltzer',
   'Dr Niccolò Valmori': None},
  'Professional Support Staff': {'Oli Harrison': None,
   'Kamilah Hassan': None,
   'Helena Ivins': None,
   'Tracy Keefe': None,
   'Jennie Stayner': None,
   'Darren Townsend': None},
  'PhD students': {'Luisa Bicalho Ritzkat': 'https://www.lse.ac.uk/Economic-History/People/PhDs/Luisa-Bicalho-Ritzkat',
   'Ruoran Cheng': None,
   'Marco Cokic': None,
   'Mario Cuenda Garcia': 'https://www.lse.ac.uk/Economic-History/People/PhDs/Mario-Cuenda-Garcia',
   'Eoin Dignam': None,
   'Junqi': None,
   'Nick Fitzhenry': 'https://www.lse.ac.uk/Economic-History/People/PhDs/Nick-Fitzhenry',
   'Florentine Friedrich': None,
   'Hampton Gaddy': None,
   'Alex Green': 'https://www.lse.ac.uk/Economic-History/People/PhDs/Alex-Green',
   'Andrés Irarrázaval': 'https://www.lse.ac.uk/Economic-History/People/PhDs/Andrés-Irarrázaval',
   'Zane Jennings': 'https://www.lse.ac.uk/Economic-History/People/PhDs/Zane-Jennings',
   'Jennifer Kohler': 'https://www.lse.ac.uk/Economic-History/People/PhDs/Jennifer-Kohler',
   'Julius Koschnick': 'https://www.lse.ac.uk/Economic-History/People/PhDs/Julius-Koschnick',
   'Mikhail Kolosov': None,
   'Tom Learmouth': 'https://www.lse.ac.uk/Economic-History/People/PhDs/Tom-Learmouth',
   'Yangyang Liu': None,
   'Xizi Luo': 'https://www.lse.ac.uk/Economic-History/People/PhDs/Xizi-Luo',
   'Viren Mahurkar': None,
   'Aurelius Noble': 'https://www.lse.ac.uk/Economic-History/People/PhDs/Aurelius-Noble',
   "Bumjin 'Andy' Park": 'https://www.lse.ac.uk/Economic-History/People/PhDs/Bumjin-Andy-Park',
   'Victor Perez Sanchez': None,
   'Nick Peyton': 'https://www.lse.ac.uk/Economic-History/People/PhDs/Nick-Peyton',
   'Sheila Pugh': None,
   'Matthew Purcell': 'https://www.lse.ac.uk/Economic-History/People/PhDs/MatthewPurcell',
   'Greg Salter': None,
   "Ziyue 'Zoey' Shen": None,
   'Charles Smith': None,
   'Mek Ajay Sribhibhadh': None,
   'Noah Sutter': 'https://www.lse.ac.uk/Economic-History/People/PhDs/noah-sutter',
   'David Teeters': None,
   'Charles Udale': None,
   'Hillary Vipond': None,
   'Weizhe Zhang': None,
   'Tianning Zhu': None,
   'Ziming Zhu': 'https://www.lse.ac.uk/Economic-History/People/PhDs/Ziming-Zhu'},
  'Head': {'Professor Patrick Wallis ': 'https://www.lse.ac.uk/Economic-History/People/Faculty-and-teachers/Wallis/Professor-Patrick-Wallis',
   'Professor Neil Cummins\xa0': 'https://www.lse.ac.uk/Economic-History/People/Faculty-and-teachers/Professor-Neil-Cummins',
   'Professor Sara Horrell ': 'https://www.lse.ac.uk/Economic-History/People/Faculty-and-teachers/Professor-Sara-Horrell',
   'Jennie Stayner ': None}},
 'European Institute': {'European Institute Leadership': {'Professor Simon Glendinning': 'https://www.lse.ac.uk/european-institute/people/glendinning-simon',
   'Dr Jonathan White': 'https://www.lse.ac.uk/european-institute/people/white-jonathan',
   'Carl-Ludwig Campbell': 'https://www.lse.ac.uk/european-institute/people/campbell-carl-ludwig-',
   'Professor Kevin Featherstone': 'https://www.lse.ac.uk/european-institute/people/featherstone-kevin',
   'Yaprak Gürsoy': 'https://www.lse.ac.uk/european-institute/people/gursoy-yaprak',
   'Denisa Kostovicova': 'https://www.lse.ac.uk/european-institute/people/kostovicova-denisa',
   'Professor Jonathan Hopkin': 'https://www.lse.ac.uk/european-institute/people/hopkin-jonathan',
   'Dr. Abby Innes': 'https://www.lse.ac.uk/european-institute/people/innes-abby',
   'Dr Mareike Kleine': 'https://www.lse.ac.uk/european-institute/people/kleine-mareike',
   'Dr. Patrick McGovern': 'https://www.lse.ac.uk/sociology/people/patrick-mcgovern'},
  'Academic and Teaching Staff\xa0': {'Chris Anderson': 'https://www.lse.ac.uk/european-institute/people/anderson-chris',
   'Nicholas Barr': 'https://www.lse.ac.uk/european-institute/people/barr-nicholas',
   'Johann Robert Basedow': 'https://www.lse.ac.uk/european-institute/people/basedow-johann-robert',
   'Professor Iain Begg': 'https://www.lse.ac.uk/european-institute/people/begg-iain',
   'Chris Bick': 'https://www.lse.ac.uk/european-institute/people/bick-chris-teacher',
   'Professor Stefan Collignon': 'https://www.lse.ac.uk/european-institute/people/collignon-stefan',
   'Professor Paul De Grauwe': 'https://www.lse.ac.uk/european-institute/people/de-grauwe-paul',
   'Dr Spyros Economides': 'https://www.lse.ac.uk/european-institute/people/economides-spyros',
   'Professor Kevin Featherstone': 'https://www.lse.ac.uk/european-institute/people/featherstone-kevin',
   'Gianmarco Fifi': 'https://www.lse.ac.uk/european-institute/people/fifi-gianmarco',
   'garibay-petersen-cristobal': 'https://www.lse.ac.uk/european-institute/people/Garibay-Petersen-Cristobal',
   'Dr\xa0Kira Gartzou-Katsouyanni': 'https://www.lse.ac.uk/european-institute/people/gartzou-katsouyanni-kira-teacher',
   'Professor Simon Glendinning': 'https://www.lse.ac.uk/european-institute/people/glendinning-simon',
   'Dr.\xa0Katerina Glyniadaki': 'https://www.lse.ac.uk/european-institute/people/katerina-glyniadaki/glyniadaki-katerina',
   'Yaprak Gürsoy': 'https://www.lse.ac.uk/european-institute/people/gursoy-yaprak',
   'Asha Herten-Crabb': 'https://www.lse.ac.uk/european-institute/people/Herten-Crabb-Asha',
   'Professor Jonathan Hopkin': 'https://www.lse.ac.uk/european-institute/people/hopkin-jonathan',
   'Dr Abby Innes': 'https://www.lse.ac.uk/european-institute/people/innes-abby',
   'Dr Jennifer Jackson-Preece': 'https://www.lse.ac.uk/european-institute/people/jackson-preece-jennifer',
   'Dr. Patrick Kimunguyi': 'https://www.lse.ac.uk/european-institute/people/kimunguyi-patrick-teacher',
   'Dr Mareike Kleine': 'https://www.lse.ac.uk/european-institute/people/kleine-mareike',
   'Denisa Kostovicova': 'https://www.lse.ac.uk/european-institute/people/kostovicova-denisa',
   'Marta Lorimer': 'https://www.lse.ac.uk/european-institute/people/lorimer-marta',
   'Dr Angelo Martelli': 'https://www.lse.ac.uk/european-institute/people/Martelli-Angelo',
   'Frieder Mitsch ': 'https://www.lse.ac.uk/european-institute/people/mitsch-frieder-teacher',
   'Dr Vassilis Monastiriotis': 'https://www.lse.ac.uk/european-institute/people/monastiriotis-vassilis',
   'Daniela Roxana Movileanu': 'https://www.lse.ac.uk/european-institute/people/movileanu-daniela-teacher',
   'Dr Miguel Pereira': 'https://www.lse.ac.uk/european-institute/people/pereira-miguel',
   'Eva Polonska-Kimunguyi': 'https://www.lse.ac.uk/european-institute/people/polonska-kimunguyi-eva',
   'Vesna Popovski': 'https://www.lse.ac.uk/european-institute/people/Popovski-vesna/Popovski-vesna',
   'rashid-tahir': 'https://www.lse.ac.uk/european-institute/people/rashid-tahir-teacher',
   'Dr Eiko Thielemann': 'https://www.lse.ac.uk/european-institute/people/Thielemann-Eiko',
   'Niina Vuolajärvi': 'https://www.lse.ac.uk/european-institute/people/vuolajarvi-niina',
   'Dr Jonathan White': 'https://www.lse.ac.uk/european-institute/people/white-jonathan'},
  'Professional Services Staff': {'Carl-Ludwig Campbell': 'https://www.lse.ac.uk/european-institute/people/campbell-carl-ludwig-',
   'Adam Judge': 'https://www.lse.ac.uk/european-institute/people/judge-adam',
   'Charlotte Ennis': 'https://www.lse.ac.uk/european-institute/people/Ennis-Charlotte',
   'Natalie Grace': 'https://www.lse.ac.uk/european-institute/people/grace-natalie',
   'jenner-stephen': 'https://www.lse.ac.uk/european-institute/people/jenner-stephen',
   'Wagio Waigi': 'https://www.lse.ac.uk/european-institute/people/Waigi-Wagio',
   'liouta-polly': 'https://www.lse.ac.uk/european-institute/people/liouta-polly',
   'Beenam Butt': 'https://www.lse.ac.uk/european-institute/people/Beenam-Butt',
   'Laura Malouf': 'https://www.lse.ac.uk/european-institute/people/Malouf-Laura',
   'Delphine Polidori': 'https://www.lse.ac.uk/european-institute/people/polidori-delphine',
   'Hannah Shearer': 'https://www.lse.ac.uk/european-institute/people/Hannah-Shearer',
   'Carolina Stern': 'https://www.lse.ac.uk/european-institute/people/carolina-stern',
   'samuels-florence': 'https://www.lse.ac.uk/european-institute/people/samuels-florence',
   'Arzu Kırcal Şahin': 'https://www.lse.ac.uk/european-institute/people/kircal-sahin-arzu',
   'Samson Yeung': 'https://www.lse.ac.uk/european-institute/people/yeung-samson'},
  'Research Staff\xa0': {'Kate Alexander Shaw': 'https://www.lse.ac.uk/european-institute/people/alexander-shaw-kate',
   'Marilena Anastasopoulou': 'https://www.lse.ac.uk/european-institute/people/anastasopoulou-marilena',
   'Joseph Ganderson': 'https://www.lse.ac.uk/european-institute/people/ganderson-joseph',
   'Lanabi la Lova': 'https://www.lse.ac.uk/european-institute/research/justint/lalova-lanabi'},
  'Visiting Staff': {'Dr Sebahattin Abdurrahman': 'https://www.lse.ac.uk/european-institute/people/abdurrahman-sebahattin',
   'Ms Helen J. Addison': 'https://www.lse.ac.uk/european-institute/people/addison-helen',
   'Mr. John Alty': 'https://www.lse.ac.uk/european-institute/people/alty-john',
   'Angelos Angelou': 'https://www.lse.ac.uk/european-institute/people/angelou-angelos',
   'Tito Boeri': 'https://www.lse.ac.uk/european-institute/people/Boeri-Tito',
   'Diane Bolet': 'https://www.lse.ac.uk/european-institute/people/bolet-diane',
   'Dr Eray Çaylı': 'https://www.lse.ac.uk/european-institute/people/cayli-eray',
   'Nathan Charlton': 'https://www.lse.ac.uk/european-institute/people/charlton-nathan',
   'Marina Cino Pagliarello': 'https://www.lse.ac.uk/european-institute/people/Cino-Pagliarello-Marina',
   'Mr Lorenzo Codogno': 'https://www.lse.ac.uk/european-institute/people/codogno-lorenzo',
   'Dr Steve Coulter': 'https://www.lse.ac.uk/european-institute/people/coulter-steve',
   'Mr Gijs De Vries': 'https://www.lse.ac.uk/european-institute/people/de-vries-gijs',
   'Sebastian Diessner': 'https://www.lse.ac.uk/european-institute/people/diessner-sebastian',
   'Dr Joseph Downing': 'https://www.lse.ac.uk/european-institute/people/Downing-Joseph',
   'Federico Maria Ferrara': 'https://www.lse.ac.uk/european-institute/people/ferrara-federico',
   'Dr Pasquale Foresti': 'https://www.lse.ac.uk/european-institute/people/foresti-pasquale',
   'Dr Eddie Gerba': 'https://www.lse.ac.uk/european-institute/people/visiting-staff/gerba-eddie',
   'Dr. Bob\xa0Hancké': 'https://www.lse.ac.uk/european-institute/people/hancké-bob',
   'Dr Pierre Hausemer': 'https://www.lse.ac.uk/european-institute/people/Hausemer-Pierre',
   'Mr Philippe Legrain': 'https://www.lse.ac.uk/european-institute/people/visiting-staff/legrain-philippe',
   'Dr Philipp Lutz': 'https://www.lse.ac.uk/european-institute/people/visiting-staff/lutz-philipp',
   'Dr Corrado Macchiarelli': 'https://www.lse.ac.uk/european-institute/people/mushövel-fabian',
   'Dr Selin Nasi': 'https://www.lse.ac.uk/european-institute/people/nasi-selin',
   'Professor Javier Ortega': 'https://www.lse.ac.uk/european-institute/people/ortega-javier',
   'Dr Chrysoula Papalexatou': 'https://www.lse.ac.uk/european-institute/people/visiting-staff/papalexatou-chrysoula',
   'Dr Maria Prats': 'https://www.lse.ac.uk/european-institute/people/prats-maria',
   'Teresa Pullano': 'https://www.lse.ac.uk/european-institute/people/pullano-teresa',
   'Mr. Mujtaba Rahman': None,
   'Dr Matilde Rosina': 'https://www.lse.ac.uk/european-institute/people/rosina-matilde',
   'Orkun Saka': 'https://www.lse.ac.uk/european-institute/people/saka-orkun',
   'Professor Waltraud Schelkle': 'https://www.lse.ac.uk/european-institute/people/Schelkle-Waltraud',
   'Dr Lucas Schramm': 'https://www.lse.ac.uk/european-institute/people/Schramm-Lucas',
   'Seçkin Sertdemir Özdemir': 'https://www.lse.ac.uk/european-institute/people/visiting-staff/seckin-sertdemir',
   'Dr\xa0Tolga Sinmazdemir': 'https://www.lse.ac.uk/european-institute/research/justint/sinmazdemir-tolga',
   'Dr Ivor Sokolić': 'https://www.lse.ac.uk/european-institute/people/Ivor-Sokolic/Ivor-Sokolić',
   'Mr Anthony Teasdale': 'https://www.lse.ac.uk/european-institute/people/teasdale-anthony',
   'Professor Mark Thatcher': 'https://www.lse.ac.uk/european-institute/people/thatcher-mark',
   'Zbigniew Truchlewski': 'https://www.lse.ac.uk/european-institute/people/zbigniew-truchlewski',
   'Dr. Jimena Valdez': 'https://www.lse.ac.uk/european-institute/people/valdez-jimena',
   'Toon Van Overbeke': 'https://www.lse.ac.uk/european-institute/people/vanOverbeke-toon',
   'voss-dustin': 'https://www.lse.ac.uk/european-institute/people/voss-dustin',
   'Mr. Klaus Welle': None,
   'Martin Westlake': 'https://www.lse.ac.uk/european-institute/people/westlake-martin',
   'Dr. Auke Willems': None,
   'Guney Yildiz': 'https://www.lse.ac.uk/european-institute/people/yildiz-guney'},
  'Research Students ': {'Chris Bick': 'https://www.lse.ac.uk/european-institute/people/bick-chris',
   'Cocaj-venera': 'https://www.lse.ac.uk/european-institute/people/Cocaj-venera',
   'michael cottakis': 'https://www.lse.ac.uk/european-institute/people/cottakis-michael',
   'Tommaso Crescioli': 'https://www.lse.ac.uk/european-institute/people/crescioli-tommaso',
   'Virginia Crespi de Valldaura': 'https://www.lse.ac.uk/european-institute/people/crespi-de-valldaura-virginia',
   'Sean Deel': 'https://www.lse.ac.uk/european-institute/people/deel-sean',
   'Andréa Delestrade': 'https://www.lse.ac.uk/european-institute/people/erspamer-melanie',
   'Sarah Gerwens': 'https://www.lse.ac.uk/european-institute/people/gerwens-sarah',
   'Nilufer Gunes': 'https://www.lse.ac.uk/european-institute/people/gunes-nilufer',
   'Paul Kindermann': 'https://www.lse.ac.uk/european-institute/people/kindermann-paul',
   'Noa Krikler': 'https://www.lse.ac.uk/european-institute/people/krikler-noa',
   'Yonatan Levi': 'https://www.lse.ac.uk/european-institute/people/levi-yonatan',
   'Jacob Lypp': 'https://www.lse.ac.uk/european-institute/people/lypp-jacob',
   'Benedetta Morari': 'https://www.lse.ac.uk/european-institute/people/morari-benedetta',
   'Daniela Roxana Movileanu': 'https://www.lse.ac.uk/european-institute/people/movileanu-daniela',
   'Elena Pro': 'https://www.lse.ac.uk/european-institute/people/pro-elena',
   'Friedrich Püttmann': 'https://www.lse.ac.uk/european-institute/people/püttmann-friedrich/püttmann-friedrich',
   'rashid-tahir': 'https://www.lse.ac.uk/european-institute/people/rashid-tahir',
   'Haile Zola': 'https://www.lse.ac.uk/european-institute/people/zola-haile'}},
 'Department of Finance': {'Finance faculty': {'Ashwini Agrawal': 'https://www.lse.ac.uk/finance/people/faculty/Agrawal',
   'Ulf Axelson': 'https://www.lse.ac.uk/finance/people/faculty/Axelson',
   'Balloch': 'https://www.lse.ac.uk/finance/people/faculty/Balloch',
   'Mike Burkart': 'https://www.lse.ac.uk/finance/people/faculty/Burkart',
   'Georgy  Chabakauri': 'https://www.lse.ac.uk/finance/people/faculty/Chabakauri',
   'Constantin Charles': 'https://www.lse.ac.uk/finance/people/faculty/Charles',
   'Kim Fe Cramer': 'https://www.lse.ac.uk/finance/people/faculty/Cramer',
   'Vicente Cuñat': 'https://www.lse.ac.uk/finance/people/faculty/Cunat',
   'Jon Danielsson': 'https://www.lse.ac.uk/finance/people/faculty/Danielsson',
   'Amil Dasgupta': 'https://www.lse.ac.uk/finance/people/faculty/Dasgupta',
   'Daniel Ferreira': 'https://www.lse.ac.uk/finance/people/faculty/Ferreira',
   'Juanita Gonzalez-Uribe': 'https://www.lse.ac.uk/finance/people/faculty/Gonzalez-Uribe',
   'Dirk Jenter': 'https://www.lse.ac.uk/finance/people/faculty/Jenter',
   'Julliard': 'https://www.lse.ac.uk/finance/people/faculty/Julliard',
   'Peter  Kondor': 'https://www.lse.ac.uk/finance/people/faculty/Kondor',
   'Paula Lopes-Cocco': 'https://www.lse.ac.uk/finance/people/faculty/Lopes-Cocco',
   'Dong Lou': 'https://www.lse.ac.uk/finance/people/faculty/Lou',
   'Igor Makarov': 'https://www.lse.ac.uk/finance/people/faculty/Makarov',
   'Ian Martin': 'https://www.lse.ac.uk/finance/people/faculty/Martin',
   'Martin Oehmke': 'https://www.lse.ac.uk/finance/people/faculty/Oehmke',
   'Daniel Paravisini': 'https://www.lse.ac.uk/finance/people/faculty/Paravisini',
   'Cameron Peng': 'https://www.lse.ac.uk/finance/people/faculty/Peng',
   'Christopher  Polk': 'https://www.lse.ac.uk/finance/people/faculty/Polk',
   'Rohit Rahi': 'https://www.lse.ac.uk/finance/people/faculty/Rahi',
   'Walker Ray': 'https://www.lse.ac.uk/finance/people/faculty/Ray',
   'Cristina Scherrer': 'https://www.lse.ac.uk/finance/people/faculty/Scherrer',
   'Dimitri Vayanos': 'https://www.lse.ac.uk/finance/people/faculty/Vayanos',
   'Liliana Varela': 'https://www.lse.ac.uk/finance/people/faculty/Varela',
   'Michela Verardo': 'https://www.lse.ac.uk/finance/people/faculty/Verardo',
   'David Webb': 'https://www.lse.ac.uk/finance/people/faculty/Webb',
   'Kathy Yuan': 'https://www.lse.ac.uk/finance/people/faculty/Yuan',
   'Linyan Zhu': 'https://www.lse.ac.uk/finance/people/faculty/Zhu',
   'Jean-Pierre Zigrand': 'https://www.lse.ac.uk/finance/people/faculty/Zigrand'},
  'Visiting and Emeritus Academics': {'Dr Angie Adrikogiannapoulou': None,
   'Professor Alex Edmans': None,
   'Dr Yves Nosbusch': None,
   'Dr Dimitris Papadimitriou': None,
   'Dr Jojo Paul': None,
   'Dr Alberto Pellicioli': None,
   'Professor Pedro Saffi': None,
   'Ron Anderson': 'https://www.lse.ac.uk/finance/people/faculty/Anderson'},
  'Class teachers': {'Oliver Ashtari Tafti': 'https://www.lse.ac.uk/finance/people/phd-students/Ashtari-Tafti',
   'Bijan Aghdasi': 'https://www.lse.ac.uk/finance/people/phd-students/Aghdasi',
   'Dr Konstantinos Baltas': None,
   'Dr Guido Bodrato': None,
   'Yanna Cai': 'https://www.lse.ac.uk/finance/people/phd-students/Cai',
   'Benjamin Chen': 'https://www.lse.ac.uk/finance/people/phd-students/ChenB',
   'Ali Choubdaran Varnosfaderani': 'https://www.lse.ac.uk/finance/people/phd-students/Choubdaran-Varnosfaderani',
   'James Clark': 'https://www.lse.ac.uk/finance/people/teaching-faculty/Clark',
   'Dr Gustavo Dias': None,
   'Nico Garrido-Sureda': 'https://www.lse.ac.uk/finance/people/phd-students/Garrido-Sureda',
   'Marcus Vinícius Gomes De Castro': 'https://www.lse.ac.uk/finance/people/phd-students/Gomes-De-Castro',
   'Chris Greiner': 'https://www.lse.ac.uk/finance/people/phd-students/Greiner',
   'Yang Guo': 'https://www.lse.ac.uk/finance/people/phd-students/Guo',
   'Magnus Hjortfors Irie': 'https://www.lse.ac.uk/finance/people/phd-students/Hjortfors-Irie',
   'Davide La Cara': 'https://www.lse.ac.uk/finance/people/phd-students/La-Cara',
   'Jialiang Lou': 'https://www.lse.ac.uk/finance/people/phd-students/Lou',
   'Xinchen Ma': 'https://www.lse.ac.uk/finance/people/phd-students/Ma',
   'Guido Maia Da Cunha': 'https://www.lse.ac.uk/finance/people/phd-students/Maia-Da-Cunha',
   'Nithin Mannil': 'https://www.lse.ac.uk/finance/people/phd-students/Mannil',
   'Maithili Modi': 'https://www.lse.ac.uk/finance/people/phd-students/Modi',
   'Mark Morley': 'https://www.lse.ac.uk/finance/people/phd-students/Morley',
   'Dr Min Park': None,
   'Dr Marco Pelosi': None,
   'Robert Rogers': 'https://www.lse.ac.uk/finance/people/phd-students/Rogers',
   'Vahid Rostamkhani': 'https://www.lse.ac.uk/finance/people/phd-students/Rostamkhani',
   'Jiahong Shi': 'https://www.lse.ac.uk/finance/people/phd-students/ShiJ',
   'Elizaveta Smorodenkova': 'https://www.lse.ac.uk/finance/people/phd-students/Smorodenkova',
   'Yanhuan Tang': 'https://www.lse.ac.uk/finance/people/phd-students/TangY',
   'TianY': 'https://www.lse.ac.uk/finance/people/phd-students/TianY',
   'Niels Wagner': 'https://www.lse.ac.uk/finance/people/phd-students/Wagner',
   'Yibing Wang': 'https://www.lse.ac.uk/finance/people/phd-students/WangY',
   'Song Xiao': 'https://www.lse.ac.uk/finance/people/phd-students/Xiao',
   'Zheyuan Yang': 'https://www.lse.ac.uk/finance/people/phd-students/Yang'},
  'Professional services staff': {'Harriet Baylore': None,
   'Brocklehurst': 'https://www.lse.ac.uk/finance/people/administrative-staff/Brocklehurst',
   'Rhys Cadman': None,
   'Hei-Man Chan': None,
   'Elena Christophorou': None,
   'Mary Comben': None,
   'Oliver Dowling': None,
   'Eleanor Eiserman': None,
   'Francesca Gil': None,
   'Henrietta Griffith': None,
   'Jasper Heeks': None,
   'Miranda Imperial-Sanchez': None,
   'Nicole Jashapara': None,
   'Ella Lucas': None,
   'Yana Marchuk': None,
   'Joe Meegan': None,
   'Isabella Peebles': None,
   'Neesha Rai': 'http://www.lse.ac.uk/finance/people/administrative-staff/Rai',
   'Osmana Raie': None,
   'Jamie Rishworth': None,
   'Dr Domingos Romualdo': None,
   'Sahajpal': 'https://www.lse.ac.uk/finance/people/administrative-staff/Sahajpal',
   'Paul Stoker': None,
   'Simon Tuck': None,
   'Grace Williams': None},
  'Research students': {'Oliver Ashtari Tafti': 'https://www.lse.ac.uk/finance/people/phd-students/Ashtari-Tafti',
   'Bijan Aghdasi': 'https://www.lse.ac.uk/finance/people/phd-students/Aghdasi',
   'Otavio Bitu': 'https://www.lse.ac.uk/finance/people/phd-students/Bitu',
   'Yanna Cai': 'https://www.lse.ac.uk/finance/people/phd-students/Cai',
   'Benjamin Chen': 'https://www.lse.ac.uk/finance/people/phd-students/ChenB',
   'Ali Choubdaran Varnosfaderani': 'https://www.lse.ac.uk/finance/people/phd-students/Choubdaran-Varnosfaderani',
   'Garrido-Sureda': 'https://www.lse.ac.uk/finance/people/phd-students/Garrido-Sureda',
   'Marc Gischer': 'https://www.lse.ac.uk/finance/people/phd-students/Gischer',
   'Marcus Vinícius Gomes De Castro': 'https://www.lse.ac.uk/finance/people/phd-students/Gomes-De-Castro',
   'Chris Greiner': 'https://www.lse.ac.uk/finance/people/phd-students/Greiner',
   'Yang Guo': 'https://www.lse.ac.uk/finance/people/phd-students/Guo',
   'Magnus Hjortfors Irie': 'https://www.lse.ac.uk/finance/people/phd-students/Hjortfors-Irie',
   'Davide La Cara': 'https://www.lse.ac.uk/finance/people/phd-students/La-Cara',
   'Mengqi Li': 'https://www.lse.ac.uk/finance/people/phd-students/Li',
   'Jialiang Lou': 'https://www.lse.ac.uk/finance/people/phd-students/Lou',
   'Xinchen Ma': 'https://www.lse.ac.uk/finance/people/phd-students/Ma',
   'Guido Maia Da Cunha': 'https://www.lse.ac.uk/finance/people/phd-students/Maia-Da-Cunha',
   'Nithin Mannil': 'https://www.lse.ac.uk/finance/people/phd-students/Mannil',
   'Maithili Modi': 'https://www.lse.ac.uk/finance/people/phd-students/Modi',
   'Mark Morley': 'https://www.lse.ac.uk/finance/people/phd-students/Morley',
   'Robert Rogers': 'https://www.lse.ac.uk/finance/people/phd-students/Rogers',
   'Vahid Rostamkhani': 'https://www.lse.ac.uk/finance/people/phd-students/Rostamkhani',
   'Jiahong Shi': 'https://www.lse.ac.uk/finance/people/phd-students/ShiJ',
   'Elizaveta Smorodenkova': 'https://www.lse.ac.uk/finance/people/phd-students/Smorodenkova',
   'Fernando Soares': 'https://www.lse.ac.uk/finance/people/phd-students/Soares',
   'Irina-Eusignia Stanciu': 'https://www.lse.ac.uk/finance/people/phd-students/Stanciu',
   'Yanhuan Tang': 'https://www.lse.ac.uk/finance/people/phd-students/TangY',
   'Jiaxing Tian': 'https://www.lse.ac.uk/finance/people/phd-students/Tian',
   'TianY': 'https://www.lse.ac.uk/finance/people/phd-students/TianY',
   'Goncalo Vieira Da Luz': 'https://www.lse.ac.uk/finance/people/phd-students/Vieira-Da-Luz',
   'Niels Wagner': 'https://www.lse.ac.uk/finance/people/phd-students/Wagner',
   'Jiaming Wang': 'https://www.lse.ac.uk/finance/people/phd-students/Wang',
   'Yibing Wang': 'https://www.lse.ac.uk/finance/people/phd-students/WangY',
   'Yue Wu': 'https://www.lse.ac.uk/finance/people/phd-students/Wu',
   'Song Xiao': 'https://www.lse.ac.uk/finance/people/phd-students/Xiao',
   'Bolin Xu': 'https://www.lse.ac.uk/finance/people/phd-students/Xu',
   'Yun Xue': 'https://www.lse.ac.uk/finance/people/phd-students/Xue',
   'Zheyuan Yang': 'https://www.lse.ac.uk/finance/people/phd-students/Yang',
   'Jianing Yuan': 'https://www.lse.ac.uk/finance/people/phd-students/YuanJ',
   'Claudia Zhao': 'https://www.lse.ac.uk/finance/people/phd-students/Zhao'}},
 'Firoz Lalji Institute for Africa': {'Internal board': {'Robtel Neajai Pailey': 'https://www.lse.ac.uk/social-policy/people/academic-staff/dr-robtel-neajai-pailey',
   'Claire Mercer\xa0': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/claire-mercer',
   'Catherine Boone': 'https://www.lse.ac.uk/international-development/people/catherine-boone',
   'Laura Mann': 'https://www.lse.ac.uk/international-development/people/laura-mann',
   'Wendy Willems': 'https://www.lse.ac.uk/media-and-communications/people/academic-staff/wendy-willems',
   'Katrin Flikschuh': 'https://www.lse.ac.uk/government/people/academic-staff/katrin-flikschuh',
   'Lucy Kanya': 'https://www.lse.ac.uk/health-policy/people/dr-lucy-kanya',
   'Declan Conway': 'https://www.lse.ac.uk/granthaminstitute/profile/declan-conway/',
   'Deborah James': 'https://www.lse.ac.uk/anthropology/people/deborah-james',
   'Leigh Gardner': 'https://www.lse.ac.uk/Economic-History/People/Faculty-and-teachers/Dr-Leigh-Gardner',
   'Joanna Lewis': 'https://www.lse.ac.uk/International-History/People/academicStaff/lewis/lewis',
   'Chaloka Beyani': 'https://www.lse.ac.uk/law/people/academic-staff/chaloka-beyani',
   'Jo Beall': 'https://www.lse.ac.uk/cities/staff-profiles/Jo-Beall',
   'George Ofosu': 'https://www.lse.ac.uk/government/people/academic-staff/george-ofosu'},
  'Affiliated faculty ': {'Laura Mann': 'https://www.lse.ac.uk/international-development/people/laura-mann',
   'Catherine Boone': 'https://www.lse.ac.uk/international-development/people/catherine-boone',
   'Laura Bear': 'https://www.lse.ac.uk/anthropology/people/laura-bear',
   'Teddy Brett': 'https://www.lse.ac.uk/international-development/people/teddy-brett',
   'Dr Simidele Dosekun': 'https://www.lse.ac.uk/media-and-communications/people/academic-staff/simidele-dosekun'},
  'Research staff ': {'Costanza Torre': 'https://www.lse.ac.uk/africa/people/Researchers/ConstanzaTorre',
   'Tom Kirk': 'https://www.lse.ac.uk/africa/centre-for-public-authority-and-international-development/Researcher-spotlight/Spotlight-Tom-Kirk',
   'Elizabeth Ngutuku': 'https://www.lse.ac.uk/africa/people/Researchers/Elizabeth-Ngutuku',
   'Colette van der Ven': None,
   'Oluwasola Omoju': 'https://www.lse.ac.uk/africa/people/Researchers/Oluwasola-Omoju',
   'Geoffroy Guepie': None,
   'Anne-Line Rodriguez': 'https://www.lse.ac.uk/africa/people/Researchers/Anne-Line',
   'Iliana Sarafian': 'https://www.lse.ac.uk/africa/people/Researchers/Iliana',
   'Julian Hopwood': 'https://www.lse.ac.uk/africa/people/Researchers/Julian-Hopwood',
   'Innocent Anguyo': 'https://www.lse.ac.uk/africa/people/Researchers/Innocent',
   'Gedion Onyango ': None},
  'Centennial and visiting professors ': {'Professor Alcinda Honwana\xa0': None,
   'Professor Leonard Wantchekon\xa0': 'https://africanschoolofeconomics.com/about-us/president-founder/',
   'Tony Barnett': 'https://www.lse.ac.uk/africa/people/Researchers/Tony-Barnett',
   'Professor Gibril Faal': 'https://www.lse.ac.uk/africa/people/Researchers/Gibril-Faal',
   'Dr Piroska Nagy Mohácsi': 'https://www.lse.ac.uk/africa/people/Researchers/Piroska-Nagy-Mohacsi'},
  'Visiting senior fellows ': {'Desné Masie': 'https://www.lse.ac.uk/africa/people/Researchers/Desné-Masie',
   'Dr Louise Arimatsu': 'https://www.lse.ac.uk/women-peace-security/people/louise-arimatsu',
   'Souad Mohamed': 'https://www.lse.ac.uk/africa/people/Researchers/Souad-Mohamed',
   'Dr Fatima el Issawi': 'https://www.lse.ac.uk/africa/people/Researchers/Dr-Fatima-el-Issawi',
   'Dr. Vanessa Iwowo': 'https://www.lse.ac.uk/africa/people/Researchers/Dr.-Vanessa-Iwowo',
   'Anna Macdonald': 'https://www.lse.ac.uk/international-development/people/fellows/anna-macdonald',
   'Holly Porter\xa0': None,
   'Shirley Ze Yu': 'https://www.lse.ac.uk/africa/people/Researchers/Shirley-Yu'},
  'Visiting fellows': {'William Davis': 'https://www.lse.ac.uk/africa/people/Researchers/William-Davis',
   'Kasper Vrolijk\xa0': None,
   'Francesca Beausang': 'https://www.lse.ac.uk/africa/people/Researchers/Francesca-Beausang',
   'Kara Blackmore': 'https://www.lse.ac.uk/international-development/people/research-students/kara-blackmore',
   'Dr\xa0Mebratu Kelecha': 'https://www.lse.ac.uk/africa/people/Researchers/Mebratu-Kelecha',
   'Naomi Pendle': 'https://www.lse.ac.uk/international-development/people/fellows/naomi-pendle',
   'Juliet Bedford': 'https://www.lse.ac.uk/africa/people/Researchers/Juliet-Bedford',
   'Paroma Bhattacharya': 'https://www.lse.ac.uk/africa/people/Researchers/Paroma-Bhattacharya',
   'Ponsiano Bimeny': 'https://www.lse.ac.uk/africa/people/Researchers/Ponsiano-Bimeny',
   'Dr Uche Igwe': 'https://www.lse.ac.uk/africa/people/Researchers/Uche-Igwe',
   'Theresa Jones': 'https://www.lse.ac.uk/africa/people/Researchers/Dr-Theresa-Jones',
   'Esther Marijnen': 'https://www.lse.ac.uk/africa/people/Researchers/Esther-Marijnen',
   'Flora McCrone': 'https://www.lse.ac.uk/africa/people/Researchers/Flora-McCrone',
   'Georgina Pearson': 'http://www.lse.ac.uk/international-development/people/fellows/georgina-pearson',
   'Maria del Pilar Lopez-Uribe': 'https://www.lse.ac.uk/africa/people/Researchers/Maria-Lopez-Uribe',
   'Cristin Alexis Fergus ': None,
   'Claire Elder ': None,
   'Nikita Simpson': 'https://www.lse.ac.uk/africa/people/Researchers/Nikita',
   'Jonathan Bashi': 'https://www.lse.ac.uk/africa/people/Researchers/Jonathan-bashi',
   'Maudo Jallow': 'https://www.lse.ac.uk/africa/people/Researchers/Maudo-Jallow',
   'Elizabeth Storer': 'https://www.lse.ac.uk/africa/people/Researchers/Elizabeth',
   "Ryan O'Byrne": 'https://www.lse.ac.uk/africa/people/Researchers/Ryan',
   'Jonah Lipton': 'https://www.lse.ac.uk/africa/people/Researchers/JonahLipton'},
  'PhD candidates': {'Alice Robinson': 'https://www.lse.ac.uk/international-development/people/research-students/alice-robinson',
   'Charlotte Brown\xa0': None},
  'Head': {'Professor Tim Allen': 'http://www.lse.ac.uk/international-development/people/tim-allen',
   'Fadil Elobeid\xa0': 'http://www.lse.ac.uk/africa/people/Staff/Fadil-Elobeid',
   'Dr Martha Geiger Mwenitete': 'http://www.lse.ac.uk/africa/people/Staff/martha-g',
   'Dr David Luke': 'https://www.lse.ac.uk/africa/people/Staff/David-Luke',
   'Lesley Orero': 'http://www.lse.ac.uk/africa/people/Staff/Lesley',
   'Anna Williams': 'http://www.lse.ac.uk/africa/people/Staff/Anna-Williams',
   'Tosin Adebisi': 'http://www.lse.ac.uk/africa/people/Staff/Tosin',
   'Mark Briggs': 'http://www.lse.ac.uk/africa/people/Staff/Mark',
   'Sofija Spasenoska': 'http://www.lse.ac.uk/africa/people/Staff/Sofija-Spasenoska',
   'Eunice Asantewaa Hansen-Sackey': 'http://www.lse.ac.uk/africa/people/Staff/Eunice-Asantewaa-Hansen-Sackey'}},
 'Department of Gender Studies': {'Faculty': {'Carrie Hamilton': 'https://www.lse.ac.uk/gender/people-profiles/faculty/carrie-hamilton/Carrie-Hamilton',
   'Clare Hemmings': 'https://www.lse.ac.uk/gender/people-profiles/faculty/clare-hemmings',
   'Aiko Holvikivi': 'https://www.lse.ac.uk/gender/people-profiles/faculty/aiko-holvikivi/Aiko-Holvikivi',
   'Emrah Karakus': 'https://www.lse.ac.uk/gender/people-profiles/faculty/emrah-karakus',
   'Zeynep Kilicoglu': 'https://www.lse.ac.uk/gender/people-profiles/faculty/zeynep-kilicoglu',
   'Sumi Madhok': 'https://www.lse.ac.uk/gender/people-profiles/faculty/sumi-madhok',
   'Milo Bettocchi': 'https://www.lse.ac.uk/gender/people-profiles/faculty/milo-miller/Milo-Miller',
   'Gloria Novovic': 'https://www.lse.ac.uk/gender/people-profiles/faculty/gloria-novovic/Gloria-Novovic',
   'Sharmila Parmanand': 'https://www.lse.ac.uk/gender/people-profiles/faculty/sharmila-parmanand/Sharmila-Parmanand',
   'Ania Plomien': 'https://www.lse.ac.uk/gender/people-profiles/faculty/ania-plomien',
   'Maria Rashid': 'https://www.lse.ac.uk/gender/people-profiles/faculty/maria-rashid',
   'SM Rodriguez': 'https://www.lse.ac.uk/gender/people-profiles/faculty/sm-rodriguez',
   'Leticia Sabsay': 'https://www.lse.ac.uk/gender/people-profiles/faculty/leticia-sabsay',
   'Wendy Sigle': 'https://www.lse.ac.uk/gender/people-profiles/faculty/wendy-sigle',
   'Sadie Wearing\xa0': 'https://www.lse.ac.uk/gender/people-profiles/faculty/sadie-wearing'},
  'Professional Services Staff': {'Henrietta Burr': None,
   'Rob Kirkland': None,
   'Catherine Perry': None,
   'Annie Robinson': None,
   'Violet Fox': None},
  'Visitors, Affiliates and Emeritus': {'Mary Evans': 'https://www.lse.ac.uk/gender/people-profiles/Visitors/mary-evans',
   'Marsha Henry': 'https://www.lse.ac.uk/gender/people-profiles/Visitors/marsha-henry',
   'Diane Perrons': 'https://www.lse.ac.uk/gender/people-profiles/Visitors/diane-perrons',
   'Alexandra Pugh': 'https://info.lse.ac.uk/current-students/lse100/people/alexandra-pugh/Alexandra-Pugh',
   'Shirin Rai': 'https://www.lse.ac.uk/gender/people-profiles/Visitors/Shirin-Rai',
   'Vron Ware': 'https://www.lse.ac.uk/gender/people-profiles/Visitors/Vron-Ware',
   'Irina Zherebkina': 'https://www.lse.ac.uk/gender/people-profiles/Visitors/Irina-Zherebkina',
   'Melissa Chacon': 'https://www.lse.ac.uk/gender/people-profiles/Visitors/Melissa-Chacon/Melissa-Chacon'},
  'Doctoral Researchers': {'Aynura Akbas': 'https://www.lse.ac.uk/gender/people-profiles/PhD/Aynura-Akbas/Aynura-Akbaš',
   'Nour Almazidi': 'https://www.lse.ac.uk/gender/people-profiles/PhD/Nour-Almazidi/Nour-Almazidi',
   'Alia Amirali': 'https://www.lse.ac.uk/gender/people-profiles/PhD/Alia-Amirali/Alia-Amirali',
   'Malena Bastida Antich': 'https://www.lse.ac.uk/gender/people-profiles/PhD/Malena-Bastida-Antich/Malena-Bastida-Antich',
   'Zuzana Dančíková': 'https://www.lse.ac.uk/gender/people-profiles/PhD/Zuzana-Dančíková/Zuzana-Dančíková',
   'Lizzie Hobbs': 'https://www.lse.ac.uk/gender/people-profiles/PhD/Lizzie-Hobbs/Lizzie-Hobbs',
   'Ting-Sian-Liu': 'https://www.lse.ac.uk/gender/people-profiles/PhD/Ting-Sian-Lui/Ting-Sian-Liu',
   'Luma (Lucas) Mantilla Garino': 'https://www.lse.ac.uk/gender/people-profiles/PhD/Luma-Lucas-Mantilla-Garino',
   'Alanah Mortlock': 'https://www.lse.ac.uk/gender/people-profiles/PhD/Alanah-Mortlock',
   'Magda Muter': 'https://www.lse.ac.uk/gender/people-profiles/PhD/Magda-Muter/Magda-Muter',
   'Oksana Potapova': None,
   'Florence Waller-Carr': 'https://www.lse.ac.uk/gender/people-profiles/PhD/Florence-Waller-Carr/Florence-Waller-Carr',
   'Senel Wanniarachchi': 'https://www.lse.ac.uk/gender/people-profiles/PhD/Senel-Wanniarachchi/Senel-Wanniarachchi',
   'Claire Wilmot': 'https://www.lse.ac.uk/gender/people-profiles/PhD/Claire-Wilmot/Claire-Wilmot'},
  'Advisory Committee': {'Suki Ali': 'http://www.lse.ac.uk/sociology/people/suki-ali',
   'Sarah Ashwin': 'http://www.lse.ac.uk/management/people/academic-staff/sashwin?from_serp=1',
   'Shakuntala Banaji': 'http://www.lse.ac.uk/media-and-communications/people/academic-staff/shakuntala-banaji',
   'Lilie Chouliaraki': 'http://www.lse.ac.uk/media-and-communications/people/academic-staff/lilie-chouliaraki',
   'Ernestina Coast': 'http://www.lse.ac.uk/international-development/people/ernestina-coast',
   'Emily Jackson': 'https://www.lse.ac.uk/law/people/academic-staff/emily-jackson',
   'Denisa Kostovicova': 'https://www.lse.ac.uk/european-institute/people/kostovicova-denisa',
   'Nicola Lacey': 'http://www.lse.ac.uk/law/people/academic-staff/nicola-lacey',
   'Katharine Millar': 'https://www.lse.ac.uk/international-relations/people/millar',
   'Irini Moustaki': 'http://www.lse.ac.uk/Statistics/People/Professor-Irini-Moustaki',
   'Anne Phillips': 'http://www.lse.ac.uk/government/people/academic-staff/anne-phillips',
   'Coretta Phillips': 'https://www.lse.ac.uk/social-policy/people/academic-staff/Professor-Coretta-Phillips',
   'Sara Salem': 'https://www.lse.ac.uk/sociology/people/sara-salem',
   'Hakan Seckinelgin': 'http://www.lse.ac.uk/social-policy/people/academic-staff/Dr-Hakan-Seckinelgin',
   'Alpa Shah': 'http://www.lse.ac.uk/anthropology/people/alpa-shah',
   'Imaobong Umoren': 'http://www.lse.ac.uk/International-History/People/academicStaff/umoren/umoren'}},
 'Department of Geography and Environment': {'Senior Management Team': {'Prof\xa0Hyun Bang Shin': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/hyun-bang-shin',
   'Christian Hilber': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/christian-hilber',
   'Claire Mercer': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/claire-mercer',
   'Simon Dietz': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/simon-dietz',
   'Nancy Holman': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/nancy-holman',
   'Olmo Silva': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/olmo-silva',
   'Sam Colegate': None},
  'A-Z': {'Laura Antona': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/laura-antona',
   'Giles Atkinson': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/giles-atkinson',
   'Aretousa Bloom': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/aretousa-bloom',
   'filippo boeri': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/filippo-boeri',
   'Felipe Carozzi': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/felipe-carozzi',
   'Ryan Centner': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/ryan-centner',
   'Anomitro Chatterjee': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/anomitro-chatterjee',
   'Paul Cheshire': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/paul-cheshire',
   'Julia Corwin': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/julia-corwin',
   'Riccardo Crescenzi': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/riccardo-crescenzi',
   'Muna Dajani': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/muna-dajani',
   'Simon Dietz': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/simon-dietz',
   'Eugenie Dugoua': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/eugenie-dugoua',
   'Pooya Ghoddousi': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/pooya-ghoddousi',
   'Steve Gibbons': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/steve-gibbons',
   'Ian Gordon': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/ian-gordon',
   'Vernon Henderson': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/vernon-henderson',
   'Christian Hilber': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/christian-hilber',
   'Nancy Holman': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/nancy-holman',
   'Carolin Hulke': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/carolin-hulke',
   'Stephen Jarvis': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/stephen-jarvis',
   'David Jones': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/david-jones',
   'Gareth A Jones': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/gareth-a-jones',
   'Neil Lee': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/neil-lee',
   'Murray Low': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/murray-low',
   'Alan Mace': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/alan-mace',
   'Michael Mason': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/michael-mason',
   'Dr Tanya Matthan': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/tanya-matthan',
   'Claire Mercer': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/claire-mercer',
   'Susana Mourato': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/susana-mourato',
   'Eric Neumayer': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/eric-neumayer',
   'Henry Overman': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/henry-overman',
   'Dr Jeffrey Pagel': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/jeff-pagel',
   'Charles Palmer': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/charles-palmer',
   'Erica Pani': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/erica-pani',
   'Kasia Paprocki': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/kasia-paprocki',
   'Richard Perkins': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/richard-perkins',
   'Laura Pulido': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/laura-pulido',
   'Elena Renzullo': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/elena-renzullo',
   'davide rigo': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/davide-rigo',
   'Andres Rodriguez-Pose': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/andres-rodriguez-pose',
   'Sefi Roth': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/sefi-roth',
   'Romola Sanyal': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/romola-sanyal',
   'Rodolfo Sejas-Portillo': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/rodolfo-sejas-portillo',
   'Hyun Bang Shin': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/hyun-bang-shin',
   'Olmo Silva': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/olmo-silva',
   'Thomas Smith': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/thomas-smith',
   'Jessie Speer': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/jessie-speer',
   'Michael Storper': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/michael-storper',
   'ana verala verala': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/ana-varela-varela',
   'meredith whitten': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/meredith-whitten',
   'hendrick-wolff': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/hendrik-wolff',
   'Austin Zeiderman': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/austin-zeiderman'},
  'Economic Geography': {'filippo boeri': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/filippo-boeri',
   'Felipe Carozzi': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/felipe-carozzi',
   'Paul Cheshire': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/paul-cheshire',
   'Riccardo Crescenzi': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/riccardo-crescenzi',
   'Steve Gibbons': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/steve-gibbons',
   'Vernon Henderson': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/vernon-henderson',
   'Christian Hilber': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/christian-hilber',
   'Carolin Hulke': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/carolin-hulke',
   'Neil Lee': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/neil-lee',
   'Prof\xa0Vassilis Monastiriotis': 'https://www.lse.ac.uk/european-institute/people/monastiriotis-vassilis',
   'Henry Overman': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/henry-overman',
   'Erica Pani': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/erica-pani',
   'Elena Renzullo': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/elena-renzullo',
   'davide rigo': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/davide-rigo',
   'Andres Rodriguez-Pose': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/andres-rodriguez-pose',
   'Rodolfo Sejas-Portillo': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/rodolfo-sejas-portillo',
   'Olmo Silva': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/olmo-silva',
   'Michael Storper': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/michael-storper',
   'ana verala verala': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/ana-varela-varela',
   'Prof Christine Whitehead': 'https://www.lse.ac.uk/economics/people/faculty/christine-whitehead'},
  'Environment': {'Giles Atkinson': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/giles-atkinson',
   'Anomitro Chatterjee': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/anomitro-chatterjee',
   'Julia Corwin': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/julia-corwin',
   'Muna Dajani': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/muna-dajani',
   'Simon Dietz': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/simon-dietz',
   'Eugenie Dugoua': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/eugenie-dugoua',
   'Dr Matteo M Galizzi': 'https://www.lse.ac.uk/PBS/People/Dr-Matteo-M-Galizzi',
   'Nancy Holman': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/nancy-holman',
   'Stephen Jarvis': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/stephen-jarvis',
   'David Jones': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/david-jones',
   'Michael Mason': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/michael-mason',
   'Dr Tanya Matthan': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/tanya-matthan',
   'Susana Mourato': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/susana-mourato',
   'Eric Neumayer': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/eric-neumayer',
   'Dr Jeffrey Pagel': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/jeff-pagel',
   'Charles Palmer': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/charles-palmer',
   'Kasia Paprocki': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/kasia-paprocki',
   'Richard Perkins': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/richard-perkins',
   'Professor Laura Pulido': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/laura-pulido',
   'Sefi Roth': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/sefi-roth',
   'Dr Ganga Shreedhar': 'https://www.lse.ac.uk/PBS/People/Dr-Ganga-Shreedhar',
   'Thomas Smith': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/thomas-smith',
   'meredith whitten': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/meredith-whitten',
   'hendrick-wolff': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/hendrik-wolff'},
  'Urbanisation, Planning and Development\xa0': {'Laura Antona': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/laura-antona',
   'Aretousa Bloom': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/aretousa-bloom',
   'Ryan Centner': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/ryan-centner',
   'Julia Corwin': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/julia-corwin',
   'Pooya Ghoddousi': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/pooya-ghoddousi',
   'Ian Gordon': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/ian-gordon',
   'Nancy Holman': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/nancy-holman',
   'Gareth A Jones': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/gareth-a-jones',
   'Murray Low': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/murray-low',
   'Alan Mace': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/alan-mace',
   'Claire Mercer': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/claire-mercer',
   'Erica Pani': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/erica-pani',
   'Laura Pulido': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/laura-pulido',
   'Romola Sanyal': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/romola-sanyal',
   'Kath Scanlon': 'http://www.lse.ac.uk/business-and-consultancy/consulting/experts/kath-scanlon',
   'Hyun Bang Shin': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/hyun-bang-shin',
   'Jessie Speer': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/jessie-speer',
   'Austin Zeiderman': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/austin-zeiderman'},
  'Graduate Teaching Assistants and Guest Teachers/Lecturers A-Z': {'Ignacio Aravena Gonzalez': 'https://www.lse.ac.uk/geography-and-environment/people/phd-students/ignacio-aravena-gonzález',
   'Antonio Avila-Uribe': 'https://www.lse.ac.uk/geography-and-environment/people/phd-students/antonio-avila-uribe',
   'Emmanuel C. P. Awohouedji': 'https://www.lse.ac.uk/geography-and-environment/people/phd-students/emmanuel-c.-p.-awohouedji',
   'Ivana Bevilacqua': 'https://www.kcl.ac.uk/people/ivana-bevilacqua',
   'Erica Borg': None,
   'Paul Brandily': 'https://cep.lse.ac.uk/_new/people/person.asp?id=11146',
   'Álvaro Carbonell Rodríguez': None,
   'Amrita DasGupta': 'https://www.soas.ac.uk/about/amrita-dasgupta',
   'Dr Magdalena Dominguez': None,
   'Tea Gamtkitsulashvili': 'https://www.lse.ac.uk/geography-and-environment/people/phd-students/tea-gamtkitsulashvili',
   'Ali Glisson': 'https://www.lse.ac.uk/geography-and-environment/people/phd-students/ali-glisson',
   'Nikolaus Hastreiter\xa0': 'https://www.lse.ac.uk/granthaminstitute/profile/nikolaus-hastreiter/',
   'Andrea Herrera': None,
   'Merrill Hopper': None,
   'Violet Lasdun': None,
   'Woong Ki Lee': 'https://www.lse.ac.uk/geography-and-environment/people/phd-students/woong-ki-lee',
   'Ali Kokbudak': None,
   'Till Meisssner': None,
   'jonathan mille': 'https://www.lse.ac.uk/geography-and-environment/people/GTAs-and-guest-teachers/jonathan-mille',
   'William Mitchell\xa0': 'https://www.lse.ac.uk/International-History/People/GTA-GT/mitchell/mitchell',
   'Morgan Olive-Carmellini': None,
   'Shaonlee Patranabis': 'https://www.lse.ac.uk/geography-and-environment/people/phd-students/shaonlee-patranabis',
   'Line Relisieux': None,
   'Capucine Riom': 'https://cep.lse.ac.uk/_new/people/person.asp?id=10369',
   'Martina Rotolo': None,
   'Hanadi Samhan': 'https://www.ucl.ac.uk/bartlett/development/people/mphilphd-students/hanadi-samhan',
   'Sraman Sircar': None,
   'Tom Siddall': None,
   'Anna Smith': None,
   'Romano Tarsia': 'https://www.lse.ac.uk/geography-and-environment/people/phd-students/romano-tarsia',
   'Hanh-An Trinh': 'https://www.lse.ac.uk/geography-and-environment/people/phd-students/hanh-an-trinh',
   'Ran Wei': None,
   'Kapil Yadav': None,
   'Jingyuan Zeng': 'https://www.lse.ac.uk/geography-and-environment/people/phd-students/jingyuan-zeng',
   'Ailin Zhang': None},
  'PhD students': {'Kevin M. Adams': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/kevin-m-adams',
   'Juan Alvarez-Vilanova': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/juan-alvarez-vilanova',
   'Pia Andres': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/pia-andres',
   'Ignacio Aravena González': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/ignacio-aravena-gonzález',
   'Antonio Avila-Uribe': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/antonio-avila-uribe',
   'Emmanuel C P Awohouedji': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/emmanuel-c.-p.-awohouedji',
   'Daniela Baeza Breinbauer': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/daniela-baeza-breinbauer',
   'Lucrecia Bertelli': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/lucrecia-bertelli',
   'Martin Brown Munene': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/martin-brown-munene',
   'Joan Camilo Lopez': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/joan-camilo-lopez',
   'Morgan Olive Carmellini': None,
   'Vincent Chung Jie Lun': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/vincent-chung-jie-lun',
   'Mattie Cox': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/mattie-cox',
   'Ana De Menezes': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/ana-de-menezes',
   'Alaa Dia': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/alaa-dia',
   'Ina Drouven': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/ina-drouven',
   'Sarah Elven': 'https://www.lse.ac.uk/granthaminstitute/profile/sarah-elven/',
   'Tea Gamtkitsulashvili': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/tea-gamtkitsulashvili',
   'Dalia Gebrial': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/dalia-gebrial',
   'Fernanda Gimenes': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/fernanda-gimenes',
   'Ali Glisson': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/ali-glisson',
   'Nikolaus Hastreiter': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/nikolaus-hastreiter',
   'Andrea Herrera Bórquez': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/andrea-herrera-bórquez',
   'Alaa Iktash': None,
   'Anu Jogesh': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/anu-jogesh',
   'Jeanette L. Kaiser': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/jeanette-l.-kaiser',
   'Woong Ki Lee': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/woong-ki-lee',
   'Violet Lasdun': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/violet-lasdun',
   'Vivian Liu': None,
   'Yarden Manelzon': 'https://www.lse.ac.uk/geography-and-environment/people/phd-students/yarden-manelzon/home',
   'Sandiswa Mapukata': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/sandiswa-mapukata',
   'Till Meissner': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/till-meissner',
   'Annalyse Moskeland': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/annalyse-moskeland',
   'Su Su Myat': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/su-su-myat',
   'Marina Nazneen': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/marina-nazneen',
   'Gabriela Neves De Lima': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/gabriela-neves-de-lima/Gabriela-Neves-De-Lima',
   'Martina Pardy': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/martina-pardy/Martina-Pardy',
   'Shaonlee Patranabis': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/shaonlee-patranabis',
   'Muhammad Yorga Permana': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/muhammad-yorga-permana',
   'Gabriele Piazza': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/gabriele-piazza',
   'Julien Picard': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/julien-picard',
   'Vishnu Prasad': None,
   'Line Relisieux': None,
   'Lindiwe Rennert': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/lindiwe-rennert',
   'Capucine Riom': 'https://cep.lse.ac.uk/_new/people/person.asp?id=10369',
   'Heini-Emilia Saari': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/heini-emilia-saari',
   'M. Adil Sait': 'https://www.lse.ac.uk/geography-and-environment/people/phd-students/m.-adil-sait',
   'Fizzah Sajjad': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/fizzah-sajjad',
   'Sraman Sircar': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/sraman-sircar',
   'Martina Vittoria Sottini': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/martina-vittoria-sottini',
   'Romano Tarsia': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/romano-tarsia',
   'Frida Timan': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/frida-timan',
   'Hanh-An Trinh': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/hanh-an-trinh',
   'Ran Wei': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/ran-wei',
   'Melissa Weihmayer': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/melissa-weihmayer',
   'Xian Wu': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/xian-wu',
   'Leiboyu Xiang': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/leiboyu-xiang',
   'Ka Chi Yip': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/ka-chi-yip',
   'Qing Yuan Guo': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/qing-yuan-guo',
   'Jingyuan Zeng': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/jingyuan-zeng'},
  'Our Professional Services Teams': {'Sam Colegate': None,
   'Ivanilza Da Mata': None,
   'Lee Mager': None,
   'Maya King': None,
   'Amy Cox': None,
   'Ruby Bennett': None,
   'Jodie Bispham': None,
   'Hanna Wolodzko': None,
   'Tom Jones': None,
   'Zia Simpson': None,
   'Catherine Mitchell': None,
   'Isobel Jones': None,
   'Emily Douglas': None},
  'Affiliate staff': {'Dr Matteo M Galizzi': 'https://www.lse.ac.uk/PBS/People/Dr-Matteo-M-Galizzi',
   'Dr Vassilis Monastiriotis': 'https://www.lse.ac.uk/european-institute/people/monastiriotis-vassilis',
   'Kerstin J. Schaefer': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/kerstin-schaefer',
   'Dr Ganga Shreedhar': 'https://www.lse.ac.uk/PBS/People/Dr-Ganga-Shreedhar',
   'Monika Streule': 'https://www.lse.ac.uk/geography-and-environment/people/visiting-staff/monika-streule'},
  'Visiting staff': {'Gabriel Ahlfeldt': 'https://www.wiwi.hu-berlin.de/de/professuren/vwl/oe',
   'Caroline Cornish': None,
   'Kate Dawson': 'https://twitter.com/kateedaws?lang=en',
   'Lewis Dijkstra': 'https://scholar.google.com/citations?user=PLPeZaoAAAAJ&hl=en',
   'Mara Ferreri': 'https://www.researchgate.net/profile/Mara-Ferreri',
   'Deborah Fromm': 'https://www.lse.ac.uk//geography-and-environment/people/visiting-staff/deborah-fromm',
   'Michael Harloe': 'https://www.kellogg.ox.ac.uk/our-people/michael-harloe/',
   'Kyoung-Ae Han': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/didi-han',
   'Simona Iammarino': 'https://www.lse.ac.uk//geography-and-environment/people/academic-staff/simona-iammarino',
   'Carolin Ioramashvili': 'https://www.lse.ac.uk//geography-and-environment/people/phd-students/carolin-ioramashvili',
   'Yi Jin': 'https://www.researchgate.net/profile/Yi-Jin-32',
   'Mohsen Khezri': 'https://www.lse.ac.uk//geography-and-environment/people/visiting-staff/mohsen-khezri',
   'Loretta Lees': 'https://www.bu.edu/ioc/profile/loretta-lees/',
   'Paul Metcalfe': 'https://www.linkedin.com/in/pauljmetcalfe/',
   'Guy Morrell': None,
   'Mark Nesbitt': None,
   'Danielle Purifoy': 'http://www.daniellepurifoy.com/',
   'Camilla Royle': 'https://camillaroyle.com/',
   'Asato Saito': 'https://er-web.ynu.ac.jp/html/SAITO_Asato/en.html',
   'Francis Salway': 'https://en.wikipedia.org/wiki/Francis_Salway',
   'Deen Shariff Sharp': 'https://www.deensharp.com/',
   'Rory Sullivan': 'https://www.lse.ac.uk/geography-and-environment/people/visiting-staff/rory-sullivan',
   'Jayaraj Sundaresan': 'https://www.lse.ac.uk//geography-and-environment/people/academic-staff/jayaraj-sundaresan',
   'Callum Ward': None,
   'Jason Wong': 'https://www.lse.ac.uk//geography-and-environment/people/academic-staff/jason-wong',
   'Baek Yung Kim': 'https://www.lse.ac.uk//geography-and-environment/people/visiting-staff/baek-yung-kim'}},
 'Department of Government': {'Key Contacts': {'Professor Paul Kelly': None,
   'Imran Iqbal': None,
   'Natalie Paris': None},
  'Academic Staff': {'Mr Victor Agboga': 'https://www.lse.ac.uk/government/people/academic-staff/victor-agboga',
   'Elise Antoine': 'https://www.lse.ac.uk/government/people/academic-staff/elise-antoine',
   'Dr Paul Apostolidis': 'https://www.lse.ac.uk/government/people/academic-staff/paul-apostolidis',
   'Tom Bailey': 'https://www.lse.ac.uk/government/people/academic-staff/tom-bailey',
   'Dr Daniel Berliner': 'https://www.lse.ac.uk/government/people/academic-staff/daniel-berliner',
   'Professor Catherine Boone': 'https://www.lse.ac.uk/government/people/academic-staff/catherine-boone',
   'Dr Sarah Brierley': 'https://www.lse.ac.uk/government/people/academic-staff/sarah-brierley',
   'Professor Michael Bruter': 'https://www.lse.ac.uk/government/people/academic-staff/michael-bruter',
   'Professor John Chalcraft': 'https://www.lse.ac.uk/government/people/academic-staff/john-chalcraft',
   'Dr Nicholas Devlin': 'https://www.lse.ac.uk/government/people/academic-staff/nicholas-devlin',
   'Professor Torun Dewan': 'https://www.lse.ac.uk/government/people/academic-staff/torun-dewan',
   'Dr Carolin Dieterle': 'https://www.lse.ac.uk/government/people/academic-staff/carolin-dieterle',
   'Dr Vesselin Dimitrov': 'https://www.lse.ac.uk/government/people/academic-staff/vesselin-dimitrov',
   'Professor Katrin Flikschuh': 'https://www.lse.ac.uk/government/people/academic-staff/katrin-flikschuh',
   'Dr Florian Foos': 'https://www.lse.ac.uk/government/people/academic-staff/florian-foos',
   'Dr David Foster': 'https://www.lse.ac.uk/government/people/academic-staff/david-foster',
   'Dr Thalia Gerzso': 'https://www.lse.ac.uk/government/people/academic-staff/thalia-gerzso',
   'Dr Steffen Hertog': 'https://www.lse.ac.uk/government/people/academic-staff/steffen-hertog',
   'Professor Sara Hobolt': 'https://www.lse.ac.uk/government/people/academic-staff/sara-hobolt',
   'Dr Jonathan Hopkin': 'https://www.lse.ac.uk/government/people/academic-staff/jonathan-hopkin',
   'Dr Rafael Hortala-Vallve': 'https://www.lse.ac.uk/government/people/academic-staff/rafael-hortala-vallve',
   'Professor James Hughes': 'https://www.lse.ac.uk/government/people/academic-staff/james-hughes',
   'Dr Ryan Jablonski': 'https://www.lse.ac.uk/government/people/academic-staff/ryan-jablonski',
   'Dr Rehan Rafay Jamil': 'https://www.lse.ac.uk/government/people/academic-staff/rehan-rafay-jamil',
   'Dr Nirvikar Jassal': 'https://www.lse.ac.uk/government/people/academic-staff/nirvikar-jassal',
   'Leigh Jenco': 'https://www.lse.ac.uk/government/people/academic-staff/leigh-jenco',
   'Professor Paul Kelly': 'https://www.lse.ac.uk/government/people/academic-staff/paul-kelly',
   'Dr Bill Kissane': 'https://www.lse.ac.uk/government/people/academic-staff/bill-kissane',
   'Dr Mathias Koenig-Archibugi': 'https://www.lse.ac.uk/government/people/academic-staff/mathias-koenig-archibugi',
   'Professor Valentino Larcinese': 'https://www.lse.ac.uk/government/people/academic-staff/valentino-larcinese',
   'Dr Michael Lerner': 'https://www.lse.ac.uk/government/people/academic-staff/michael-lerner',
   'Professor Martin Lodge': 'https://www.lse.ac.uk/government/people/academic-staff/martin-lodge',
   'Dr Jacklyn Majnemer': 'https://www.lse.ac.uk/government/people/academic-staff/jacklyn-majnemer',
   'Dr Omar McDoom': 'https://www.lse.ac.uk/government/people/academic-staff/omar-mcdoom',
   'Dr Paul Mitchell': 'https://www.lse.ac.uk/government/people/academic-staff/paul-mitchell',
   'Dr Carl Müller-Crepon': 'https://www.lse.ac.uk/government/people/academic-staff/carl-muller-crepon',
   'Dr George Ofosu': 'https://www.lse.ac.uk/government/people/academic-staff/george-ofosu',
   'Professor Edward Page (FBA)': 'https://www.lse.ac.uk/government/people/academic-staff/edward-page',
   'Professor Francisco Panizza': 'https://www.lse.ac.uk/government/people/academic-staff/francisco-panizza',
   'Dr Mathias Poertner': 'https://www.lse.ac.uk/government/people/academic-staff/mathias-poertner',
   'Dr Stephanie Rickard': 'https://www.lse.ac.uk/government/people/academic-staff/stephanie-rickard',
   'Dr Melissa Sands': 'https://www.lse.ac.uk/government/people/academic-staff/melissa-sands',
   'Professor Cheryl Schonhardt-Bailey (FBA)': 'https://www.lse.ac.uk/government/people/academic-staff/cheryl-schonhardt-bailey',
   'John Sidel': 'https://www.lse.ac.uk/government/people/academic-staff/john-sidel',
   'Dr Kai Spiekermann': 'https://www.lse.ac.uk/government/people/academic-staff/kai-spiekermann',
   'Pavithra Suryanarayan': 'https://www.lse.ac.uk/government/people/academic-staff/pavithra-suryanarayan',
   'Dr Felipe Torres Raposo': 'https://www.lse.ac.uk/government/people/academic-staff/felipe-torres-raposo',
   'Dr Aliz Tóth': 'https://www.lse.ac.uk/government/people/academic-staff/aliz-toth',
   'Professor Tony Travers': 'https://www.lse.ac.uk/government/people/academic-staff/tony-travers',
   'Dr Deirdre Troy': 'https://www.lse.ac.uk/government/people/academic-staff/deirdre-troy',
   'Tamara Tubakovic': 'https://www.lse.ac.uk/government/people/academic-staff/tamara-tubakovic',
   'Dr Joachim Wehner': 'https://www.lse.ac.uk/government/people/academic-staff/joachim-wehner',
   'Dr Stephane Wolton': 'https://www.lse.ac.uk/government/people/academic-staff/stephane-wolton',
   'Dr David Woodruff': 'https://www.lse.ac.uk/government/people/academic-staff/david-woodruff',
   'Professor Lea Ypi': 'https://www.lse.ac.uk/government/people/academic-staff/lea-ypi'},
  'Professional Services Staff': {'Imran Iqbal': None,
   'Natalie Paris': None,
   'Charlie Tickle': None,
   'Katerina Collins': None,
   'BellaRose Lennon': None,
   'Laure Van Rensburg': None,
   'Emily Crook': None,
   'Serena James': None,
   'Karen Dickenson': None,
   "Kay O'Neill": None,
   'Clair Harrison': None,
   'James Robins': None,
   'Kevin Shields': None,
   'Shelly Wilkley': None},
  'Research Staff': {'Zachary Dickson': 'https://www.lse.ac.uk/government/people/research-staff/zachary-dickson',
   'Dr Sarah Harrison': 'https://www.lse.ac.uk/government/people/research-staff/sarah-harrison'},
  'Guest Teachers and GTAs': {'Sebastian Balfour': None,
   'Sarah Goff': None,
   'Geoffrey Myers': None,
   'Jill Stuart': None,
   'Maximillian Afnan': None,
   'Ville Aula': None,
   'Luis Bosshart': None,
   'Morris Chan': None,
   'Cristina Conesa Pla': None,
   'Julia-Anne Costet': None,
   'Tommaso Crescioli': None,
   'Andrea Delestrade': None,
   'Tessa Devereaux-Evans': None,
   'Caroline Elak': None,
   'Melanie Erspamer': None,
   'Robert Falconer': None,
   'Ugo Gaudino': None,
   'Kristof Horvath': None,
   'Conor Kelly': None,
   'Yusuf Khan': None,
   'Paul Kindermann': None,
   'Matthias Kroenke': None,
   'Evgeniya Kudinova': None,
   'Tiffany Lau': None,
   'Nick Lewis': None,
   'Xufan Ma': None,
   'Alexander McKenna': None,
   'Stefano Merlo': None,
   'Lennard Metson': None,
   'Emma Obermair': None,
   'Asfa Shakeel': None,
   'Alexander Soldatkin': None,
   'Sarah Tustin': None,
   'Stephanie Wanga': None,
   'Jan Wasserziehr': None,
   'Felix Westeren': None,
   'Peter Wyckoff': None,
   'Alex Yeandle': None,
   'Antoine Zerbini': None},
  'Research Students': {'Luqman Abu El Foul': 'https://www.lse.ac.uk//government/people/research-students/luqman-abu-el-foul',
   'Meshal Alkhowaiter': 'https://www.lse.ac.uk//government/people/research-students/meshal-abdulaziz-alkhowaiter',
   'Laurence Antao': None,
   'Tom Bailey': None,
   'Luis Bosshart': 'https://www.lse.ac.uk//government/people/research-students/luis-bosshart',
   'Morris Chan': None,
   'Vanessa Cheng-Matsuno': 'https://www.lse.ac.uk//government/people/research-students/vanessa-cheng-matsuno',
   'Joel Chong': 'https://www.lse.ac.uk//government/people/research-students/joel-chong',
   'Asli Ceren Cinar': 'https://www.lse.ac.uk//government/people/research-students/asli-ceren-cinar',
   'Cristina Conesa Pla': None,
   'Julia Costet': 'https://www.lse.ac.uk//government/people/research-students/julia-costet',
   'Reuben Finighan': None,
   'Maximilian Fink': None,
   'Hilke Gudel': None,
   'Vincent Harting': 'https://www.lse.ac.uk//government/people/research-students/vincent-harting',
   'Martin Haus': 'https://www.lse.ac.uk//government/people/research-students/martin-haus',
   'Isolde Hegemann': None,
   'Angga Indraswara': 'https://www.lse.ac.uk//government/people/research-students/angga-indraswara',
   'Gaetano Inglese': None,
   'Sarah Jewett': None,
   'Yusuf Imaad Khan': 'https://www.lse.ac.uk//government/people/research-students/yusuf-imaad-khan',
   'Rune Wriedt Larsen': 'https://www.lse.ac.uk//government/people/research-students/rune-wriedt-larsen',
   'Tiffany Lau': 'https://www.lse.ac.uk//government/people/research-students/tiffany-lau',
   'Nick Lewis': 'https://www.lse.ac.uk//government/people/research-students/nick-lewis',
   'Xufan': 'https://www.lse.ac.uk//government/people/research-students/xufan-ma',
   'Andrew McNeil': 'https://www.lse.ac.uk//government/people/research-students/andrew-mcneil',
   'Lennard Metson': None,
   'Frieder Mitsch': None,
   'Miran Norderland': None,
   'Oscar Nowlan': None,
   'Emma Obermair': 'https://www.lse.ac.uk//government/people/research-students/emma-obermair',
   'Alberto Parmigiani': 'https://www.lse.ac.uk//government/people/research-students/alberto-parmigiani',
   'Michal Pasovsky': 'https://www.lse.ac.uk//government/people/research-students/michal-pasovsky',
   'Ariel Perkins': None,
   'Diego Sazo': 'https://www.lse.ac.uk//government/people/research-students/diego-sazo',
   'Asfa Shakeel': 'https://www.lse.ac.uk//government/people/research-students/asfa-shakeel',
   'Andreas-Johann Sorger': 'https://www.lse.ac.uk//government/people/research-students/andreas-johann-sorger',
   'Andreas-Johann': None,
   'Carol Thanki': None,
   'Theint Theint Thu': 'https://www.lse.ac.uk//government/people/research-students/theint-theint-thu',
   'Tarsha Vasu': 'https://www.lse.ac.uk//government/people/research-students/tarsha-vasu',
   'Lorenzo Vicari': 'https://www.lse.ac.uk//government/people/research-students/lorenzo-vicari',
   'Stephanie Wanga': 'https://www.lse.ac.uk//government/people/research-students/stephanie-wanga',
   'Jan Henrik Wasserziehr': 'https://www.lse.ac.uk//government/people/research-students/jan-henrik-wasserziehr',
   'Felix Westerén': 'https://www.lse.ac.uk//government/people/research-students/felix-westeren',
   'Peter Wyckoff': 'https://www.lse.ac.uk//government/people/research-students/peter-wyckoff',
   'Alex Yeandle': 'https://www.lse.ac.uk//government/people/research-students/alex-yeandle',
   'Khairulanwar Zaini': None,
   'Antoine Zerbini': 'https://www.lse.ac.uk//government/people/research-students/antoine-zerbini'},
  'Emeritus, Affiliated & Visiting Academic Staff': {'Sebastian Balfour': 'https://www.lse.ac.uk//government/people/emeritus-academic-staff/sebastian-balfour',
   'John Barnes': None,
   'Rodney Barker': 'https://www.lse.ac.uk//government/people/emeritus-academic-staff/rodney-barker',
   'John Breuilly': 'https://www.lse.ac.uk//government/people/emeritus-academic-staff/john-breuilly',
   'John Charvet': 'https://www.lse.ac.uk//government/people/emeritus-academic-staff/john-charvet',
   'Janet Coleman': 'https://www.lse.ac.uk//government/people/emeritus-academic-staff/janet-coleman',
   'Patrick Dunleavy': 'https://www.lse.ac.uk//government/people/emeritus-academic-staff/patrick-dunleavy',
   'John Gray': None,
   'Chun Lin': 'https://www.lse.ac.uk//government/people/emeritus-academic-staff/chun-lin',
   'John Madeley': None,
   'Anne Phillips': 'https://www.lse.ac.uk//government/people/emeritus-academic-staff/anne-phillips',
   'David Soskice': 'https://www.lse.ac.uk//government/people/emeritus-academic-staff/david-soskice',
   'Ernest Thorp': None,
   'Michael Barzelay': 'http://www.lse.ac.uk/management/people/academic-staff/mbarzelay',
   'Charlie Beckett': 'http://www.lse.ac.uk/researchAndExpertise/Experts/profile.aspx?KeyValue=c.h.beckett@lse.ac.uk',
   'Kenneth Benoit': 'http://www.lse.ac.uk/Methodology/People/Academic-Staff/Kenneth-Benoit/Kenneth-Benoit',
   'Jean-Paul Faguet': 'http://www.lse.ac.uk/international-development/people/jean-paul-faguet',
   'Tasha Fairfeld': 'http://www.lse.ac.uk/international-development/people/tasha-fairfield',
   'Kevin Featherstone': 'http://www.lse.ac.uk/european-institute/people/academic-staff/featherstone-kevin',
   'Benjamin Lauderdale': 'http://www.lse.ac.uk/Methodology/People/Academic-Staff/Benjamin-Lauderdale/Benjamin-Lauderdale',
   'Javier Ortega': 'https://www.kingston.ac.uk/staff/profile/professor-javier-ortega-786/',
   'James Putzel': 'http://www.lse.ac.uk/international-development/people/james-putzel',
   'Ken Shadlen': 'http://www.lse.ac.uk/international-development/people/ken-shadlen',
   'Eiko Thielemann': 'https://www.lse.ac.uk//european-institute/people/Thielemann-Eiko',
   'Jonathan White': 'http://www.lse.ac.uk/european-institute/People/Academic-Staff/White-Jonathan',
   'David Banisar': None,
   'Sevinc Bermek': None,
   'Andrea Capussela': None,
   'Daniel de Kadt': None,
   'Thoraya El-Rayyes': 'https://www.lse.ac.uk//government/people/visiting-academic-staff/thoraya-el-rayyes',
   'Dominik Hangartner': None,
   'Helios Herrera': None,
   'Bruno Leipold': 'https://www.lse.ac.uk//government/people/academic-staff/bruno-leipold',
   'Julia Leschke': 'https://www.lse.ac.uk//government/people/visiting-academic-staff/julia-leschke',
   'Geoffrey Myers': None,
   'Elena Pupaza': 'https://www.lse.ac.uk//government/people/visiting-academic-staff/elena-pupaza',
   'Antoni Rodon-Casarramona': None,
   'Nelson Ruiz': 'https://www.lse.ac.uk//government/people/visiting-academic-staff/nelson-ruiz',
   'Lukas Slothuus': None,
   'Tinghua Yu': None}},
 'Department of Health Policy': {'Senior Management Team': {'Professor Andrew Street': 'https://www.lse.ac.uk/health-policy/people/professor-andrew-street',
   'Dr Mylene Lagarde': 'https://www.lse.ac.uk/health-policy/people/dr-mylene-lagarde',
   'Dr Justin Parkhurst': 'https://www.lse.ac.uk/health-policy/people/dr-justin-parkhurst',
   'Amanda Gaddes': None},
  'Academic Staff': {'Marcello Antonini': 'https://www.lse.ac.uk/health-policy/people/Marcello-Antonini',
   'Miqdad Asaria': 'https://www.lse.ac.uk/health-policy/people/miqdad-asaria',
   'Dr Mrigesh Bhatia': 'https://www.lse.ac.uk/health-policy/people/dr-mrigesh-bhatia',
   'Dr-Alex-Carter': 'https://www.lse.ac.uk/health-policy/people/Dr-Alex-Carter',
   'Emilie Courtin': 'https://www.lse.ac.uk/health-policy/people/dr-emilie-courtin',
   'Professor Joan Costa-i-Font': 'https://www.lse.ac.uk/health-policy/people/professor-joan-costa-font',
   'Dr Rocco Friebel': 'https://www.lse.ac.uk/health-policy/people/dr-rocco-friebel',
   'Genevieve-Jeffrey': 'https://www.lse.ac.uk/health-policy/people/Genevieve-Jeffrey',
   'Dr-Catherine-M-Jones': 'https://www.lse.ac.uk/health-policy/people/Catherine-M-Jones',
   'Dr Panos Kanavos': 'https://www.lse.ac.uk/health-policy/people/dr-panos-kanavos',
   'Ilias Kyriopoulos': 'https://www.lse.ac.uk/health-policy/people/Ilias-Kyriopoulos',
   'Professor Martin Knapp': 'https://www.lse.ac.uk/health-policy/people/professor-martin-knapp',
   'Professor Alistair McGuire': 'https://www.lse.ac.uk/health-policy/people/professor-alistair-mcguire',
   'Dr Mylene Lagarde': 'https://www.lse.ac.uk/health-policy/people/dr-mylene-lagarde',
   'Dr-Ritesh-Maharaj': 'https://www.lse.ac.uk/health-policy/people/Dr-Ritesh-Maharaj',
   'Professor Elias Mossialos': 'https://www.lse.ac.uk/health-policy/people/professor-elias-mossialos',
   'Huseyin Naci PhD MHS': 'https://www.lse.ac.uk/health-policy/people/huseyin-naci-phd-mhs',
   'Dr Irene Papanicolas': 'https://www.lse.ac.uk/health-policy/people/dr-irene-papanicolas',
   'Dr Justin Parkhurst': 'https://www.lse.ac.uk/health-policy/people/dr-justin-parkhurst',
   'Nilesh Raut': 'https://www.lse.ac.uk/health-policy/people/Nilesh-Raut',
   'Dr Divya Srivastava': 'https://www.lse.ac.uk/health-policy/people/dr-divya-srivastava',
   'Professor Andrew Street': 'https://www.lse.ac.uk/health-policy/people/professor-andrew-street',
   'Dr Ranjeeta Thomas': 'https://www.lse.ac.uk/health-policy/people/dr-ranjeeta-thomas',
   'Dr Clare Wenham': 'https://www.lse.ac.uk/health-policy/people/dr-clare-wenham',
   'George A Wharton': 'https://www.lse.ac.uk/health-policy/people/george-a-wharton',
   'Dr Olivier Wouters': 'https://www.lse.ac.uk/health-policy/people/dr-olivier-wouters'},
  'Department Managers': {'Amanda Gaddes': None,
   'Tash Dalling': None,
   'Rana James': None},
  'Postgraduate Taught Programmes': {'Muheez Busari': None,
   'Dr Dina Davaki': None,
   'Christopher Roberts': None,
   'Zaina Salama': None},
  'Executive Programmes': {'Jacqueline McAndrew': None,
   'Hayley Briars': None,
   'Alex Rose': None},
  'Postgraduate Research Programmes': {'Muheez Busari': None,
   'Christopher Roberts': None,
   'Zaina Salama': None},
  'Research Management and Administration': {'Jo Brown': 'https://www.lse.ac.uk/cpec/people/jo-brown',
   'Janice Dickson': None,
   'Alexia Honore': None,
   'Angela Mehta': None,
   'Sergio Rotundo': None,
   'Susana Sidders': None,
   'Rebekah Smith': None,
   'Sofia Vyzantiadou': 'https://www.lse.ac.uk/lse-health/people/Vyzantiadou-Sofia'},
  'Communications and Marketing': {'Xiaoran Xu': None},
  'Guest Lecturers': {'Dr Jeroen Bax': 'https://www.universiteitleiden.nl/en/staffmembers/jeroen-bax#tab-1',
   'Dr Paul Carter': None,
   'Dr Martin Cowie': 'https://www.imperial.ac.uk/people/m.cowie',
   'Dr Eduardo Gomez': 'https://www.kcl.ac.uk/sspp/departments/did/people/academic-staff/eduardo-gomez/index.aspx',
   'Dr Allan Hackshaw': 'https://iris.ucl.ac.uk/iris/browse/profile?upi=AHACK11',
   'Dr Michael Holland': 'https://www.slam.nhs.uk/about-us/trust-board/dr-michael-holland',
   'Professor Ashish Jha': 'https://www.hsph.harvard.edu/profile/ashish-jha/',
   'Dr Brittany Jones': 'https://www.linkedin.com/in/brittany-jones-723235b5/?originalSubdomain=uk',
   'Dr Joseph Kim': None,
   'Professor Paulus Kirkhof': 'https://www.birmingham.ac.uk/staff/profiles/cardiovascular-sciences/Kirchhof-Paulus.aspx',
   'Professor Mandeep Mehra': 'https://www.lse.ac.uk/lse-health/people/Visiting-Fellows/mandeep-mehra',
   'Dr Mark Newman': 'http://iris.ucl.ac.uk/iris/browse/profile?upi=MJNEW38',
   'Dr Jennifer Palmer': 'https://www.lshtm.ac.uk/aboutus/people/palmer.jennifer',
   'Dr Maria Raikou': 'https://www.unipi.gr/unipi/en/mraikou.html',
   'Professor Sir Mike Richards': None,
   'Professor Lise Rochaix': None,
   'Dr Victoria Serra-Sastre': None,
   'Dr Matthew Skellern': None,
   'Dr Harpreet Sood': None,
   'Christos Stylianides': None},
  'LSE Health': {'Michael_Anderson': 'https://www.lse.ac.uk/lse-health/people/Michael-Anderson',
   'Carlos A. Bana e Costa': 'https://www.lse.ac.uk/lse-health/people/Carlos-A.-Bana-e-Costa',
   'Maeve Bognini': 'https://www.lse.ac.uk/lse-health/people/maeve-bognini',
   'Danitza Chávez-Montoya': 'https://www.lse.ac.uk/lse-health/people/Danitza-Ch%C3%A1vez-Montoya',
   'Avi Cherla': None,
   'Dorothy Chisare': 'https://www.lse.ac.uk/lse-health/people/dorothy-chisare',
   'Dr Lesong Conteh': 'https://www.lse.ac.uk/health-policy/people/Dr-Lesong-Conteh',
   'Jonathan Cylus': 'https://www.lse.ac.uk/lse-health/people/jonathan-cylus',
   'Arianna Gentilini': 'https://www.lse.ac.uk/lse-health/people/Arianna-Gentilini',
   'Jennifer Gill\xa0': 'https://www.lse.ac.uk/business-and-consultancy/consulting/experts/jennifer-gill',
   'Madeleine Haig': 'https://www.lse.ac.uk/lse-health/people/Madeleine-Haig',
   'Cristina Hernandez Quevedo': 'https://www.lse.ac.uk/lse-health/people/cristina-hernandez-quevedo',
   'Martilord Ifeanyichi': 'https://www.lse.ac.uk/lse-health/people/martilord-ifeanyichi',
   'Sahan Jayawardana': 'https://www.lse.ac.uk/lse-health/people/sahan-jayawardana',
   'Charlotte Johnston-Webber': 'https://www.lse.ac.uk/lse-health/people/charlotte-johnston-webber',
   'Dr Lucy Kanya': 'http://www.lse.ac.uk/health-policy/people/dr-lucy-kanya',
   'Meskerem Kebede': 'https://www.lse.ac.uk/lse-health/people/meskerem-kebede',
   'Beth Kreling': 'http://www.lse.ac.uk/health-policy/people/beth-kreling',
   'Sherry Merkur': None,
   'Caitlin Main': 'https://www.lse.ac.uk/lse-health/people/Caitlin-Main',
   'Mackenzie Mills': 'http://www.lse.ac.uk/health-policy/people/PhD/mackenzie-mills',
   'Aurelio Miracolo': 'https://www.lse.ac.uk/lse-health/people/Aurelio-Miracolo',
   'Jean Mossman': 'https://www.lse.ac.uk/lse-health/people/Jean-Mossman',
   'Zlatko Nikoloski': 'https://www.lse.ac.uk/lse-health/people/zlatko-nikoloski',
   'Nilesh Raut': 'https://www.lse.ac.uk/health-policy/people/Nilesh-Raut',
   'Konstantina Politopoulou': 'https://www.lse.ac.uk/lse-health/people/Konstantina-Politopoulou',
   'Elisa Sicuri': 'https://www.lse.ac.uk/lse-health/people/Elisa-Sicuri',
   'Bryony Simmons': 'https://www.lse.ac.uk/lse-health/people/Bryony-Simmons',
   'Jannis Stockel': 'https://www.lse.ac.uk/lse-health/people/jannis-stockel',
   'Gemma Williams': None},
  'Care Policy and Evaluation Centre (CPEC)': {'Annette Bauer': 'https://www.lse.ac.uk/cpec/people/annette-bauer',
   'Eva-Maria Bonin': 'https://www.lse.ac.uk/cpec/people/eva-maria-bonin',
   'Camille Bou': 'https://www.lse.ac.uk/cpec/people/camille-bou',
   'Nicola Brimblecombe': 'https://www.lse.ac.uk/cpec/people/nicola-brimblecombe',
   'Javiera Cartagena-Farias': 'https://www.lse.ac.uk/cpec/people/javiera-cartagena-farias',
   'Michael Clark': 'https://www.lse.ac.uk/cpec/people/michael-clark',
   'Adelina Comas Herrera': 'https://www.lse.ac.uk/cpec/people/adelina-comas-herrera',
   "Francesco D'Amico": 'https://www.lse.ac.uk/cpec/people/francesco-damico',
   'Jacqueline Damant': 'https://www.lse.ac.uk/cpec/people/jacqueline-damant',
   'Margaret Dangoor': 'https://www.lse.ac.uk/cpec/people/margaret-dangoor',
   'Bleddyn Davies': 'https://www.lse.ac.uk/cpec/people/bleddyn-davies',
   'Josie Dixon': 'https://www.lse.ac.uk/cpec/people/josie-dixon',
   'Sara Evans-Lacko': 'https://www.lse.ac.uk/cpec/people/sara-evans-lacko',
   'José Luis Fernández': 'https://www.lse.ac.uk/cpec/people/jose-luis-fernandez',
   'Paul Freddolino': 'https://www.lse.ac.uk/cpec/people/paul-freddolino',
   'Emily Freeman': 'https://www.lse.ac.uk/cpec/people/emily-freeman',
   'Philipp Friemann': None,
   'Dr Alan Glanz': None,
   'Daniel Gulliford': 'https://www.lse.ac.uk/cpec/people/daniel-gulliford',
   'Professor Ruth Hancock': None,
   'Catherine Henderson': 'https://www.lse.ac.uk/cpec/people/catherine-henderson',
   'Dr Ties Hoomans': 'https://www.lse.ac.uk/cpec/people/ties-hoomans',
   'Bo Hu': 'https://www.lse.ac.uk/cpec/people/bo-hu',
   'Sarah Jasim': None,
   'Maria Karagiannidou': 'https://www.lse.ac.uk/cpec/people/maria-karagiannidou',
   'Derek King': 'https://www.lse.ac.uk/cpec/people/derek-king',
   'Martin Knapp': 'https://www.lse.ac.uk/cpec/people/martin-knapp',
   'Klara Lorenz': 'https://www.lse.ac.uk/cpec/people/klara-lorenz',
   'Dr Juliette Malley': 'https://www.lse.ac.uk/cpec/people/juliette-malley',
   'Joanna Marczak': 'https://www.lse.ac.uk/cpec/people/joanna-marczak',
   'Tihana Matosevic': 'https://www.lse.ac.uk/cpec/people/tihana-matosevic',
   'David McDaid': 'https://www.lse.ac.uk/cpec/people/david-mcdaid',
   'A-La Park': 'https://www.lse.ac.uk/cpec/people/a-la-park',
   'Dr Sanna Read': 'https://www.lse.ac.uk/cpec/people/archive/sanna-read',
   'Amritpal Rehill': 'https://www.lse.ac.uk/cpec/people/amripal-rehill',
   'Sam Rickman': 'https://www.lse.ac.uk/cpec/people/sam-rickman',
   'Maximilian Salcher-Konrad': 'https://www.lse.ac.uk/cpec/people/maximilian-salcher-konrad',
   'Wagner Silva-Ribeiro': 'https://www.lse.ac.uk/cpec/people/wagner-silva-ribeiro',
   'Madeleine Stevens': 'https://www.lse.ac.uk/cpec/people/madeleine-stevens',
   'Dr Michela Tinelli': 'https://www.lse.ac.uk/cpec/people/michela-tinelli',
   'Gerald Wistow': 'https://www.lse.ac.uk/cpec/people/gerald-wistow',
   'Raphael Wittenberg': 'https://www.lse.ac.uk/cpec/people/raphael-wittenberg',
   'Valentina Zigante': 'https://www.lse.ac.uk/cpec/people/valentina-zigante'},
  'Vacancies for Academics': {'LSE Fellow in Health Policy': 'https://jobs.lse.ac.uk/Vacancies/W/7328/0/425591/15539/lse-fellow-in-health-policy'}},
 'Department of International Development': {'Academic staff ': {'Allen_Tim': 'https://www.lse.ac.uk/international-development/people/tim-allen',
   'Lydia Assouad': 'https://www.lse.ac.uk/international-development/people/Lydia-Assouad',
   'Professor Catherine Boone': 'http://www.lse.ac.uk/government/people/academic-staff/catherine-boone',
   'Ernestina Coast': 'https://www.lse.ac.uk/international-development/people/ernestina-coast',
   'Professor Jean-Paul Faguet': 'https://www.lse.ac.uk/international-development/people/jean-paul-faguet',
   'Dr Tasha Fairfield': 'https://www.lse.ac.uk/international-development/people/tasha-fairfield',
   'Professor Tim Forsyth': 'https://www.lse.ac.uk/international-development/people/tim-forsyth',
   'Arjan Gjonça': 'https://www.lse.ac.uk/international-development/people/arjan-gjonça',
   'Stuart Gordon': 'https://www.lse.ac.uk/international-development/people/stuart-gordon',
   'Dr Duncan Green': 'https://www.lse.ac.uk/international-development/people/duncan-green',
   'Elliott Green': 'https://www.lse.ac.uk/international-development/people/elliott-green',
   'Dr Lloyd Gruber': 'https://www.lse.ac.uk/international-development/people/lloyd-gruber',
   'tine hanrieder': 'https://www.lse.ac.uk/international-development/people/tine-hanrieder',
   'Professor Kathryn Hochstetler': 'https://www.lse.ac.uk/international-development/people/kathy-hochstetler',
   'Myfanwy James': 'https://www.lse.ac.uk/international-development/people/Myfanwy-James',
   'Naila Kabeer': 'https://www.lse.ac.uk/international-development/people/naila-kabeer',
   'Sohini Kar': 'https://www.lse.ac.uk/international-development/people/sohini-kar',
   'Professor David Keen': 'https://www.lse.ac.uk/international-development/people/david-keen',
   'Tiziana Leone': 'https://www.lse.ac.uk/international-development/people/Professor-David-Lewis',
   'Dr Shirin Madon': 'https://www.lse.ac.uk/international-development/people/shirin-madon',
   'Dr Laura Mann': 'https://www.lse.ac.uk/international-development/people/laura-mann',
   'Professor Kate Meagher': 'https://www.lse.ac.uk/international-development/people/kate-meagher',
   'Philipa Mladovsky': 'https://www.lse.ac.uk/international-development/people/philipa-mladovsky',
   'Joana Naritomi': 'https://www.lse.ac.uk/international-development/people/joana-naritomi',
   'Professor James Putzel': 'https://www.lse.ac.uk/international-development/people/james-putzel',
   'Dr Sandra Sequeira': 'https://www.lse.ac.uk/international-development/people/sandra-sequeira',
   'Professor Ken Shadlen\xa0': 'https://www.lse.ac.uk/international-development/people/ken-shadlen',
   'Mahvish Shami': 'https://www.lse.ac.uk/international-development/people/mahvish-shami',
   'Rajesh Venugopal': 'https://www.lse.ac.uk/international-development/people/rajesh-venugopal',
   'Robert Wade': 'https://www.lse.ac.uk/international-development/people/robert-wade',
   'Diana Weinhold': 'https://www.lse.ac.uk/international-development/people/diana-weinhold'},
  'Fellows and Guest Teachers': {'Carolin Dieterle': 'https://www.lse.ac.uk/international-development/people/fellows/Carolin-Dieterle',
   'Tom Kirk': 'https://www.lse.ac.uk/africa/centre-for-public-authority-and-international-development/Researcher-spotlight/Spotlight-Tom-Kirk',
   'Dr Stephanie Levy': 'https://www.lse.ac.uk/international-development/people/fellows/stephanie-levy',
   'Jon Lunn': 'https://www.lse.ac.uk/international-development/people/fellows/Dr-Jon-Lunn',
   'Dr Jerome Roos': 'https://www.lse.ac.uk/international-development/people/fellows/dr-jerome-roos',
   'Greta Seibel': 'https://www.lse.ac.uk/international-development/people/fellows/Greta-Seibel',
   'Bookang Seol': 'https://www.lse.ac.uk/international-development/people/fellows/Bookang-Seol',
   'Joe Strong': 'https://www.lse.ac.uk/international-development/people/fellows/Joe-Strong',
   'Jennifer Melvin': 'https://www.lse.ac.uk/international-development/people/fellows/Jennifer-Melvin'},
  'Visiting Fellows': {'Professor Alaka Basu': 'https://sociology.cornell.edu/alaka-basu',
   'Professor Radhika Desai': 'https://geopoliticaleconomy.org/people/radhika-desai/',
   'Dr Joseph Hanlon': 'https://www.open.ac.uk/people/jh46',
   'Dr Rishita Nandagiri Nandagiri': 'https://www.kcl.ac.uk/people/rishita-nandagiri'},
  'Professors in Practice': {'Duncan Green': 'https://www.lse.ac.uk/international-development/people/duncan-green',
   'Kate Gilmore': 'https://www.ohchr.org/EN/AboutUs/Pages/KateGilmore.aspx',
   'Mark Lowcock': None,
   'Professor\xa0David Luke': 'https://www.lse.ac.uk/africa/people/Staff/David-Luke'},
  'Affiliated researchers': {'Madhuri Agarwal': 'https://www.lse.ac.uk/international-development/people/fellows/Madhuri-Agarwal',
   'Professor E A Brett': 'https://www.lse.ac.uk/international-development/people/teddy-brett',
   'Allison Benson Hernandez': None,
   'Chao-Yo Cheng': 'https://ccheng11.github.io/',
   'rim turkmani': 'https://www.lse.ac.uk/international-development/people/tim-dyson',
   'Francisco Ferreira': 'https://www.lse.ac.uk/International-Inequalities/People/Francisco-H.-G.-Ferreira',
   'Annie Kuse': None,
   'Elizabeth Ngutuku': 'http://www.linkedin.com/in/elizabeth-eliza-n-a9b39336',
   'Reece Sisto': None,
   'James Walters': None},
  'Professional service staff': {'Matthew Redfearn': None,
   'Drucilla Nelson': None,
   'Andrea Merino-Mayayo': None,
   'Charlotte McKay': None,
   'Maria Do-Prado': None,
   'Ayman Hassan': None,
   'Dipa Patel': None,
   'Maya Bullen': None,
   'Dr Monika Kruesmann': None},
  'Research Students': {'Camilo Acero Vargas': 'https://www.lse.ac.uk/international-development/people/research-students/Camilo-Acero-Vargas',
   'Rahma Ahmed': 'https://www.lse.ac.uk/international-development/people/research-students/Rahma-Ahmed',
   'Faris Al-Sulayman': None,
   'Marietta Angeli': 'https://www.lse.ac.uk/international-development/people/research-students/Marietta-Angeli',
   'Lucas D. Arce\xa0': None,
   'Ritika Arora': None,
   'Arbie Baguios': None,
   'Parth Bhatia': None,
   'Charlotte Brown': None,
   'Javiera Caceres': None,
   'Chiara-Chiavaroli': 'https://www.lse.ac.uk/international-development/people/research-students/Chiara-Chiavaroli',
   'Tin Hinane El Kadi': 'https://www.lse.ac.uk/international-development/people/research-students/Tin-Hinane-El-Kadi',
   'Isaac Haruna': 'https://www.lse.ac.uk/international-development/people/research-students/Isaac-Haruna',
   'Claudia Horn': 'https://sites.google.com/view/claudiahorn',
   'Hosna Jahan': 'https://www.lse.ac.uk/international-development/people/research-students/Hosna-Jahan',
   'Shinzani Jain': 'https://www.lse.ac.uk/international-development/people/research-students/Shinzani-Jain',
   'Pauline Jerrentrup': 'https://www.lse.ac.uk/international-development/people/research-students/Pauline-Jerrentrup',
   'Mina Kozluca': 'https://www.lse.ac.uk/international-development/people/research-students/Mina-Kozluca',
   'Sophie Legros': 'https://www.lse.ac.uk/international-development/people/research-students/sophie-legros',
   'Jorich Johann Loubser': 'https://www.lse.ac.uk/international-development/people/research-students/Jorich-Loubser',
   'Richard Mallett': 'https://www.lse.ac.uk/international-development/people/research-students/richard-mallett',
   'Bincy Mathew': 'https://www.lse.ac.uk/international-development/people/research-students/Bincy-Mathew',
   'Aoife McCullough': 'https://www.lse.ac.uk/international-development/people/research-students/Aoife-McCullough',
   'eduardo-mercadante': 'https://www.lse.ac.uk/international-development/people/research-students/eduardo-mercadante',
   'Aisha Modibo': None,
   'Michael Mugisha': 'https://www.lse.ac.uk/international-development/people/research-students/michael-mugisha',
   'tony neil': 'https://www.lse.ac.uk/international-development/people/research-students/tony-neil',
   'Evan Oliner': None,
   'Alice Robinson': 'https://www.lse.ac.uk/international-development/people/research-students/alice-robinson',
   'claudia-rodriguez-castellanos': 'https://www.lse.ac.uk/international-development/people/research-students/claudia-rodriguez-castellanos',
   'Gijs van Selm': 'https://www.lse.ac.uk/international-development/people/research-students/Gijs-van-Selm',
   'Ciara Silke': None,
   'harshita-sinha': 'https://www.lse.ac.uk/international-development/people/research-students/harshita-sinha',
   'Marika Theros': None,
   'Costanza Torre': None}},
 'Department of International History': {'Permanent academic staff': {'Dr Roham Alvandi': 'https://www.lse.ac.uk/International-History/People/AcademicStaff/alvandi/alvandi',
   'Professor Nigel J. Ashton': 'https://www.lse.ac.uk/International-History/People/AcademicStaff/ashton/ashton',
   'Professor Marc David Baer': 'https://www.lse.ac.uk/International-History/People/AcademicStaff/baer/baer',
   'Dr Antony Best': 'https://www.lse.ac.uk/International-History/People/AcademicStaff/best/best',
   'Dr Anna Cant': 'https://www.lse.ac.uk/International-History/People/AcademicStaff/cant/cant',
   'Professor Steven Casey': 'https://www.lse.ac.uk/International-History/People/AcademicStaff/casey/casey',
   'Dr Dina Gusejnova': 'https://www.lse.ac.uk/International-History/People/AcademicStaff/gusejnova/gusejnova',
   'Dr Tanya Harmer': 'https://www.lse.ac.uk/International-History/People/AcademicStaff/harmer/harmer',
   'Dr Tim Hochstrasser': 'https://www.lse.ac.uk/International-History/People/AcademicStaff/hochstrasser/hochstrasser',
   'Dr Elizabeth Ingleson': 'https://www.lse.ac.uk/International-History/People/AcademicStaff/ingleson/ingleson',
   'Professor Matthew Jones': 'https://www.lse.ac.uk/International-History/People/AcademicStaff/jones/jones',
   'Dr Paul Keenan': 'https://www.lse.ac.uk/International-History/People/AcademicStaff/keenan/keenan',
   'Dr Joanna Lewis': 'https://www.lse.ac.uk/International-History/People/AcademicStaff/lewis/lewis',
   'Professor N. Piers Ludlow': 'https://www.lse.ac.uk/International-History/People/AcademicStaff/ludlow/ludlow',
   'Dr. Alex Mayhew': 'https://www.lse.ac.uk/International-History/People/AcademicStaff/mayhew/mayhew',
   'Dr David Motadel': 'https://www.lse.ac.uk/International-History/People/AcademicStaff/motadel/motadel',
   'Dr Artemis Photiadou': 'https://www.lse.ac.uk/International-History/People/AcademicStaff/photiadou/photiadou',
   'Dr Ronald C. Po': 'https://www.lse.ac.uk/International-History/People/AcademicStaff/po/po',
   'Professor Paul Preston': 'https://www.lse.ac.uk/International-History/People/Emeritus-Distinguished-Staff/preston/preston',
   'Dr Svetozar Rajak': 'https://www.lse.ac.uk/International-History/People/AcademicStaff/rajak/rajak',
   'Dr Jake Subryan Richards': 'https://www.lse.ac.uk/International-History/People/AcademicStaff/richards/richards',
   'Dr Kirsten E. Schulze': 'https://www.lse.ac.uk/International-History/People/AcademicStaff/schulze/schulze',
   'Dr Gagandeep S. Sood': 'https://www.lse.ac.uk/International-History/People/AcademicStaff/sood/sood',
   'Dr Aino Rosa Kristina Spohr': 'https://www.lse.ac.uk/International-History/People/AcademicStaff/spohr/spohr',
   'Dr Paul Stock': 'https://www.lse.ac.uk/International-History/People/AcademicStaff/stock/stock',
   'Dr Imaobong Umoren': 'https://www.lse.ac.uk/International-History/People/AcademicStaff/umoren/umoren',
   'Dr Qingfei Yin': 'https://www.lse.ac.uk/International-History/People/AcademicStaff/yin/yin',
   'Professor Vladislav Zubok': 'https://www.lse.ac.uk/International-History/People/AcademicStaff/zubok/zubok'},
  'Professional services team ': {'Ms Demetra Frini': None,
   'Mrs Jacquie Minter': None,
   'Mrs Milada Fomina': None,
   'Mrs Nayna Bhatti': None,
   'Mr Mathew Betts': None,
   'Dr Edlira Gjonca': None,
   'Mr Maximilian': None},
  'Graduate teaching assistants and guest teachers ': {'Maliha Ahmed': None,
   'Dr Sara Al-Qaiwani': None,
   'Armand Azra bin Azlira': 'https://www.lse.ac.uk/International-History/People/GTA-GT/azlira/azlira',
   'Medha Bhattacharya': 'https://www.lse.ac.uk/International-History/People/PhD/bhattacharya',
   'chua': 'https://www.lse.ac.uk/International-History/People/GTA-GT/chua/chua',
   'Dr Rosalind Coffey': 'https://www.lse.ac.uk/International-History/People/GTA-GT/coffey',
   'Pedro Correa Martin-Arroyo': 'https://www.lse.ac.uk/International-History/People/GTA-GT/correa-martin-arroyo/pedro-correa-martin-arroyo',
   'Alexandre Dab': 'https://www.lse.ac.uk/International-History/People/GTA-GT/dab/dab',
   'Charlotte Eaton': 'https://www.lse.ac.uk/International-History/People/PhD/eaton',
   'Jonathan Franco': 'https://www.lse.ac.uk/International-History/People/GTA-GT/franco/franco',
   'Dr Sajjan Gohel': 'https://www.lse.ac.uk/International-History/People/GTA-GT/gohel/gohel',
   'Dr Giovanni Graglia': 'https://www.lse.ac.uk/International-History/People/GTA-GT/graglia/graglia',
   'Dr Anton Harder': 'https://www.lse.ac.uk/International-History/People/GTA-GT/harder/harder',
   'Jeff Hawn': 'https://www.lse.ac.uk/International-History/People/PhD/hawn',
   'Sandip Kana': 'https://www.lse.ac.uk/International-History/People/GTA-GT/kana/kana',
   'Ronan Mainprize': 'https://www.lse.ac.uk/International-History/People/GTA-GT/mainprize/mainprize',
   'Amir Mehdi': None,
   'Dr William Mitchell': 'https://www.lse.ac.uk/International-History/People/GTA-GT/mitchell/mitchell',
   "Fionntan O'Hara": None,
   'Michele Pajero': None,
   'Daniele Giuseppe Palmer': 'https://www.lse.ac.uk/International-History/People/GTA-GT/palmer/Daniele-Giuseppe-Palmer',
   'Dr Agis Papageorgiou': 'https://www.lse.ac.uk/International-History/People/GTA-GT/papageorgiou/papageorgiou',
   'Dr Ben Sayle': 'https://www.lse.ac.uk/International-History/People/GTA-GT/sayle/sayle',
   'Gray Sergeant': 'https://www.lse.ac.uk/International-History/People/GTA-GT/sergeant/sergeant'}},
 'International Inequalities Institute': {'Faculty Associates': {'Professor Tim Allen': 'https://www.lse.ac.uk/international-development/people/tim-allen',
   'Professor Oriana Bandiera': 'https://www.lse.ac.uk/economics/people/faculty/oriana-bandiera',
   'Professor Tim Besley': 'https://www.lse.ac.uk/economics/people/faculty/tim-besley',
   'Professor Catherine Boone': 'https://www.lse.ac.uk/international-development/people/catherine-boone',
   'Dr Pawel Bukowski': 'https://cep.lse.ac.uk/_new/people/person.asp?id=10291',
   'Dr Tania Burchardt': 'https://www.lse.ac.uk/social-policy/people/academic-staff/Dr-Tania-Burchardt',
   'Dr Flora Cornish': 'https://www.lse.ac.uk/Methodology/People/Academic-Staff/Flora-Cornish/Flora-Cornish',
   'Professor Joan Costa-Font': 'https://www.lse.ac.uk/health-policy/people/professor-joan-costa-font',
   'Professor Frank Cowell': 'https://www.lse.ac.uk/economics/people/faculty/frank-cowell',
   'Professor Neil Cummins': 'https://www.lse.ac.uk/Economic-History/People/Faculty-and-teachers/Professor-Neil-Cummins',
   'Professor Jean-Paul Faguet': 'https://www.lse.ac.uk/international-development/people/jean-paul-faguet',
   'Sam Friedman': 'https://www.lse.ac.uk/sociology/people/sam-friedman',
   'Professor\xa0Ellen Helsper\xa0': 'http://www.lse.ac.uk/media-and-communications/people/academic-staff/ellen-helsper',
   'Professor\xa0Jonathan Hopkin': 'https://www.lse.ac.uk/government/people/academic-staff/jonathan-hopkin',
   'Professor Simona Iammarino': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/simona-iammarino',
   'Professor Deborah James': 'https://www.lse.ac.uk/anthropology/people/deborah-james',
   'Dr Xavier Jaravel': 'https://www.lse.ac.uk/economics/people/faculty/xavier-jaravel',
   'Professor Stephen Jenkins': 'https://www.lse.ac.uk/social-policy/people/academic-staff/Professor-Stephen-Jenkins',
   'Professor Gareth Jones': 'https://www.lse.ac.uk/lacc/people/gareth-jones',
   'Professor Naila Kabeer': 'https://www.lse.ac.uk/international-development/people/naila-kabeer',
   'Dr Eleni Karagiannaki': 'https://sticerd.lse.ac.uk/CASE/_new/people/person.asp?id=1671',
   'Dr Insa Koch': 'https://www.lse.ac.uk/law/people/academic-staff/insa-koch',
   'Professor Nicola Lacey': 'https://www.lse.ac.uk/law/people/academic-staff/nicola-lacey',
   'Professor Camille Landais': 'https://www.lse.ac.uk/economics/people/faculty/camille-landais',
   'Professor Neil Lee': 'https://www.lse.ac.uk/geography-and-environment/people/academic-staff/neil-lee',
   'Dr\xa0Marion Lieutaud': 'https://www.lse.ac.uk/Methodology/People/Academic-Staff/Marion-Lieutaud/Marion-Lieutaud',
   'Professor Sumi Madhok': 'https://www.lse.ac.uk/gender/people-profiles/faculty/sumi-madhok',
   'Dr Abigail McKnight': 'https://sticerd.lse.ac.uk/_new/staff/person.asp?id=833',
   'Dr Joana Naritomi': 'https://www.lse.ac.uk/international-development/people/joana-naritomi',
   'Professor\xa0Berkay Ozcan': 'https://www.lse.ac.uk/social-policy/people/academic-staff/Professor-Berkay-Ozcan',
   'Dr Robtel Neajai Pailey': 'https://www.lse.ac.uk/social-policy/people/academic-staff/dr-robtel-neajai-pailey',
   'Professor Lucinda Platt': 'https://www.lse.ac.uk/social-policy/people/academic-staff/Professor-Lucinda-Platt',
   'Dr Ania Plomien': 'https://www.lse.ac.uk/gender/people-profiles/faculty/ania-plomien',
   'Dr Eleanor Power': 'https://www.lse.ac.uk/Methodology/People/Academic-Staff/Eleanor-Power/Eleanor-Power',
   'Dr Aliya Rao': 'https://www.lse.ac.uk/Methodology/People/Academic-Staff/Aliya-Rao/Aliya-Rao',
   'Dr Daniel Reck': 'https://www.lse.ac.uk/economics/people/faculty/daniel-reck',
   'Professor Mike Savage': 'http://www.lse.ac.uk/sociology/people/mike-savage',
   'Professor\xa0Alpa Shah': 'https://www.lse.ac.uk/anthropology/people/alpa-shah',
   'Dr Jennifer Sheehy-Skeffington': 'https://www.lse.ac.uk/PBS/People/Dr-Jennifer-Sheehy-Skeffington',
   'Dr\xa0Isabel Shutes': 'https://www.lse.ac.uk/social-policy/people/academic-staff/Dr-Isabel-Shutes',
   'Professor David Soskice': 'http://www.lse.ac.uk/government/people/academic-staff/david-soskice',
   'Professor\xa0Johannes Spinnewijn': 'https://www.lse.ac.uk/economics/people/faculty/johannes-spinnewijn',
   'Dr Andy Summers': 'https://www.lse.ac.uk/law/people/academic-staff/andrew-summers',
   'Dr Kate Summers': 'https://www.lse.ac.uk/Methodology/People/Academic-Staff/Kate-Summers/Kate-Summers',
   'Dr Kristin Surak': 'https://www.lse.ac.uk/sociology/people/kristin-surak',
   'Dr\xa0Chana Teeger': 'https://www.lse.ac.uk/Methodology/People/Academic-Staff/Chana-Teeger/Chana-Teeger',
   'Dr Milena Tsvetkova': 'https://www.lse.ac.uk/Methodology/People/Academic-Staff/Milena-Tsvetkova/Milena-Tsvetkova',
   'Dr Imaobong Umoren': 'https://www.lse.ac.uk/International-History/People/academicStaff/umoren/umoren'},
  'Research Staff': {'Facundo Alvaredo': 'https://www.lse.ac.uk/International-Inequalities/People/Facundo-Alvaredo/Facundo-Alvaredo',
   'Dr Paolo Brunori': 'https://www.lse.ac.uk/International-Inequalities/People/Paolo-Brunori',
   'Hazel V. Carby': 'https://www.lse.ac.uk/International-Inequalities/People/Hazel-V.-Carby/Hazel-V.-Carby',
   'Valentina Contreras': 'https://www.lse.ac.uk/International-Inequalities/People/Valentina-Contreras/Valentina-Contreras',
   'Caroline Falkman Olsson': 'https://www.lse.ac.uk/International-Inequalities/People/Caroline-Falkman-Olsson/Caroline-Falkman-Olsson',
   'Dr\xa0Shalini Grover': 'https://www.lse.ac.uk/International-Inequalities/People/Shalini-Grover/Shalini-Grover',
   'Max Herbertson': 'https://www.lse.ac.uk/International-Inequalities/People/Max-Herbertson',
   'Johnathan Inkley': 'https://www.lse.ac.uk/International-Inequalities/People/Johnathan-Inkley',
   'Beatriz Jambrina Canseco': 'https://www.lse.ac.uk/International-Inequalities/People/Beatriz-Jambrina-Canseco/Beatriz-Jambrina-Canseco',
   'H Xavier Jara': 'https://www.lse.ac.uk/International-Inequalities/People/Xavier-Jara/H-Xavier-Jara',
   'Sarah Kerr': 'https://www.lse.ac.uk/International-Inequalities/People/Sarah-Kerr',
   'Aygen Kurt-Dickson': 'https://www.lse.ac.uk/International-Inequalities/People/Aygen-Kurt-Dickson',
   'Maël Lavenaire': 'https://www.lse.ac.uk/International-Inequalities/People/Maël-Lavenaire/Maël-Lavenaire',
   'Andrew Lonsdale ': 'https://www.lse.ac.uk/International-Inequalities/People/Andrew-Lonsdale',
   'Mina Mahmoudzadeh': 'https://www.lse.ac.uk/International-Inequalities/People/Mina-Mahmoudzadeh',
   'Fabrício Mendes Fialho': 'https://www.lse.ac.uk/International-Inequalities/People/Fabrício-Mendes-Fialho',
   'Frieder\xa0Mitsch': 'http://www.lse.ac.uk/International-Inequalities/People/frieder-mitsch/Home.aspx',
   'Annalena Oppel': 'https://www.lse.ac.uk/International-Inequalities/People/Annalena-Oppel/Annalena-Oppel',
   'Amaia Palencia-Esteban': 'https://www.lse.ac.uk/International-Inequalities/People/Amaia-Palencia-Esteban',
   'Martina Pardy': 'https://www.lse.ac.uk/International-Inequalities/People/Martina-Pardy',
   'César Poux': 'https://www.lse.ac.uk/International-Inequalities/People/César-Poux/César-Poux',
   'Pedro Salas-Rojo': 'https://www.lse.ac.uk/International-Inequalities/People/Pedro-Salas-Rojo/Pedro-Salas-Rojo',
   'Violetta van Veen': 'https://www.lse.ac.uk/International-Inequalities/People/Violetta-van-Veen/Violetta-van-Veen',
   'Dr Michael Vaughan': 'https://www.lse.ac.uk/International-Inequalities/People/Michael-Vaughan/Michael-Vaughan',
   'Nora Waitkus': 'https://www.lse.ac.uk/International-Inequalities/People/Nora-Waitkus/Nora-Waitkus'},
  'Support Staff': {'Emma Ceccato': None,
   'Peter Kane': None,
   'Jacqueline Mujico': 'https://www.lse.ac.uk/International-Inequalities/People/Jacqui-Mujico/Jacqui-Mujico',
   'Sallyann Oates': None,
   'Dan Nixon': None,
   'Miranda Saul': None,
   'Kate Steward': 'https://www.lse.ac.uk/International-Inequalities/People/Kate-Steward'},
  'Visiting Professors': {'Sanghamitra Bandyopadhyay': 'https://www.lse.ac.uk/International-Inequalities/People/Sanghamitra-Bandyopadhyay/Sanghamitra-Bandyopadhyay',
   'Madeleine Bunting': 'https://www.lse.ac.uk/International-Inequalities/People/Madeleine-Bunting/Madeleine-Bunting',
   'Emma Chamberlain': 'https://www.lse.ac.uk/International-Inequalities/People/Emma-Chamberlain/Emma-Chamberlain',
   'Dave Coady': 'https://www.lse.ac.uk/International-Inequalities/People/Dave-Coady/David-Coady',
   'Ashwini Deshpande': 'https://www.lse.ac.uk/International-Inequalities/People/Ashwini-Deshpande/Ashwini-Deshpande',
   'James Foster': 'https://www.lse.ac.uk/International-Inequalities/People/James-Foster/James-Foster',
   'Carola Frege': 'https://www.lse.ac.uk/International-Inequalities/People/Carola-Frege/Carola-Frege',
   'Linda Hantrais': 'https://www.lse.ac.uk/International-Inequalities/People/Linda-Hantrais/Linda-Hantrais',
   'Lee Elliot Major': 'https://www.lse.ac.uk/International-Inequalities/People/Lee-Elliot-Major',
   'Professor Branko Milanovic': 'https://www.lse.ac.uk/International-Inequalities/People/Branko-Milanovic',
   'Jenny Pearce': 'https://www.lse.ac.uk/International-Inequalities/People/Jenny-Pearce',
   'Professor Aaron Reeves': 'https://www.lse.ac.uk/International-Inequalities/People/Aaron-Reeves',
   'Dr Liz Sayce': 'https://www.lse.ac.uk/International-Inequalities/People/Liz-Sayce',
   'Dr Faiza Shaheen': 'https://www.lse.ac.uk/International-Inequalities/People/Faiza-Shaheen',
   'Professor Susanne Wessendorf': 'https://www.lse.ac.uk/International-Inequalities/People/Susanne-Wessendorf'},
  'Visiting Fellows': {'Dr\xa0Arun Advani': 'http://www.lse.ac.uk/International-Inequalities/People/Arun-Advani',
   'Mauricio Apablaza': 'https://www.lse.ac.uk/International-Inequalities/People/Mauricio-Apablaza/Mauricio-Apablaza',
   'Dr Yonatan Berman': 'https://www.lse.ac.uk/International-Inequalities/People/Yonatan-Berman',
   'Ingrid Bleynat': 'https://www.lse.ac.uk/International-Inequalities/People/Ingrid-Bleynat',
   'David Burgherr': 'https://www.lse.ac.uk/International-Inequalities/People/David-Burgherr',
   'Rafael Carranza': 'https://www.lse.ac.uk/International-Inequalities/People/Rafael-Carranza',
   'Lucas Chancel': 'https://www.lse.ac.uk/International-Inequalities/People/Lucas-Chancel/Lucas-Chancel',
   'Matthew Collin': 'https://www.lse.ac.uk/International-Inequalities/People/Matthew-Collin/Matthew-Collin',
   'Hai-Anh H. Dang': 'https://www.lse.ac.uk/International-Inequalities/People/Hai-Anh-H.-Dang/Hai-Anh-H.-Dang',
   'Nimesh Dhungana': 'https://www.lse.ac.uk/International-Inequalities/People/Nimesh-Dhungana/Nimesh-Dhungana',
   'Niccolo Durazzi': 'https://www.lse.ac.uk/International-Inequalities/People/Niccolo-Durazzi/Niccolo-Durazzi',
   'Dr Mark Fransham': 'https://www.lse.ac.uk/International-Inequalities/People/Mark-Fransham',
   'Dr Luna Glucksberg': 'https://www.lse.ac.uk/International-Inequalities/People/Luna-Glucksberg',
   'Dr Charlotte Haberstroh': 'https://www.lse.ac.uk/government/people/academic-staff/charlotte-haberstroh',
   'Gaby Harris': 'https://www.lse.ac.uk/International-Inequalities/People/Gaby-Harris/Gaby-Harris',
   'Dr\xa0Katharina Hecht': 'http://www.lse.ac.uk/International-Inequalities/People/Katharina-Hecht/Katharina-Hecht',
   'Jason Hickel': 'https://www.lse.ac.uk/International-Inequalities/People/Jason-Hickel',
   'Dr David Hope': 'https://www.lse.ac.uk/International-Inequalities/People/David-Hope',
   'Helen Hughson': 'https://www.lse.ac.uk/International-Inequalities/People/Helen-Hughson',
   'Roberto Iacono': 'https://www.lse.ac.uk/International-Inequalities/People/Roberto-Iacono/Roberto-Iacono',
   'Ruth Kattumuri': 'https://www.lse.ac.uk/International-Inequalities/People/Ruth-Kattumuri',
   'Dr Tom Kemeny': 'http://www.lse.ac.uk/International-Inequalities/People/Tom-Kemeny',
   'Kristina Kolbe': 'https://www.lse.ac.uk/International-Inequalities/People/Kristina-Kolbe',
   'Niyathi R. Krishna': 'https://www.lse.ac.uk/International-Inequalities/People/Niyathi-R.-Krishna',
   'Dr Erica Lagalisse': 'http://www.lse.ac.uk/International-Inequalities/People/Erica-Lagalisse',
   'Paul Lagneau-Ymonet': 'https://www.lse.ac.uk/International-Inequalities/People/Paul-Lagneau-Ymonet/Paul-Lagneau-Ymonet',
   'Marion-Lieutaud': 'https://www.lse.ac.uk/International-Inequalities/People/Marion-Lieutaud',
   'Davide Luca': 'https://www.lse.ac.uk/International-Inequalities/People/Davide-Luca',
   'Dr Jonathan Mijs': 'http://www.lse.ac.uk/International-Inequalities/People/Jonathan-Mijs',
   'Lara Minkus': 'https://www.lse.ac.uk/International-Inequalities/People/Lara-Minkus/Lara-Minkus',
   'Salvatore Morelli': 'https://www.lse.ac.uk/International-Inequalities/People/Salvatore-Morelli',
   'Philippa Mullins': 'https://www.lse.ac.uk/International-Inequalities/People/Philippa-Mullins/Philippa-Mullins',
   'Dr Guido Neidhöfer': 'https://www.lse.ac.uk/International-Inequalities/People/Guido-Neidhöfer',
   'Dr Clive Nwonka': 'https://www.ucl.ac.uk/institute-of-advanced-studies/dr-clive-nwonka',
   'Dr\xa0Tahnee Ooms': 'https://www.lse.ac.uk/International-Inequalities/People/Tahnee-Ooms',
   'Poornima Paidipaty': 'https://www.lse.ac.uk/International-Inequalities/People/Poornima-Paidipaty',
   'Alessandra Radicati': 'https://www.lse.ac.uk/International-Inequalities/People/Alessandra-Radicati',
   'Giuliano Resce': 'https://www.lse.ac.uk/International-Inequalities/People/Giuliano-Resce/Giuliano-Resce',
   'Rebecca Simson': 'https://www.lse.ac.uk/International-Inequalities/People/Rebecca-Simson',
   'Anna Powell-Smith': 'https://www.lse.ac.uk/International-Inequalities/People/Anna-Powell-Smith',
   'Joaquin Prieto': 'https://www.lse.ac.uk/International-Inequalities/People/Joaquin-Prieto',
   'Marco Ranaldi': 'https://www.lse.ac.uk/International-Inequalities/People/Marco-Ranaldi',
   'Soham Sahoo': 'https://www.lse.ac.uk/International-Inequalities/People/Soham-Sahoo/Soham-Sahoo',
   'Giovanna Scarchilli': 'https://www.lse.ac.uk/International-Inequalities/People/Giovanna-Scarchilli/Giovanna-Scarchilli',
   'Elisabeth Schimpfössl': 'https://www.lse.ac.uk/International-Inequalities/People/Elisabeth-Schimpf%C3%B6ssl',
   'Dr Paul Segal': 'http://www.lse.ac.uk/International-Inequalities/People/Paul-Segal',
   'Dr Laura Sochas': 'https://www.lse.ac.uk/International-Inequalities/People/Laura-Sochas',
   'Dhananjayan Sriskandarajah': 'https://www.lse.ac.uk/International-Inequalities/People/Dhananjayan-Sriskandarajah/Dhananjayan-Sriskandarajah',
   'Dr\xa0Joel Suss': 'https://www.lse.ac.uk/International-Inequalities/People/Joel-Suss',
   'Dr Gaston Yalonetzky': 'https://www.lse.ac.uk/International-Inequalities/People/Gaston-Yalonetzky'},
  'Atlantic Fellows for Social and Economic Equity': {'Asmaa Akhtar': 'http://www.lse.ac.uk/International-Inequalities/People/Asmaa-Akhtar',
   'Dr Sara Camacho Felix': 'http://www.lse.ac.uk/International-Inequalities/People/Sara-Camacho-Felix',
   'Grace Farrell-Twiney': None,
   'Professor\xa0Armine Ishkanian': 'https://www.lse.ac.uk/social-policy/people/academic-staff/Professor-Armine-Ishkanian',
   'George Kunnath': 'https://www.lse.ac.uk/International-Inequalities/People/George-Kunnath/George-Kunnath',
   'Saaga Leppänen': None,
   'Anastasia Nazaryan': 'https://www.lse.ac.uk/International-Inequalities/People/Anastasia-Nazaryan/Anastasia-Nazaryan',
   'Adele Oliver': None,
   'Julia Ryng': None},
  'Head': {'Professor Francisco H. G. Ferreira': 'https://www.lse.ac.uk/International-Inequalities/People/Francisco-H.-G.-Ferreira',
   'Professor\xa0Kirsten Sehnbruch': 'https://www.lse.ac.uk/International-Inequalities/People/Kirsten-Sehnbruch',
   'Professor\xa0Armine Ishkanian': 'https://www.lse.ac.uk/social-policy/people/academic-staff/Professor-Armine-Ishkanian',
   'Liza Ryan': 'https://www.lse.ac.uk/International-Inequalities/People/Liza-Ryan'}},
 'Department of International Relations': {'Academic staff': {'Professor J. Chris Alden': 'https://www.lse.ac.uk/international-relations/people/alden',
   'Martin Bayly': 'https://www.lse.ac.uk/international-relations/people/bayly',
   'Federica Bicchi': 'https://www.lse.ac.uk/international-relations/people/bicchi',
   'Ida Roland Birkvad': 'https://www.lse.ac.uk/international-relations/people/birkvad',
   'Tarsis Daylan Brito Sepulveda-Coelho': 'https://www.lse.ac.uk/international-relations/people/brito-tarsis',
   'Lucinda Cadzow': 'https://www.lse.ac.uk/international-relations/people/cadzow',
   'William Callahan': 'https://www.lse.ac.uk/international-relations/people/callahan',
   'Jeffrey Chwieroth': 'https://www.lse.ac.uk/international-relations/people/chwieroth',
   'Thomas Da Costa Vieira': 'https://www.lse.ac.uk/international-relations/people/da-costa-vieira',
   'Katerina Dalacoura': 'https://www.lse.ac.uk/international-relations/people/dalacoura',
   'Shikha Dilawri': 'https://www.lse.ac.uk/international-relations/people/dilawri',
   'Toby Dodge': 'https://www.lse.ac.uk/international-relations/people/dodge',
   'Dr Spyros Economides': 'https://www.lse.ac.uk/european-institute/people/economides-spyros',
   'Alice Engelhard': 'https://www.lse.ac.uk/international-relations/phd-students/engelhard-alice',
   'Robert Falkner': 'https://www.lse.ac.uk/international-relations/people/falkner',
   'Fawaz Gerges': 'https://www.lse.ac.uk/international-relations/people/gerges',
   'getmansky': 'https://www.lse.ac.uk/international-relations/people/getmansky',
   'Patrick Gill-Tiney': 'https://www.lse.ac.uk/international-relations/people/gill-tiney',
   'Sinja Graf': 'https://www.lse.ac.uk/international-relations/people/graf',
   'Jurgen Haacke': 'https://www.lse.ac.uk/international-relations/people/haacke',
   'Jonny Hall': 'https://www.lse.ac.uk/international-relations/phd-job-market/hall-jonny',
   'Mirko Heinzel': 'https://www.lse.ac.uk/international-relations/people/heinzel',
   'Joe Dacre Higton': 'https://www.lse.ac.uk/international-relations/people/higton',
   'Mr Mark Hoffman': None,
   'Kira Huju': 'https://www.lse.ac.uk/international-relations/people/huju',
   'Dr Jennifer Jackson-Preece': 'https://www.lse.ac.uk/european-institute/people/jackson-preece-jennifer',
   'Nikhil Kalyanpur': 'https://www.lse.ac.uk/international-relations/people/kalyanpur',
   'Koenig-Archibugi': 'https://www.lse.ac.uk/international-relations/people/koenig-archibugi',
   'Milli Lake': 'https://www.lse.ac.uk/international-relations/people/lake',
   'Tomila Lankina': 'https://www.lse.ac.uk/international-relations/people/lankina',
   'Boram Lee': 'https://www.lse.ac.uk/international-relations/people/lee',
   'Jens Meierhenrich': 'https://www.lse.ac.uk/international-relations/people/meierhenrich',
   'Katharine M Millar': 'https://www.lse.ac.uk/international-relations/people/millar',
   'Irene Morlino': 'https://www.lse.ac.uk/international-relations/people/morlino-irene',
   'Dr James Morrison': 'http://www.jamesashleymorrison.com/',
   'Dr Rohan Mukherjee': 'https://www.lse.ac.uk/international-relations/people/mukherjee',
   'Natalya Naqvi': 'https://www.lse.ac.uk/international-relations/people/naqvi',
   'Victoria Paniagua': 'https://www.lse.ac.uk/international-relations/people/paniagua',
   'Fabian Pape': 'https://www.lse.ac.uk/international-relations/people/pape',
   'McKenzie Ratner': 'https://www.lse.ac.uk/international-relations/phd-job-market/ratner-mckenzie',
   'Howie Rechavia-Taylor': 'https://www.lse.ac.uk/international-relations/people/rechavia-taylor',
   'Stephanie Schwartz': 'https://www.lse.ac.uk/international-relations/people/schwartz',
   'Giulia Sciorati': 'https://www.lse.ac.uk/international-relations/people/sciorati',
   'Ulrich Sedelmeier': 'https://www.lse.ac.uk/international-relations/people/sedelmeier',
   'Professor John Sidel': 'https://www.lse.ac.uk/government/people/academic-staff/john-sidel',
   'Karen Smith': 'https://www.lse.ac.uk/international-relations/people/smith',
   'Marta Soprana': 'https://www.lse.ac.uk/international-relations/people/soprana',
   'Theresa Squatrito': 'https://www.lse.ac.uk/international-relations/people/squatrito',
   'Dimitrios Stroikos': 'https://www.lse.ac.uk/international-relations/people/stroikos',
   'Lauren Sukin': 'https://www.lse.ac.uk/international-relations/people/sukin',
   'Luca Tardelli': 'https://www.lse.ac.uk/international-relations/people/tardelli',
   'Peter Trubowitz': 'https://www.lse.ac.uk/international-relations/people/trubowitz',
   'James Walters': 'https://www.lse.ac.uk/international-relations/people/walters',
   'Peter Wilson': 'https://www.lse.ac.uk/international-relations/people/wilson',
   'Alexandros Zachariades': 'https://www.lse.ac.uk/international-relations/phd-job-market/zachariades-alexandros',
   'Mariia Zolkina': 'https://www.lse.ac.uk/international-relations/people/dinam/zolkina-mariia',
   'Noah Zucker': 'https://www.lse.ac.uk/international-relations/people/zucker'},
  'Professional services staff': {'Amy Brook': 'https://www.lse.ac.uk/international-relations/people/pss/brook',
   'Mark Bryceland': 'https://www.lse.ac.uk/international-relations/people/pss/bryceland',
   'Hermione Burnett': 'https://www.lse.ac.uk/international-relations/people/pss/burnett',
   'Alison Carter': 'https://www.lse.ac.uk/international-relations/people/pss/carter',
   'Matthew Gaisford St Lawrence': 'https://www.lse.ac.uk/international-relations/people/pss/gaisford-st-lawrence',
   'Joyce Harvey': 'https://www.lse.ac.uk/international-relations/people/pss/harvey',
   'Sarah Hélias': 'https://www.lse.ac.uk/international-relations/people/pss/helias',
   'Katy Hill': 'https://www.lse.ac.uk/international-relations/people/pss/hill',
   'Adnan Khan': 'https://www.lse.ac.uk/international-relations/people/pss/khan',
   'Romy Mokogwu': 'https://www.lse.ac.uk/international-relations/people/pss/mokogwu',
   'Nicholas Ortiz': 'https://www.lse.ac.uk/international-relations/people/pss/ortiz',
   'Claire Ridge': 'https://www.lse.ac.uk/international-relations/people/pss/ridge',
   'Amanda Schultz': 'https://www.lse.ac.uk/international-relations/people/pss/schultz',
   'Andrew Sherwood': 'https://www.lse.ac.uk/international-relations/people/pss/sherwood',
   'Rosie Springett': 'https://www.lse.ac.uk/international-relations/people/pss/springett'},
  'Academic Visitors': {'Giovanni Angioni': 'https://www.lse.ac.uk/international-relations/people/visiting-fellows/angioni-giovanni',
   'Dr Mia Certo': None,
   'Hassan Damluji': 'https://www.lse.ac.uk/international-relations/people/visiting-fellows/damluji-hassan',
   'Eyüp Ersoy': 'https://www.lse.ac.uk/international-relations/people/visiting-fellows/ersoy-eyup',
   'Gubad Ibadoghlu': 'https://www.lse.ac.uk/international-relations/people/visiting-fellows/ibadoghlu-gubad',
   'Paul Kirby': 'https://www.lse.ac.uk/international-relations/people/visiting-fellows/kirby-paul',
   'Anahita Motazed Rad': 'https://www.lse.ac.uk/international-relations/people/visiting-fellows/motazed-rad-anahita',
   'Michael Reynolds': 'https://www.lse.ac.uk/international-relations/people/visiting-fellows/reynolds-michael',
   'Geoffrey Swenson': 'https://www.lse.ac.uk/international-relations/people/visiting-fellows/swenson-geoffrey',
   'Jianyong Yue': 'https://www.lse.ac.uk/international-relations/people/visiting-fellows/yue-jianyong'},
  'Emeritus and retired staff': {'Chris Brown': 'https://www.lse.ac.uk/international-relations/people/brown',
   'Barry Buzan': 'https://www.lse.ac.uk/international-relations/people/buzan',
   'Professor Michael Cox': 'http://www.lse.ac.uk/ideas/people/cox-mick',
   'Chris Hughes': 'https://www.lse.ac.uk/international-relations/people/hughes',
   'Dr John Kent': None,
   'Margot Light': 'https://www.lse.ac.uk/international-relations/people/light',
   'Professor James Mayall': 'https://www.polis.cam.ac.uk/Staff_and_Students/professor-james-mayall',
   'Mr Nicholas Sims': None,
   'Paul Taylor': 'https://www.lse.ac.uk/international-relations/people/taylor-p',
   'Professor William Wallace': None,
   'Stephen Woolcock': 'https://www.lse.ac.uk/international-relations/people/woolcock',
   'Michael Yahuda': 'https://www.lse.ac.uk/international-relations/people/yahuda'},
  'Graduate teaching assistants (GTAs)': {'Albert Cullell Cano': 'https://www.lse.ac.uk//international-relations/phd-students/cullell-cano-albert',
   'Caroline Elak': 'https://www.lse.ac.uk//international-relations/phd-students/elak-caroline',
   'Emanuele Errichiello': 'https://www.lse.ac.uk//international-relations/phd-students/errichiello-emanuele',
   'Elizabeth Humphrey': 'https://www.lse.ac.uk//international-relations/phd-students/humphrey-elizabeth',
   'Julie Lespinasse': 'https://www.lse.ac.uk//international-relations/phd-students/lespinasse-julie',
   'Shireen Manocha': 'https://www.lse.ac.uk//international-relations/phd-students/manocha-shireen',
   'Chloé Raïd': 'https://www.lse.ac.uk//international-relations/phd-students/raid-chloe',
   'Hadrien T. Saperstein': 'https://www.lse.ac.uk//international-relations/phd-students/saperstein-hadrien',
   'Woohyeok Seo': 'https://www.lse.ac.uk//international-relations/phd-students/seo-woohyeok',
   'Mehdi Shakarchi': 'https://www.lse.ac.uk//international-relations/phd-students/shakarchi-mehdi',
   'Mariah Thornton': 'https://www.lse.ac.uk//international-relations/phd-students/thornton-mariah',
   'Sara Wong': 'https://www.lse.ac.uk//international-relations/phd-students/wong-sara',
   'Agnes Yu': 'https://www.lse.ac.uk//international-relations/phd-students/yu-agnes'},
  'Guest teachers': {'Arthur Kilgore': None,
   'Oksana Levkovych': None,
   'David Rampton': None,
   'James Strong': None}},
 'Language Centre': {'Director': {'Dr Neil McLean': 'https://www.lse.ac.uk//language-centre/people/neil-mclean'},
  'Deputy Director': {'Dr Alison Standring': 'https://www.lse.ac.uk//language-centre/people/alison-standring'},
  'Centre Manager': {'John Heyworth': 'https://www.lse.ac.uk//language-centre/people/john-heyworth'},
  'Course Administrators': {'Suzie Beckley': 'https://www.lse.ac.uk//language-centre/people/suzie-beckley',
   'Sabrina Rabi': 'https://www.lse.ac.uk//language-centre/people/sabrina-rabi'},
  'Staff and Student Relations Administrator': {'Maggie Evans': 'https://www.lse.ac.uk//language-centre/people/margaret-evans'},
  'Teaching and Learning Facilitator': {'Inés Alonso-García': 'https://www.lse.ac.uk//language-centre/people/ines-alonso-garcia'},
  'Language Community and Communications Manager': {'Helen Mayer': 'https://www.lse.ac.uk//language-centre/people/helen-mayer'},
  'Arabic': {'Lourdes Hernández-Martín': 'https://www.lse.ac.uk//language-centre/people/lourdes-hernández-martín',
   'Luay Hasan': 'https://www.lse.ac.uk//language-centre/people/luay-hasan',
   'Nadim Ben-Mohamed-Mahjoub': 'https://www.lse.ac.uk//language-centre/people/nadim-ben-mohamed-mahjoub'},
  'East Asian Languages': {'Dr. Catherine Hua Xiang': 'https://www.lse.ac.uk//language-centre/people/catherine-xiang',
   'Dr Lijing Shi': 'https://www.lse.ac.uk//language-centre/people/lijing-shi',
   'Hongyi Xin': 'https://www.lse.ac.uk//language-centre/people/hongyi-xin',
   'Fang Pan': 'https://www.lse.ac.uk//language-centre/people/fang-pan',
   'Rong Guo': 'https://www.lse.ac.uk//language-centre/people/rong-guo',
   'Alice Yao': 'https://www.lse.ac.uk//language-centre/people/alice-yao',
   'Fei Yuan': 'https://www.lse.ac.uk//language-centre/people/fei-yuan',
   'Yoko Aldous': 'https://www.lse.ac.uk//language-centre/people/yoko-aldous',
   'Yukie Yagioka': 'https://www.lse.ac.uk//language-centre/people/yukie-yagioka',
   'Mihi Kang': 'https://www.lse.ac.uk//language-centre/people/mihi-Kang'},
  'English for Academic Purposes': {'Dr Alison Standring': 'https://www.lse.ac.uk//language-centre/people/alison-standring',
   'Mike McGarvey': 'https://www.lse.ac.uk//language-centre/people/mike-mcgarvey',
   'Simon Roberts': 'https://www.lse.ac.uk//language-centre/people/simon-roberts',
   'Jim Pavitt': 'https://www.lse.ac.uk//language-centre/people/jim-pavitt',
   'Simon Mathews': 'https://www.lse.ac.uk//language-centre/people/simon-mathews'},
  'French': {'François Simon': 'https://www.lse.ac.uk//language-centre/people/françois-simon',
   'Sandrine Victor': 'https://www.lse.ac.uk//language-centre/people/sandrine-victor',
   'Bérengère Genest': 'https://www.lse.ac.uk//language-centre/people/bérengère-genest',
   'Virginie Rodriguez': 'https://www.lse.ac.uk//language-centre/people/Virginie-Rodriguez',
   'Magali Brenac': 'https://www.lse.ac.uk//language-centre/people/magali-brenac',
   'Cécile Alais': 'https://www.lse.ac.uk//language-centre/people/cecile-alais',
   'Pierre Dagonnot': 'https://www.lse.ac.uk//language-centre/people/pierre-dagonnot',
   'Catherine Gaitte': 'https://www.lse.ac.uk//language-centre/people/catherine-gaitte',
   'Patricia Gaudron': 'https://www.lse.ac.uk//language-centre/people/patricia-gaudron',
   'Patricia Morabo': 'https://www.lse.ac.uk//language-centre/people/patricia-morabo'},
  'German': {'Dr. Peter Skrandies': 'https://www.lse.ac.uk//language-centre/people/peter-skrandies',
   'Florian Fischer': 'https://www.lse.ac.uk//language-centre/people/florian-fischer',
   'Dr Manuela von Papen': 'https://www.lse.ac.uk//language-centre/people/manuela-von-papen',
   'Angela Curtis-Landolt': 'https://www.lse.ac.uk//language-centre/people/angela-curtis-landolt'},
  'Literature and Linguistics': {'Dr. Angus Wrenn': 'https://www.lse.ac.uk//language-centre/people/angus-wrenn',
   'Dr. Olga Sobolev': 'https://www.lse.ac.uk//language-centre/people/olga-sobolev',
   'Dr. Peter Skrandies': 'https://www.lse.ac.uk//language-centre/people/peter-skrandies',
   'Dr Neil McLean': 'https://www.lse.ac.uk//language-centre/people/neil-mclean'},
  'Russian': {'Dr. Olga Sobolev': 'https://www.lse.ac.uk//language-centre/people/olga-sobolev',
   'Natasha Bershadski': 'https://www.lse.ac.uk//language-centre/people/natasha-bershadski',
   'Irina Forbes': 'https://www.lse.ac.uk//language-centre/people/irina-forbes',
   'Elena Dearden': 'https://www.lse.ac.uk//language-centre/people/elena-dearden',
   'Larisa Leifland Berntsson': 'https://www.lse.ac.uk//language-centre/people/larisa-leifland-berntsson'},
  'Spanish & Italian': {'Mercedes Coca': 'https://www.lse.ac.uk//language-centre/people/mercedes-coca',
   'Lourdes Hernández-Martín': 'https://www.lse.ac.uk//language-centre/people/lourdes-hernández-martín',
   'Dr. Roser Martinez-Sanchez': 'https://www.lse.ac.uk//language-centre/people/roser-martinez-sanchez',
   'Margarita Rois-Quiroga': 'https://www.lse.ac.uk//language-centre/people/margarita-rois-quiroga',
   'Pablo Sánchez-García': 'https://www.lse.ac.uk//language-centre/people/Pablo-Sánchez-García',
   'Anna Maria Giuffria': 'https://www.lse.ac.uk//language-centre/people/anna-maria-giuffria',
   "Flavia D'Angelantonio": 'https://www.lse.ac.uk//language-centre/people/flavia-d-angelantonio'}},
 'LSE Law School': {'LSE Law School Leadership Team': {'David Kershaw': 'https://www.lse.ac.uk/law/people/academic-staff/david-kershaw',
   'Professor Veerle Heyvaert': 'https://www.lse.ac.uk/law/people/academic-staff/veerle-heyvaert',
   'Paul Sullivan': None},
  'Academic staff\xa0by surname: A > H': {'Cressida Auckland': 'https://www.lse.ac.uk/law/people/academic-staff/cressida-auckland',
   'Professor Susanne Baer': 'https://www.lse.ac.uk/law/people/visiting-professors/susanne-baer',
   'Eduardo Baistrocchi': 'https://www.lse.ac.uk/law/people/academic-staff/eduardo-baistrocchi',
   'Chaloka Beyani': 'https://www.lse.ac.uk/law/people/academic-staff/chaloka-beyani',
   'Julia Black': 'https://www.lse.ac.uk/law/people/academic-staff/julia-black',
   'Jacco Bomhoff': 'https://www.lse.ac.uk/law/people/academic-staff/jacco-bomhoff',
   'Jo Braithwaite': 'https://www.lse.ac.uk/law/people/academic-staff/jo-braithwaite',
   'Nafay Choudhury': 'https://www.lse.ac.uk/law/people/academic-staff/nafay-choudhury',
   'Hugh Collins': 'https://www.lse.ac.uk/law/people/academic-staff/hugh-collins',
   'Sir Ross Cranston': 'https://www.lse.ac.uk/law/people/academic-staff/ross-cranston',
   'Niamh Dunne': 'https://www.lse.ac.uk/law/people/academic-staff/niamh-dunne',
   'Neil Duxbury': 'https://www.lse.ac.uk/law/people/academic-staff/neil-duxbury',
   'Tatiana Flessas': 'https://www.lse.ac.uk/law/people/academic-staff/tatiana-flessas',
   'Conor Gearty': 'https://www.lse.ac.uk/law/people/academic-staff/conor-gearty',
   'Hannah Gibbs': 'https://www.lse.ac.uk/law/people/academic-staff/hannah-gibbs',
   'Suren Gomtsyan': 'https://www.lse.ac.uk/law/people/academic-staff/suren-gomtsyan',
   'Alperen Gözlügöl': 'https://www.lse.ac.uk/law/people/academic-staff/alperen-gozlugol',
   'Oliver Hailes': 'https://www.lse.ac.uk/law/people/academic-staff/oliver-hailes',
   'Veerle Heyvaert': 'https://www.lse.ac.uk/law/people/academic-staff/veerle-heyvaert',
   'Jeremy Horder': 'https://www.lse.ac.uk/law/people/academic-staff/jeremy-horder',
   'Devika Hovell': 'https://www.lse.ac.uk/law/people/academic-staff/devika-hovell',
   'Elizabeth Howell': 'https://www.lse.ac.uk/law/people/academic-staff/elizabeth-howell',
   'Stephen Humphreys': 'https://www.lse.ac.uk/law/people/academic-staff/stephen-humphreys',
   'Martin Husovec': 'https://www.lse.ac.uk/law/people/academic-staff/martin-husovec'},
  'Academic staff\xa0by surname:\xa0I > O': {'Pablo Ibáñez Colomo': 'https://www.lse.ac.uk/law/people/academic-staff/pablo-ibanez-colomo',
   'Emily Jackson': 'https://www.lse.ac.uk/law/people/academic-staff/emily-jackson',
   'David Kershaw': 'https://www.lse.ac.uk/law/people/academic-staff/david-kershaw',
   'Tarun Khaitan': 'https://www.lse.ac.uk/law/people/academic-staff/tarun-khaitan',
   'Professor Larry Kramer': 'https://www.lse.ac.uk/law/people/academic-staff/larry-kramer',
   'Nicola Lacey': 'https://www.lse.ac.uk/law/people/academic-staff/nicola-lacey',
   'Giulia Leonelli': 'https://www.lse.ac.uk/law/people/academic-staff/giulia-claudia-leonelli',
   'Rachel Leow': 'https://www.lse.ac.uk/law/people/academic-staff/rachel-leow',
   'Tim Liau': 'https://www.lse.ac.uk/law/people/academic-staff/timothy-liau',
   'Martin Loughlin': 'https://www.lse.ac.uk/law/people/academic-staff/martin-loughlin',
   'Orla Lynskey': 'https://www.lse.ac.uk/law/people/academic-staff/orla-lynskey',
   'Paul MacMahon': 'https://www.lse.ac.uk/law/people/academic-staff/paul-macmahon',
   'Susan Marks': 'https://www.lse.ac.uk/law/people/academic-staff/susan-marks',
   'Richard Martin': 'https://www.lse.ac.uk/law/people/academic-staff/richard-martin',
   'Luke McDonagh': 'https://www.lse.ac.uk/law/people/academic-staff/luke-mcdonagh',
   'Eva Micheler': 'https://www.lse.ac.uk/law/people/academic-staff/eva-micheler',
   'Kai Moller': 'https://www.lse.ac.uk/law/people/academic-staff/kai-moller',
   'Niamh Moloney': 'https://www.lse.ac.uk/law/people/academic-staff/niamh-moloney',
   'Jo Eric Khushal Murkens': 'https://www.lse.ac.uk/law/people/academic-staff/jo-murkens',
   'Professor Veerle Heyvaert': 'https://www.lse.ac.uk/law/people/academic-staff/andrew-murray',
   'Sonya Onwu': 'https://www.lse.ac.uk/law/people/academic-staff/sonya-onwu/sonya-onwu',
   'Abenaa Owusu-Bempah': 'https://www.lse.ac.uk/law/people/academic-staff/abenaa-owusu-bempah'},
  'Academic staff by surname: P > Z': {'Philipp Paech': 'https://www.lse.ac.uk/law/people/academic-staff/philipp-paech',
   'Sarah Paterson': 'https://www.lse.ac.uk/law/people/academic-staff/sarah-paterson',
   'Mona Paulsen': 'https://www.lse.ac.uk/law/people/academic-staff/mona-paulsen',
   'Marie Petersmann': 'https://www.lse.ac.uk/law/people/academic-staff/marie-petersmann',
   'Federico Picinali': 'https://www.lse.ac.uk/law/people/academic-staff/federico-picinali',
   'Thomas Poole': 'https://www.lse.ac.uk/law/people/academic-staff/thomas-poole',
   'Peter Ramsay': 'https://www.lse.ac.uk/law/people/academic-staff/peter-ramsay',
   'Nick Sage': 'https://www.lse.ac.uk/law/people/academic-staff/nick-sage',
   'Margot Salomon': 'https://www.lse.ac.uk/law/people/academic-staff/margot-salomon',
   'Astrid Sanders': 'https://www.lse.ac.uk/law/people/academic-staff/astrid-sanders',
   'Edmund Schuster': 'https://www.lse.ac.uk/law/people/academic-staff/edmund-philipp-schuster',
   'Andrew Scott': 'https://www.lse.ac.uk/law/people/academic-staff/andrew-scott',
   'Gerry Simpson': 'https://www.lse.ac.uk/law/people/academic-staff/gerry-simpson',
   'Joseph Spooner': 'https://www.lse.ac.uk/law/people/academic-staff/joseph-spooner',
   'Igor Stramignoni': 'https://www.lse.ac.uk/law/people/academic-staff/igor-stramignoni',
   'Andrew Summers': 'https://www.lse.ac.uk/law/people/academic-staff/andrew-summers',
   'Sivaramjani Thambisetty Ramakrishna': 'https://www.lse.ac.uk/law/people/academic-staff/sivaramjani-thambisetty',
   'Sarah Trotter': 'https://www.lse.ac.uk/law/people/academic-staff/sarah-trotter',
   'Emmanuel Voyiakis': 'https://www.lse.ac.uk/law/people/academic-staff/emmanuel-voyiakis',
   'Alex Waghorn': 'https://www.lse.ac.uk/law/people/academic-staff/alex-waghorn',
   'Charlie Webb': 'https://www.lse.ac.uk/law/people/academic-staff/charlie-webb',
   'Michael Wilkinson': 'https://www.lse.ac.uk/law/people/academic-staff/michael-wilkinson',
   'Roxana Willis': 'https://www.lse.ac.uk/law/people/academic-staff/roxana-willis',
   'Floris de Witte': 'https://www.lse.ac.uk/law/people/academic-staff/floris-de-witte',
   'Dame Sarah Worthington': 'https://www.lse.ac.uk/law/people/academic-staff/sarah-worthington',
   'Ayse Gizem Yasar': 'https://www.lse.ac.uk/law/people/academic-staff/ayse-gizem-yasar',
   'Jan Zglinski': 'https://www.lse.ac.uk/law/people/academic-staff/jan-zglinski'},
  'Academic staff (Emeritus)\xa0': {'Robert Baldwin': 'https://www.lse.ac.uk//law/people/academic-staff/robert-baldwin',
   'Anne Barron': 'https://www.lse.ac.uk//law/people/academic-staff/anne-barron',
   'Joanna Benjamin': 'https://www.lse.ac.uk//law/people/academic-staff/joanna-benjamin',
   'David Bradley': None,
   'Michael Bridge': 'https://www.lse.ac.uk//law/people/academic-staff/michael-bridge',
   'Vanessa Finch': 'https://www.lse.ac.uk//law/people/academic-staff/vanessa-finch',
   'Carol Harlow': 'https://www.lse.ac.uk//law/people/academic-staff/carol-harlow',
   'Trevor Hartley': 'https://www.lse.ac.uk//law/people/academic-staff/trevor-hartley',
   'Joe Jacob': None,
   'Tim Murphy': 'https://www.lse.ac.uk//law/people/academic-staff/tim-murphy',
   'Jill Peay': 'https://www.lse.ac.uk//law/people/academic-staff/jill-peay',
   'Robert Reiner': 'https://www.lse.ac.uk//law/people/academic-staff/robert-reiner',
   'Michael Zander': 'https://www.lse.ac.uk//law/people/academic-staff/michael-zander'},
  'LSE Fellows': {'Grigorios Bacharis': 'https://www.lse.ac.uk/law/people/academic-staff/grigorios-bacharis',
   'Alex C. Evans': 'https://www.lse.ac.uk/law/people/academic-staff/alex-evans',
   'Ciara Hurley': 'https://www.lse.ac.uk/law/people/academic-staff/ciara-hurley',
   'Lora Izvorova': 'https://www.lse.ac.uk/law/people/academic-staff/lora-izvorova',
   'Szymon Osmola': 'https://www.lse.ac.uk/law/people/academic-staff/syzmon-osmola'},
  'Professional services staff': {'Karla Barca Marrero': None,
   'Megan Bennett': None,
   'Ayana Brimacombe-Sakey': None,
   'Ruairi Cahill-Fleury': None,
   'Giu': None,
   'Laura Carseldine': None,
   'Denise De Castro': None,
   'Lewina Coote': None,
   'Jeffrey Franklin': None,
   'Sophia Freckmann': None,
   'Alison Grant': None,
   'Zalika Henry': None,
   'Abby Howard': None,
   'Lee Jackson': None,
   'Alexandra Klegg': None,
   'Megha Krishnakumar': None,
   'Rebecca Newman': None,
   'Niamh Sheehan': None,
   'Saiful Siddeky': None,
   'Paul Sullivan': None,
   'Fiona Thomas': None,
   'Amanda Tinnams': None,
   'Mike Twyman': None,
   'Naomi Warren': None,
   'Lucy Wright': None},
  'Senior Fellows': {'Jonathan Fisher QC': 'https://www.lse.ac.uk//law/people/academic-staff/jonathan-fisher',
   'Mary Stokes': 'https://www.lse.ac.uk//law/people/senior-fellows/mary-stokes',
   'Simon Witney': 'https://www.lse.ac.uk//law/people/academic-staff/simon-witney'},
  'Visiting Professors & Visiting Professors in Practice': {'Karen Banks': 'https://www.lse.ac.uk//law/people/visiting-professors/karen-banks',
   'Carl Baudenbacher': None,
   'Baroness Chakrabarti': 'http://www.parliament.uk/biographies/lords/member/4579',
   'Sarah Bauerle Danzman': 'https://www.sarahbauerledanzman.com/',
   'Dr. Michal Agmon-Gonnen': 'https://www.lse.ac.uk//law/people/visiting-professors/michal-agmon-gonnen',
   'Thomas Grant KC': 'https://www.lse.ac.uk//law/people/visiting-professors/thomas-grant',
   'Christos Hadjiemmanuil': 'https://www.lse.ac.uk//law/people/academic-staff/christos-hadjiemmanuil',
   'Dr Thorsten Kaeseberg': 'https://www.lse.ac.uk//law/people/visiting-professors/thorsten-kaeseberg',
   'Baroness Helena Kennedy': 'https://www.helenakennedy.co.uk/about/main.html',
   'Jan Kleinheisterkamp': 'https://www.lse.ac.uk//law/people/academic-staff/jan-kleinheisterkamp',
   'Insa Koch': 'https://www.lse.ac.uk/anthropology/people/insa-koch/Insa-Koch',
   'David Lammy': 'https://www.davidlammy.co.uk',
   'LTS Research directory': 'https://www.lse.ac.uk//law/research/law-technology-society/research-directory.aspx#MarkLewis',
   'David Lock': 'https://www.landmarkchambers.co.uk/people/david-lock-qc/',
   'Stephanie Maguire': 'https://www.lse.ac.uk//law/people/visiting-professors/stephanie-maguire',
   'Dr David Murphy': 'https://www.lse.ac.uk//law/people/visiting-professors/david-murphy',
   'Elisabeth Noble': 'https://www.lse.ac.uk//law/people/visiting-professors/elisabeth-noble',
   'Conor Quigley': None,
   'Jessica Simor QC': 'https://www.lse.ac.uk//law/people/visiting-professors/jessica-simor'},
  'Visiting Fellows': {'Diego Acosta': 'https://www.bristol.ac.uk/people/person/Diego-Acosta%20Arcarazo-bf9950e8-0f70-4be3-958e-3bca777c38cc/',
   'Dr Stephanie Classmann': 'https://www.lse.ac.uk//law/people/visiting-fellows/stephanie-classmann',
   'Daniel Joyce': 'https://www.unsw.edu.au/staff/daniel-joyce',
   'Michael Lobban': 'https://www.lse.ac.uk//law/people/academic-staff/michael-lobban',
   'Joana Mendes': 'https://www.uni.lu/en/person/NTAwMjI5MDFfX0pvYW5hIE1FTkRFUw==/',
   'Doreen Lustig': 'https://en-law.tau.ac.il/profile/lustigd',
   'Meredith Rossner': 'https://researchers.anu.edu.au/researchers/rossner-m',
   'Ian Roxan': 'https://www.lse.ac.uk//law/people/academic-staff/ian-roxan',
   'Grégoire Webber': 'https://www.lse.ac.uk//law/people/academic-staff/gregoire-webber',
   'Ting Xu': 'https://www.essex.ac.uk/people/x/t/xutin01207'},
  'Visiting Fellows in Practice': {'Nath Gbikpi': 'https://www.leighday.co.uk/about-us/our-people/senior-staff/nath-gbikpi/',
   'Kathryn Davis': 'https://www.lse.ac.uk//law/people/visiting-fellows/kathryn-davis',
   'Adrien de Hauteclocque': 'https://www.lse.ac.uk//law/people/visiting-fellows/adrien-de-hauteclocque',
   'Dr Michael Kanu': None,
   'Sam McAlister': 'https://www.lse.ac.uk//law/people/visiting-fellows/sam-mcalister'}},
 'Department of Management': {'Find a member of academic staff': {'Dr Sarah Ashwin': 'https://www.lse.ac.uk/management/people/academic-staff/sarah-ashwin',
   'Professor Connson Locke': 'https://www.lse.ac.uk/management/people/academic-staff/connson-locke',
   'Professor Yona Rubinstein': 'https://www.lse.ac.uk/management/people/academic-staff/yona-rubinstein',
   'Dr Bethania Antunes': 'https://www.lse.ac.uk/management/people/academic-staff/bethania-antunes',
   'Dr Jonathan E. Booth': 'https://www.lse.ac.uk/management/people/academic-staff/jonathan-booth',
   'Wafaa Elmezraoui': 'https://www.lse.ac.uk/management/people/academic-staff/wafaa-elmezraoui',
   'Karin King': 'https://www.lse.ac.uk/management/people/academic-staff/karin-king',
   'Dr Chunyun Li': 'https://www.lse.ac.uk/management/people/academic-staff/chunyun-li',
   'Dr Francesca Manzi': 'https://www.lse.ac.uk/management/people/academic-staff/francesca-manzi',
   'Dr Elizabeth Stillwell': 'https://www.lse.ac.uk/management/people/academic-staff/elizabeth-stillwell',
   'ksun': 'https://www.lse.ac.uk/management/people/academic-staff/jiaqing-kathy-sun',
   'Dr Jeffrey Thomas': 'https://www.lse.ac.uk/management/people/academic-staff/jeffrey-thomas',
   'Dr Frido Wenten': 'https://www.lse.ac.uk/management/people/academic-staff/frido-wenten',
   'Professor Chrisanthi Avgerou': 'https://www.lse.ac.uk/management/people/academic-staff/chrisanthi-avgerou',
   'Professor Michael Barzelay': 'https://www.lse.ac.uk/management/people/academic-staff/michael-barzelay',
   'Dr Aaron Cheng': 'https://www.lse.ac.uk/management/people/academic-staff/aaron-cheng',
   'Dr Antonio Cordella': 'https://www.lse.ac.uk/management/people/academic-staff/antonio-cordella',
   'jcgiraldomora': 'https://www.lse.ac.uk/management/people/academic-staff/juan-camilo-giraldo-mora',
   'Dr Francesco Gualdi': 'https://www.lse.ac.uk/management/people/academic-staff/francesco-gualdi',
   'Dr Ela Klecun': 'https://www.lse.ac.uk/management/people/academic-staff/ela-klecun',
   'Dr Antti Lyyra': 'https://www.lse.ac.uk/management/people/academic-staff/antti-lyyra',
   'Dr Shirin Madon': 'https://www.lse.ac.uk/management/people/academic-staff/shirin-madon',
   'Professor Susan Scott': 'https://www.lse.ac.uk/management/people/academic-staff/susan-scott',
   'Dr Carsten Sørensen': 'https://www.lse.ac.uk/management/people/academic-staff/carsten-sorensen',
   'Dr Will Venters': 'https://www.lse.ac.uk/management/people/academic-staff/will-venters',
   'Dr Edgar Whitley': 'https://www.lse.ac.uk/management/people/academic-staff/edgar-whitley',
   'Dr Ricardo Alonso': 'https://www.lse.ac.uk/management/people/academic-staff/ricardo-alonso',
   'Dr Jordi Blanes i Vidal': 'https://www.lse.ac.uk/management/people/academic-staff/jordi-blanes-i-vidal',
   'Dr Raquel Campos': 'https://www.lse.ac.uk/management/people/academic-staff/raquel-campos',
   'Nilesh Dittani': 'https://www.lse.ac.uk/management/people/academic-staff/nilesh-dattani',
   'David de Meza': 'https://www.lse.ac.uk/management/people/academic-staff/david-de-meza',
   'Shadi Farahzadi': 'https://www.lse.ac.uk/management/people/academic-staff/shadi-farahzadi',
   'Professor Daniel Gottlieb': 'https://www.lse.ac.uk/management/people/academic-staff/daniel-gottlieb',
   'Dr Satoshi Kanazawa': 'https://www.lse.ac.uk/management/people/academic-staff/satoshi-kanazawa',
   'Dr Rocco Macchiavello': 'https://www.lse.ac.uk/management/people/academic-staff/rocco-macchiavello',
   'Dr Kristóf Madarász': 'https://www.lse.ac.uk/management/people/academic-staff/kristof-madarasz',
   'Dr Veronica Rappoport': 'https://www.lse.ac.uk/management/people/academic-staff/veronica-rappoport',
   'Professor Diane Reyniers': 'https://www.lse.ac.uk/management/people/academic-staff/diane-reyniers',
   'Dr Jörn Rothe': 'https://www.lse.ac.uk/management/people/academic-staff/jorn-rothe',
   'Dr Catherine Thomas': 'https://www.lse.ac.uk/management/people/academic-staff/catherine-thomas',
   'Dr Jose P. Vasquez': 'https://www.lse.ac.uk/management/people/academic-staff/jose-vasquez',
   'Professor Noam Yuchtman': 'https://www.lse.ac.uk/management/people/academic-staff/noam-yuchtman',
   'Professor Amitav Chakravarti': 'https://www.lse.ac.uk/management/people/academic-staff/amitav-chakravarti',
   'Dr Heather Kappes': 'https://www.lse.ac.uk/management/people/academic-staff/heather-kappes',
   'Dr Hye-Young Kim': 'https://www.lse.ac.uk/management/people/academic-staff/hye-young-kim',
   'Dr Xiaolin LI': 'https://www.lse.ac.uk/management/people/academic-staff/xiaolin-li',
   'Professor Om Narasimhan': 'https://www.lse.ac.uk/management/people/academic-staff/om-narasimhan',
   'Dr Nicolette Sullivan': 'https://www.lse.ac.uk/management/people/academic-staff/nicolette-sullivan',
   'Professor Naufel Vilcassim': 'https://www.lse.ac.uk/management/people/academic-staff/naufel-vilcassim',
   'Dr Nayat Horozoglu': 'https://www.lse.ac.uk/management/people/academic-staff/nayat-horozoglu',
   'Professor Richard Steinberg': 'https://www.lse.ac.uk/management/people/academic-staff/richard-steinberg',
   'Dr Nicole Abi-Esber': 'https://www.lse.ac.uk/management/people/academic-staff/nicole-abi-esber',
   'Dr Yally Avrahampour': 'https://www.lse.ac.uk/management/people/academic-staff/yally-avrahampour',
   'Professor Harry Barkema': 'https://www.lse.ac.uk/management/people/academic-staff/harry-barkema',
   'Dr Rebecca Campbell': 'https://www.lse.ac.uk/management/people/academic-staff/rebecca-campbell',
   'Dr Christine Cote': 'https://www.lse.ac.uk/management/people/academic-staff/christine-cote',
   'Dr Shoshana Dobrow Riza': 'https://www.lse.ac.uk/management/people/academic-staff/shoshana-dobrow',
   'Dr Barbara Fasolo': 'https://www.lse.ac.uk/management/people/academic-staff/barbara-fasolo',
   'Helen Hu': 'https://www.lse.ac.uk/management/people/academic-staff/helen-hu',
   'Dr Xiaoran Hu': 'https://www.lse.ac.uk/management/people/academic-staff/xiaoran-hu',
   'Dr Niranjan Janardhanan': 'https://www.lse.ac.uk/management/people/academic-staff/niranjan-janardhanan',
   'Dr Hyun-Jung Lee': 'https://www.lse.ac.uk/management/people/academic-staff/hyun-jung-lee',
   'Dr Xu Li': 'https://www.lse.ac.uk/management/people/academic-staff/xu-li',
   'Dr Nadia Millington': 'https://www.lse.ac.uk/management/people/academic-staff/nadia-millington',
   'Dr Lidiia Pletneva': 'https://www.lse.ac.uk/management/people/academic-staff/lidiia-pletneva',
   'Dr Dorottya Sallai': 'https://www.lse.ac.uk/management/people/academic-staff/dorottya-sallai',
   'Luc Schneider': 'https://www.lse.ac.uk/management/people/academic-staff/luc-schneider',
   'Dr Emma Soane': 'https://www.lse.ac.uk/management/people/academic-staff/emma-soane',
   'Dr Lourdes Sosa': 'https://www.lse.ac.uk/management/people/academic-staff/lourdes-sosa',
   'avenkatesan': 'https://www.lse.ac.uk/management/people/academic-staff/aparna-venkatesan',
   'Chunlin Yu': 'https://www.lse.ac.uk/management/people/academic-staff/chunlin-yu',
   'Dr Jonathan Liebenau': 'https://www.lse.ac.uk/management/people/academic-staff/jonathan-liebenau'},
  'Find a member of professional services staff': {'David Meech Mazumdar': None,
   'Martha Langan': None,
   'Kelly Benson': None,
   'Sharon Barnes': None,
   'Nicola Foster': None,
   'Jas Dhanda': None,
   'Katrina Millet': None,
   'Paulette Annon': None,
   'Becci Ayling': None,
   'Mallika Bandyopadhyay': None,
   'Alex Banks': None,
   'Kati Cooper': None,
   'Daniella Duarte': None,
   'Charlene Fairs': None,
   'Sarah Greene': None,
   'Freya Grisoni': None,
   'Charlotte Harris': None,
   'Olivia Keane': None,
   'Dr Stephanie Lambert': None,
   'Astrid Laurent': None,
   'Lucy Ariyadasa': None,
   'Nicholas Martin': None,
   'Sharan Purewal': None,
   'Laura Rivers': None,
   'Kareema Carrington': None,
   'Annie Wang': None,
   'Langyi Ye': None,
   'Alessia Zaja': None,
   'Becky Coggins': None,
   'Sophie Gavriel': None,
   'Barbara Miller Manzanares': None,
   'Julie Gordon Shipley': None,
   'Hrilina Lock': None,
   'Cheryl Johnson': None,
   'Linda Sclanders': None,
   'Nayani Gunatilake': None,
   'Alice Johns': None,
   'Wasima Anwari': None,
   'Hiba Dabis': None,
   'Bethan Briggs': None,
   'Ardy Cheung': None,
   'Molly Healy': None,
   'Emma Mullen': None,
   'Raheela Shah': None,
   'Laura Knowles': None,
   'Sam Hunt': None,
   'Paddy Hyde': None,
   'Tevec Ibrahim': None,
   'Daniel Marenghi': None,
   'Désirée Meyerding': None,
   'Nino Nizharadze': None,
   'Leah Murray': None,
   'Dina Rabie': None,
   'Patience Ayalogu': None,
   'Ellie Fleming': None,
   'Camilla Kennedy Harper': None,
   'Emma Ward': None,
   'Emma Balaguette': None,
   'Nenna Opara': None,
   'Jessica Harrison': None},
  'Find a PhD Student': {'Sazid Ahmad': 'https://www.lse.ac.uk/management/people/phd-student/sazid-ahmad',
   'Tanya Alaaldin': 'https://www.lse.ac.uk/management/people/phd-student/tanya-alaaldin',
   'Aaron Aujla': 'https://www.lse.ac.uk/management/people/phd-student/aaron-aujla',
   'Alastair Buckle': 'https://www.lse.ac.uk/management/people/phd-student/alastair-buckle',
   'Yiting Dong': 'https://www.lse.ac.uk/management/people/phd-student/yiting-dong',
   'Sun Woo Nam': 'https://www.lse.ac.uk/management/people/phd-student/sun-woo-nam',
   'Roop Bhadury': 'https://www.lse.ac.uk/management/people/phd-student/roop-bhadury',
   'Lisa Di Natale': 'https://www.lse.ac.uk/management/people/phd-student/lisa-di-natale',
   'Philipp Gagel': 'https://www.lse.ac.uk/management/people/phd-student/philipp-gagel',
   'Maximilian Goehmann': None,
   'Yingjia Hu': 'https://www.lse.ac.uk/management/people/phd-student/yingjia-hu',
   'Susanne Klausing': 'https://www.lse.ac.uk/management/people/phd-student/susanne-klausing',
   'Georgia Meyer': 'https://www.lse.ac.uk/management/people/phd-student/georgia-meyer',
   'Rohit Nair': 'https://www.lse.ac.uk/management/people/phd-student/rohit-nair',
   'Anulekha Nandi': 'https://www.lse.ac.uk/management/people/phd-student/anulekha-nandi',
   'Muhammad Aqeel Awan': 'https://www.lse.ac.uk/management/people/phd-student/muhammad-aqeel-awan',
   'Anna Czaplewska-Jaffery': 'https://www.lse.ac.uk/management/people/phd-student/anna-czaplewska-jaffery',
   'Ozan Kirtac': None,
   'Caleb Phillips': 'https://www.lse.ac.uk/management/people/phd-student/caleb-phillips',
   'Rashi Sonal': 'https://www.lse.ac.uk/management/people/phd-student/rashi-sonal',
   'Ravneet Bawa': 'https://www.lse.ac.uk/management/people/phd-student/ravneet-bawa',
   'Areum Cho': 'https://www.lse.ac.uk/management/people/phd-student/areum-cho',
   'Ceylin Ertekin': 'https://www.lse.ac.uk/management/people/phd-student/ceylin-petek-ertekin',
   'Earl Panganiban': 'https://www.lse.ac.uk/management/people/phd-student/earl-panganiban',
   'Zoe Ziqi Zhong': 'https://www.lse.ac.uk/management/people/phd-student/ziqi-zhong',
   'Luca Barbato': 'https://www.lse.ac.uk/management/people/phd-student/luca-barbato',
   'David Cai': 'https://www.lse.ac.uk/management/people/phd-student/david-cai',
   'José Ignacio González-Rojas': 'https://www.lse.ac.uk/management/people/phd-student/jose-ignacio-gonzalez-rojas',
   'Fabrizio Leone': None,
   'Romaine Loubes': 'https://www.lse.ac.uk/management/people/phd-student/romaine-loubes',
   'Ottavia Pesenti': 'https://www.lse.ac.uk/management/people/phd-student/ottavia-pesenti'},
  'Find a member of visiting staff': {'Professor Sarosh Kuruvilla': 'https://www.lse.ac.uk/management/people/visiting-staff/sarosh-kuruvilla',
   'Professor\xa0Saonee Sarker': 'https://www.lunduniversity.lu.se/lucat/user/178eb835da039906559dfd39e98f59ff',
   'Sir Geoffrey Owen': 'https://policyexchange.org.uk/author/geoffowen/',
   'Dr Susanna Khavul': 'https://www.lse.ac.uk/management/people/visiting-staff/susanna-khavul',
   'Dr\xa0Ihor Lantukh': 'https://www.researchgate.net/profile/Ihor-Lantukh',
   'Dr Rebecca Newton': 'http://rebeccanewton.co.uk/',
   'Dr Katie Best': 'https://www.lse.ac.uk/management/people/visiting-staff/katie-best',
   'Dr Tal Harel': None,
   'Dr Nurul Liyana Mohd Kamil': 'https://umexpert.um.edu.my/nurulliyana',
   'Dr Daniela Lup': 'https://www.escp.eu/lup-daniela',
   'Dr Michele Fioretti': 'https://www.sciencespo.fr/department-economics/en/researcher/michele-fioretti.html',
   'Dr\xa0Henry Hang Shen': 'https://www.linkedin.com/in/henryhangshen/?originalSubdomain=fr',
   'Dr Dina Rabie': 'https://www.lse.ac.uk/management/people/visiting-staff/dina-rabie'},
  'Find a member of emeritus staff': {'Professor Peter Abell': 'https://www.lse.ac.uk/management/people/emeriti/peter-abell',
   'Professor Ian Angell': 'https://www.lse.ac.uk/management/people/emeriti/ian-angell',
   'Professor Gautam Appa': 'https://www.lse.ac.uk/management/people/emeriti/gautam-appa',
   'Dr James Backhouse': 'https://www.lse.ac.uk/management/people/emeriti/james-backhouse',
   'Professor Gwyn Bevan': 'https://www.lse.ac.uk/management/people/emeriti/gwyn-bevan',
   'Professor Jacqueline Coyle-Shapiro': 'https://www.lse.ac.uk/management/people/emeriti/jacqueline-coyle-shapiro',
   'Professor Saul Estrin': 'https://www.lse.ac.uk/management/people/emeriti/saul-estrin',
   'Professor Carola Frege': 'https://www.lse.ac.uk/management/people/emeriti/carola-frege',
   'Professor Richard Hyman': 'https://www.lse.ac.uk/management/people/emeriti/richard-hyman',
   'Professor Jannis Kallinikos': 'https://www.lse.ac.uk/management/people/emeriti/jannis-kallinikos',
   'Professor Frank Land': 'https://www.lse.ac.uk/management/people/emeriti/frank-land',
   'Professor David Metcalf': 'https://www.lse.ac.uk/management/people/emeriti/david-metcalf',
   'Professor Alexander (Sandy) Pepper': 'https://www.lse.ac.uk/management/people/emeriti/alexander-sandy-pepper',
   'Professor Larry Phillips': 'https://www.lse.ac.uk/management/people/emeriti/larry-phillips',
   'Professor Jonathan Rosenhead': 'https://www.lse.ac.uk/management/people/emeriti/jonathan-rosenhead',
   'Professor Leslie Willcocks': 'https://www.lse.ac.uk/management/people/emeriti/leslie-willcocks',
   'Professor Paul Williams': 'https://www.lse.ac.uk/management/people/emeriti/paul-williams',
   'Professor Paul Willman': 'https://www.lse.ac.uk/management/people/emeriti/paul-willman'}},
 'Marshall Institute': {'Directors': {'Stephan Chambers': 'https://www.lse.ac.uk/marshall-institute/people/schambers',
   'Jonathan Roberts': 'https://www.lse.ac.uk/marshall-institute/people/jroberts'},
  'Teaching and Research ': {'Nava Ashraf': 'https://www.lse.ac.uk/marshall-institute/people/nashraf',
   'Sir Thomas Hughes-Hallett': None,
   'Kerryn Krige': 'https://www.lse.ac.uk/marshall-institute/people/Kerryn-Krige',
   'Julian Le Grand': 'https://www.lse.ac.uk/marshall-institute/people/jlegrand',
   'Jack Winterton': None},
  'Operations': {'Emma Bayes': None,
   'Amelia Bradley': None,
   'Shanzeh Mahmood': None,
   'Corrina Summers': None,
   'Anna Townsend': None,
   'Alexander Wright': None,
   'Julia Ziemer': None},
  '100x Impact Accelerator': {'Amelia Bradley': None,
   'Kayleigh Cunoosamy': None,
   'Fan Gu': None,
   'Nell Lemaistre': None,
   'Shanzeh Mahmood': None,
   'Carl Moldestad': None,
   'Aggrey Nyondwa': None,
   'Keri Rowsell': None},
  'Current Practitioners in Residence and Visiting Fellows ': {'Manuel Antunes': None,
   'Swatee Deepak': None,
   'Saul Estrin': None,
   'Luke Fletcher': None,
   'Sonal Sachdev Patel': None},
  'Past Practitioners in Residence and Visiting Fellows': {'Daniel Hulme': None,
   'Matthew Bishop': None,
   'Christian Busch': 'http://www.lse.ac.uk/business-and-consultancy/consulting/experts/christian-busch',
   'Veronika Kapustina': None,
   'Emily Kasriel': None,
   'Jude Kelly': None,
   'Katherine Lorenz': None,
   'Sonia Medina': None,
   'Jerome Misso': None,
   'Eva Neitzert': None,
   'Will Page': None,
   'Aunnie Patton Power': None,
   'William Perrin': None,
   'Chelsea Phipps': None,
   'Tom Rippin': None,
   'David Robinson': None,
   'Mabel Van Oranje': None}},
 'Department of Mathematics': {'Academic Faculty': {'Dr Ahmad Abdi': 'http://www.lse.ac.uk/Mathematics/people/Ahmad-Abdi',
   'Professor Peter Allen': 'http://www.lse.ac.uk/Mathematics/people/Peter-Allen',
   'Professor Martin Anthony': 'http://www.lse.ac.uk/Mathematics/people/Martin-Anthony',
   'Galit Ashkenazi-Golan': 'https://www.lse.ac.uk/Mathematics/People/Galit-Ashkenazi-Golan',
   'Dr Tuğkan\xa0Batu': 'http://www.lse.ac.uk/Mathematics/people/Tugkan-Batu',
   'Professor Julia\xa0Böttcher': 'http://www.lse.ac.uk/Mathematics/people/Julia-Boettcher',
   'Professor Graham Brightwell': 'http://www.lse.ac.uk/Mathematics/people/Graham-Brightwell',
   'Dr Christoph\xa0Czichowsky': 'http://www.lse.ac.uk/Mathematics/people/Christoph-Czichowsky',
   'Dr Albina Danilova': 'http://www.lse.ac.uk/Mathematics/people/Albina-Danilova',
   'Dr Pavel Gapeev': 'http://www.lse.ac.uk/Mathematics/people/Pavel-Gapeev',
   'Professor Olivier Gossner': 'http://www.lse.ac.uk/Mathematics/people/Olivier-Gossner',
   'Jan van den Heuvel': 'https://www.lse.ac.uk/Mathematics/People/Jan-van-den-Heuvel',
   'Grammateia Kotsialou': 'https://www.lse.ac.uk/Mathematics/People/Grammateia-Kotsialou',
   'Dr Ioannis Kouletsis': 'http://www.lse.ac.uk/Mathematics/people/Ioannis-Kouletsis',
   'Professor Andrew Lewis-Pye': 'http://www.lse.ac.uk/Mathematics/people/Andrew-Lewis-Pye',
   'Dr Arne Lokka': 'http://www.lse.ac.uk/Mathematics/people/Arne-Lokka',
   'Neil Olver': 'https://www.lse.ac.uk/Mathematics/People/Neil-Olver',
   'Professor Adam Ostaszewski': 'http://www.lse.ac.uk/Mathematics/people/Adam-Ostaszewski',
   'Dr Katerina Papadaki': 'http://www.lse.ac.uk/Mathematics/people/Katerina-Papadaki',
   'Emilio Pierro': 'https://www.lse.ac.uk/Mathematics/People/Emilio-Pierro',
   'Professor Johannes Ruf': 'http://www.lse.ac.uk/Mathematics/people/Johannes-Ruf',
   'Professor Amol Sasane': 'http://www.lse.ac.uk/Mathematics/people/Amol-Sasane',
   'Dr Robert Simon': 'http://www.lse.ac.uk/Mathematics/people/Robert-Simon',
   'Professor Jozef Skokan': 'http://www.lse.ac.uk/Mathematics/people/Jozef-Skokan',
   'Professor Gregory Sorkin': 'http://www.lse.ac.uk/Mathematics/people/Gregory-Sorkin',
   'Bernhard Von Stengel': 'https://www.lse.ac.uk/Mathematics/People/Bernhard-Von-Stengel',
   'Professor Konrad Swanepoel': 'http://www.lse.ac.uk/Mathematics/people/Konrad-Swanepoel',
   'Professor László Végh': 'http://www.lse.ac.uk/Mathematics/people/Laszlo-Vegh',
   'Professor Luitgard Veraart': 'http://www.lse.ac.uk/Mathematics/people/Luitgard-Veraart',
   'James Ward': 'https://www.lse.ac.uk/Mathematics/People/James-Ward',
   'Aled Williams': 'https://www.lse.ac.uk/Mathematics/People/Aled-Williams',
   'Giacomo Zambelli': 'https://www.lse.ac.uk/Mathematics/People/Giacomo-Zambelli',
   'Mihalis Zervos': 'https://www.lse.ac.uk/Mathematics/People/Mihail-Zervos'},
  'Academic Associates -\xa0Emeritus professors and visiting staff ': {'Elad Aigner-Horev': 'https://www.lse.ac.uk/Mathematics/People/Associate-Academics/Elad-Aigner-Horev/Elad-Aigner-Horev',
   'Steve Alpern': 'https://www.lse.ac.uk/Mathematics/People/Associate-Academics/Steve-Alpern/Steve-Alpern',
   'June Barrow-Green': 'https://www.lse.ac.uk/Mathematics/People/Associate-Academics/June-Barrow-Green/June-Barrow-Green',
   'Norman Biggs': 'https://www.lse.ac.uk/Mathematics/People/Associate-Academics/Norman-Biggs/Norman-Biggs',
   'Professor Nick Bingham': 'https://www.lse.ac.uk/Mathematics/People/Associate-Academics/Nick-Bingham/Nick-Bingham',
   'Richard Cole': 'https://www.lse.ac.uk/Mathematics/People/Associate-Academics/Richard-Cole/Richard-Cole',
   'David Collier': 'https://www.lse.ac.uk/Mathematics/People/Associate-Academics/David-Collier/David-Collier',
   'Paul Duetting': 'https://www.lse.ac.uk/Mathematics/People/Associate-Academics/Paul-Duetting/Paul-Duetting',
   'Michal Feldman': 'https://www.lse.ac.uk/Mathematics/People/Associate-Academics/Michal-Feldman/Michal-Feldman',
   'Thomas Lidbetter': 'https://www.lse.ac.uk/Mathematics/People/Associate-Academics/Thomas-Lidbetter/Thomas-Lidbetter',
   'Benny Sudakov': 'https://www.lse.ac.uk/Mathematics/People/Associate-Academics/Benny-Sudakov/Benny-Sudakov',
   'Peyton Young': 'https://www.lse.ac.uk/Mathematics/People/Associate-Academics/Peyton-Young/Peyton-Young'},
  'Class Teachers': {'Jose Armando Vivero': 'https://www.lse.ac.uk/Mathematics/People/Teachers/Jose-Armando-Vivero',
   'Rowland Bennett': 'http://www.lse.ac.uk/Mathematics/people/Teachers/Rowland-Bennett',
   'Johannes Brustle': 'https://www.lse.ac.uk/Mathematics/People/Research-Students/Johannes-Brustle',
   'Jun Cheng': 'https://www.lse.ac.uk/Mathematics/People/Teachers/Jun-Cheng',
   'Viet Dang': 'https://www.lse.ac.uk/Mathematics/People/Research-Students/Mahsa-Dalirrooyfard',
   'Francesco Di Braccio': 'https://www.lse.ac.uk/Mathematics/People/Research-Students/Francesco-Di-Braccio',
   'Carmelo Di Natale': 'https://www.lse.ac.uk/Mathematics/People/Teachers/Carmelo-Di-Natale',
   'Sam Fendrich': 'http://www.lse.ac.uk/Mathematics/people/Teachers/Sam-Fendrich',
   'Áron Fülöp': 'https://www.lse.ac.uk/Mathematics/People/Research-Students/Aron-Fulop',
   'Erin Hales': 'https://www.lse.ac.uk/Mathematics/People/Teachers/Erin-Hales',
   'Eleni Katirtzoglou': 'http://www.lse.ac.uk/Mathematics/people/Teachers/Eleni-Katirtzoglou',
   'Jasmin Katz': 'https://www.lse.ac.uk/Mathematics/People/Jasmin-Katz',
   'Boris Krippa': 'https://www.lse.ac.uk/Mathematics/People/Teachers/Boris-Krippa',
   'Arthur Lloyd-Phillips': 'https://www.lse.ac.uk/Mathematics/People/Teachers/Arthur-Lloyd-Phillips',
   'Daniil Mamaev': 'https://www.lse.ac.uk/Mathematics/People/Teachers/Daniil-Mamaev',
   'Mihir Neve': 'https://www.lse.ac.uk/Mathematics/People/Mihir-Neve',
   "Laurence O'Toole": 'https://www.lse.ac.uk/Mathematics/People/Teachers/Laurence-OToole',
   'Vasos Pavlika': 'https://www.lse.ac.uk/Mathematics/People/Teachers/Vasos-Pavlika',
   'Emilio Pierro': 'https://www.lse.ac.uk/Mathematics/People/Emilio-Pierro',
   'Calum Robson': 'https://www.lse.ac.uk/Mathematics/People/Teachers/Calum-Robson',
   'Niccolo Salvatori': 'https://www.lse.ac.uk/Mathematics/People/Teachers/Niccolò-Salvatori',
   'Amedeo Sgueglia': 'https://www.lse.ac.uk/Mathematics/People/Teachers/Amedeo-Sgueglia',
   'David Solomon': 'https://www.lse.ac.uk/Mathematics/People/Teachers/David-Solomon',
   'Yueying Sun': 'https://www.lse.ac.uk/Mathematics/People/Research-Students/Yueying-Sun',
   'Mohamed Tawfik': 'https://www.lse.ac.uk/Mathematics/People/Teachers/Mohamed-Tawfik',
   'Haoxiang Wang': 'https://www.lse.ac.uk/Mathematics/People/Teachers/Haoxiang-Wang',
   'Tony Whelan': 'http://www.lse.ac.uk/Mathematics/people/Teachers/Tony-Whelan',
   'Michael Yiasemides': 'https://www.lse.ac.uk/Mathematics/People/Teachers/Michael-Yiasemides',
   'Hengjian Zhang': 'https://www.lse.ac.uk/Mathematics/People/Research-Students/Hengjian-Zhang',
   'Lu-Ming Zhang': 'https://www.lse.ac.uk/Mathematics/People/Research-Students/Lu-Ming-Zhang',
   'Dr George Zouros': 'https://www.lse.ac.uk/Mathematics/People/Teachers/George-Zouros'},
  'Professional Services Staff': {'Kate Barker': 'https://www.lse.ac.uk/Mathematics/People/Kate-Barker',
   'Rebecca Batey': 'https://www.lse.ac.uk/Mathematics/People/Rebecca-Batey',
   'Jasna Begić': 'https://www.lse.ac.uk/Mathematics/People/Jasna-Begić',
   'Sharon Donegal-Grant': 'https://www.lse.ac.uk/Mathematics/People/Sharon-Donegal-Grant',
   'Emily Jackson': 'https://www.lse.ac.uk/Mathematics/People/Emily-Jackson',
   'Sarah Massey': 'https://www.lse.ac.uk/Mathematics/People/Joe-Faulkner',
   'Elvina Moonien': 'https://www.lse.ac.uk/Mathematics/People/Elvina-Moonien'},
  'Post-doctoral Researchers': {'Dr Sharat Ibrahimpur': 'https://www.lse.ac.uk/Mathematics/People/Sharat-Ibrahimpur',
   'Mehdi Makhul': 'https://www.lse.ac.uk/Mathematics/People/Mehdi-Makhul',
   'Dr Yani Pehova': 'https://www.lse.ac.uk/Mathematics/People/Yani-Pehova'},
  'Research Students': {'Johannes Brustle': 'https://www.lse.ac.uk/Mathematics/People/Research-Students/Johannes-Brustle',
   'Jun Cheng': 'https://www.lse.ac.uk/Mathematics/People/Research-Students/Jun-Cheng',
   'Mahsa Dalirrooy-Fard': 'https://www.lse.ac.uk/Mathematics/People/Research-Students/Mahsa-Dalirrooyfard',
   'Francesco Di Braccio': 'https://www.lse.ac.uk/Mathematics/People/Research-Students/Francesco-Di-Braccio',
   'Áron Fülöp': 'https://www.lse.ac.uk/Mathematics/People/Research-Students/Aron-Fulop',
   'Justin Gwee': 'http://www.lse.ac.uk/Mathematics/people/Research-Students/Justin-Gwee',
   'Sahar Jahani': 'https://www.lse.ac.uk/Mathematics/People/Research-Students/Sahar-Jahani',
   'Jasmin Katz': 'https://www.lse.ac.uk/Mathematics/People/Jasmin-Katz',
   'Joanna Lada': 'https://www.lse.ac.uk/Mathematics/People/Research-Students/Joanna-Lada',
   'Zhesheng Liu': 'https://www.lse.ac.uk/Mathematics/People/Research-Students/Zhesheng-Liu',
   'Haoyuan Ma': 'https://www.lse.ac.uk/Mathematics/People/Haoyuan-Ma',
   'Domenico Mergoni': 'https://www.lse.ac.uk/Mathematics/People/Research-Students/Domenico-Mergoni',
   'Mihir Neve': 'https://www.lse.ac.uk/Mathematics/People/Mihir-Neve',
   'Raymond Pang': 'https://www.lse.ac.uk/Mathematics/People/Research-Students/Raymond-Pang',
   'Edward Plumb': 'https://www.lse.ac.uk/Mathematics/People/Research-Students/Edward-Plumb',
   'Karl Stickler': 'https://www.lse.ac.uk/Mathematics/People/Research-Students/Karl-Stickler',
   'Cameron Strachan': 'https://www.lse.ac.uk/Mathematics/People/Research-Students/Cameron-Strachan',
   'Yueying Sun': 'https://www.lse.ac.uk/Mathematics/People/Research-Students/Yueying-Sun',
   'Ruilan Wang': 'http://www.lse.ac.uk/Mathematics/people/Research-Students/Ruilan-Wang',
   'Hengjian Zhang': 'https://www.lse.ac.uk/Mathematics/People/Research-Students/Hengjian-Zhang',
   'Luming Zhang': 'https://www.lse.ac.uk/Mathematics/People/Research-Students/Lu-Ming-Zhang',
   'Yuliang Zhang': 'https://www.lse.ac.uk/Mathematics/People/Research-Students/Yuliang-Zhang'}},
 'Department of Media and Communications': {'Department Leadership Team': {'Professor Bart Cammaerts': 'https://www.lse.ac.uk/media-and-communications/people/academic-staff/bart-cammaerts',
   'Professor Lee Edwards': 'https://www.lse.ac.uk/media-and-communications/people/academic-staff/lee-edwards',
   'Dr Seeta Peña Gangadharan': 'https://www.lse.ac.uk/media-and-communications/people/academic-staff/seeta-pena-gangadharan',
   'James Deeley': None},
  'Academic staff': {'Omar Al-Ghazzi': 'https://www.lse.ac.uk/media-and-communications/people/academic-staff/omar-al-ghazzi',
   'Dr Asif Ali Akhtar': 'https://www.lse.ac.uk/media-and-communications/people/academic-staff/asif-ali-akhtar',
   'Nick Anstead': 'https://www.lse.ac.uk/media-and-communications/people/academic-staff/nick-anstead',
   'Shakuntala Banaji': 'https://www.lse.ac.uk/media-and-communications/people/academic-staff/shakuntala-banaji',
   'Charlie Beckett': 'https://www.lse.ac.uk/media-and-communications/people/academic-staff/charlie-beckett',
   'Professor Bart Cammaerts': 'https://www.lse.ac.uk/media-and-communications/people/academic-staff/bart-cammaerts',
   'Lilie Chouliaraki': 'https://www.lse.ac.uk/media-and-communications/people/academic-staff/lilie-chouliaraki',
   'Nick Couldry': 'https://www.lse.ac.uk/media-and-communications/people/academic-staff/nick-couldry',
   'Dr Simidele Dosekun': 'https://www.lse.ac.uk/media-and-communications/people/academic-staff/simidele-dosekun',
   'Dr Lee Edwards': 'https://www.lse.ac.uk/media-and-communications/people/academic-staff/lee-edwards',
   'Seeta Peña Gangadharan': 'https://www.lse.ac.uk/media-and-communications/people/academic-staff/seeta-pena-gangadharan',
   'Professor Myria Georgiou': 'https://www.lse.ac.uk/media-and-communications/people/academic-staff/myria-georgiou',
   'Suzanne Temwa Gondwe Harris': 'https://www.lse.ac.uk/media-and-communications/people/academic-staff/suzanne-temwa-gondwe-harris',
   'Ellen Helsper': 'https://www.lse.ac.uk/media-and-communications/people/academic-staff/ellen-helsper',
   'César Jiménez Martínez': 'https://www.lse.ac.uk/media-and-communications/people/academic-staff/césar-jiménez-martínez',
   'Sonia Livingstone': 'https://www.lse.ac.uk/media-and-communications/people/academic-staff/sonia-livingstone',
   'Robin Mansell': 'https://www.lse.ac.uk/media-and-communications/people/academic-staff/robin-mansell',
   'Bingchun Meng': 'https://www.lse.ac.uk/media-and-communications/people/academic-staff/bingchun-meng',
   'Dr Dylan Mulvin': 'https://www.lse.ac.uk/media-and-communications/people/academic-staff/dylan-mulvin',
   "Dr Rachel O'Neill": 'https://www.lse.ac.uk/media-and-communications/people/academic-staff/rachel-oneill',
   'Shani Orgad': 'https://www.lse.ac.uk/media-and-communications/people/academic-staff/shani-orgad',
   'Jean-Christophe Plantin': 'https://www.lse.ac.uk/media-and-communications/people/academic-staff/jean-christophe-plantin',
   'Alison Powell': 'https://www.lse.ac.uk/media-and-communications/people/academic-staff/alison-powell',
   'Terhi Rantanen': 'https://www.lse.ac.uk/media-and-communications/people/academic-staff/terhi-rantanen',
   'Philipp Seuferling': 'https://www.lse.ac.uk/media-and-communications/people/academic-staff/philipp-seuferling',
   'Damian Tambini': 'https://www.lse.ac.uk/media-and-communications/people/academic-staff/damian-tambini',
   'Wendy Willems': 'https://www.lse.ac.uk/media-and-communications/people/academic-staff/wendy-willems',
   'jungmo youn': 'https://www.lse.ac.uk/media-and-communications/people/academic-staff/jungmo-youn'},
  'Professional services staff': {'Nicole Garnier': None,
   'Ethan Liu': None,
   'Ben McDonald': None,
   'Martina Zuccarella': None,
   'Leo Beattie': None,
   'Danielle Blasse': None,
   'Brianna Crummy': None,
   'Lwam Tesfay': None,
   'James Deeley': None},
  'Guest teaching staff': {'Alessandro Castellini': 'https://www.lse.ac.uk/media-and-communications/people/guest-teachers/alessandro-castellini',
   'Dr Kate Gilchrist': 'https://www.lse.ac.uk/media-and-communications/people/guest-teachers/kate-gilchrist',
   'Dr Leslie Haddon': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/leslie-haddon',
   'Dr Jess Hannah': 'https://www.lse.ac.uk/media-and-communications/people/guest-teachers/jess-hannah',
   'Dr Maria Paola Pofi': 'https://www.lse.ac.uk/media-and-communications/people/guest-teachers/maria-paola-pofi'},
  'Visiting and associate staff': {'Professor Ahmet Atay': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/ahmet-atay',
   'Meghanne Barker': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/meghanne-barker',
   'Professor Sarah Banet-Weiser': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/sarah-banet-weiser',
   'Professor Benedetta Brevini': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/benedetta-brevini',
   'Dr Patricio Cabello': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/patricio-cabello',
   'Dr Helena Chmielewska-Szlajfer': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/helena-chmielewska-szlajfer',
   'Dr Bei Deng': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/bei-deng',
   'Dr Torsten Geelan': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/torsten-geelan',
   'Dr Kate Gilchrist': 'https://www.lse.ac.uk/media-and-communications/people/guest-teachers/kate-gilchrist',
   'Dr Yael Gordon': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/yael-gordon',
   'Dr Leslie Haddon': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/leslie-haddon',
   'Alison Harcourt': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/alison-harcourt',
   'Velislava Hillman': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/velislava-hillman',
   'Dr Weizi Huang': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/weizi-huang',
   'Yasemin İnceoğlu': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/yasemin-İnceoğlu',
   'Professor Baharul Islam': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/baharul-islam',
   'Reidar S Jessen': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/reidar-s-jessen',
   'Professor Christian Katzenbach': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/christian-katzenbach',
   'Dr Anthony Kelly': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/anthony-kelly',
   'Baroness Beeban Kidron OBE': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/beeban-kidron',
   'Angelos Kissas': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/angelos-kissas',
   'Professor Jim Macnamara': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/jim-macnamara',
   'Dr Ewa Majczak': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/ewa-majczak',
   'Dr Jose-Carlos Mariategui': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/jose-carlos-mariategui',
   'Dr Terese Mendiguren': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/terese-mendiguren',
   'Claire Milne': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/claire-milne',
   'Dr Rian Mulcahy': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/rian-mulcahy',
   'Professor Hannu Nieminen': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/hannu-nieminen',
   'Jedrzej Niklas': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/jerdrzej-niklas',
   'Dr Gabriel Pereira': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/gabriel-pereira',
   'Kruakae Pothong': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/kruakae-pothong',
   'Miriam Rahali': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/Miriam-Rahali',
   'Dr Maggie Scammell': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/margaret-scammell',
   'Professor Philip Schlesinger': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/philip-schlesinger',
   'Professor Hannele Seeck': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/hannele-seeck',
   'Professor Julian Sefton-Green': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/julian-sefton-green',
   'Professor Katherine Sender': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/katherine-sender',
   'Jonny Shipp': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/jonny-shipp',
   'Dr Johanna Sumiala': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/johanna-sumiala',
   'Dr Jane Vincent': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/jane-vincent',
   'Dr Nina Vindum Rasmussen': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/nina-rasmussen',
   'Dr\xa0Grace Yuehan Wang': 'https://www.lse.ac.uk/media-and-communications/people/visiting-and-associate-staff/grace-yuehan-wang',
   'Dr Jun Yu': None},
  'Research and project staff': {'Fiona Abades Barclay': 'https://www.lse.ac.uk/media-and-communications/people/research-staff/fiona-abades-barclay',
   'Dr Fabian Broeker': 'https://www.lse.ac.uk/media-and-communications/people/research-staff/fabian-broeker',
   'Emma Goodman': 'https://www.lse.ac.uk/media-and-communications/people/research-staff/emma-goodman',
   'Diego Martinez': 'https://www.lse.ac.uk/media-and-communications/people/research-staff/diego-martinez',
   'Dr Pablo Morales': 'https://www.lse.ac.uk/media-and-communications/people/research-staff/pablo-morales',
   'Esra Ozkan': 'https://www.lse.ac.uk/media-and-communications/people/research-staff/esra-ozkan',
   'Didem Ozkul': 'https://www.lse.ac.uk/media-and-communications/people/research-staff/didem-ozkul',
   'Sanne Stevens': 'https://www.lse.ac.uk/media-and-communications/people/research-staff/sanne-stevens',
   'Mariya Stoilova': 'https://www.lse.ac.uk/media-and-communications/people/research-staff/mariya-stoilova',
   'Tshepo Tshabalala': 'https://www.lse.ac.uk/media-and-communications/people/research-staff/tshepo-tshabalala'},
  'Job opportunities': {'Postdoctoral research fellowships': 'https://www.lse.ac.uk//media-and-communications/research/postdoctoral-research-fellowships'}},
 'Department of Methodology': {'Academic Staff': {'Johan Ahlback': 'https://www.lse.ac.uk/Methodology/People/Academic-Staff/Johan-Ahlback/Johan-Ahlback',
   'Audrey Alejandro': 'https://www.lse.ac.uk/Methodology/People/Academic-Staff/Audrey-Alejandro/Audrey-Alejandro',
   'Eileen Alexander': 'https://www.lse.ac.uk/Methodology/People/Academic-Staff/Eileen-Alexander/Eileen-Alexander',
   'Professor Kenneth Benoit': 'https://www.lse.ac.uk/Methodology/People/Academic-Staff/Kenneth-Benoit/Kenneth-Benoit',
   'Dr. Flora Cornish': 'https://www.lse.ac.uk/Methodology/People/Academic-Staff/Flora-Cornish/Flora-Cornish',
   'Daniel De Kadt': 'https://www.lse.ac.uk/Methodology/People/Academic-Staff/Daniel-De-Kadt/Daniel-De-Kadt',
   'Zach Dickson': 'https://www.lse.ac.uk/Methodology/People/Academic-Staff/Zach-Dickson/Zach-Dickson',
   'Michael Ganslmeier': 'https://www.lse.ac.uk/Methodology/People/Academic-Staff/Michael-Ganslmeier/Michael-Ganslmeier',
   'Professor George Gaskell': 'https://www.lse.ac.uk/Methodology/People/Academic-Staff/George-Gaskell/George-Gaskell',
   'Friedrich Geiecke': 'https://www.lse.ac.uk/Methodology/People/Academic-Staff/Friedrich-Geiecke/Friedrich-Geiecke',
   'Patrick Gildersleve': 'https://www.lse.ac.uk/Methodology/People/Academic-Staff/Patrick-Gildersleve/Patrick-Gildersleve',
   'Isaac Haruna Ziaba': 'https://www.lse.ac.uk/Methodology/People/Academic-Staff/Isaac-Haruna/Isaac-Haruna-Ziaba',
   'Professor Jonathan Jackson': 'https://www.lse.ac.uk/Methodology/People/Academic-Staff/Jonathan-Jackson/Jonathan-Jackson',
   'Dr. Ellie Knott': 'https://www.lse.ac.uk/Methodology/People/Academic-Staff/Ellie-Knott/Ellie-Knott',
   'Professor Jouni Kuha': 'https://www.lse.ac.uk/statistics/people/jouni-kuha',
   'Anne-Laure Mahe': 'https://www.lse.ac.uk/Methodology/People/Academic-Staff/Anne-Laure-Mahe/Anne-Laure-Mahe',
   'Blake Miller': 'https://www.lse.ac.uk/Methodology/People/Academic-Staff/Blake-Miller/Blake-Miller',
   'Seila Panizzolo': 'https://www.lse.ac.uk/Methodology/People/Academic-Staff/Seila-Panizzolo/Seila-Panizzolo',
   'Elly Power': 'https://www.lse.ac.uk/Methodology/People/Academic-Staff/Eleanor-Power/Eleanor-Power',
   'Aliya Rao': 'https://www.lse.ac.uk/Methodology/People/Academic-Staff/Aliya-Rao/Aliya-Rao',
   'Thomas Robinson': 'https://www.lse.ac.uk/Methodology/People/Academic-Staff/Thomas-Robinson/Thomas-Robinson',
   'Sally Stares': 'https://www.lse.ac.uk/Methodology/People/Academic-Staff/Sally-Stares/Sally-Stares',
   'Raphael Susewind': 'https://www.lse.ac.uk/Methodology/People/Academic-Staff/Raphael-Susewind/Raphael-Susewind',
   'Patrick Sturgis': 'https://www.lse.ac.uk/Methodology/People/Academic-Staff/Patrick-Sturgis/Patrick-Sturgis',
   'Dr. Chana Teeger': 'https://www.lse.ac.uk/Methodology/People/Academic-Staff/Chana-Teeger/Chana-Teeger',
   'Dr. Milena Tsvetkova': 'https://www.lse.ac.uk/Methodology/People/Academic-Staff/Milena-Tsvetkova/Milena-Tsvetkova'},
  'Professional Services Staff': {'Crystal Chia': None,
   'John Curtis': None,
   'Chau Le': None,
   'Camilya Maleh': None,
   'Serena Ngai': None,
   'Lucia Pedrioli': None,
   'Olivia Talland': None},
  'Research Staff': {'Siân Brooke': 'https://www.lse.ac.uk/Methodology/People/Academic-Staff/Siân-Brooke/Siân-Brooke',
   'Dr Kate Summers': 'https://www.lse.ac.uk/Methodology/People/Academic-Staff/Kate-Summers/Kate-Summers'},
  'Research Students': {'Islam al Khatib': 'https://www.lse.ac.uk//Methodology/People/Research-Students/Islam-al-Khatib/Islam-al-Khatib',
   'Amal Ali': 'https://www.lse.ac.uk//Methodology/People/Research-Students/Amal-Ali/Amal-Ali',
   'Akriti Mehta': 'https://www.lse.ac.uk//Methodology/People/Research-Students/Akriti-Mehta/Akriti-Mehta',
   'Nancy Breton': '/Methodology/People/Research-Students/Nancy-Breton/Nancy-Breton',
   'Izzi Carter': 'https://www.lse.ac.uk//Methodology/People/Research-Students/Izzi-Carter/Izzi-Carter',
   'Christy Coulson': 'https://www.lse.ac.uk//Methodology/People/Research-Students/Christy-Coulson/Christy-Coulson',
   'Qi Cui': 'https://www.lse.ac.uk//Methodology/People/Research-Students/Qi-Cui/Qi-Cui',
   'Midanna de Almada': 'https://www.lse.ac.uk//Methodology/People/Research-Students/Midanna-De-Almada/Midanna-de-Almada',
   'Yuanmo He': 'https://www.lse.ac.uk//Methodology/People/Research-Students/Yuanmo-He/Yuanmo-He',
   'Katya Kostadintcheva': 'https://www.lse.ac.uk//Methodology/People/Research-Students/Katya-Kostadintcheva/Katya-Kostadintcheva',
   'Roni Küppers': 'https://www.lse.ac.uk//Methodology/People/Research-Students/Roni-Küppers/Roni-Küppers',
   'Rosanna Le Voir': 'https://www.lse.ac.uk//Methodology/People/Research-Students/Rosie-Le-Voir/Rosanna-Le-Voir',
   'Christian Mueller': 'https://www.lse.ac.uk//Methodology/People/Research-Students/Christian-Mueller/Christian-Mueller',
   'Jasmine Oware': 'https://www.lse.ac.uk//Methodology/People/Research-Students/Jasmine-Oware/Jasmine-Oware',
   'Livio Miles Silva Muller profile page': 'https://www.lse.ac.uk//Methodology/People/Research-Students/Livio-Miles-Silva-Muller/Livio-Miles-Silva-Muller-profile-page',
   'Chuyao Wang': 'https://www.lse.ac.uk//Methodology/People/Research-Students/Chuyao-Wang/Chuyao-Wang',
   'Wanting Wang': 'https://www.lse.ac.uk//Methodology/People/Research-Students/Wanting-Wang/Wanting-Wang'},
  'Visiting Staff': {'Ian Brunton-Smith': 'https://www.lse.ac.uk/Methodology/People/Visiting-Staff/Ian-Brunton-Smith/Ian-Brunton-Smith',
   'Aroop Chatterjee': 'https://www.lse.ac.uk/Methodology/People/Visiting-Fellows/Aroop-Chatterjee',
   'Patrick English': 'https://www.lse.ac.uk/Methodology/People/Visiting-Staff/Patrick-English/Patrick-English',
   'Daniele Fanelli': 'https://www.lse.ac.uk/Methodology/People/Visiting-Fellows/Daniele-Fanelli',
   'Monica Gerber': 'https://www.lse.ac.uk/Methodology/People/Visiting-Fellows/Monica-Gerber',
   'Jennifer Hochschild': 'https://www.lse.ac.uk/Methodology/People/Visiting-Staff/Jennifer-Hochschild/Jennifer-Hochschild',
   'Florian Kern': 'https://www.lse.ac.uk/Methodology/People/Visiting-Fellows/Florian-Kern',
   'Emeritus Professor Michael Laver': None,
   'Professor Benjamin Lauderdale': None,
   'Sonja Marzi': 'https://www.lse.ac.uk/Methodology/People/Visiting-Fellows/Sonja-Marzi',
   'Ulises Moreno-Tabarez': 'https://www.lse.ac.uk/Methodology/People/Visiting-Fellows/Ulises-Moreno-Tabarez',
   'Matthew Mulford': 'https://www.lse.ac.uk/Methodology/People/Visiting-Staff/Matthew-Mulford/Matthew-Mulford',
   'Laura Pantzerhielm': 'https://www.lse.ac.uk/Methodology/People/Visiting-Fellows/Laura-Pantzerhielm',
   'Chris Posch': 'https://www.lse.ac.uk/Methodology/People/Visiting-Fellows/Chris-Posch',
   'Caroline Roberts': 'https://www.lse.ac.uk/Methodology/People/Visiting-Fellows/Caroline-Roberts',
   'Cohen Simpson': 'https://www.lse.ac.uk/Methodology/People/Visiting-Fellows/Cohen-Simpson',
   'Martina Testori': 'https://www.lse.ac.uk/Methodology/People/Visiting-Fellows/Martina-Testori',
   'Joshua Townsley': 'https://www.lse.ac.uk/Methodology/People/Visiting-Fellows/Joshua-Townsley',
   'Dr Ben Wilson': 'http://www.socialdemography.net/'}},
 'Department of Philosophy, Logic and Scientific Method': {'head': {'Shira Ahissar': None,
   'Prof J. McKenzie Alexander': 'http://jmckalex.org/home/Home.php',
   'Catherine Audard': 'http://blogs.lse.ac.uk/theforum/',
   'Prof Nick Baigent': 'http://www.uni-graz.at/fwiwww/2010/engl/faculty/baig.html',
   'Lukas Beck': 'https://www.lukasbeck.co/',
   'Prof Jonathan Birch': 'http://personal.lse.ac.uk/birchj1/',
   'Anika Bloomfield': None,
   'Ali Boyle': 'http://www.aliboyle.com/',
   'Prof Richard Bradley': 'http://personal.lse.ac.uk/bradleyr/',
   'Liam Kofi Bright': 'http://www.liamkofibright.com/',
   'Dr Campbell Brown': None,
   'Prof Nancy Cartwright': None,
   'Harriet Fagerberg': 'https://www.hfagerberg.co.uk',
   'Talita Ferrantelli': None,
   'Prof Roman Frigg': 'http://www.romanfrigg.org/',
   'Prof Wulf Gaertner': None,
   'Prof Fernand Gobet': 'http://chrest.info/fg/home.htm',
   'Dr. Daniel Guillery': 'https://www.danielguillery.com/',
   'Remco Heesen': 'https://remcoheesen.eu',
   'Isabel Lacurie': None,
   'Prof Christian List': 'http://personal.lse.ac.uk/list/',
   'Linnea Luuppala': 'https://www.linnealuuppala.com/%20',
   'Prof Anna Mahtani': 'http://personal.lse.ac.uk/mahtania/',
   'Becky Matthams': None,
   'Dr Marie Milofsky': None,
   'Dr Kieran Oberman': None,
   'Dr Jonathan Parry': None,
   'Prof Miklós Rédei': 'http://personal.lse.ac.uk/redeim/',
   'Bryan Roberts': 'http://personal.lse.ac.uk/ROBERT49/',
   'Dr. Paola Romero': None,
   'Lewis Ross': 'http://lewisdylanross.com/',
   'Dr Charles Sherwood': None,
   'Max Steuer': None,
   'Johanna Thoma': 'https://johannathoma.com/',
   'Mar Torró': None,
   'Nikhil Venkatesh': 'https://nikhilvenkateshphilosophy.com',
   'Prof Alex Voorhoeve': 'http://personal.lse.ac.uk/voorhoev/',
   'Kate Vredenburgh': 'https://katevredenburgh.com/',
   'Prof Charlotte Werndl': 'http://charlottewerndl.net/',
   'Prof John Worrall': 'http://johnworrall.org/',
   'Wesley Wrigley': 'https://www.wesleywrigley.com/',
   'Jingyi Wu': 'https://www.jingyiwu.org/'}},
 'Department of Psychological and Behavioural Science': {'PBS Management': {'Professor Liam Delaney': 'https://www.lse.ac.uk/PBS/People/Professor-Liam-Delaney',
   'Professor Bradley Franks': 'https://www.lse.ac.uk/PBS/People/Professor-Bradley-Franks',
   'Dr Alex Gillespie': 'https://www.lse.ac.uk/PBS/People/Professor-Alex-Gillespie',
   'Champa Heidbrink': 'https://www.lse.ac.uk/PBS/People/Champa-Heidbrink'},
  'Faculty': {'Dr Deema Awad': 'https://www.lse.ac.uk/PBS/People/Dr-Deema-Awad',
   'Dr Frédéric Basso': 'https://www.lse.ac.uk/PBS/People/Dr-Frederic-Basso',
   'Professor Martin W Bauer': 'https://www.lse.ac.uk/PBS/People/Professor-Martin-W-Bauer',
   'Dr Thomas Curran': 'https://www.lse.ac.uk/PBS/People/Dr-Thomas-Curran',
   'Professor Liam Delaney': 'https://www.lse.ac.uk/PBS/People/Professor-Liam-Delaney',
   'Professor Paul Dolan': 'https://www.lse.ac.uk/PBS/People/Professor-Paul-Dolan',
   'Professor Bradley Franks': 'https://www.lse.ac.uk/PBS/People/Professor-Bradley-Franks',
   'Dr Matteo M Galizzi': 'https://www.lse.ac.uk/PBS/People/Dr-Matteo-M-Galizzi',
   'Dr Lucia Garcia-Lorenzo': 'https://www.lse.ac.uk/PBS/People/Dr-Lucia-Garcia-Lorenzo',
   'Dr Alex Gillespie': 'https://www.lse.ac.uk/PBS/People/Professor-Alex-Gillespie',
   'Jeremy Ginges': 'https://www.lse.ac.uk/PBS/People/Professor-Jeremy-Ginges',
   'Dr Laura M Giurge': 'https://www.lse.ac.uk/PBS/People/Dr-Laura-M-Giurge',
   'Dr Ilka Gleibs\xa0': 'https://www.lse.ac.uk/PBS/People/Dr-Ilka-Gleibs',
   'Dr Maxi Heitmayer': 'https://www.lse.ac.uk/PBS/People/Dr-Maxi-Heitmayer',
   'Dr Ishan Jalan': 'https://www.lse.ac.uk/PBS/People/Dr-Ishan-Jalan',
   'Professor Sandra Jovchelovitch': 'https://www.lse.ac.uk/PBS/People/Professor-Sandra-Jovchelovitch',
   'Dr Christian Krekel': 'https://www.lse.ac.uk/PBS/People/Dr-Christian-Krekel',
   'Dr Dario Krpan': 'https://www.lse.ac.uk/PBS/People/Dr-Dario-Krpan',
   'Dr Kate Laffan': 'https://www.lse.ac.uk/PBS/People/Dr-Kate-Laffan',
   'Professor Saadi Lahlou': 'https://www.lse.ac.uk/PBS/People/Professor-Saadi-Lahlou',
   'Dr Grace Lordan': 'https://www.lse.ac.uk/PBS/People/Dr-Grace-Lordan',
   'Dr Jens Koed Madsen': 'https://www.lse.ac.uk/PBS/People/Dr-Jens-Koed-Madsen',
   'Dr Michael Muthukrishna': 'https://www.lse.ac.uk/PBS/People/Dr-Michael-Muthukrishna',
   'Dr Tom Reader': 'https://www.lse.ac.uk/PBS/People/Dr-Tom-Reader',
   'Dr Jet G.Sanders': 'https://www.lse.ac.uk/PBS/People/Dr-Jet-G.Sanders',
   'Dr Jennifer Sheehy-Skeffington': 'https://www.lse.ac.uk/PBS/People/Dr-Jennifer-Sheehy-Skeffington',
   'Dr Ganga Shreedhar': 'https://www.lse.ac.uk/PBS/People/Dr-Ganga-Shreedhar',
   'Professor Elizabeth Stokoe': 'https://www.lse.ac.uk/PBS/People/Professor-Elizabeth-Stokoe',
   'Dr Miriam Tresh': 'https://www.lse.ac.uk/PBS/People/Dr-Miriam-Tresh',
   'Alina Velias': 'https://www.lse.ac.uk/PBS/People/Alina-Velias',
   'Dr Edoardo Zulato': 'https://www.lse.ac.uk/PBS/People/Dr-Edoardo-Zulato'},
  'Professional Services Staff (PSS)': {'Gosia Brown': 'https://www.lse.ac.uk/PBS/People/Gosia-Brown',
   'Martha Clarke': None,
   'Karine Gay': None,
   'Carl Goodwin': 'https://www.lse.ac.uk/PBS/People/Carl-Goodwin',
   'Champa Heidbrink': 'https://www.lse.ac.uk/PBS/People/Champa-Heidbrink',
   'Therese Holmqvist': 'https://www.lse.ac.uk/PBS/People/Therese-Holmqvist',
   'Louise Millar': 'https://www.lse.ac.uk/PBS/People/Louise-Millar',
   'Salv Ridino': 'https://www.lse.ac.uk/PBS/People/Salv-Ridino',
   'Sean Rooney': None,
   'Will Stubbs': 'https://www.lse.ac.uk/PBS/People/Will-Stubbs',
   'Dimitris Thomopoulos': None},
  'Research Staff': {'Dr Bankole Falade': 'https://www.lse.ac.uk/PBS/People/Dr-Bankole-Falade',
   'Dr Ivan Kroupin': 'https://www.lse.ac.uk/PBS/People/Dr-Ivan-Kroupin',
   'Dr Georgios Melios': 'https://www.lse.ac.uk/PBS/People/Dr-Georgios-Melios',
   'Dr Maria Cecilia Dedios-Sanguineti': 'https://www.lse.ac.uk/PBS/People/Maria-Cecilia-Dedios-Sanguineti',
   'Rachel Spicer': 'https://www.lse.ac.uk/PBS/People/Rachel-Spicer'},
  'PhD Students': {'Karin Ahlbäck': 'https://www.lse.ac.uk/PBS/People/Karin-Ahlbäck',
   'Robin Bachmann': 'https://www.lse.ac.uk/PBS/People/Robin-Bachmann',
   'Aishwarya Bellam': 'https://www.lse.ac.uk/PBS/People/Aishwarya-Bellam',
   'Erika Brodnock': 'https://www.lse.ac.uk/PBS/People/Erika-Brodnock',
   'Julia Buzan': 'https://www.lse.ac.uk/PBS/People/Julia-Buzan',
   'Ivan Cano-Gomez': 'https://www.lse.ac.uk/PBS/People/Ivan-Cano-Gomez',
   'Jaesun Chae': 'https://www.lse.ac.uk/PBS/People/Jaesun-Chae',
   'Hannah Chappell': 'https://www.lse.ac.uk/PBS/People/Hannah-Chappell',
   'Alexandra Chesterfield': 'https://www.lse.ac.uk/PBS/People/Alexandra-Chesterfield',
   'Adam Davidson': 'https://www.lse.ac.uk/PBS/People/Adam-Davidson',
   "Dallas O'Dell": 'https://www.lse.ac.uk/PBS/People/Dallas-ODell',
   'Ethan Dias': 'https://www.lse.ac.uk/PBS/People/Ethan-Dias',
   'Virginia Fedrigo': 'https://www.lse.ac.uk/PBS/People/Virginia-Fedrigo',
   'Laura Fonseca-Durán': 'https://www.lse.ac.uk/PBS/People/Laura-Fonseca-Duran',
   'Alex Goddard': 'https://www.lse.ac.uk/PBS/People/Alex-Goddard',
   'Benno Guenther': 'https://www.lse.ac.uk/PBS/People/Benno-Guenther',
   'Julie Hald': 'https://www.lse.ac.uk/PBS/People/Julie-Hald',
   'Katharina Heby': 'https://www.lse.ac.uk/PBS/People/Katharina-Heby',
   'Juliet Hodges': 'https://www.lse.ac.uk/PBS/People/Juliet-Hodges',
   'Sharon Jackson': None,
   'Pinar Jenkins': 'https://www.lse.ac.uk/PBS/People/Pinar-Jenkins',
   'Bessi Jonsdottir': 'https://www.lse.ac.uk/PBS/People/Bessi-Jonsdottir',
   'Alexandra Kirienko': 'https://www.lse.ac.uk/PBS/People/Alexandra-Kirienko',
   'Fatima Koaik': 'https://www.lse.ac.uk/PBS/People/Fatima-Koaik',
   'Veronika Luptáková': 'https://www.lse.ac.uk/PBS/People/Veronika-Luptakova',
   'Nils Mallock': 'https://www.lse.ac.uk/PBS/People/Nils-Mallock',
   'Nikita': 'https://www.lse.ac.uk/tii/people/nikita',
   'Atrina Oraee': 'https://www.lse.ac.uk/PBS/People/Atrina-Oraee',
   'Alyssa Pandolfo': 'https://www.lse.ac.uk/PBS/People/Alyssa-Pandolfo',
   'Leah Jeongwon Park': 'https://www.lse.ac.uk/PBS/People/Leah-Jeongwon-Park',
   'Daniele Pollicino': 'https://www.lse.ac.uk/PBS/People/Daniele-Pollicino',
   'Pia Marie Pose': 'https://www.lse.ac.uk/PBS/People/Pia-Marie-Pose',
   'Sharon Raj': 'https://www.lse.ac.uk/PBS/People/Sharon-Raj',
   'Anandita Sabherwal': 'https://www.lse.ac.uk/PBS/People/Anandita-Sabherwal',
   'Eric Schnell': 'https://www.lse.ac.uk/PBS/People/Eric-Schnell',
   'Joseph Sherlock': 'https://www.lse.ac.uk/PBS/People/Joseph-Sherlock',
   'Haoran Shi': 'https://www.lse.ac.uk/PBS/People/Haoran-Shi',
   'Anisha Singh': 'https://www.lse.ac.uk/PBS/People/Anisha-Singh',
   'Andy Stewart': 'https://www.lse.ac.uk/PBS/People/Andy-Stewart',
   'Chen-Ta Sung': 'https://www.lse.ac.uk/PBS/People/Chen-Ta-Sung',
   'Audrey Van Hoecke': 'https://www.lse.ac.uk/PBS/People/Audrey-Van-Hoecke',
   'Feiyang Wang': 'https://www.lse.ac.uk/PBS/People/Feiyang-Wang',
   'Martin Whitehead': None,
   'Heidi Zamzow': 'https://www.lse.ac.uk/PBS/People/Heidi-Zamzow',
   'Katherine Ziegelbauer': 'https://www.lse.ac.uk/PBS/People/Katherine-Ziegelbauer'},
  'Emeritus Professors and Honorary Fellows': {'Professor Catherine Campbell': 'https://www.lse.ac.uk/PBS/People/Professor-Catherine-Campbell',
   'Professor George Gaskell': 'https://www.lse.ac.uk/Methodology/People/Academic-Staff/George-Gaskell/George-Gaskell',
   'Professor Patrick Humphreys': 'https://www.lse.ac.uk/PBS/People/Professor-Patrick-Humphreys',
   'Dr Janet Stockdale': 'https://www.lse.ac.uk/PBS/People/Dr-Janet-Stockdale'},
  'Visiting Fellows': {'Dr Kavita Abraham-Dowsing': 'https://www.lse.ac.uk/PBS/People/Dr-Kavita-Abraham-Dowsing',
   'Paul Adams': 'https://www.lse.ac.uk/PBS/People/Paul-Adams',
   'Nihan Albayrak-Aydemir': 'https://www.lse.ac.uk/PBS/People/Dr-Nihan-Albayrak-Aydemir',
   'dr sanchayan banerjee': 'https://www.lse.ac.uk/PBS/People/dr-sanchayan-banerjee',
   'Dr Denise Baron': 'https://www.lse.ac.uk/PBS/People/Dr-Denise-Baron',
   'Dr Natalia Concha': 'https://www.lse.ac.uk/PBS/People/Dr-Natalia-Concha',
   'Elisabeth Costa': 'https://www.lse.ac.uk/PBS/People/Elisabeth-Costa',
   'Dr Fangming Cui': 'https://www.lse.ac.uk/PBS/People/Dr-Fangming-Cui',
   'Dr Marta Entradas': 'https://www.lse.ac.uk/PBS/People/Dr-Marta-Entradas',
   'Professor Natalie Gold': 'https://www.lse.ac.uk/PBS/People/Professor-Natalie-Gold',
   'Dr Caroline Howarth': 'https://www.lse.ac.uk/PBS/People/Dr-Caroline-Howarth',
   'Dr Edward Hughes': 'https://www.lse.ac.uk/PBS/People/dr-edward-hughes',
   'Dr Anni Kajanus': 'https://www.lse.ac.uk/PBS/People/dr-anni-kajanus',
   'Dr Meena Kotecha': 'https://www.lse.ac.uk/PBS/People/Dr-Meena-Kotecha',
   'Dr Ploutarchos Kourtidis': 'https://www.lse.ac.uk/PBS/People/Dr-Ploutarchos-Kourtidis',
   'George Loewenstein': 'https://www.lse.ac.uk/PBS/People/george-loewenstein',
   'Dr Lucia Macchia': 'https://www.lse.ac.uk/PBS/People/Dr-Lucia-Macchia',
   'Professor Ivana Markova': 'https://www.lse.ac.uk/PBS/People/Professor-Ivana-Markova',
   'Dr Aleksandar Matic': 'https://www.lse.ac.uk/PBS/People/Dr-Aleksandar-Matic',
   'Ms Mary McLennan': None,
   'Dr Liora Moskovitz': 'https://www.lse.ac.uk/PBS/People/Dr-Liora-Moskovitz',
   'Dr Jeroen Nieboer': 'https://www.lse.ac.uk/PBS/People/Dr-Jeroen-Nieboer',
   'Dr Cathy Nicholson': 'https://www.lse.ac.uk/PBS/People/Dr-Cathy-Nicholson',
   'Dr Sandra Obradović': 'https://www.lse.ac.uk/PBS/People/Dr-Sandra-Obradovic',
   'Dr Celestin Okoroji': 'https://www.lse.ac.uk/PBS/People/Dr-Celestin-Okoroji',
   'Dr Marcelo Ramella': 'https://www.lse.ac.uk/PBS/People/Dr-Marcelo-Ramella',
   'Dr Jessica Rea': 'https://www.lse.ac.uk/PBS/People/Dr-Jessica-Rea',
   'Dr Barry Rogers': 'https://www.lse.ac.uk/PBS/People/Dr-Barry-Rogers',
   'Dr Gordon Sammut': 'https://www.lse.ac.uk/PBS/People/Dr-Gordon-Sammut',
   'Dr Lucia Sell-Trujillo': 'https://www.lse.ac.uk/PBS/People/Dr-Lucia-Sell-Trujillo',
   'Dr Ben Shenoy': 'https://www.lse.ac.uk/PBS/People/Dr-Ben-Shenoy',
   'Professor Ahmet Suerdem': 'https://www.lse.ac.uk/PBS/People/Professor-Ahmet-Suerdem',
   'Dr Umar Taj': 'https://www.lse.ac.uk/PBS/People/Dr-Umar-Taj',
   'Dr Chris Tennant': 'https://www.lse.ac.uk/PBS/People/Dr-Chris-Tennant',
   'Dr Alessia Tosi': 'https://www.lse.ac.uk/PBS/People/Dr-Alessia-Tosi',
   'Professor Ben G. Voyer': 'https://www.lse.ac.uk/PBS/People/Professor-Ben-G.-Voyer',
   'Dr Paulius Yamin-Slotkus': 'https://www.lse.ac.uk/PBS/People/Dr-Paulius-Yamin-Slotkus',
   'Scott Young': 'https://www.lse.ac.uk/PBS/People/Scott-Young'},
  'Graduate Teaching Assistants (GTAs)': {'Aishwarya Bellam': 'https://www.lse.ac.uk/PBS/People/Aishwarya-Bellam',
   'Lazaros': None,
   'Adam Davidson': 'https://www.lse.ac.uk/PBS/People/Adam-Davidson',
   'Alexandra Kirienko': 'https://www.lse.ac.uk/PBS/People/Alexandra-Kirienko',
   'Wangjingyi Liao': None,
   'Nils Mallock': 'https://www.lse.ac.uk/PBS/People/Nils-Mallock',
   'Atrina Oraee': 'https://www.lse.ac.uk/PBS/People/Atrina-Oraee',
   'Alyssa Pandolfo': 'https://www.lse.ac.uk/PBS/People/Alyssa-Pandolfo',
   'Sharon Raj': 'https://www.lse.ac.uk/PBS/People/Sharon-Raj',
   'Anandita Sabherwal': 'https://www.lse.ac.uk/PBS/People/Anandita-Sabherwal',
   'Haoran Shi': 'https://www.lse.ac.uk/PBS/People/Haoran-Shi',
   'Da’Quallon Smith': None,
   'Chen-Ta Sung': 'https://www.lse.ac.uk/PBS/People/Chen-Ta-Sung',
   'Wendy Wang': None,
   'Sayeh Yousefi': None,
   'Katherine Ziegelbauer': 'https://www.lse.ac.uk/PBS/People/Katherine-Ziegelbauer'},
  'Department committees ': {'Professor Bradley Franks': None,
   'Dr Frederic Basso': None,
   'Martha Clarke': None,
   'Dr Lucia Garcia-Lorenzo': None,
   'Karine Gay': None,
   'Champa Heidbrink': None,
   'Dr Christian Krekel': None,
   'Dr Dario Krpan': None,
   'Louise Millar': None,
   'Dr Michael Muthukrishna': None,
   'Will Stubbs': None,
   'Dimitris Thomopoulos': None,
   'Dr Miriam Tresh': None,
   'Dr Deema Awad': None,
   'Dr Thomas Curran': None,
   'Salv Ridino': None,
   'Dr Jens Madsen': None,
   'Sean Rooney': None,
   'Dr Jet Sanders': None,
   'Professor Martin Bauer': None,
   'Professor Liam Delaney': None,
   'Dr Matteo M Galizzi': None,
   'Professor Alex Gillespie': None,
   'Dr Laura M. Giurge': None,
   'Therese Holmqvist': None,
   'Dr Rachel Spicer': None,
   'Carl Goodwin': None,
   'Dr Ilka Gleibs': None,
   'Kate Laffan': None}},
 'School of Public Policy': {'Academic Leadership': {'Andrés Velasco': 'https://www.lse.ac.uk/school-of-public-policy/people/Andrés-Velasco',
   'Professor Tony Travers': 'https://www.lse.ac.uk/government/people/academic-staff/tony-travers',
   'Vanessa Rubio-Márquez': 'https://www.lse.ac.uk/school-of-public-policy/people/Vanessa-Rubio-Márquez',
   'Dr Joana Naritomi': 'https://www.lse.ac.uk/international-development/people/joana-naritomi',
   'Lloyd Gruber': 'https://www.lse.ac.uk/international-development/people/lloyd-gruber',
   'Professor Alexander Evans': 'https://www.lse.ac.uk/school-of-public-policy/people/Alexander-Evans',
   'Dr Simon Bastow': 'https://www.lse.ac.uk/school-of-public-policy/people/simon-bastow',
   'Charles Bean': 'https://www.lse.ac.uk/economics/people/faculty/charles-bean',
   'Dr Daniel Berliner': 'https://www.lse.ac.uk/government/people/academic-staff/daniel-berliner'},
  'Faculty': {'Professor Nava Ashraf': 'https://www.lse.ac.uk/economics/people/faculty/nava-ashraf',
   'Nicholas Barr': 'https://www.lse.ac.uk/european-institute/people/barr-nicholas',
   'Dr Simon Bastow': 'https://www.lse.ac.uk/school-of-public-policy/people/simon-bastow',
   'Tim Besley': 'https://www.lse.ac.uk/economics/people/faculty/tim-besley',
   'Professor Gwyn Bevan': 'https://www.lse.ac.uk/management/people/emeriti/gwyn-bevan',
   'Dr Gharad Bryan': 'https://www.lse.ac.uk/economics/people/faculty/gharad-bryan',
   'Professor Robin Burgess': 'https://www.lse.ac.uk/economics/people/faculty/robin-burgess',
   'Professor Stephan Chambers': 'https://www.lse.ac.uk/marshall-institute/people/schambers',
   'Richard Davies': 'https://www.lse.ac.uk/school-of-public-policy/people/Richard-Davies',
   'Jeremiah Dittmar': 'https://www.lse.ac.uk/economics/people/faculty/jeremiah-dittmar',
   'Professor Alexander Evans': 'https://www.lse.ac.uk/school-of-public-policy/people/Alexander-Evans',
   'Dr Tasha Fairfield': 'http://www.lse.ac.uk/international-development/people/tasha-fairfield',
   'Chenyang He': 'https://www.lse.ac.uk/school-of-public-policy/people/Luis-Garicano',
   'Professor Sir Julian Le Grand': 'http://www.lse.ac.uk/marshall-institute/people/jlegrand',
   'Lloyd Gruber': 'https://www.lse.ac.uk/international-development/people/lloyd-gruber',
   'Professor Rafael Hortala-Vallve': 'http://www.lse.ac.uk/government/people/academic-staff/rafael-hortala-vallve',
   'Dr Ethan Ilzetzki': 'https://www.lse.ac.uk/economics/people/faculty/ethan-ilzetzki',
   'Professor Stephen Jenkins': 'https://www.lse.ac.uk/social-policy/people/academic-staff/Professor-Stephen-Jenkins',
   'Casey Kearney': 'https://www.lse.ac.uk/school-of-public-policy/people/Casey-Kearney',
   'Adnan Khan': 'https://www.lse.ac.uk/school-of-public-policy/people/adnan-khan/adnan-khan',
   'Professor Valentino Larcinese': 'https://www.lse.ac.uk/government/people/academic-staff/valentino-larcinese',
   'Professor Richard Layard': 'https://cep.lse.ac.uk/_new/people/person.asp?id=970',
   'Professor Martin Lodge': 'http://www.lse.ac.uk/government/people/academic-staff/martin-lodge',
   'Frank Muci': 'https://www.lse.ac.uk/school-of-public-policy/people/frank-muci',
   'Dr Berkay Ozcan': 'https://www.lse.ac.uk/social-policy/people/academic-staff/Professor-Berkay-Ozcan',
   'Professor John Van Reenen': 'https://www.lse.ac.uk/economics/people/faculty/john-van-reenen',
   'Dr Jonathan Roberts': 'http://www.lse.ac.uk/marshall-institute/people/jroberts',
   'Dr Philipp Rode': 'https://www.lse.ac.uk/Cities/staff-profiles/Philipp-Rode',
   'Dr Lewis Ross': 'https://www.lse.ac.uk/research/research-for-the-world/profiles/lewis-ross',
   'Dr Sandra Sequeira': 'https://www.lse.ac.uk/international-development/people/sandra-sequeira',
   'Professor Mark Schankerman': 'http://www.lse.ac.uk/economics/people/faculty/mark-schankerman',
   'Dr Johannes Spinnewijn': 'https://www.lse.ac.uk/economics/people/faculty/johannes-spinnewijn',
   'Professor Daniel Sturm': 'https://www.lse.ac.uk/economics/people/faculty/daniel-sturm',
   'Andrés Velasco': 'https://www.lse.ac.uk/school-of-public-policy/people/Andrés-Velasco',
   'Dr Savvas Verdis': 'https://www.lse.ac.uk/business/consulting/experts/savvas-verdis',
   'Professor Alex Voorhoeve': 'https://www.lse.ac.uk/cpnss/people/alex-voorhoeve',
   'Dr Joachim Wehner': 'https://www.lse.ac.uk/government/people/academic-staff/joachim-wehner'},
  'Fellows and Teaching Fellows': {'Ignacio Banares-Sanchez': 'https://www.lse.ac.uk/economics/people/research-students/ignacio-banares-sanchez',
   'Dr Daniel Brieba': 'https://www.lse.ac.uk/school-of-public-policy/people/Daniel-Brieba',
   'Adam Brzezinski': None,
   'Dr Nuno F da Cruz': 'https://www.lse.ac.uk/cities/staff-profiles/Nuno-F.-da-Cruz',
   'Andres Fajardo-Ramirez': 'https://www.lse.ac.uk/economics/people/research-students/andres-fajardo-ramirez',
   'Miguel Fajardo-Steinhauser': 'https://www.lse.ac.uk/economics/people/research-students/miguel-fajardo-steinh%C3%A4user',
   'Dr Omar Hammoud Gallego': 'https://www.lse.ac.uk/school-of-public-policy/people/Omar-Hammoud-Gallego',
   'Marta Guasch-Rusinol': 'https://www.lse.ac.uk/economics/people/research-students/marta-guasch-rusi%C3%B1ol',
   'Daniel Guillery': 'https://www.lse.ac.uk/school-of-public-policy/people/guillery-daniel',
   'Aung Hein': 'https://www.lse.ac.uk/school-of-public-policy/people/hein-aung',
   'Aidan Hughes': None,
   'Kirstine La Cour': None,
   'Benedikt Margraf': 'https://www.lse.ac.uk/school-of-public-policy/people/Margraf-Benedikt',
   'Johannes Matt': 'https://www.lse.ac.uk/economics/people/research-students/johannes-matt',
   'Thomas Monk': 'https://www.lse.ac.uk/economics/people/research-students/thomas-monk',
   'Marta Morando': 'https://www.lse.ac.uk/economics/people/research-students/marta-morando',
   'Bernardo Mottironi': 'https://www.lse.ac.uk/economics/people/research-students/bernardo-mottironi',
   'Dr Zhamilya Mukasheva': 'https://www.lse.ac.uk/school-of-public-policy/people/Mukasheva-Zhamilya',
   'Canishk Naik': 'https://www.lse.ac.uk/economics/people/research-students/canishk-naik',
   'Dr Eugenia Nazrullaeva': 'https://www.lse.ac.uk/school-of-public-policy/people/Eugenia-Nazrullaeva',
   'Ekaterina Oparina': 'https://cep.lse.ac.uk/_new/people/person.asp?id=10813',
   'Michelle Rao': 'https://www.lse.ac.uk/economics/people/research-students/michelle-rao',
   'Veronica Salazar-Restrepo': 'https://www.lse.ac.uk/economics/people/research-students/veronica-salazar-restrepo',
   'Patrick Schneider\xa0': 'https://www.lse.ac.uk/economics/people/research-students/patrick-schneider',
   'Jayatheerth Seethraman\xa0': 'https://www.lse.ac.uk/school-of-public-policy/people/jayatheerth-Seetharaman',
   'Pol Simpson': 'https://www.lse.ac.uk/economics/people/research-students/pol-simpson',
   'Ameek Singh': 'https://www.lse.ac.uk/economics/people/research-students/ameek-singh',
   'Caterina Soto-Vieira': 'https://www.lse.ac.uk/economics/people/research-students/caterina-soto-vieira',
   'Thomas Stephens': 'https://www.lse.ac.uk/social-policy/people/research-students/thomas-stephens',
   'Hamza Syed': 'https://www.lse.ac.uk/economics/people/research-students/hamza-syed',
   'Peter Ward-Griffin': 'https://www.lse.ac.uk/economics/people/research-students/peter-ward-griffin',
   'Linchuan Xu': 'https://www.lse.ac.uk/school-of-public-policy/people/xu-linchuan',
   'Yuanhang': 'https://cep.lse.ac.uk/_new/people/person.asp?id=11176',
   'Dr Tamar Zeilberger': None},
  'Visiting Appointments': {'Rt Hon Sir Vince Cable': 'https://www.lse.ac.uk/school-of-public-policy/people/vince-cable',
   'Hugh Cole': 'https://www.lse.ac.uk/school-of-public-policy/people/Hugh-Cole',
   'Dimitri Demekas': 'https://www.lse.ac.uk/school-of-public-policy/people/Dimitri-Demekas',
   'Ricardo Hausmann': 'https://www.lse.ac.uk/school-of-public-policy/people/ricardo-hausmann',
   'Laura Gilbert': 'https://www.lse.ac.uk/school-of-public-policy/people/gilbert-laura',
   'Valeria Gontareva': 'https://www.lse.ac.uk/school-of-public-policy/people/Valeria-Gontareva',
   'Chenyang He': 'https://www.lse.ac.uk/school-of-public-policy/people/Chenyang-He',
   'Tim Leunig': 'https://www.lse.ac.uk/school-of-public-policy/people/Tim-Leunig',
   'Ousmène Jacques Mandeng': 'https://www.lse.ac.uk/school-of-public-policy/people/ousmène-jacques-mandeng',
   'Geoffrey Myers': 'https://www.lse.ac.uk/school-of-public-policy/people/Geoffrey-Myers',
   'Dzhamilya Nigmatulina': 'https://www.lse.ac.uk/school-of-public-policy/people/Dzhamilya-Nigmatulina',
   'Charles Okehalam': 'https://www.lse.ac.uk/school-of-public-policy/people/Charles-Okehalam',
   'Lant Pritchett': 'https://www.lse.ac.uk/school-of-public-policy/people/Lant-Pritchett',
   'Valerio Riavez': 'https://www.lse.ac.uk/school-of-public-policy/people/Valerio-Riavez',
   'Michael Reid': 'https://www.lse.ac.uk/school-of-public-policy/people/michael-reid',
   'Dr Christopher Sabatini ': 'https://www.lse.ac.uk/school-of-public-policy/people/Christopher-Sabatini',
   'Christopher Schildt': 'https://www.lse.ac.uk/school-of-public-policy/people/schildt-christopher',
   'Elod Takats': 'https://www.lse.ac.uk/school-of-public-policy/people/Elod-Takats',
   'Tinghua Yu': 'https://www.lse.ac.uk/school-of-public-policy/people/Tinghua-Yu'},
  'Skills Associates': {'Philip Colins': None},
  'LSE Cities Executive Group': {'Professor Ricky Burdett': 'https://www.lse.ac.uk/sociology/people/ricky-burdett',
   'Dr Philipp Rode': 'https://www.lse.ac.uk/Cities/staff-profiles/Philipp-Rode',
   'Professor Jo Beall': 'https://www.lse.ac.uk/Cities/staff-profiles/Jo-Beall'},
  'LSE-Fudan\xa0Global Public Policy Hub ': {'Dr Binchung Meng': 'https://www.lse.ac.uk/media-and-communications/people/academic-staff/bingchun-meng',
   'Dr Timothy Hildebrandt': 'https://www.lse.ac.uk/social-policy/people/academic-staff/Dr-Timothy-Hildebrandt',
   'Dr Tanvi Deshpande': 'https://www.lse.ac.uk/school-of-public-policy/people/Tanvi-Deshpande',
   'Yuezhou Yang': 'https://www.lse.ac.uk/school-of-public-policy/people/Yuezhou-Yang',
   'Linnea Luuppala': 'https://www.lse.ac.uk/school-of-public-policy/people/Linnea-Luuppala'},
  'Growth Co-Lab at LSE': {'Richard Davies': 'https://www.lse.ac.uk/school-of-public-policy/people/Richard-Davies',
   'Rahat Siddique': 'https://www.lse.ac.uk/school-of-public-policy/people/Rahat-Siddique',
   'Charlie Meyrick': None,
   'Andrea Correa': 'https://www.lse.ac.uk/school-of-public-policy/people/Correa-Andrea',
   'Camila Arroyo From': 'https://www.lse.ac.uk/school-of-public-policy/people/Camila-Arroyo-From',
   'Sheyla Enciso-Valdivia': 'https://www.lse.ac.uk/school-of-public-policy/people/Sheyla-Enciso',
   'Fernando Pino': None,
   'Josh Hellings': None,
   'Finn McEvoy': None},
  'Professional Services Staff': {'James Deeley': 'https://www.lse.ac.uk/school-of-public-policy/people/James-Deeley',
   'Jim Osborne': 'https://www.lse.ac.uk/school-of-public-policy/people/jim-osborne',
   'pilar-frogley': 'https://www.lse.ac.uk/school-of-public-policy/people/pilar-frogley',
   'Caroline Boswell': 'https://www.lse.ac.uk/school-of-public-policy/people/caroline-milton',
   'Michelle Batten': None,
   'Victoria Bryant': None,
   'Teresa Chung': None,
   'Heather Gorrie': None,
   'Zuzana Navratilova': None,
   'Emma Nevell': None,
   'Andrew Park': None,
   'Alex Rose': None,
   'Jemima Warren': 'https://www.lse.ac.uk/school-of-public-policy/people/Jemima-Swallow',
   'Verity Wilkes': None,
   'Doreen Thompson-Addo': 'https://www.lse.ac.uk/school-of-public-policy/people/Doreen-Thompson-Addo',
   'Polly Liouta': 'https://www.lse.ac.uk/school-of-public-policy/people/liouta-polly',
   'Linnea Luuppala': 'https://www.lse.ac.uk/school-of-public-policy/people/Linnea-Luuppala',
   'Laura Malouf': 'https://www.lse.ac.uk/school-of-public-policy/people/Malouf-Laura',
   'Delphine Polidori': 'https://www.lse.ac.uk/school-of-public-policy/people/polidori-delphine',
   'Hannah Shearer': 'https://www.lse.ac.uk/school-of-public-policy/people/Hannah-Shearer',
   'carolina-stern': 'https://www.lse.ac.uk/school-of-public-policy/people/carolina-stern',
   'Andrew Brennen': 'https://www.lse.ac.uk/school-of-public-policy/people/andrew-brennen',
   'Georgina Whitham': 'https://www.lse.ac.uk/school-of-public-policy/people/georgina-whitham',
   'Sarah-Massey': 'https://www.lse.ac.uk/school-of-public-policy/people/Sarah-Massey',
   'Irene Bucelli': 'https://www.lse.ac.uk/school-of-public-policy/people/Bucelli-Irene'}},
 'Department of Social Policy': {'Academic staff': {'Dr Fabio Battaglia': 'https://www.lse.ac.uk/social-policy/people/academic-staff/dr-fabio-battaglia',
   'Dr Liam Beiser-McGrath': 'https://www.lse.ac.uk/social-policy/people/academic-staff/dr-liam-beiser-mcgrath',
   'Dr Thomas Biegert': 'https://www.lse.ac.uk/social-policy/people/academic-staff/Dr-Thomas-Biegert',
   'Dr Tania Burchardt': 'https://www.lse.ac.uk/social-policy/people/academic-staff/Dr-Tania-Burchardt',
   'Dr Shekhar Chandra': 'https://www.lse.ac.uk/social-policy/people/academic-staff/Dr-Shekhar-Chandra',
   'Dr Leonidas Cheliotis': 'https://www.lse.ac.uk/social-policy/people/academic-staff/Dr-Leonidas-Cheliotis',
   'Dr Shuang Chen': 'https://www.lse.ac.uk/social-policy/people/academic-staff/dr-shuang-chen',
   'Dr Sonia Exley': 'https://www.lse.ac.uk/social-policy/people/academic-staff/Dr-Sonia-Exley',
   'Professor Timo Fleckenstein': 'https://www.lse.ac.uk/social-policy/people/academic-staff/Professor-Timo-Fleckenstein',
   'Fiona Gogescu': 'https://www.lse.ac.uk/social-policy/people/research-students/Fiona-Gogescu',
   'Dr Timothy Hildebrandt': 'https://www.lse.ac.uk/social-policy/people/academic-staff/Dr-Timothy-Hildebrandt',
   'Dr Mobarak Hossain': 'https://www.lse.ac.uk/social-policy/people/academic-staff/dr-mobarak-hossain',
   'Professor Armine Ishkanian': 'https://www.lse.ac.uk/social-policy/people/academic-staff/Professor-Armine-Ishkanian',
   'Professor Stephen Jenkins': 'https://www.lse.ac.uk/social-policy/people/academic-staff/Professor-Stephen-Jenkins',
   'Dr Johann Koehler': 'https://www.lse.ac.uk/social-policy/people/academic-staff/Dr-Johann-Koehler',
   'Dr Sunil Kumar': 'https://www.lse.ac.uk/social-policy/people/academic-staff/Dr-Sunil-Kumar',
   'Dr Youngcho Lee': 'https://www.lse.ac.uk/social-policy/people/academic-staff/dr-youngcho-lee',
   'Professor Sir Julian Le Grand': 'http://www.lse.ac.uk/marshall-institute/people/jlegrand',
   'Dr Zahid Mumtaz': 'https://www.lse.ac.uk/social-policy/people/academic-staff/dr-zahid-mumtaz',
   'Professor Tim Newburn': 'https://www.lse.ac.uk/social-policy/people/academic-staff/Professor-Tim-Newburn',
   'Professor Adam Oliver': 'https://www.lse.ac.uk/social-policy/people/academic-staff/Professor-Adam-Oliver',
   'Professor Berkay Ozcan': 'https://www.lse.ac.uk/social-policy/people/academic-staff/Professor-Berkay-Ozcan',
   'Dr Robtel Neajai Pailey': 'https://www.lse.ac.uk/social-policy/people/academic-staff/dr-robtel-neajai-pailey',
   'Professor Coretta Phillips': 'https://www.lse.ac.uk/social-policy/people/academic-staff/Professor-Coretta-Phillips',
   'Professor Lucinda Platt': 'https://www.lse.ac.uk/social-policy/people/academic-staff/Professor-Lucinda-Platt',
   'Dr Virgi Sari': 'https://www.lse.ac.uk/social-policy/people/academic-staff/dr-virgi-sari',
   'Dr Hakan Seckinelgin': 'https://www.lse.ac.uk/social-policy/people/academic-staff/Dr-Hakan-Seckinelgin',
   'Professor Almudena Sevilla': 'https://www.lse.ac.uk/social-policy/people/academic-staff/Professor-Almudena-Sevilla',
   'Dr Michael Shiner': 'https://www.lse.ac.uk/social-policy/people/academic-staff/Dr-Michael-Shiner',
   'Dr Isabel Shutes': 'https://www.lse.ac.uk/social-policy/people/academic-staff/Dr-Isabel-Shutes',
   'Dr Kitty Stewart': 'https://www.lse.ac.uk/social-policy/people/academic-staff/Dr-Kitty-Stewart',
   'Dr Margaux Suteau': 'https://www.lse.ac.uk/social-policy/people/academic-staff/Margaux-Suteau',
   'Dr Iva Tasseva': 'https://www.lse.ac.uk/social-policy/people/academic-staff/dr-iva-tasseva',
   'Professor Anne West': 'https://www.lse.ac.uk/social-policy/people/academic-staff/Professor-Anne-West'},
  'Affiliated staff': {'Professor Francisco H. G. Ferreira': 'https://www.lse.ac.uk/International-Inequalities/People/Francisco-H.-G.-Ferreira'},
  'Professional services staff': {'Damian Roberts': None,
   'Clare Gorman': None,
   'Sevilay Erdogan': None,
   'Adrian Knight': None,
   'Dr Joe Macdonald': None,
   'Nathaniel Ocquaye': None,
   'Maria Schlegel': None,
   'Dr Craig Stewart': None,
   'Jake Watkins': None,
   'Dr Hao Wu': None},
  'Research Staff and Clusters': {'Raegan Booth': 'https://www.lse.ac.uk/social-policy/people/research-staff/raegan-booth',
   'Pilar Cuevas-Ruiz': 'https://www.lse.ac.uk/social-policy/people/research-staff/Pilar-Cuevas-Ruiz'},
  'Guest teachers': {'Ritika Arora': None,
   'Chiara Chiavaroli': 'https://www.lse.ac.uk/international-development/people/research-students/Chiara-Chiavaroli',
   'Joel Chong': 'https://www.lse.ac.uk/government/people/research-students/joel-chong',
   'Robert Falconer': 'https://www.lse.ac.uk/social-policy/people/research-students/robert-falconer',
   'Franco Fernandez Fleming': None,
   'Maria Karagiannidou': 'https://www.lse.ac.uk/cpec/people/maria-karagiannidou',
   'Roni\xa0Küppers': 'https://www.lse.ac.uk/Methodology/People/Research-Students/Roni-K%C3%BCppers/Roni-K%C3%Bcppers',
   'Julie Lespinasse': 'https://www.lse.ac.uk/international-relations/phd-students/lespinasse-julie',
   'Joaquin Mayora Camus': 'https://www.lse.ac.uk/social-policy/people/research-students/Joaqu%C3%ADn-Mayorga-Camus',
   'Manna Mostaghim': None},
  'Visiting Staff': {'Professor Waqar Ahmad': None,
   'Dr Piera Bello': 'https://www.lse.ac.uk/social-policy/people/visiting-staff/dr-piera-bello',
   'Dr Mike Brewer': 'https://www.lse.ac.uk/social-policy/people/visiting-staff/dr-mike-brewer',
   'Dr Tymofii Brik': 'https://www.lse.ac.uk/social-policy/people/visiting-staff/dr-tymofii-brik',
   'Dr Alessandra Casarico': 'https://www.lse.ac.uk/social-policy/people/visiting-staff/dr-alessandra-casarico',
   'Dr Francesca Foliano': 'https://www.lse.ac.uk/social-policy/people/visiting-staff/Dr-Francesca-Foliano',
   'Professor Paul Frijters': 'https://www.lse.ac.uk/social-policy/people/visiting-staff/professor-paul-frijters',
   'Professor Cecilia Garcia Penalosa': 'https://www.lse.ac.uk/social-policy/people/visiting-staff/professor-cecilia-garcia-penalosa',
   'Dr Tony Hockley': 'https://www.lse.ac.uk/social-policy/people/visiting-staff/Dr-Tony-Hockley',
   'Dr Vanessa Hughes': 'https://www.lse.ac.uk/social-policy/people/visiting-staff/Dr-Vanessa-Hughes',
   'Dr Ching Leong': 'https://www.lse.ac.uk/social-policy/people/visiting-staff/dr-ching-leong',
   'Dr Greta Morando': 'https://www.lse.ac.uk/social-policy/people/visiting-staff/dr-greta-morando',
   'Dr Chiara Orsini': 'https://www.lse.ac.uk/social-policy/people/visiting-staff/dr-chiara-orsini',
   'Dr Deborah Outhwaite': 'https://www.lse.ac.uk/social-policy/people/visiting-staff/Dr-Deborah-Outhwaite',
   'Dr Lynette Rawlings': 'https://www.lse.ac.uk/social-policy/people/visiting-staff/dr-lynette-rawlings',
   'Professor Ismael Sanz': 'https://www.lse.ac.uk/social-policy/people/visiting-staff/professor-ismael-sanz',
   'Dr Amanda Sheely': 'https://www.lse.ac.uk/social-policy/people/visiting-staff/Dr-Amanda-Sheely'},
  'Emeritus': {'Dr John Carrier': 'https://www.lse.ac.uk//social-policy/people/Emeritus/Dr-John-Carrier',
   'Professor Hartley Dean': 'https://www.lse.ac.uk//social-policy/people/Emeritus/Professor-Hartley-Dean',
   'Professor David Downes': 'https://www.lse.ac.uk//social-policy/people/Emeritus/Professor-David-Downes',
   'Professor Howard Glennerster': 'https://www.lse.ac.uk//social-policy/people/Emeritus/Professor-Howard-Glennerster',
   'Professor Jane Lewis': 'https://www.lse.ac.uk//social-policy/people/Emeritus/Professor-Jane-Lewis',
   'Professor Eileen Munro': 'https://www.lse.ac.uk//social-policy/people/Emeritus/Professor-Eileen-Munro',
   'Professor Michael Murphy': 'https://www.lse.ac.uk//social-policy/people/Emeritus/Professor-Michael-Murphy',
   'Professor David Piachaud': None,
   'Professor Anne Power': 'https://www.lse.ac.uk//social-policy/people/Emeritus/Professor-Anne-Power',
   'Dr Gail Wilson': None},
  'Head': {'Professor Stephen Jenkins': 'http://www.lse.ac.uk/social-policy/people/academic-staff/Professor-Stephen-Jenkins',
   'Dr Leonidas Cheliotis': 'https://www.lse.ac.uk/social-policy/people/academic-staff/Dr-Leonidas-Cheliotis',
   'Damian Roberts': None}},
 'Department of Sociology': {'Academic staff': {'Mahvish Ahmad': 'https://www.lse.ac.uk/sociology/people/mahvish-ahmad',
   'Suki Ali': 'https://www.lse.ac.uk/sociology/people/suki-ali',
   'Robin Archer': 'https://www.lse.ac.uk/sociology/people/robin-archer',
   'Chetan Bhatt': 'https://www.lse.ac.uk/sociology/people/chetan-bhatt',
   'Ayça Çubukçu': 'https://www.lse.ac.uk/sociology/people/ayca-cubukcu',
   'Rebecca Elliott': 'https://www.lse.ac.uk/sociology/people/rebecca-elliott',
   'Janet Foster': 'https://www.lse.ac.uk/sociology/people/janet-foster',
   'Sam Friedman': 'https://www.lse.ac.uk/sociology/people/sam-friedman',
   'Carrie Friese': 'https://www.lse.ac.uk/sociology/people/carrie-friese',
   'Suzi Hall': 'https://www.lse.ac.uk/sociology/people/suzi-hall',
   'Ursula Henz': 'https://www.lse.ac.uk/sociology/people/ursula-henz',
   'Monika Krause': 'https://www.lse.ac.uk/sociology/people/monika-krause',
   'David Madden': 'https://www.lse.ac.uk/sociology/people/david-madden',
   'Patrick McGovern': 'https://www.lse.ac.uk/sociology/people/patrick-mcgovern',
   'Jana Melkumova-Reynolds': 'https://www.lse.ac.uk/sociology/people/jana-melkumova-reynolds',
   'Claire Moon': 'https://www.lse.ac.uk/sociology/people/claire-moon',
   'David Pinzur': 'https://www.lse.ac.uk/sociology/people/david-pinzur',
   'Olivia Umurerwa Rutazibwa': 'https://www.lse.ac.uk/sociology/people/olivia-rutazibwa',
   'Sara Salem': 'https://www.lse.ac.uk/sociology/people/sara-salem',
   'Mike Savage': 'https://www.lse.ac.uk/sociology/people/mike-savage',
   'Don Slater': 'https://www.lse.ac.uk/sociology/people/don-slater',
   'Kristin Surak': 'https://www.lse.ac.uk/sociology/people/kristin-surak',
   'Fran Tonkiss': 'https://www.lse.ac.uk/sociology/people/fran-tonkiss',
   'Mai Taha': 'https://www.lse.ac.uk/sociology/people/mai-taha'},
  'Research staff': {'Ioanna Gouseti': 'https://www.lse.ac.uk/sociology/people/ioanna-gouseti',
   'baljit kaur': 'https://www.lse.ac.uk/sociology/people/baljit-kaur',
   'Lareau, Annette': None},
  'LSE Fellows and Course Tutors': {'Qilyu Hong': 'https://www.lse.ac.uk/sociology/people/qilyu-hong',
   'Nils Peters': 'https://www.lse.ac.uk/sociology/people/nils-peters',
   'Dena Qaddumi': 'https://www.lse.ac.uk/sociology/people/dena-qaddumi',
   'Shaheen, Faiza': 'https://www.lse.ac.uk/sociology/people/hakan-sandal-wilson',
   'Sandal-Wilson, Hakan': 'https://www.lse.ac.uk/sociology/people/hakan-sandal-wilson',
   'Leili Sreberny-Mohammadi': 'https://www.lse.ac.uk/sociology/people/leili-sreberny-mohammadi'},
  'Emeritus': {'Christopher Robert Badcock': 'https://www.lse.ac.uk/sociology/people/christopher-robert-badcock',
   'Eileen Barker': 'https://www.lse.ac.uk/sociology/people/eileen-barker',
   'Tony Giddens': 'https://www.lse.ac.uk/sociology/people/tony-giddens',
   'frances-heidensohn': 'https://www.lse.ac.uk/sociology/people/frances-heidensohn',
   'Christopher T Husbands': 'https://www.lse.ac.uk/sociology/people/christopher-husbands',
   'Paul Rock': 'https://www.lse.ac.uk/sociology/people/paul-rock',
   'Leslie Sklair': 'https://www.lse.ac.uk/sociology/people/leslie-sklair',
   'Charis Thompson': 'https://www.lse.ac.uk/sociology/people/charis-thompson',
   'Judy Wajcman FBA': 'https://www.lse.ac.uk/sociology/people/judy-wajcman'},
  'Professional services staff': {'Borowska, Emilia': None,
   'Christofferson, Andrew': None,
   'Clarke, Evie': None,
   'Codjoe, Vivienne': None,
   'Giles, Maddie': None,
   'Lawrence, Louisa': None,
   'Lyons, Clara': None,
   'Parvez, Adeel': None,
   'Rahman, Yadna': None,
   'Tomlinson, Claire': None,
   'Wain, Laura': None,
   'Walker, Monika': None},
  'Research students': {'Maya Adereth': 'https://www.lse.ac.uk/sociology/people/research-students/maya-adereth/maya-adereth',
   'christiana-ajai-thomas': 'https://www.lse.ac.uk/sociology/people/research-students/christiania-ajai-thomas',
   'Akdag, Muhammed': None,
   'Özgün Aksakal': 'https://www.lse.ac.uk/sociology/people/research-students/özgün-aksakal',
   'Amitay, Nitzan': None,
   'Nicolás Arenas': 'https://www.lse.ac.uk/sociology/people/research-students/nicolás-arenas/nicolás-arenas',
   'Rajesh Bhattacherjee': 'https://www.lse.ac.uk/sociology/people/research-students/rajesh-bhattacherjee',
   'Amani Bhobe': 'https://www.lse.ac.uk/sociology/people/research-students/amani-bhobe',
   'Dan Brown': 'https://www.lse.ac.uk/sociology/people/research-students/dan-brown/dan-brown',
   'Brundu-Gonzalez, Benjamin': None,
   'Asif Butt': 'https://www.lse.ac.uk/sociology/people/research-students/asif-butt',
   'Donna Carmichael': 'https://www.lse.ac.uk/sociology/people/research-students/donna-carmichael/donna-carmichael',
   'Iulia-Clara Cirdan': 'https://www.lse.ac.uk/sociology/people/research-students/iulia-clara-cirdan/iulia-clara-cirdan',
   'Rodrigo Círigo-Jiménez': 'https://www.lse.ac.uk/sociology/people/research-students/rodrigo-cirigo/rodrigo-círigo-jiménez',
   'Sigrid Corry': 'https://www.lse.ac.uk/sociology/people/research-students/sigrid-corry',
   'Cutts, Danielle': None,
   'Sherelle Davids': 'https://www.lse.ac.uk/sociology/people/research-students/sherelle-davids',
   'Doğukan Dere': 'https://www.lse.ac.uk/sociology/people/research-students/doğukan-dere',
   'Shanae Dyer': 'https://www.lse.ac.uk/sociology/people/research-students/shanae-dyer/shanae-dyer',
   'Lucy Garbett': 'https://www.lse.ac.uk/sociology/people/research-students/lucy-garbett',
   'Jan Gilles': 'https://www.lse.ac.uk/sociology/people/research-students/jan-gilles',
   'Gray, Selena': None,
   'Adam Greenfield': 'https://www.lse.ac.uk/sociology/people/research-students/adam-greenfield',
   'Victoria Gronwald': 'https://www.lse.ac.uk/sociology/people/research-students/victoria-gronwald/victoria-gronwald',
   'Sacha Hilhorst': 'https://www.lse.ac.uk/sociology/people/research-students/sacha-hilhorst/sacha-hilhorst',
   'Johannes Hollenhorst': 'https://www.lse.ac.uk/sociology/people/research-students/johannes-hollenhorst',
   'Will Kendall': 'https://www.lse.ac.uk/sociology/people/research-students/will-kendall/will-kendall',
   'Kherfi, Yasmine': None,
   'Lally, Amraj': 'http://www.amrajlally.co.uk',
   'Helen Mackreath': 'https://www.lse.ac.uk/sociology/people/research-students/helen-mackreath/helen-mackreath',
   'Hobeth I Martínez Carrillo': 'https://www.lse.ac.uk/sociology/people/research-students/hobeth-i-martínez-carrillo',
   'Babette May': 'https://www.lse.ac.uk/sociology/people/research-students/babette-may',
   'Jack McGinn': 'https://www.lse.ac.uk/sociology/people/research-students/jack-mcginn/jack-mcginn',
   'Anaïde Nahikian': 'https://www.lse.ac.uk/sociology/people/research-students/anaïde-nahikian',
   'Noura Nasser': 'https://www.lse.ac.uk/sociology/people/research-students/noura-nasser/Noura-Nasser',
   'Nichols, Georgia': None,
   'Marta Pagnini': 'https://www.lse.ac.uk/sociology/people/research-students/marta-pagnini/marta-pagnini',
   'Daniela Perez Aguilar': 'https://www.lse.ac.uk/sociology/people/research-students/daniela-perez-aguilar',
   'Matt Reynolds': 'https://www.lse.ac.uk/sociology/people/research-students/matt-reynolds/matt-reynolds',
   'Carla Rivera Blanco': 'https://www.lse.ac.uk/sociology/people/research-students/carla-rivera-blanco',
   'Rémy-Paulin Twahirwa': 'https://www.lse.ac.uk/sociology/people/research-students/rémy-paulin-twahirwa/rémy-paulin-twahirwa',
   'Neil Warner': 'https://www.lse.ac.uk/sociology/people/research-students/neil-warner/neil-warner',
   'Hannah Wilson': 'https://www.lse.ac.uk/sociology/people/research-students/hannah-wilson',
   'Jawaher Al Sudairy': 'https://www.lse.ac.uk/sociology/people/research-students/jawaher-al-sudairy',
   'Anthony Miro Born': 'https://www.lse.ac.uk/sociology/people/research-students/anthony-miro-born',
   'Harris, Gabrielle': None,
   'Kampmann, David': None,
   'Tim White': 'https://www.lse.ac.uk/sociology/people/research-students/tim-white'},
  'Academic visitors': {'Bordonaro, Elettra': None,
   'Dasgupta, Rohit K': None,
   'Gabriel Eidelman': 'https://www.lse.ac.uk/sociology/people/academic-visitors/gabriel-eidelman/gabriel-eidelman',
   'Ghaziani, Amin': None,
   'Miriam Glucksmann': 'https://www.lse.ac.uk/sociology/people/academic-visitors/miriam-glucksmann/miriam-glucksmann',
   'Hen-Smith, May': None,
   'Kampmann, David': None,
   'Kajta, Justyna': None,
   'Kramer, Maria': None,
   'Küçük, Bermal': None,
   'Ian Patel': 'https://www.lse.ac.uk/sociology/people/ian-patel',
   'Perolini, Marco': None,
   'Thierry Rossier': 'https://www.lse.ac.uk/sociology/people/academic-visitors/thierry-rossier/thierry-rossier',
   'Svetlana Ruseishvili': 'https://www.lse.ac.uk/sociology/people/academic-visitors/svetlana-ruseishvili/svetlana-ruseishvili',
   'Taylor, Emma': None,
   'Watt, Paul': None,
   'Zherebkin, Sergey V.': None,
   'De Keere, Kobe': None},
  'Afilliated Research Fellows': {'Mallett, Victoria': None,
   'McCurdy, Martha': None,
   'Partyga, Dominika': None,
   'White, Tim': None}},
 'Department of Statistics': {'Academic faculty': {'James Abdey': 'https://www.lse.ac.uk/statistics/people/james-abdey',
   'Mona Azadkia': 'https://www.lse.ac.uk/statistics/people/mona-azadkia',
   'Marcos Barreto': 'https://www.lse.ac.uk/statistics/people/marcos-barreto',
   'Pauline Barrieu': 'https://www.lse.ac.uk/statistics/people/pauline-barrieu',
   'Erik Baurdoux': 'https://www.lse.ac.uk/statistics/people/erik-baurdoux',
   'Professor Wicher Bergsma': 'https://www.lse.ac.uk/statistics/people/wicher-bergsma',
   'Umut Cetin': 'https://www.lse.ac.uk/statistics/people/umut-cetin',
   'Yining Chen': 'https://www.lse.ac.uk/statistics/people/yining-chen',
   'Yunxiao Chen': 'https://www.lse.ac.uk/statistics/people/yunxiao-chen',
   'Professor Angelos Dassios': 'https://www.lse.ac.uk/statistics/people/angelos-dassios',
   'Piotr Fryzlewicz': 'https://www.lse.ac.uk/statistics/people/piotr-fryzlewicz',
   'Sara Geneletti': 'https://www.lse.ac.uk/statistics/people/sara-geneletti',
   'Kostas Kalogeropoulos': 'https://www.lse.ac.uk/statistics/people/kostas-kalogeropoulos',
   'Kostas Kardaras': 'https://www.lse.ac.uk/statistics/people/kostas-kardaras',
   'Jouni Kuha': 'https://www.lse.ac.uk/statistics/people/jouni-kuha',
   'Clifford Lam': 'https://www.lse.ac.uk/statistics/people/clifford-lam',
   'Pik Liew': 'https://www.lse.ac.uk/statistics/people/pik-liew',
   'Giulia Livieri': 'https://www.lse.ac.uk/statistics/people/giulia-livieri',
   'Joshua Loftus': 'https://www.lse.ac.uk/statistics/people/joshua-loftus',
   'Gelly Mitrodima': 'https://www.lse.ac.uk/statistics/people/gelly-mitrodima',
   'Irini Moustaki': 'https://www.lse.ac.uk/statistics/people/irini-moustaki',
   'Francesca Panero': 'https://www.lse.ac.uk/statistics/people/francesca-panero',
   '\xa0Xinghao Qiao': 'https://www.lse.ac.uk/statistics/people/xinghao-qiao',
   'Chengchun Shi': 'https://www.lse.ac.uk/statistics/people/chengchun-shi',
   'Andreas Søjmark': 'https://www.lse.ac.uk/statistics/people/andreas-søjmark',
   'Fiona Steele': 'https://www.lse.ac.uk/statistics/people/fiona-steele',
   'Zoltan Szabo': 'https://www.lse.ac.uk/statistics/people/zoltan-szabo',
   'Milan Vojnović': 'https://www.lse.ac.uk/statistics/people/milan-vojnovic',
   'Tengyao Wang': 'https://www.lse.ac.uk/statistics/people/tengyao-wang',
   'Qiwei Yao': 'https://www.lse.ac.uk/statistics/people/qiwei-yao',
   'Christine Yuen': 'https://www.lse.ac.uk/statistics/people/christine-yuen',
   'Xiaolin Zhu': 'https://www.lse.ac.uk/statistics/people/xiaolin-zhu'},
  'Academic associates -\xa0Emeritus Professors, Guest Lecturers, Fellows and Visiting staff ': {'Anthony Atkinson': 'https://www.lse.ac.uk/statistics/people/anthony-atkinson',
   'Dr Camilo Cárdenas Hurtado': 'https://www.lse.ac.uk/statistics/people/camilo-cardenas-hurtado',
   'Phil Chan': 'https://www.lse.ac.uk/statistics/people/phil-chan',
   'Yudong Chen': 'https://www.lse.ac.uk/statistics/people/yudong-chen',
   'Moez Draief': 'https://www.lse.ac.uk/statistics/people/moez-draief',
   'Tomasz Dubiel-Teleszynski': 'https://www.lse.ac.uk/statistics/people/tomasz-dubiel-teleszynski',
   'Shakeel\xa0Gavioli-Akilagun': 'https://www.lse.ac.uk/Statistics/People/Shakeel-Gavioli-Akilagun',
   'Haziq Jamil': 'https://www.lse.ac.uk/statistics/people/haziq-jamil',
   'Anastasia Kakou': 'https://www.lse.ac.uk/statistics/people/anastasia-kakou',
   'Dr Anica Kostic': 'https://www.lse.ac.uk/statistics/people/anica-kostic',
   'Yirui Liu': 'https://www.lse.ac.uk/statistics/people/yirui-liu',
   'Martin Knott': 'https://www.lse.ac.uk/statistics/people/martin-knott',
   'Milt Mavrakakis': 'https://www.lse.ac.uk/statistics/people/milt-mavrakakis',
   'Professor Colm O’Muircheartaigh': None,
   'Dr Roel Oomen': None,
   'Karsten Shaw': 'https://www.lse.ac.uk/statistics/people/karsten-shaw',
   'Howell Tong': 'https://www.lse.ac.uk/statistics/people/howell-tong',
   'Konstantinos Vamvourellis': 'https://www.lse.ac.uk/statistics/people/konstantinos-vamvourellis',
   'Gabriel Wallin': 'https://www.lse.ac.uk/statistics/people/gabriel-wallin',
   'Henry Wynn': 'https://www.lse.ac.uk/statistics/people/henry-wynn',
   'Kaifang Zhou': 'https://www.lse.ac.uk/statistics/people/kaifang-zhou',
   'George Zouros': 'https://www.lse.ac.uk/statistics/people/george-zouros'},
  'Research staff': {'Mingcong Wu': 'https://www.lse.ac.uk/statistics/people/mingcong-wu',
   'Xuewen Yu': 'https://www.lse.ac.uk/statistics/people/xuewen-yu',
   'Jin Zhu': 'https://www.lse.ac.uk/statistics/people/jin-zhu'},
  'Professional services staff': {'Steve Ellis': None,
   'Muhammed Sabih Iqbal': None,
   'Yanli Ji': None,
   'Sarah McManus': None,
   'Penny Montague': None,
   'Charlotte Morgan': None,
   'Imelda Noble Andolfo': None,
   'Penelope Smith': None,
   'Montana Tracey': None},
  'Data Science group': {'Sakina Hansen': 'https://www.lse.ac.uk/statistics/people/sakina-hansen',
   'Ziqing Ho': 'https://www.lse.ac.uk/statistics/people/ziqing-ho',
   'Liyuan Hu': 'https://www.lse.ac.uk/statistics/people/liyuan-hu',
   'Tao Ma': 'https://www.lse.ac.uk/statistics/people/tao-ma',
   'Pingfan Su': 'https://www.lse.ac.uk/statistics/people/pingfan-su',
   'Xiaoyi Wen': 'https://www.lse.ac.uk/statistics/people/xiaoyi-wen',
   'Xuzhi Yang': 'https://www.lse.ac.uk/statistics/people/xuzhi-yang'},
  'Probability in Finance and Insurance group': {'Eduardo Ferioli-Gomes': 'https://www.lse.ac.uk/Statistics/People/Eduardo-Ferioli-Gomes',
   'Mingwei Lin': 'https://www.lse.ac.uk/Statistics/People/Mingwei-Lin',
   'Alexandros Pavlis': 'https://www.lse.ac.uk/Statistics/People/Alexandros-Pavlis',
   'Pietro Maria Sparago': 'https://www.lse.ac.uk/statistics/people/pietro-maria-sparago',
   'Mohammad Fadil Sumun': 'https://www.lse.ac.uk/statistics/people/mohammad-fadil-sumun',
   'Zeng Zhang': 'https://www.lse.ac.uk/statistics/people/zeng-zhang'},
  'Social Statistics group': {'Zackary Allison': 'https://www.lse.ac.uk/statistics/people/zackary-allinson',
   'Sahoko Ishida': 'https://www.lse.ac.uk/Statistics/People/Sahoko-Ishida',
   'Sze Ming Lee': 'https://www.lse.ac.uk/statistics/people/sze-ming-lee',
   'Xinyi Liu': 'https://www.lse.ac.uk/Statistics/People/Xinyi-Liu',
   'Seyedpouya Mirrezaeiroudaki': 'https://www.lse.ac.uk/statistics/people/seyedpouya-mirrezaeiroudaki',
   'Motonori Oka': 'https://www.lse.ac.uk/statistics/people/motonori-oka',
   'August Shen': 'https://www.lse.ac.uk/Statistics/People/Zhichao-Shen'},
  'Time Series and Statistical Learning group': {'Zetai Cen': 'https://www.lse.ac.uk/statistics/people/zetai-cen',
   'Weilin Chen': 'https://www.lse.ac.uk/Statistics/People/Weilin-Chen',
   'Wenyu Cheng': 'https://www.lse.ac.uk/Statistics/People/Wenyu-Cheng',
   'Shakeel\xa0Gavioli-Akilagun': 'https://www.lse.ac.uk/Statistics/People/Shakeel-Gavioli-Akilagun',
   'Sixing Hao': 'https://www.lse.ac.uk/Statistics/People/Sixing-Hao',
   'Di Su': 'https://www.lse.ac.uk/statistics/people/di-su',
   'Toby Wade': 'https://www.lse.ac.uk/statistics/people/toby-wade',
   'Yutong Wang': 'https://www.lse.ac.uk/statistics/people/yutong-wang',
   'Shuhan Yang': 'https://www.lse.ac.uk/Statistics/People/Shuhan-Yang',
   'Xianghe Zhu': 'https://www.lse.ac.uk/statistics/people/xianghe-zhu'}}}

___
## Data Cleaning

In [11]:
cleaned_all_departments_dict = {}

for department, department_data in all_dep_people_dict.items():
    cleaned_department = department.replace('\xa0', ' ')
    cleaned_department_data = {}
    for role, people_dict in department_data.items():
        cleaned_role = role.replace('\xa0', ' ').lower().replace('find a member of ', '')
        cleaned_people_dict = {}
        for person, link in people_dict.items():
            cleaned_person = person.replace('\xa0', ' ')
            cleaned_link = link.replace('\xa0', '') if link else None
            cleaned_people_dict[cleaned_person] = cleaned_link
        cleaned_department_data[cleaned_role] = cleaned_people_dict
    cleaned_all_departments_dict[cleaned_department] = cleaned_department_data

correct_data=cleaned_all_departments_dict
correct_data


{'Department of Accounting': {'academic faculty': {'Mr Per Ahblom': 'https://www.lse.ac.uk/accounting/people/per-ahblom',
   'Professor Alnoor Bhimani': 'https://www.lse.ac.uk/accounting/people/al-bhimani',
   'Dr Jose Carabias Palmeiro': 'https://www.lse.ac.uk/accounting/people/jose-carabias-palmeiro',
   'Dr Stefano Cascino': 'https://www.lse.ac.uk/accounting/people/stefano-cascino',
   'Furkan Cetin': 'https://www.lse.ac.uk/accounting/people/Furkin-Cetin/Furkan-Cetin',
   'Dr Maria Correia': 'https://www.lse.ac.uk/accounting/people/maria-correia',
   'Pascal Frantz': 'https://www.lse.ac.uk/accounting/people/pascal-frantz',
   'Saipriya Kamath': 'https://www.lse.ac.uk/accounting/people/saipriya-kamath',
   'Gulnaz Khamidullina': 'https://www.lse.ac.uk/accounting/people/Gulnaz-Khamidullina/Gulnaz-Khamidullina',
   'Jeroen Koenraadt': 'https://www.lse.ac.uk/accounting/people/Jeroen-Koenraadt/Jeroen-Koenraadt',
   'Dr Xi Li': 'https://www.lse.ac.uk/accounting/people/xi-li',
   'Dr Nadia

___
## Getting the Gender Data

We start by putting all the names and their links into a gender dict

In [12]:
gender_dict={}
for dict_ in correct_data.values():
  for role in dict_.values():
    for person, link in role.items():
      gender_dict[person]={'Link': link, 'Gender':None}
gender_dict

{'Mr Per Ahblom': {'Link': 'https://www.lse.ac.uk/accounting/people/per-ahblom',
  'Gender': None},
 'Professor Alnoor Bhimani': {'Link': 'https://www.lse.ac.uk/accounting/people/al-bhimani',
  'Gender': None},
 'Dr Jose Carabias Palmeiro': {'Link': 'https://www.lse.ac.uk/accounting/people/jose-carabias-palmeiro',
  'Gender': None},
 'Dr Stefano Cascino': {'Link': 'https://www.lse.ac.uk/accounting/people/stefano-cascino',
  'Gender': None},
 'Furkan Cetin': {'Link': 'https://www.lse.ac.uk/accounting/people/Furkin-Cetin/Furkan-Cetin',
  'Gender': None},
 'Dr Maria Correia': {'Link': 'https://www.lse.ac.uk/accounting/people/maria-correia',
  'Gender': None},
 'Pascal Frantz': {'Link': 'https://www.lse.ac.uk/accounting/people/pascal-frantz',
  'Gender': None},
 'Saipriya Kamath': {'Link': 'https://www.lse.ac.uk/accounting/people/saipriya-kamath',
  'Gender': None},
 'Gulnaz Khamidullina': {'Link': 'https://www.lse.ac.uk/accounting/people/Gulnaz-Khamidullina/Gulnaz-Khamidullina',
  'Gender

If we check the length of the dict, it roughly corresponds to the number of staff LSE have (3300).

In [13]:
len(gender_dict)

3159

Below is the current_gender_dict, we achieved this by deducing the gender from the person's link if they had one. Otherwise, we used a name API. If you want, you can skip this part and run the entire code, but it will take a very long time as it will have to deduce the names of 3100+ people.

In [14]:
current_gender_dict={'Peter Miller': 'Male',
 'Professor Rita Astuti': 'Female',
 'Dr Mukulika Banerjee': 'Female',
 'Professor Laura Bear': 'Female',
 'Dr Clara Devlieger': 'Female',
 'Dr Yazan Doughan': 'Male',
 'Professor Katy Gardner': 'Female',
 'Dr Nicholas Long': 'Male',
 'Dr Mathijs Pelkmans': 'Male',
 'Andrea Pia': 'Male',
 'Dr Alpa Shah': 'Female',
 'Professor Charles Stafford': 'Male',
 'Dr Hans Steinmüller': 'Male',
 'Dr Harry Walker': 'Male',
 'Dr Mareike Winchell': 'Female',
 'Dr Noah Walker-Crawford': 'Male',
 'Dr Anjana Bala': 'Female',
 'Dr Gareth Breen': 'Male',
 'Dr Agathe Faure': 'Female',
 'Dr Stephanie Postar': 'Female',
 'Dr Sofía Ugarte': 'Female',
 'Dr Frederick Wojnarowski': 'Male',
 'Dr Teodor Zidaru': 'Male',
 'Professor Karin Barber': 'Female',
 'Eona Bell': 'Female',
 'Hélène Bloch': 'Female',
 'Dr Rebecca Bowers': 'Female',
 'Sanda Caracentev': 'Female',
 'Dr Chris Chaplin': 'Male',
 'Professor Harriet Evans': 'Female',
 'Dr Luis Garcia Briceno': 'Male',
 'Dr Charlotte Hawkins': 'Female',
 'Dr Mathilde Heslon': 'Female',
 'Insa Koch': 'Female',
 'Professor Adam Kuper': 'Male',
 'Dr Loretta Lou': 'Female',
 'Dr William Matthews': 'Male',
 'Claire Moll Namas': 'Female',
 'Dr Fuad Musallam': 'Male',
 'Dr Ottavia Paterno': 'Female',
 'Dr Dagna Rams': 'Female',
 'Zimran Samuel': 'Male',
 'Dr Marina Sapritsky-Nahum': 'Female',
 'Dr Mitchell Sedgwick': 'Male',
 'Dr Imani Strong': 'Female',
 'Kelzang Tashi': 'Male',
 'Professor Sylvia Yanagisako': 'Female',
 'Professor Maurice Bloch': 'Male',
 'Professor Stephan Feuchtwang': 'Male',
 'Professor Chris Fuller': 'Male',
 'Professor Martha Mundy': 'Female',
 'Professor Jonathan Parry': 'Male',
 'Kenneth Benoit': 'Male',
 'Jonathan Cardoso-Silva': 'Male',
 'Dr Ghita Berrada': 'Female',
 'Professor Zoltán Szabó': 'Male',
 'Vincent Chung': 'Male',
 'Dr Miqdad Asaria': 'Male',
 'Dr Marcos Barreto': 'Male',
 'Dr Siân Brooke': 'Female',
 'Professor Umut Cetin': 'Male',
 'Dr Yining Chen': 'Male',
 'Zachary Dickson': 'Male',
 'Professor Alexander Evans': 'Male',
 'Dr Giulia Ferrari': 'Female',
 'Dr Alexandra Gomes': 'Female',
 'Dr Patrick Gildersleve': 'Male',
 'Dr Anushri Gupta': 'Female',
 'Dr Kostas Kalogeropoulos': 'Male',
 'Dr Jens Koed Madsen': 'Male',
 'Dr Carl Müller-Crepon': 'Male',
 'Dr Francesca Panero': 'Female',
 'Dr Jean-Christophe Plantin': 'Male',
 'Dr Alison Powell': 'Female',
 'Dr Eleanor Power': 'Female',
 'Dr Dorottya Sallai': 'Female',
 'Dr Melissa Sands': 'Female',
 'Dr Divya Srivastrava': 'Female',
 'Dr Milena Tsvetkova': 'Female',
 'Dr Will Venters': 'Male',
 'Professor Henry Wynn': 'Male',
 'Dr Hadi Alagha': 'Male',
 'John Burn-Murdoch': 'Male',
 'Professor Patrick Sturgis': 'Male',
 'Professor Olmo Silva': 'Male',
 'Professor Susana Mourato': 'Female',
 'Professor Irini Moustaki': 'Female',
 'Dr Thomas Smith': 'Male',
 'Professor Emma McCoy': 'Female',
 'Nava Ashraf': 'Female',
 'Tim Besley': 'Male',
 'Christine Whitehead': 'Female',
 'Ola Aboukhsaiwan': 'Female',
 'Rasif Alakbarov': 'Male',
 'Laure Anique': 'Female',
 'Dimitris Athanasiou': 'Male',
 'Sreevidya Ayyar': 'Female',
 'Ignacio Banares Sanchez': 'Male',
 'jasper-bechtold': 'Male',
 'Alex Ben Hassine': 'Male',
 'Shania Bhalotia': 'Female',
 'raffaele-blasone': 'Male',
 'Julio Brandao Roll': 'Male',
 'Adrien Couturier': 'Male',
 'ben-dahmen': 'Male',
 'Chloe De Meulenaer': 'Female',
 'Tim Dobermann': 'Male',
 'Sebastian Ernst': 'Male',
 'Tiernan Evans': 'Female',
 'Miguel Fajardo Steinhäuser': 'Male',
 'Shadi Farahzadi': 'Female',
 'salome-fofana': 'Female',
 'Marie Fuchs': 'Female',
 'Thomas Glinnan': 'Male',
 'Marta Guasch Rusiñol': 'Female',
 'Nilmini Herath': 'Female',
 'Stephan Hobler': 'Male',
 'yu-hui': 'Male',
 'Felix Iglhaut': 'Male',
 'Muskan Jain': 'Female',
 'Amen Jalal': 'Female',
 'Animesh Jayant': 'Male',
 'Peter Lambert': 'Male',
 'Gabriel Leite Mariante': 'Male',
 'hongting-leng': 'Male',
 'mars-leung': 'Male',
 'Francisco Libano-Monteiro': 'Male',
 'delia-macaluso': 'Female',
 'Beatriz Machado Ribeiro': 'Female',
 'Covadonga Machicado Alvarez': 'Female',
 'Isaac Martinez-Centeno': 'Male',
 'Johannes Matt': 'Male',
 'Roberto Maura-Rivero': 'Male',
 'Negar Mohammadi Jazi': 'Female',
 'Sidharth Moktan': 'Male',
 'Thomas Monk': 'Male',
 'Mateus Morais': 'Male',
 'Marta Morando': 'Female',
 'Bernardo Mottironi': 'Male',
 'ellen-munroe': 'Female',
 'Canishk Naik': 'Male',
 'charles-nourse': 'Male',
 "Loughlan O'Doherty": 'Male',
 'Shinji Okazaki': 'Male',
 'Lisa Beihy Pacheco': 'Female',
 'William Parker': 'Male',
 'Paula Patzelt': 'Female',
 'Carlos Pérez-Cavero': 'Male',
 'Michelle Rao': 'Female',
 'Khawaja Arsalan Rasheed': 'Male',
 'Giorgio Ravalli': 'Male',
 'Maria Santos Feliciano': 'Female',
 'Yannick Schindler': 'Male',
 'Patrick Schneider': 'Male',
 'mitch-scott': 'Male',
 'meet-shah': 'Male',
 'Javad Shamsi': 'Male',
 'John Shannon': 'Male',
 'Callum Shaw': 'Male',
 'Lachi Singh': 'Female',
 'Caterina Soto-Vieira': 'Female',
 'Hamza Syed': 'Male',
 'Abhijit Tagade': 'Male',
 'andreas-teichgraeber': 'Male',
 'Xavier Tierney': 'Male',
 'Enrico Duilio Turri': 'Male',
 'wei-wang': 'Male',
 'Zixin Wang': 'Male',
 'Lukas Wiedemann': 'Male',
 'Yoshiki Wiskamp': 'Male',
 'Jeongkyung Won': 'Male',
 'Linchuan Xu': 'Male',
 'Tianhao Yin': 'Male',
 'Omry Yoresh': 'Male',
 'alfred-zhang': 'Male',
 'Weijie Zhang': 'Male',
 'Eddy (Weijian) Zou': 'Male',
 'Dr Gerben Bakker': 'Male',
 'Professor Kent Deng': 'Male',
 'Professor Sara Horrell': 'Female',
 'Professor Jane Humphries': 'Female',
 'Tehreem Husain': 'Female',
 'Jason Lennard': 'Male',
 'Professor Mary Morgan': 'Female',
 'Professor Albrecht Ritschl': 'Male',
 'Dr Anne Ruderman': 'Female',
 'Dr Mohamed Saleh': 'Male',
 'Dr Sabine Schneider': 'Female',
 'Christopher Kissane': 'Male',
 'Mario Cuenda Garcia': 'Male',
 'Professor Robert Bennett': 'Male',
 'Professor Youssef Cassis': 'Male',
 'Leslie Hannah': 'Male',
 'Professor Andrew Seltzer': 'Male',
 'Luisa Bicalho Ritzkat': 'Female',
 'Nick Fitzhenry': 'Male',
 'Andrés Irarrázaval': 'Male',
 'Zane Jennings': 'Male',
 'Jennifer Kohler': 'Female',
 'Julius Koschnick': 'Male',
 'Tom Learmouth': 'Male',
 'Xizi Luo': 'Female',
 'Aurelius Noble': 'Male',
 "Bumjin 'Andy' Park": 'Male',
 'Nick Peyton': 'Male',
 'Matthew Purcell': 'Male',
 'Noah Sutter': 'Male',
 'Ziming Zhu': 'Male',
 'Professor Sara Horrell ': 'Female',
 'Professor Simon Glendinning': 'Male',
 'Dr Jonathan White': 'Male',
 'Professor Kevin Featherstone': 'Male',
 'Yaprak Gürsoy': 'Female',
 'Denisa Kostovicova': 'Female',
 'Professor Jonathan Hopkin': 'Male',
 'Dr. Abby Innes': 'Female',
 'Dr Mareike Kleine': 'Female',
 'Dr. Patrick McGovern': 'Male',
 'Chris Anderson': 'Male',
 'Nicholas Barr': 'Male',
 'Johann Robert Basedow': 'Male',
 'Professor Iain Begg': 'Male',
 'Chris Bick': 'Male',
 'Professor Stefan Collignon': 'Male',
 'Professor Paul De Grauwe': 'Male',
 'Dr Spyros Economides': 'Male',
 'Gianmarco Fifi': 'Male',
 'garibay-petersen-cristobal': 'Male',
 'Dr Kira Gartzou-Katsouyanni': 'Female',
 'Dr. Katerina Glyniadaki': 'Female',
 'Asha Herten-Crabb': 'Female',
 'Dr Abby Innes': 'Female',
 'Dr Jennifer Jackson-Preece': 'Female',
 'Dr. Patrick Kimunguyi': 'Male',
 'Marta Lorimer': 'Female',
 'Dr Angelo Martelli': 'Male',
 'Frieder Mitsch ': 'Male',
 'Dr Vassilis Monastiriotis': 'Male',
 'Daniela Roxana Movileanu': 'Female',
 'Dr Miguel Pereira': 'Male',
 'Eva Polonska-Kimunguyi': 'Female',
 'Vesna Popovski': 'Female',
 'rashid-tahir': 'Male',
 'Dr Eiko Thielemann': 'Male',
 'Niina Vuolajärvi': 'Female',
 'Adam Judge': 'Male',
 'Charlotte Ennis': 'Female',
 'Natalie Grace': 'Female',
 'jenner-stephen': 'Male',
 'Wagio Waigi': 'Female',
 'liouta-polly': 'Female',
 'Beenam Butt': 'Female',
 'Laura Malouf': 'Female',
 'Delphine Polidori': 'Female',
 'Hannah Shearer': 'Female',
 'Carolina Stern': 'Female',
 'samuels-florence': 'Female',
 'Arzu Kırcal Şahin': 'Female',
 'Samson Yeung': 'Male',
 'Kate Alexander Shaw': 'Female',
 'Marilena Anastasopoulou': 'Female',
 'Joseph Ganderson': 'Male',
 'Dr Sebahattin Abdurrahman': 'Male',
 'Ms Helen J. Addison': 'Female',
 'Mr. John Alty': 'Male',
 'Angelos Angelou': 'Male',
 'Tito Boeri': 'Male',
 'Diane Bolet': 'Female',
 'Dr Eray Çaylı': 'Male',
 'Nathan Charlton': 'Male',
 'Marina Cino Pagliarello': 'Female',
 'Mr Lorenzo Codogno': 'Male',
 'Dr Steve Coulter': 'Male',
 'Mr Gijs De Vries': 'Male',
 'Sebastian Diessner': 'Male',
 'Dr Joseph Downing': 'Male',
 'Federico Maria Ferrara': 'Male',
 'Dr Pasquale Foresti': 'Male',
 'Dr Eddie Gerba': 'Male',
 'Dr. Bob Hancké': 'Male',
 'Dr Pierre Hausemer': 'Male',
 'Mr Philippe Legrain': 'Male',
 'Dr Philipp Lutz': 'Male',
 'Dr Corrado Macchiarelli': 'Male',
 'Dr Selin Nasi': 'Female',
 'Professor Javier Ortega': 'Male',
 'Dr Chrysoula Papalexatou': 'Female',
 'Dr Maria Prats': 'Female',
 'Teresa Pullano': 'Female',
 'Dr Matilde Rosina': 'Female',
 'Orkun Saka': 'Male',
 'Professor Waltraud Schelkle': 'Female',
 'Dr Lucas Schramm': 'Male',
 'Seçkin Sertdemir Özdemir': 'Female',
 'Dr Tolga Sinmazdemir': 'Male',
 'Dr Ivor Sokolić': 'Male',
 'Mr Anthony Teasdale': 'Male',
 'Professor Mark Thatcher': 'Male',
 'Zbigniew Truchlewski': 'Male',
 'Dr. Jimena Valdez': 'Female',
 'Toon Van Overbeke': 'Male',
 'voss-dustin': 'Male',
 'Martin Westlake': 'Male',
 'Cocaj-venera': 'Female',
 'michael cottakis': 'Male',
 'Virginia Crespi de Valldaura': 'Female',
 'Sean Deel': 'Male',
 'Andréa Delestrade': 'Female',
 'Sarah Gerwens': 'Female',
 'Nilufer Gunes': 'Female',
 'Noa Krikler': 'Female',
 'Yonatan Levi': 'Male',
 'Jacob Lypp': 'Male',
 'Benedetta Morari': 'Female',
 'Elena Pro': 'Female',
 'Friedrich Püttmann': 'Male',
 'Haile Zola': 'Female',
 'Neesha Rai': 'Female',
 'Sahajpal': 'Female',
 'Robtel Neajai Pailey': 'Female',
 'Claire Mercer ': 'Female',
 'Catherine Boone': 'Female',
 'Laura Mann': 'Female',
 'Wendy Willems': 'Female',
 'Katrin Flikschuh': 'Female',
 'Declan Conway': 'Male',
 'Chaloka Beyani': 'Male',
 'Jo Beall': 'Female',
 'George Ofosu': 'Male',
 'Laura Bear': 'Female',
 'Teddy Brett': 'Male',
 'Dr Simidele Dosekun': 'Female',
 'Tom Kirk': 'Female',
 'Oluwasola Omoju': 'Male',
 'Anne-Line Rodriguez': 'Female',
 'Iliana Sarafian': 'Female',
 'Innocent Anguyo': 'Male',
 'Tony Barnett': 'Male',
 'Professor Gibril Faal': 'Male',
 'Dr Piroska Nagy Mohácsi': 'Female',
 'Desné Masie': 'Female',
 'Dr Louise Arimatsu': 'Female',
 'Souad Mohamed': 'Female',
 'Dr Fatima el Issawi': 'Female',
 'Dr. Vanessa Iwowo': 'Female',
 'Anna Macdonald': 'Female',
 'Shirley Ze Yu': 'Female',
 'Francesca Beausang': 'Female',
 'Kara Blackmore': 'Female',
 'Dr Mebratu Kelecha': 'Male',
 'Naomi Pendle': 'Female',
 'Juliet Bedford': 'Female',
 'Paroma Bhattacharya': 'Female',
 'Ponsiano Bimeny': 'Male',
 'Dr Uche Igwe': 'Male',
 'Theresa Jones': 'Female',
 'Esther Marijnen': 'Female',
 'Flora McCrone': 'Female',
 'Georgina Pearson': 'Female',
 'Maria del Pilar Lopez-Uribe': 'Female',
 'Nikita Simpson': 'Female',
 'Jonathan Bashi': 'Male',
 'Maudo Jallow': 'Male',
 'Elizabeth Storer': 'Female',
 "Ryan O'Byrne": 'Male',
 'Jonah Lipton': 'Male',
 'Alice Robinson': 'Female',
 'Professor Tim Allen': 'Male',
 'Fadil Elobeid ': 'Male',
 'Dr Martha Geiger Mwenitete': 'Female',
 'Dr David Luke': 'Male',
 'Lesley Orero': 'Female',
 'Anna Williams': 'Female',
 'Tosin Adebisi': 'Male',
 'Mark Briggs': 'Male',
 'Sofija Spasenoska': 'Female',
 'Eunice Asantewaa Hansen-Sackey': 'Female',
 'Carrie Hamilton': 'Female',
 'Clare Hemmings': 'Female',
 'Emrah Karakus': 'Male',
 'Sumi Madhok': 'Female',
 'Milo Bettocchi': 'Male',
 'Maria Rashid': 'Female',
 'Leticia Sabsay': 'Female',
 'Sadie Wearing ': 'Female',
 'Mary Evans': 'Female',
 'Marsha Henry': 'Female',
 'Diane Perrons': 'Female',
 'Alexandra Pugh': 'Female',
 'Shirin Rai': 'Female',
 'Vron Ware': 'Female',
 'Irina Zherebkina': 'Female',
 'Melissa Chacon': 'Female',
 'Aynura Akbas': 'Female',
 'Nour Almazidi': 'Female',
 'Malena Bastida Antich': 'Female',
 'Lizzie Hobbs': 'Female',
 'Alanah Mortlock': 'Female',
 'Magda Muter': 'Female',
 'Florence Waller-Carr': 'Female',
 'Senel Wanniarachchi': 'Male',
 'Claire Wilmot': 'Female',
 'Suki Ali': 'Female',
 'Sarah Ashwin': 'Female',
 'Shakuntala Banaji': 'Female',
 'Lilie Chouliaraki': 'Female',
 'Ernestina Coast': 'Female',
 'Emily Jackson': 'Female',
 'Nicola Lacey': 'Female',
 'Katharine Millar': 'Female',
 'Irini Moustaki': 'Female',
 'Anne Phillips': 'Female',
 'Coretta Phillips': 'Female',
 'Sara Salem': 'Female',
 'Hakan Seckinelgin': 'Male',
 'Alpa Shah': 'Female',
 'Imaobong Umoren': 'Female',
 'Prof Hyun Bang Shin': 'Male',
 'Christian Hilber': 'Male',
 'Claire Mercer': 'Female',
 'Simon Dietz': 'Male',
 'Nancy Holman': 'Female',
 'Olmo Silva': 'Male',
 'Cornelia Agyenim – Boateng': 'Female',
 'Laura Antona': 'Female',
 'Prof Gabriel Ahlfeldt': 'Male',
 'Giles Atkinson': 'Male',
 'Aretousa Bloom': 'Female',
 'filippo boeri': 'Male',
 'Felipe Carozzi': 'Male',
 'Ryan Centner': 'Male',
 'Anomitro Chatterjee': 'Male',
 'Paul Cheshire': 'Male',
 'Julia Corwin': 'Female',
 'Riccardo Crescenzi': 'Male',
 'Muna Dajani': 'Female',
 'Pooya Ghoddousi': 'Male',
 'Steve Gibbons': 'Male',
 'Ian Gordon': 'Male',
 'Vernon Henderson': 'Male',
 'Carolin Hulke': 'Female',
 'Stephen Jarvis': 'Male',
 'David Jones': 'Male',
 'Gareth A Jones': 'Male',
 'Neil Lee': 'Male',
 'Murray Low': 'Male',
 'Alan Mace': 'Male',
 'Michael Mason': 'Male',
 'Dr Tanya Matthan': 'Female',
 'Susana Mourato': 'Female',
 'Eric Neumayer': 'Male',
 'Henry Overman': 'Male',
 'Dr Jeffrey Pagel': 'Male',
 'Erica Pani': 'Female',
 'Kasia Paprocki': 'Female',
 'Richard Perkins': 'Male',
 'Laura Pulido': 'Female',
 'Elena Renzullo': 'Female',
 'davide rigo': 'Male',
 'Sefi Roth': 'Male',
 'Romola Sanyal': 'Female',
 'Rodolfo Sejas-Portillo': 'Male',
 'Hyun Bang Shin': 'Male',
 'Thomas Smith': 'Male',
 'Jessie Speer': 'Female',
 'Michael Storper': 'Male',
 'ana verala verala': 'Female',
 'meredith whitten': 'Female',
 'hendrick-wolff': 'Male',
 'Austin Zeiderman': 'Male',
 'Prof Vassilis Monastiriotis': 'Male',
 'Prof Christine Whitehead': 'Female',
 'Professor Laura Pulido': 'Female',
 'Dr Ganga Shreedhar': 'Female',
 'Kath Scanlon': 'Female',
 'Ignacio Aravena Gonzalez': 'Male',
 'Emmanuel C. P. Awohouedji': 'Male',
 'Ivana Bevilacqua': 'Female', 'Charles Palmer': 'Male', 'Andres Rodriguez-Pose': 'Male', 'Mr Per Ahblom': 'Male',
 'Professor Alnoor Bhimani': 'Male',
 'Dr Jose Carabias Palmeiro': 'Male',
 'Dr Stefano Cascino': 'Male',
 'Furkan Cetin': 'Male',
 'Dr Maria Correia': 'Female',
 'Pascal Frantz': 'Male',
 'Saipriya Kamath': 'Female',
 'Gulnaz Khamidullina': 'Female',
 'Jeroen Koenraadt': 'Male',
 'Dr Xi Li': 'Female',
 'Dr Nadia Matringe': 'Female',
 'Andrea Mennicken': 'Female',
 'Dr Julia Morley': 'Female',
 'Jacob Ott': 'Male',
 'Tommaso Palermo': 'Male',
 'Michael Power': 'Male',
 'Dr Aneesh Raghunandan': 'Male',
 'Alexa Scherf': 'Female',
 'Ane Tamayo': 'Female',
 'Oscar Timmermans': 'Male',
 'Professor Wim A Van der Stede': 'Male',
 'Professor Richard Macve': 'Male',
 'Emeritus Professor Michael Bromwich': 'Male',
 'Professor Peter Pope': 'Male',
 'Alistair Marsden': 'Male',
 'Yinan Li': 'Female',
 'Kathyayini Madduri': 'Female',
 'Anna Romanova': 'Female',
 'Wan Chu Cheong': 'Male',
 'Xinyun Liu': 'Female',
 'Parth Shah': 'Male',
 'Jiachen Gao': 'Male',
 'Lorenzo Pirozzi': 'Male',
 'Dr Catherine Allerton': 'Female',
 'Dr Fenella Cannell': 'Female',
 'Professor Deborah James': 'Female',
 'Dr Michael W. Scott': 'Male',
 'Dr Gisa Weszkalnys': 'Female',
 'jan david hauck': 'Male',
 'Dr Carrie Heitmeyer': 'Female',
 'Dr Megan Laws ': 'Female',
 'Dr Arthur Mason': 'Male',
 'Dr Giulio Ongaro': 'Male',
 'Jinshuai Ma': 'Male',
 'Dr Mona Azadkia': 'Female',
 'Dr Tuğkan Batu': 'Male',
 'Dr Liam Beiser-McGrath': 'Male',
 'Dr Michael Blackwell': 'Male',
 'Dr Chris Blunt': 'Male',
 'Dr Yunxiao Chen': 'Female',
 'Dr Neil Cummins': 'Male',
 'Dr Daniel De Kadt': 'Male',
 'Dr Florian Foos': 'Male',
 'Dr Michael Ganslmeier': 'Male',
 'Dr Casey Kearney': 'Male',
 'Dr Marion Lieutaud': 'Female',
 'Dr Joshua Loftus': 'Male',
 'Dr Blake Miller': 'Male',
 'Andrew Moles': 'Male',
 'Dr Marie Oldfield': 'Female',
 'Dr Katerina Papadaki': 'Female',
 'Mathias Poertner': 'Male',
 'Dr Xinghao Qiao': 'Male',
 'Professor Johannes Ruf': 'Male',
 'Dr Thomas Robinson': 'Male',
 'Dr Chengchun Shi': 'Male',
 'Professor László Végh': 'Male',
 'Dr Kate Vredenburgh': 'Female',
 'Dr Tengyao Wang': 'Male',
 'Dr Yan Wang': 'Male',
 'Dr Nicole Baerg': 'Female',
 'Dr Gökhan Çiflikli': 'Male',
 'Dr James Hamp ': 'Male',
 'Ajay Kumar ': 'Male',
 'Dr Tom Paskhalis': 'Male',
 'Professor Leonard Smith': 'Male',
 'Dr Miriam Sorace': 'Female',
 'Dr Erica Thompson': 'Female',
 'Dr Joshua Townsley': 'Male',
 'Professor Bernhard Von Stengel': 'Male',
 'Professor Michael Laver': 'Male',
 'Professor Peter Allen': 'Male',
 'Jeni Brown': 'Female',
 'Philippe Aghion': 'Male',
 'Oriana Bandiera': 'Female',
 'Clare Balboni': 'Female',
 'Charles Bean': 'Male',
 'Mohan Bijapur': 'Male',
 'Juraj Briskar': 'Male',
 'Gharad Bryan': 'Male',
 'Robin Burgess': 'Female',
 'Michael Callen': 'Male',
 'Francesco Caselli': 'Male',
 'Frank Cowell': 'Male',
 'Wouter Den Haan': 'Male',
 'Maarten De-Ridder': 'Male',
 'Swati Dhingra': 'Female',
 'Jeremiah Dittmar': 'Male',
 'Matthias Doepke': 'Male',
 'Andrew Ellis': 'Male',
 'Alessandro Gavazza': 'Male',
 'Maitreesh Ghatak': 'Female',
 'Michael Gmeiner': 'Male',
 'Vassilis Hajivassiliou': 'Male',
 'Jonathon Hazell': 'Male',
 'Javier Hidalgo': 'Male',
 'Ethan Ilzetzki': 'Male',
 'Xavier Jaravel': 'Male',
 'Keyu Jin': 'Female',
 'Katarzyna Krajniewska': 'Female',
 'Camille Landais': 'Female',
 'Jonathan Leape': 'Male',
 'Gilat Levy': 'Female',
 'Matthew Levy': 'Male',
 'Stephen Machin': 'Male',
 'Isabela Manelici': 'Female',
 'Alan Manning': 'Male',
 'Antonio Mele': 'Male',
 'Guy Michaels': 'Male',
 'Benjamin Moll': 'Male',
 'John Moore': 'Male',
 'Dmitry Mukhin': 'Male',
 'Francesco Nava': 'Male',
 'Junius Olivier': 'Male',
 'Taisuke Otsu': 'Male',
 'Martin Pesendorfer': 'Male',
 'Dimitra Petropoulou': 'Female',
 'Michele Piccione': 'Female',
 'Steve Pischke': 'Male',
 'Christopher Pissarides': 'Male',
 'Ronny Razin': 'Male',
 'Ricardo Reis': 'Male',
 'David Ren': 'Male',
 'Nico Rosetti': 'Male',
 'Jane Olmstead-Rumsey': 'Female',
 'Ragvir Sabharwal': 'Male',
 'Thomas Sampson': 'Male',
 'Christopher Sandmann': 'Male',
 'Marcia Schafgans': 'Female',
 'Mark Schankerman': 'Male',
 'Pasquale Schiraldi': 'Male',
 'Judith Shapiro': 'Female',
 'Kevin Sheedy': 'Male',
 'Kate Smith': 'Female',
 'Johannes Spinnewijn': 'Male',
 'Nicholas Stern': 'Male',
 'Daniel Sturm': 'Male',
 'Balazs Szentes': 'Male',
 'Silvana Tenreyro': 'Female',
 'John Van Reenen': 'Male',
 'Yike Wang': 'Female',
 'Alwyn Young': 'Male',
 'Shengxing Zhang': 'Male',
 'Mervyn King': 'Male',
 'John Sutton': 'Male',
 'Per Krusell': 'Male',
 'Torsten Persson': 'Male',
 'Margaret Bray': 'Female',
 'Christopher Dougherty': 'Male',
 'Richard Jackman': 'Male',
 'Lord Richard Layard': 'Male',
 'Peter Robinson': 'Male',
 'Sierene Abdelsayed': 'Female',
 'Philipp Barteska': 'Male',
 'Marco Bellifemine': 'Male',
 'Alix Bonargent': 'Female',
 'Isadora Bousquat Arabe': 'Female',
 'Daniel Chandler': 'Male',
 'wallace-de-jesus-inocencio': 'Male',
 'Zhiyao Deng': 'Male',
 'Gaia Dossi': 'Female',
 'Arnaud Dyevre': 'Male',
 'Andres Fajardo-Ramirez': 'Male',
 'Lin Fan': 'Female',
 'Tuo Fan': 'Male',
 'Bennet Luca Feld': 'Male',
 'Diego Ferreras Garrucho': 'Male',
 'betsenat-gebrewold': 'Male',
 'Michelle Harnisch': 'Female',
 'anton-heil': 'Male',
 'Shaohua Huang': 'Male',
 'Antonio Armindo Iavarone': 'Male',
 'evie-jamieson': 'Male',
 'Ningyuan Jia': 'Male',
 'zihan-jia': 'Male',
 'Luiza Kunz Aires': 'Female',
 'javier-lopez': 'Male',
 'Runhong Ma': 'Female',
 'Edward Manuel': 'Male',
 'Giovanni Minolfi': 'Male',
 'Davi Moura': 'Male',
 'Alexander Newton': 'Male',
 'Kamila Nowakowicz': 'Female',
 "Matthew O'Brien": 'Male',
 'Tatiana Pazem': 'Female',
 'gailius-praninskas': 'Male',
 'Ludovica Priviero': 'Female',
 'Hugo Reichardt': 'Male',
 'Soroush Sabet': 'Male',
 'Veronica Salazar Restrepo': 'Female',
 'brayan-segura-solano': 'Male',
 'Pol Simpson': 'Male',
 'Ameek Singh': 'Male',
 'Aditya Soenarjo': 'Male',
 'Meng Su': 'Male',
 'xiao-sun': 'Male',
 'Kazunari Tanabe': 'Male',
 'Likun Tian': 'Male',
 'Maria Ventura': 'Female',
 'Peter Ward-Griffin': 'Male',
 'Cecilia Wood': 'Female',
 'Chia-Hung Yeh': 'Male',
 'Yuanhang Yu': 'Male',
 'Dr Olivier Accominotti': 'Male',
 'Dudley Baines': 'Male',
 'Dr Jordan Claridge': 'Male',
 'Professor Neil Cummins': 'Male',
 'Dr Leigh Gardner': 'Female',
 'Andrés Guiot Isaac': 'Male',
 'Professor Janet Hunter': 'Female',
 'Dr Alejandra Irigoin': 'Female',
 'Professor Colin Lewis': 'Male',
 'Dr Chris Minns': 'Male',
 'Safya Morshed': 'Female',
 'Dr Natacha Postel-Vinay': 'Female',
 'Professor Joan Roses': 'Female',
 'Professor Tirthankar Roy': 'Male',
 'Professor Max Schulze': 'Male',
 'Dr Eric Schneider': 'Male',
 'Professor Oliver Volckart': 'Male',
 'Professor Patrick Wallis': 'Male',
 'Dr Melanie Meng Xue': 'Female',
 'Dr Jeremiah Dittmar': 'Male',
 'William Mitchell ': 'Male',
 'Dr Karolina Hutkova': 'Female',
 'Alex Green': 'Male',
 'Professor Patrick Wallis ': 'Male',
 'Professor Neil Cummins ': 'Male',
 'Carl-Ludwig Campbell': 'Male',
 'Lanabi la Lova': 'Female',
 'Guney Yildiz': 'Male',
 'Ashwini Agrawal': 'Female',
 'Ulf Axelson': 'Male',
 'Balloch': 'Male',
 'Mike Burkart': 'Male',
 'Georgy  Chabakauri': 'Male',
 'Constantin Charles': 'Male',
 'Kim Fe Cramer': 'Female',
 'Vicente Cuñat': 'Male',
 'Jon Danielsson': 'Male',
 'Amil Dasgupta': 'Male',
 'Daniel Ferreira': 'Male',
 'Juanita Gonzalez-Uribe': 'Female',
 'Dirk Jenter': 'Male',
 'Julliard': 'Male',
 'Peter  Kondor': 'Male',
 'Paula Lopes-Cocco': 'Female',
 'Dong Lou': 'Male',
 'Igor Makarov': 'Male',
 'Ian Martin': 'Male',
 'Martin Oehmke': 'Male',
 'Daniel Paravisini': 'Male',
 'Cameron Peng': 'Male',
 'Christopher  Polk': 'Male',
 'Rohit Rahi': 'Male',
 'Walker Ray': 'Male',
 'Cristina Scherrer': 'Female',
 'Dimitri Vayanos': 'Male',
 'Liliana Varela': 'Female',
 'Michela Verardo': 'Female',
 'David Webb': 'Male',
 'Kathy Yuan': 'Female',
 'Linyan Zhu': 'Female',
 'Jean-Pierre Zigrand': 'Male',
 'Ron Anderson': 'Male',
 'Oliver Ashtari Tafti': 'Male',
 'Bijan Aghdasi': 'Male',
 'Yanna Cai': 'Female',
 'Benjamin Chen': 'Male',
 'Ali Choubdaran Varnosfaderani': 'Male',
 'James Clark': 'Male',
 'Nico Garrido-Sureda': 'Male',
 'Marcus Vinícius Gomes De Castro': 'Male',
 'Chris Greiner': 'Male',
 'Yang Guo': 'Male',
 'Magnus Hjortfors Irie': 'Male',
 'Davide La Cara': 'Male',
 'Jialiang Lou': 'Male',
 'Xinchen Ma': 'Female',
 'Guido Maia Da Cunha': 'Male',
 'Nithin Mannil': 'Male',
 'Maithili Modi': 'Female',
 'Mark Morley': 'Male',
 'Robert Rogers': 'Male',
 'Vahid Rostamkhani': 'Male',
 'Jiahong Shi': 'Male',
 'Elizaveta Smorodenkova': 'Female',
 'Yanhuan Tang': 'Female',
 'TianY': 'Female',
 'Niels Wagner': 'Male',
 'Yibing Wang': 'Female',
 'Song Xiao': 'Female',
 'Zheyuan Yang': 'Male',
 'Brocklehurst': 'Male',
 'Otavio Bitu': 'Male',
 'Garrido-Sureda': 'Female',
 'Marc Gischer': 'Male',
 'Mengqi Li': 'Female',
 'Fernando Soares': 'Male',
 'Irina-Eusignia Stanciu': 'Female',
 'Jiaxing Tian': 'Male',
 'Goncalo Vieira Da Luz': 'Male',
 'Jiaming Wang': 'Male',
 'Yue Wu': 'Female',
 'Bolin Xu': 'Male',
 'Yun Xue': 'Female',
 'Jianing Yuan': 'Female',
 'Claudia Zhao': 'Female',
 'Lucy Kanya': 'Female',
 'Deborah James': 'Female',
 'Leigh Gardner': 'Female',
 'Joanna Lewis': 'Female',
 'Elizabeth Ngutuku': 'Female',
 'Julian Hopwood': 'Male',
 'Professor Leonard Wantchekon ': 'Male',
 'Aiko Holvikivi': 'Female',
 'Zeynep Kilicoglu': 'Female',
 'Gloria Novovic': 'Female',
 'Sharmila Parmanand': 'Female',
 'Ania Plomien': 'Female',
 'SM Rodriguez': 'Male',
 'Wendy Sigle': 'Female',
 'Alia Amirali': 'Female',
 'Zuzana Dančíková': 'Female',
 'Ting-Sian-Liu': 'Male',
 'Luma (Lucas) Mantilla Garino': 'Female',
 'Eugenie Dugoua': 'Female',
 'Antonio Avila-Uribe': 'Male', 'Paul Brandily': 'Male', 'Amrita DasGupta': 'Female', 'Tea Gamtkitsulashvili': 'Female', 'Ali Glisson': 'Female', 'Nikolaus Hastreiter ': 'Male', 'Violet Lasdun': 'Female', 'Woong Ki Lee': 'Male', 'jonathan mille': 'Male', 'Shaonlee Patranabis': 'Male', 'Capucine Riom': 'Female', 'Hanadi Samhan': 'Female', 'Sraman Sircar': 'Male', 'Romano Tarsia': 'Male', 'Hanh-An Trinh': 'Female', 'Ran Wei': 'Female', 'Jingyuan Zeng': 'Male', 'Kevin M. Adams': 'Male', 'Juan Alvarez-Vilanova': 'Male', 'Pia Andres': 'Female', 'Ignacio Aravena González': 'Male', 'Emmanuel C P Awohouedji': 'Male', 'Daniela Baeza Breinbauer': 'Female', 'Lucrecia Bertelli': 'Female', 'Martin Brown Munene': 'Male', 'Joan Camilo Lopez': 'Male', 'Vincent Chung Jie Lun': 'Male', 'Mattie Cox': 'Female', 'Ana De Menezes': 'Female', 'Alaa Dia': 'Male', 'Ina Drouven': 'Female', 'Sarah Elven': 'Female', 'Dalia Gebrial': 'Female', 'Fernanda Gimenes': 'Female', 'Nikolaus Hastreiter': 'Male', 'Andrea Herrera Bórquez': 'Female', 'Anu Jogesh': 'Female', 'Jeanette L. Kaiser': 'Female', 'Yarden Manelzon': 'Female', 'Sandiswa Mapukata': 'Female', 'Till Meissner': 'Male', 'Annalyse Moskeland': 'Female', 'Su Su Myat': 'Female', 'Marina Nazneen': 'Female', 'Gabriela Neves De Lima': 'Female', 'Martina Pardy': 'Female', 'Muhammad Yorga Permana': 'Male', 'Gabriele Piazza': 'Male', 'Julien Picard': 'Male', 'Lindiwe Rennert': 'Female', 'Heini-Emilia Saari': 'Female', 'M. Adil Sait': 'Male', 'Fizzah Sajjad': 'Female', 'Martina Vittoria Sottini': 'Female', 'Frida Timan': 'Female', 'Melissa Weihmayer': 'Female', 'Xian Wu': 'Male', 'Leiboyu Xiang': 'Male', 'Ka Chi Yip': 'Male', 'Qing Yuan Guo': 'Male', 'Kerstin J. Schaefer': 'Female', 'Monika Streule': 'Female', 'Kate Dawson': 'Female', 'Lewis Dijkstra': 'Male', 'Mara Ferreri': 'Female', 'Deborah Fromm': 'Female', 'Michael Harloe': 'Male', 'Kirk Hamilton': 'Male', 'Kyoung-Ae Han': 'Female', 'Simona Iammarino': 'Female', 'Carolin Ioramashvili': 'Female', 'Yi Jin': 'Female', 'Mohsen Khezri': 'Male', 'Loretta Lees': 'Female', 'Paul Metcalfe': 'Male', 'Danielle Purifoy': 'Female', 'Asato Saito': 'Male', 'Deen Shariff Sharp': 'Male', 'Rory Sullivan': 'Male', 'Jayaraj Sundaresan': 'Male', 'Jason Wong': 'Male', 'Baek Yung Kim': 'Male', 'Professor Paul Kelly': 'Male', 'Mr Victor Agboga': 'Male', 'Elise Antoine': 'Female', 'Dr Paul Apostolidis': 'Male', 'Dr Daniel Berliner': 'Male', 'Professor Catherine Boone': 'Female', 'Dr Sarah Brierley': 'Female', 'Professor Michael Bruter': 'Male', 'Professor John Chalcraft': 'Male', 'Dr Nicholas Devlin': 'Male', 'Professor Torun Dewan': 'Male', 'Dr Carolin Dieterle': 'Female', 'Dr Vesselin Dimitrov': 'Male', 'Professor Katrin Flikschuh': 'Female', 'Dr David Foster': 'Male', 'Dr Thalia Gerzso': 'Female', 'Dr Steffen Hertog': 'Male', 'Professor Sara Hobolt': 'Female', 'Dr Jonathan Hopkin': 'Male', 'Dr Rafael Hortala-Vallve': 'Male', 'Professor James Hughes': 'Male', 'Dr Ryan Jablonski': 'Male', 'Dr Rehan Rafay Jamil': 'Male', 'Dr Nirvikar Jassal': 'Male', 'Leigh Jenco': 'Female', 'Dr Bill Kissane': 'Male', 'Dr Mathias Koenig-Archibugi': 'Male', 'Professor Valentino Larcinese': 'Male', 'Dr Michael Lerner': 'Male', 'Professor Martin Lodge': 'Male', 'Dr Jacklyn Majnemer': 'Female', 'Dr Omar McDoom': 'Male', 'Dr Paul Mitchell': 'Male', 'Dr George Ofosu': 'Male', 'Professor Edward Page (FBA)': 'Male', 'Professor Francisco Panizza': 'Male', 'Dr Mathias Poertner': 'Male', 'Dr Stephanie Rickard': 'Female', 'Professor Cheryl Schonhardt-Bailey (FBA)': 'Female', 'John Sidel': 'Male', 'Dr Kai Spiekermann': 'Male', 'Pavithra Suryanarayan': 'Female', 'Dr Felipe Torres Raposo': 'Male', 'Dr Aliz Tóth': 'Female', 'Professor Tony Travers': 'Male', 'Dr Deirdre Troy': 'Female', 'Tamara Tubakovic': 'Female', 'Dr Joachim Wehner': 'Male', 'Dr Stephane Wolton': 'Male', 'Dr David Woodruff': 'Male', 'Professor Lea Ypi': 'Female', 'Dr Sarah Harrison': 'Female', 'Sebastian Balfour': 'Male', 'Geoffrey Myers': 'Male', 'Luis Bosshart': 'Male', 'Caroline Elak': 'Female', 'Robert Falconer': 'Male', 'Tiffany Lau': 'Female', 'Nick Lewis': 'Male', 'Emma Obermair': 'Female', 'Asfa Shakeel': 'Female', 'Stephanie Wanga': 'Female', 'Peter Wyckoff': 'Male', 'Alex Yeandle': 'Male', 'Antoine Zerbini': 'Male', 'Luqman Abu El Foul': 'Male', 'Meshal Alkhowaiter': 'Male', 'Vanessa Cheng-Matsuno': 'Female', 'Joel Chong': 'Male', 'Asli Ceren Cinar': 'Female', 'Julia Costet': 'Female', 'Vincent Harting': 'Male', 'Martin Haus': 'Male', 'Angga Indraswara': 'Male', 'Yusuf Imaad Khan': 'Male', 'Rune Wriedt Larsen': 'Male', 'Xufan': 'Male', 'Andrew McNeil': 'Male', 'Frieder Mitsch': 'Male', 'Alberto Parmigiani': 'Male', 'Michal Pasovsky': 'Male', 'Diego Sazo': 'Male', 'Andreas-Johann Sorger': 'Male', 'Theint Theint Thu': 'Female', 'Tarsha Vasu': 'Female', 'Lorenzo Vicari': 'Male', 'Jan Henrik Wasserziehr': 'Male', 'Felix Westerén': 'Male', 'Rodney Barker': 'Male', 'John Breuilly': 'Male', 'John Charvet': 'Male', 'Janet Coleman': 'Female', 'Patrick Dunleavy': 'Male', 'Chun Lin': 'Female', 'David Soskice': 'Male', 'Michael Barzelay': 'Male', 'Charlie Beckett': 'Male', 'Jean-Paul Faguet': 'Male', 'Tasha Fairfeld': 'Female', 'Kevin Featherstone': 'Male', 'Benjamin Lauderdale': 'Male', 'Javier Ortega': 'Male', 'James Putzel': 'Male', 'Ken Shadlen': 'Male', 'Eiko Thielemann': 'Male', 'Jonathan White': 'Male', 'Thoraya El-Rayyes': 'Female', 'Bruno Leipold': 'Male', 'Julia Leschke': 'Female', 'Elena Pupaza': 'Female', 'Nelson Ruiz': 'Male', 'Tinghua Yu': 'Female', 'Professor Andrew Street': 'Male', 'Dr Mylene Lagarde': 'Female', 'Dr Justin Parkhurst': 'Male', 'Marcello Antonini': 'Male', 'Miqdad Asaria': 'Male', 'Dr Mrigesh Bhatia': 'Male', 'Dr-Alex-Carter': 'Male', 'Emilie Courtin': 'Female', 'Professor Joan Costa-i-Font': 'Male', 'Dr Rocco Friebel': 'Male', 'Genevieve-Jeffrey': 'Female', 'Dr-Catherine-M-Jones': 'Female', 'Dr Panos Kanavos': 'Male', 'Ilias Kyriopoulos': 'Male', 'Professor Martin Knapp': 'Male', 'Professor Alistair McGuire': 'Male', 'Dr-Ritesh-Maharaj': 'Male', 'Professor Elias Mossialos': 'Male', 'Huseyin Naci PhD MHS': 'Male', 'Dr Irene Papanicolas': 'Female', 'Nilesh Raut': 'Male', 'Dr Divya Srivastava': 'Female', 'Dr Ranjeeta Thomas': 'Female', 'Dr Clare Wenham': 'Female', 'George A Wharton': 'Male', 'Dr Olivier Wouters': 'Male', 'Jo Brown': 'Female', 'Sofia Vyzantiadou': 'Female', 'Dr Jeroen Bax': 'Male', 'Dr Martin Cowie': 'Male', 'Dr Eduardo Gomez': 'Male', 'Dr Allan Hackshaw': 'Male', 'Dr Michael Holland': 'Male', 'Professor Ashish Jha': 'Male', 'Dr Brittany Jones': 'Female', 'Professor Paulus Kirkhof': 'Male', 'Professor Mandeep Mehra': 'Male', 'Dr Mark Newman': 'Male', 'Dr Jennifer Palmer': 'Female', 'Dr Maria Raikou': 'Female', 'Michael_Anderson': 'Male', 'Carlos A. Bana e Costa': 'Male', 'Maeve Bognini': 'Female', 'Danitza Chávez-Montoya': 'Female', 'Dorothy Chisare': 'Female', 'Dr Lesong Conteh': 'Female', 'Jonathan Cylus': 'Male', 'Arianna Gentilini': 'Female', 'Jennifer Gill ': 'Female', 'Madeleine Haig': 'Female', 'Cristina Hernandez Quevedo': 'Female', 'Martilord Ifeanyichi': 'Male', 'Sahan Jayawardana': 'Male', 'Charlotte Johnston-Webber': 'Female', 'Dr Lucy Kanya': 'Female', 'Meskerem Kebede': 'Female', 'Beth Kreling': 'Female', 'Caitlin Main': 'Female', 'Mackenzie Mills': 'Female', 'Aurelio Miracolo': 'Male', 'Jean Mossman': 'Female', 'Zlatko Nikoloski': 'Male', 'Konstantina Politopoulou': 'Female', 'Elisa Sicuri': 'Female', 'Bryony Simmons': 'Female', 'Jannis Stockel': 'Male', 'Annette Bauer': 'Female', 'Eva-Maria Bonin': 'Female', 'Camille Bou': 'Female', 'Nicola Brimblecombe': 'Female', 'Javiera Cartagena-Farias': 'Female', 'Michael Clark': 'Male', 'Adelina Comas Herrera': 'Female', "Francesco D'Amico": 'Male', 'Jacqueline Damant': 'Female', 'Margaret Dangoor': 'Female', 'Bleddyn Davies': 'Male', 'Josie Dixon': 'Female', 'Sara Evans-Lacko': 'Female', 'José Luis Fernández': 'Male', 'Paul Freddolino': 'Male', 'Emily Freeman': 'Female', 'Daniel Gulliford': 'Male', 'Catherine Henderson': 'Female', 'Dr Ties Hoomans': 'Male', 'Bo Hu': 'Male', 'Maria Karagiannidou': 'Female', 'Derek King': 'Male', 'Martin Knapp': 'Male', 'Klara Lorenz': 'Female', 'Dr Juliette Malley': 'Female', 'Joanna Marczak': 'Female', 'Tihana Matosevic': 'Female', 'David McDaid': 'Male', 'A-La Park': 'Female', 'Dr Sanna Read': 'Female', 'Amritpal Rehill': 'Male', 'Sam Rickman': 'Male', 'Maximilian Salcher-Konrad': 'Male', 'Wagner Silva-Ribeiro': 'Male', 'Madeleine Stevens': 'Female', 'Dr Michela Tinelli': 'Female', 'Gerald Wistow': 'Male', 'Raphael Wittenberg': 'Male', 'Valentina Zigante': 'Female', 'Allen_Tim': 'Male', 'Lydia Assouad': 'Female', 'Professor Jean-Paul Faguet': 'Male', 'Dr Tasha Fairfield': 'Female', 'Professor Tim Forsyth': 'Male', 'Arjan Gjonça': 'Male', 'Stuart Gordon': 'Male', 'Dr Duncan Green': 'Male', 'Elliott Green': 'Male', 'Dr Lloyd Gruber': 'Male', 'tine hanrieder': 'Female', 'Professor Kathryn Hochstetler': 'Female', 'Myfanwy James': 'Female', 'Naila Kabeer': 'Female', 'Sohini Kar': 'Female', 'Professor David Keen': 'Male', 'Tiziana Leone': 'Male', 'Dr Shirin Madon': 'Female', 'Dr Laura Mann': 'Female', 'Professor Kate Meagher': 'Female', 'Philipa Mladovsky': 'Female', 'Joana Naritomi': 'Female', 'Professor James Putzel': 'Male', 'Dr Sandra Sequeira': 'Female', 'Professor Ken Shadlen ': 'Male', 'Mahvish Shami': 'Female', 'Rajesh Venugopal': 'Male', 'Robert Wade': 'Male', 'Diana Weinhold': 'Female', 'Carolin Dieterle': 'Female', 'Dr Stephanie Levy': 'Female', 'Jon Lunn': 'Male', 'Dr Jerome Roos': 'Male', 'Greta Seibel': 'Female', 'Bookang Seol': 'Male', 'Joe Strong': 'Male', 'Jennifer Melvin': 'Female', 'Professor Alaka Basu': 'Female', 'Professor Radhika Desai': 'Female', 'Dr Joseph Hanlon': 'Male', 'Dr Rishita Nandagiri Nandagiri': 'Female', 'Duncan Green': 'Male', 'Kate Gilmore': 'Female', 'Professor David Luke': 'Male', 'Madhuri Agarwal': 'Female', 'Professor E A Brett': 'Male', 'Chao-Yo Cheng': 'Male', 'rim turkmani': 'Male', 'Francisco Ferreira': 'Male', 'James Walters': 'Male', 'Camilo Acero Vargas': 'Male', 'Rahma Ahmed': 'Female', 'Marietta Angeli': 'Female', 'Chiara-Chiavaroli': 'Female', 'Tin Hinane El Kadi': 'Female', 'Isaac Haruna': 'Male', 'Claudia Horn': 'Female', 'Hosna Jahan': 'Female', 'Shinzani Jain': 'Female', 'Pauline Jerrentrup': 'Female', 'Mina Kozluca': 'Female', 'Sophie Legros': 'Female', 'Jorich Johann Loubser': 'Male', 'Richard Mallett': 'Male', 'Bincy Mathew': 'Female', 'Aoife McCullough': 'Female', 'eduardo-mercadante': 'Male', 'Michael Mugisha': 'Male', 'tony neil': 'Male', 'claudia-rodriguez-castellanos': 'Female', 'Gijs van Selm': 'Male', 'harshita-sinha': 'Female', 'Dr Roham Alvandi': 'Male', 'Professor Nigel J. Ashton': 'Male', 'Professor Marc David Baer': 'Male', 'Dr Antony Best': 'Male', 'Dr Anna Cant': 'Female', 'Professor Steven Casey': 'Male', 'Dr Dina Gusejnova': 'Female', 'Dr Tanya Harmer': 'Female', 'Dr Tim Hochstrasser': 'Male', 'Dr Elizabeth Ingleson': 'Female', 'Professor Matthew Jones': 'Male', 'Dr Paul Keenan': 'Male', 'Dr Joanna Lewis': 'Female', 'Professor N. Piers Ludlow': 'Male', 'Dr. Alex Mayhew': 'Male', 'Dr David Motadel': 'Male', 'Dr Artemis Photiadou': 'Female', 'Dr Ronald C. Po': 'Male', 'Professor Paul Preston': 'Male', 'Dr Svetozar Rajak': 'Male', 'Dr Jake Subryan Richards': 'Male', 'Dr Kirsten E. Schulze': 'Female', 'Dr Gagandeep S. Sood': 'Male', 'Dr Aino Rosa Kristina Spohr': 'Female', 'Dr Paul Stock': 'Male', 'Dr Imaobong Umoren': 'Female', 'Dr Qingfei Yin': 'Female', 'Professor Vladislav Zubok': 'Male', 'Armand Azra bin Azlira': 'Male', 'Medha Bhattacharya': 'Female', 'chua': 'Female', 'Dr Rosalind Coffey': 'Female', 'Pedro Correa Martin-Arroyo': 'Male', 'Alexandre Dab': 'Male', 'Charlotte Eaton': 'Female', 'Jonathan Franco': 'Male', 'Dr Sajjan Gohel': 'Male', 'Dr Giovanni Graglia': 'Male', 'Dr Anton Harder': 'Male', 'Jeff Hawn': 'Male', 'Sandip Kana': 'Male', 'Ronan Mainprize': 'Male', 'Dr William Mitchell': 'Male', 'Daniele Giuseppe Palmer': 'Male', 'Dr Agis Papageorgiou': 'Male', 'Dr Ben Sayle': 'Male', 'Gray Sergeant': 'Male', 'Professor Oriana Bandiera': 'Female', 'Professor Tim Besley': 'Male', 'Dr Pawel Bukowski': 'Male', 'Dr Tania Burchardt': 'Female', 'Dr Flora Cornish': 'Female', 'Professor Joan Costa-Font': 'Male', 'Professor Frank Cowell': 'Male', 'Sam Friedman': 'Male', 'Professor Ellen Helsper ': 'Female', 'Professor Simona Iammarino': 'Female', 'Dr Xavier Jaravel': 'Male', 'Professor Stephen Jenkins': 'Male', 'Professor Gareth Jones': 'Male', 'Professor Naila Kabeer': 'Female', 'Dr Eleni Karagiannaki': 'Female', 'Dr Insa Koch': 'Female', 'Professor Nicola Lacey': 'Female', 'Professor Camille Landais': 'Female', 'Professor Neil Lee': 'Male', 'Professor Sumi Madhok': 'Female', 'Dr Abigail McKnight': 'Female', 'Dr Joana Naritomi': 'Female', 'Professor Berkay Ozcan': 'Male', 'Dr Robtel Neajai Pailey': 'Female', 'Professor Lucinda Platt': 'Female', 'Dr Ania Plomien': 'Female', 'Dr Aliya Rao': 'Female', 'Dr Daniel Reck': 'Male', 'Professor Mike Savage': 'Male', 'Professor Alpa Shah': 'Female', 'Dr Jennifer Sheehy-Skeffington': 'Female', 'Dr Isabel Shutes': 'Female', 'Professor David Soskice': 'Male', 'Professor Johannes Spinnewijn': 'Male', 'Dr Andy Summers': 'Male', 'Dr Kate Summers': 'Female', 'Dr Kristin Surak': 'Female', 'Dr Chana Teeger': 'Female', 'Facundo Alvaredo': 'Male', 'Dr Paolo Brunori': 'Male', 'Hazel V. Carby': 'Female', 'Valentina Contreras': 'Female', 'Caroline Falkman Olsson': 'Female', 'Dr Shalini Grover': 'Female', 'Max Herbertson': 'Male', 'Johnathan Inkley': 'Male', 'Beatriz Jambrina Canseco': 'Female', 'H Xavier Jara': 'Male', 'Sarah Kerr': 'Female', 'Aygen Kurt-Dickson': 'Female', 'Maël Lavenaire': 'Male', 'Andrew Lonsdale ': 'Male', 'Mina Mahmoudzadeh': 'Female', 'Fabrício Mendes Fialho': 'Male', 'Annalena Oppel': 'Female', 'Amaia Palencia-Esteban': 'Female', 'César Poux': 'Male', 'Pedro Salas-Rojo': 'Male', 'Violetta van Veen': 'Female', 'Dr Michael Vaughan': 'Male', 'Nora Waitkus': 'Female', 'Jacqueline Mujico': 'Female', 'Sanghamitra Bandyopadhyay': 'Female', 'Madeleine Bunting': 'Female', 'Emma Chamberlain': 'Female', 'Dave Coady': 'Male', 'Ashwini Deshpande': 'Female', 'James Foster': 'Male', 'Carola Frege': 'Female', 'Linda Hantrais': 'Female', 'Lee Elliot Major': 'Male', 'Professor Branko Milanovic': 'Male', 'Jenny Pearce': 'Female', 'Professor Aaron Reeves': 'Male', 'Dr Liz Sayce': 'Female', 'Dr Faiza Shaheen': 'Female', 'Professor Susanne Wessendorf': 'Female', 'Dr Arun Advani': 'Male', 'Mauricio Apablaza': 'Male', 'Dr Yonatan Berman': 'Male', 'Ingrid Bleynat': 'Female', 'David Burgherr': 'Male', 'Rafael Carranza': 'Male', 'Lucas Chancel': 'Male', 'Matthew Collin': 'Male', 'Hai-Anh H. Dang': 'Male', 'Nimesh Dhungana': 'Male', 'Niccolo Durazzi': 'Male', 'Dr Mark Fransham': 'Male', 'Dr Luna Glucksberg': 'Female', 'Dr Charlotte Haberstroh': 'Female', 'Gaby Harris': 'Female', 'Dr Katharina Hecht': 'Female', 'Jason Hickel': 'Male', 'Dr David Hope': 'Male', 'Helen Hughson': 'Female', 'Roberto Iacono': 'Male', 'Ruth Kattumuri': 'Female', 'Dr Tom Kemeny': 'Male', 'Kristina Kolbe': 'Female', 'Niyathi R. Krishna': 'Female', 'Dr Erica Lagalisse': 'Female', 'Paul Lagneau-Ymonet': 'Male', 'Marion-Lieutaud': 'Female', 'Davide Luca': 'Male', 'Dr Jonathan Mijs': 'Male', 'Lara Minkus': 'Female', 'Salvatore Morelli': 'Male', 'Philippa Mullins': 'Female', 'Dr Guido Neidhöfer': 'Male', 'Dr Clive Nwonka': 'Male', 'Dr Tahnee Ooms': 'Female', 'Poornima Paidipaty': 'Female', 'Alessandra Radicati': 'Female', 'Giuliano Resce': 'Male', 'Rebecca Simson': 'Female', 'Anna Powell-Smith': 'Female', 'Joaquin Prieto': 'Male', 'Marco Ranaldi': 'Male', 'Soham Sahoo': 'Male', 'Giovanna Scarchilli': 'Female', 'Elisabeth Schimpfössl': 'Female', 'Dr Paul Segal': 'Male', 'Dr Laura Sochas': 'Female', 'Dhananjayan Sriskandarajah': 'Male', 'Dr Joel Suss': 'Male', 'Dr Gaston Yalonetzky': 'Male', 'Asmaa Akhtar': 'Female', 'Dr Sara Camacho Felix': 'Female', 'Professor Armine Ishkanian': 'Female', 'George Kunnath': 'Male', 'Anastasia Nazaryan': 'Female', 'Professor Francisco H. G. Ferreira': 'Male', 'Professor Kirsten Sehnbruch': 'Female', 'Liza Ryan': 'Female', 'Professor J. Chris Alden': 'Male', 'Martin Bayly': 'Male', 'Federica Bicchi': 'Female', 'Ida Roland Birkvad': 'Female', 'Tarsis Daylan Brito Sepulveda-Coelho': 'Male', 'Lucinda Cadzow': 'Female', 'William Callahan': 'Male', 'Jeffrey Chwieroth': 'Male', 'Thomas Da Costa Vieira': 'Male', 'Katerina Dalacoura': 'Female', 'Shikha Dilawri': 'Female', 'Toby Dodge': 'Male', 'Alice Engelhard': 'Female', 'Robert Falkner': 'Male', 'Fawaz Gerges': 'Male', 'getmansky': 'Female', 'Patrick Gill-Tiney': 'Male', 'Sinja Graf': 'Female', 'Jurgen Haacke': 'Male', 'Jonny Hall': 'Male', 'Mirko Heinzel': 'Male', 'Joe Dacre Higton': 'Male', 'Kira Huju': 'Female', 'Nikhil Kalyanpur': 'Male', 'Koenig-Archibugi': 'Male', 'Milli Lake': 'Female', 'Tomila Lankina': 'Female', 'Boram Lee': 'Female', 'Jens Meierhenrich': 'Male', 'Katharine M Millar': 'Female', 'Irene Morlino': 'Female', 'Dr James Morrison': 'Male', 'Dr Rohan Mukherjee': 'Male', 'Natalya Naqvi': 'Female', 'Victoria Paniagua': 'Female', 'Fabian Pape': 'Male', 'McKenzie Ratner': 'Female', 'Howie Rechavia-Taylor': 'Male', 'Stephanie Schwartz': 'Female', 'Giulia Sciorati': 'Female', 'Ulrich Sedelmeier': 'Male', 'Professor John Sidel': 'Male', 'Karen Smith': 'Female', 'Marta Soprana': 'Female', 'Theresa Squatrito': 'Female', 'Dimitrios Stroikos': 'Male', 'Lauren Sukin': 'Female', 'Luca Tardelli': 'Male', 'Peter Trubowitz': 'Male', 'Peter Wilson': 'Male', 'Alexandros Zachariades': 'Male', 'Mariia Zolkina': 'Female', 'Noah Zucker': 'Male', 'Amy Brook': 'Female', 'Mark Bryceland': 'Male', 'Hermione Burnett': 'Female', 'Alison Carter': 'Female', 'Matthew Gaisford St Lawrence': 'Male', 'Joyce Harvey': 'Female', 'Sarah Hélias': 'Female', 'Katy Hill': 'Female', 'Adnan Khan': 'Male', 'Romy Mokogwu': 'Female', 'Nicholas Ortiz': 'Male', 'Claire Ridge': 'Female', 'Amanda Schultz': 'Female', 'Andrew Sherwood': 'Male', 'Rosie Springett': 'Female', 'Giovanni Angioni': 'Male', 'Hassan Damluji': 'Male', 'Eyüp Ersoy': 'Male', 'Gubad Ibadoghlu': 'Male', 'Paul Kirby': 'Male', 'Anahita Motazed Rad': 'Female', 'Michael Reynolds': 'Male', 'Dr Geoffrey Swenson': 'Male', 'Jianyong Yue': 'Male', 'Chris Brown': 'Male', 'Barry Buzan': 'Male', 'Professor Michael Cox': 'Male', 'Chris Hughes': 'Male', 'Margot Light': 'Female', 'Professor James Mayall': 'Male', 'Paul Taylor': 'Male', 'Stephen Woolcock': 'Male', 'Michael Yahuda': 'Male', 'Albert Cullell Cano': 'Male', 'Emanuele Errichiello': 'Male', 'Elizabeth Humphrey': 'Female', 'Julie Lespinasse': 'Female', 'Shireen Manocha': 'Female', 'Chloé Raïd': 'Female', 'Hadrien T. Saperstein': 'Male', 'Woohyeok Seo': 'Male', 'Mehdi Shakarchi': 'Male', 'Mariah Thornton': 'Female', 'Sara Wong': 'Female', 'Agnes Yu': 'Female', 'Dr Neil McLean': 'Male', 'Dr Alison Standring': 'Female', 'John Heyworth': 'Male', 'Suzie Beckley': 'Female', 'Sabrina Rabi': 'Female', 'Maggie Evans': 'Female', 'Inés Alonso-García': 'Female', 'Helen Mayer': 'Female', 'Lourdes Hernández-Martín': 'Female', 'Luay Hasan': 'Male', 'Nadim Ben-Mohamed-Mahjoub': 'Male', 'Dr. Catherine Hua Xiang': 'Male', 'Dr Lijing Shi': 'Female', 'Hongyi Xin': 'Male', 'Fang Pan': 'Female', 'Rong Guo': 'Female', 'Alice Yao': 'Female', 'Fei Yuan': 'Male', 'Yoko Aldous': 'Female', 'Yukie Yagioka': 'Female', 'Mihi Kang': 'Male', 'Mike McGarvey': 'Male', 'Simon Roberts': 'Male', 'Jim Pavitt': 'Male', 'Simon Mathews': 'Male', 'François Simon': 'Male', 'Sandrine Victor': 'Female', 'Bérengère Genest': 'Female', 'Virginie Rodriguez': 'Female', 'Cécile Alais': 'Female', 'Pierre Dagonnot': 'Male', 'Catherine Gaitte': 'Female', 'Patricia Gaudron': 'Female', 'Patricia Morabo': 'Female', 'Magali Brenac': 'Female', 'Dr. Peter Skrandies': 'Male', 'Florian Fischer': 'Male', 'Dr Manuela von Papen': 'Female', 'Angela Curtis-Landolt': 'Female', 'Dr. Angus Wrenn': 'Male', 'Dr. Olga Sobolev': 'Male', 'Natasha Bershadski': 'Female', 'Irina Forbes': 'Female', 'Elena Dearden': 'Female', 'Larisa Leifland Berntsson': 'Female', 'Mercedes Coca': 'Female', 'Dr. Roser Martinez-Sanchez': 'Male', 'Margarita Rois-Quiroga': 'Female', 'Pablo Sánchez-García': 'Male', 'Anna Maria Giuffria': 'Female', "Flavia D'Angelantonio": 'Female', 'David Kershaw': 'Male', 'Professor Veerle Heyvaert': 'Male', 'Cressida Auckland': 'Female', 'Professor Susanne Baer': 'Female', 'Eduardo Baistrocchi': 'Male', 'Julia Black': 'Female', 'Michael Blackwell': 'Male', 'Jacco Bomhoff': 'Male', 'Jo Braithwaite': 'Female', 'Nafay Choudhury': 'Male', 'Hugh Collins': 'Male', 'Sir Ross Cranston': 'Male', 'Niamh Dunne': 'Female', 'Neil Duxbury': 'Male', 'Tatiana Flessas': 'Female', 'Conor Gearty': 'Male', 'Hannah Gibbs': 'Female', 'Suren Gomtsyan': 'Male', 'Alperen Gözlügöl': 'Male', 'Oliver Hailes': 'Male', 'Veerle Heyvaert': 'Female', 'Jeremy Horder': 'Male', 'Devika Hovell': 'Female', 'Elizabeth Howell': 'Female', 'Stephen Humphreys': 'Male', 'Martin Husovec': 'Male', 'Pablo Ibáñez Colomo': 'Male', 'Tarun Khaitan': 'Male', 'Giulia Leonelli': 'Female', 'Rachel Leow': 'Female', 'Tim Liau': 'Male', 'Martin Loughlin': 'Male', 'Orla Lynskey': 'Female', 'Paul MacMahon': 'Male', 'Susan Marks': 'Female', 'Richard Martin': 'Male', 'Luke McDonagh': 'Male', 'Eva Micheler': 'Female', 'Kai Moller': 'Male', 'Niamh Moloney': 'Female', 'Jo Eric Khushal Murkens': 'Male', 'Sonya Onwu': 'Female', 'Abenaa Owusu-Bempah': 'Female', 'Philipp Paech': 'Male', 'Sarah Paterson': 'Female', 'Mona Paulsen': 'Female', 'Marie Petersmann': 'Female', 'Federico Picinali': 'Male', 'Thomas Poole': 'Male', 'Peter Ramsay': 'Male', 'Nick Sage': 'Male', 'Margot Salomon': 'Female', 'Astrid Sanders': 'Female', 'Edmund Schuster': 'Male', 'Andrew Scott': 'Male', 'Gerry Simpson': 'Male', 'Joseph Spooner': 'Male', 'Igor Stramignoni': 'Male', 'Andrew Summers': 'Male', 'Sivaramjani Thambisetty Ramakrishna': 'Female', 'Sarah Trotter': 'Female', 'Emmanuel Voyiakis': 'Male', 'Alex Waghorn': 'Male', 'Charlie Webb': 'Male', 'Michael Wilkinson': 'Male', 'Roxana Willis': 'Female', 'Floris de Witte': 'Male', 'Dame Sarah Worthington': 'Female', 'Ayse Gizem Yasar': 'Female', 'Jan Zglinski': 'Male', 'Robert Baldwin': 'Male', 'Anne Barron': 'Female', 'Joanna Benjamin': 'Female', 'Michael Bridge': 'Male', 'Vanessa Finch': 'Female', 'Carol Harlow': 'Female', 'Trevor Hartley': 'Male', 'Tim Murphy': 'Male', 'Jill Peay': 'Female', 'Robert Reiner': 'Male', 'Michael Zander': 'Male', 'Grigorios Bacharis': 'Male', 'Alex C. Evans': 'Female', 'Ciara Hurley': 'Female', 'Lora Izvorova': 'Female', 'Szymon Osmola': 'Male', 'Jonathan Fisher QC': 'Male', 'Mary Stokes': 'Female', 'Simon Witney': 'Male', 'Karen Banks': 'Female', 'Baroness Chakrabarti': 'Female', 'Sarah Bauerle Danzman': 'Female', 'Dr. Michal Agmon-Gonnen': 'Female', 'Thomas Grant KC': 'Male', 'Christos Hadjiemmanuil': 'Male', 'Dr Thorsten Kaeseberg': 'Male', 'Baroness Helena Kennedy': 'Female', 'Jan Kleinheisterkamp': 'Male', 'David Lammy': 'Male', 'LTS Research directory': 'Male', 'David Lock': 'Male', 'Stephanie Maguire': 'Female', 'Dr David Murphy': 'Male', 'Elisabeth Noble': 'Female', 'Jessica Simor QC': 'Female', 'Diego Acosta': 'Male', 'Dr Stephanie Classmann': 'Female', 'Daniel Joyce': 'Male', 'Michael Lobban': 'Male', 'Joana Mendes': 'Female', 'Doreen Lustig': 'Female', 'Meredith Rossner': 'Female', 'Ian Roxan': 'Male', 'Grégoire Webber': 'Male', 'Ting Xu': 'Female', 'Nath Gbikpi': 'Female', 'Kathryn Davis': 'Female', 'Adrien de Hauteclocque': 'Male', 'Sam McAlister': 'Female', 'Dr Bethania Antunes': 'Female', 'Dr Sarah Ashwin': 'Female', 'Dr Jonathan E. Booth': 'Male', 'Wafaa Elmezraoui': 'Female', 'Karin King': 'Female', 'Dr Chunyun Li': 'Female', 'Dr Francesca Manzi': 'Female', 'Dr Elizabeth Stillwell': 'Female', 'ksun': 'Female', 'Dr Jeffrey Thomas': 'Male', 'Dr Frido Wenten': 'Male', 'Professor Chrisanthi Avgerou': 'Female', 'Professor Michael Barzelay': 'Male', 'Dr Aaron Cheng': 'Male', 'Dr Antonio Cordella': 'Male', 'jcgiraldomora': 'Male', 'Dr Francesco Gualdi': 'Male', 'Dr Ela Klecun': 'Female', 'Dr Antti Lyyra': 'Male', 'Professor Susan Scott': 'Female', 'Dr Carsten Sørensen': 'Male', 'Dr Edgar Whitley': 'Male', 'Dr Ricardo Alonso': 'Male', 'Dr Jordi Blanes i Vidal': 'Male', 'Dr Raquel Campos': 'Female', 'Nilesh Dittani': 'Male', 'David de Meza': 'Male', 'Professor Daniel Gottlieb': 'Male', 'Dr Satoshi Kanazawa': 'Male', 'Dr Rocco Macchiavello': 'Male', 'Dr Kristóf Madarász': 'Male', 'Dr Veronica Rappoport': 'Female', 'Professor Diane Reyniers': 'Female', 'Dr Jörn Rothe': 'Male', 'Professor Yona Rubinstein': 'Male', 'Dr Catherine Thomas': 'Female', 'Dr Jose P. Vasquez': 'Male', 'Professor Noam Yuchtman': 'Male', 'Professor Amitav Chakravarti': 'Male', 'Dr Heather Kappes': 'Female', 'Dr Hye-Young Kim': 'Female', 'Dr Xiaolin LI': 'Female', 'Professor Om Narasimhan': 'Male', 'Dr Nicolette Sullivan': 'Female', 'Professor Naufel Vilcassim': 'Male', 'Dr Nayat Horozoglu': 'Female', 'Professor Richard Steinberg': 'Male', 'Dr Nicole Abi-Esber': 'Female', 'Dr Yally Avrahampour': 'Male', 'Professor Harry Barkema': 'Male', 'Dr Rebecca Campbell': 'Female', 'Dr Christine Cote': 'Female', 'Dr Shoshana Dobrow Riza': 'Female', 'Dr Barbara Fasolo': 'Female', 'Helen Hu': 'Female', 'Dr Xiaoran Hu': 'Male', 'Dr Niranjan Janardhanan': 'Male', 'Dr Hyun-Jung Lee': 'Female', 'Dr Xu Li': 'Male', 'Professor Connson Locke': 'Female', 'Dr Nadia Millington': 'Female', 'Dr Lidiia Pletneva': 'Female', 'Luc Schneider': 'Male', 'Dr Emma Soane': 'Female', 'Dr Lourdes Sosa': 'Female', 'avenkatesan': 'Female', 'Chunlin Yu': 'Male', 'Dr Jonathan Liebenau': 'Male', 'Virginia Fedrigo': 'Female', 'Professor Sarosh Kuruvilla': 'Male', 'Professor Saonee Sarker': 'Male', 'Sir Geoffrey Owen': 'Male', 'Dr Susanna Khavul': 'Female', 'Dr Ihor Lantukh': 'Male', 'Dr Rebecca Newton': 'Female', 'Dr Katie Best': 'Female', 'Dr Nurul Liyana Mohd Kamil': 'Female', 'Dr Daniela Lup': 'Female', 'Dr Michele Fioretti': 'Male', 'Dr Henry Hang Shen': 'Male', 'Dr Dina Rabie': 'Female', 'Professor Peter Abell': 'Male', 'Professor Ian Angell': 'Male', 'Professor Gautam Appa': 'Male', 'Dr James Backhouse': 'Male', 'Professor Gwyn Bevan': 'Male', 'Professor Jacqueline Coyle-Shapiro': 'Female', 'Professor Saul Estrin': 'Male', 'Professor Carola Frege': 'Female', 'Professor Richard Hyman': 'Male', 'Professor Jannis Kallinikos': 'Male', 'Professor Frank Land': 'Male', 'Professor David Metcalf': 'Male', 'Professor Alexander (Sandy) Pepper': 'Male', 'Professor Larry Phillips': 'Male', 'Professor Jonathan Rosenhead': 'Male', 'Professor Leslie Willcocks': 'Male', 'Professor Paul Williams': 'Male', 'Professor Paul Willman': 'Male', 'Dr Mohamed Abouaziza': 'Male', 'Dr Philipp Schoenegger': 'Male', 'Oliver Seager': 'Male', 'Sazid Ahmad': 'Male', 'Tanya Alaaldin': 'Female', 'Aaron Aujla': 'Male', 'Alastair Buckle': 'Male', 'Yiting Dong': 'Female', 'Sun Woo Nam': 'Female', 'Roop Bhadury': 'Male', 'Lisa Di Natale': 'Female', 'Philipp Gagel': 'Male', 'Yingjia Hu': 'Female', 'Susanne Klausing': 'Female', 'Georgia Meyer': 'Female', 'Rohit Nair': 'Male', 'Anulekha Nandi': 'Female', 'Muhammad Aqeel Awan': 'Male', 'Anna Czaplewska-Jaffery': 'Female', 'Caleb Phillips': 'Male', 'Rashi Sonal': 'Female', 'Ravneet Bawa': 'Female', 'Areum Cho': 'Female', 'Ceylin Ertekin': 'Female', 'Earl Panganiban': 'Male', 'Zoe Ziqi Zhong': 'Male', 'Luca Barbato': 'Male', 'David Cai': 'Male', 'José Ignacio González-Rojas': 'Male', 'Romaine Loubes': 'Female', 'Ottavia Pesenti': 'Female', 'Stephan Chambers': 'Male', 'Jonathan Roberts': 'Male', 'Kerryn Krige': 'Female', 'Julian Le Grand': 'Male', 'Christian Busch': 'Male', 'Dr Ahmad Abdi': 'Male', 'Professor Martin Anthony': 'Male', 'Galit Ashkenazi-Golan': 'Female', 'Professor Julia Böttcher': 'Female', 'Professor Graham Brightwell': 'Male', 'Dr Christoph Czichowsky': 'Male', 'Dr Albina Danilova': 'Female', 'Dr Pavel Gapeev': 'Male', 'Professor Olivier Gossner': 'Male', 'Jan van den Heuvel': 'Male', 'Grammateia Kotsialou': 'Female', 'Dr Ioannis Kouletsis': 'Male', 'Professor Andrew Lewis-Pye': 'Male', 'Dr Arne Lokka': 'Male', 'Neil Olver': 'Male', 'Professor Adam Ostaszewski': 'Male', 'Emilio Pierro': 'Male', 'Professor Amol Sasane': 'Male', 'Dr Robert Simon': 'Male', 'Professor Jozef Skokan': 'Male', 'Professor Gregory Sorkin': 'Male', 'Bernhard Von Stengel': 'Male', 'Professor Konrad Swanepoel': 'Male', 'Professor Luitgard Veraart': 'Female', 'James Ward': 'Male', 'Aled Williams': 'Male', 'Giacomo Zambelli': 'Male', 'Mihalis Zervos': 'Male', 'Elad Aigner-Horev': 'Male', 'Steve Alpern': 'Male', 'June Barrow-Green': 'Male', 'Norman Biggs': 'Male', 'Professor Nick Bingham': 'Male', 'Richard Cole': 'Male', 'David Collier': 'Male', 'Paul Duetting': 'Male', 'Michal Feldman': 'Female', 'Thomas Lidbetter': 'Male', 'Benny Sudakov': 'Male', 'Peyton Young': 'Male', 'Jose Armando Vivero': 'Male', 'Rowland Bennett': 'Male', 'Johannes Brustle': 'Male', 'Jun Cheng': 'Male', 'Viet Dang': 'Male', 'Francesco Di Braccio': 'Male', 'Carmelo Di Natale': 'Male', 'Sam Fendrich': 'Male', 'Áron Fülöp': 'Male', 'Erin Hales': 'Female', 'Eleni Katirtzoglou': 'Female', 'Jasmin Katz': 'Female', 'Boris Krippa': 'Male', 'Arthur Lloyd-Phillips': 'Male', 'Daniil Mamaev': 'Male', 'Mihir Neve': 'Male', "Laurence O'Toole": 'Male', 'Vasos Pavlika': 'Male', 'Calum Robson': 'Male', 'Niccolo Salvatori': 'Male', 'Amedeo Sgueglia': 'Male', 'David Solomon': 'Male', 'Yueying Sun': 'Female', 'Mohamed Tawfik': 'Male', 'Haoxiang Wang': 'Male', 'Tony Whelan': 'Male', 'Michael Yiasemides': 'Male', 'Hengjian Zhang': 'Male', 'Lu-Ming Zhang': 'Female', 'Dr George Zouros': 'Male', 'Kate Barker': 'Female', 'Rebecca Batey': 'Female', 'Jasna Begić': 'Female', 'Sharon Donegal-Grant': 'Female', 'Sarah Massey': 'Male', 'Elvina Moonien': 'Female', 'Dr Sharat Ibrahimpur': 'Male', 'Mehdi Makhul': 'Male', 'Dr Yani Pehova': 'Female', 'Mahsa Dalirrooy-Fard': 'Female', 'Justin Gwee': 'Male', 'Sahar Jahani': 'Female', 'Joanna Lada': 'Female', 'Zhesheng Liu': 'Male', 'Haoyuan Ma': 'Male', 'Domenico Mergoni': 'Male', 'Raymond Pang': 'Male', 'Edward Plumb': 'Male', 'Karl Stickler': 'Male', 'Cameron Strachan': 'Male', 'Ruilan Wang': 'Female', 'Luming Zhang': 'Female', 'Yuliang Zhang': 'Male', 'Professor Bart Cammaerts': 'Male', 'Professor Lee Edwards': 'Female', 'Dr Seeta Peña Gangadharan': 'Female', 'James Deeley': 'Male', 'Omar Al-Ghazzi': 'Male', 'Dr Asif Ali Akhtar': 'Male', 'Nick Anstead': 'Male', 'Nick Couldry': 'Male', 'Dr Lee Edwards': 'Female', 'Seeta Peña Gangadharan': 'Female', 'Professor Myria Georgiou': 'Female', 'Suzanne Temwa Gondwe Harris': 'Female', 'Ellen Helsper': 'Female', 'César Jiménez Martínez': 'Male', 'Sonia Livingstone': 'Female', 'Robin Mansell': 'Female', 'Bingchun Meng': 'Female', 'Dr Dylan Mulvin': 'Male', "Dr Rachel O'Neill": 'Female', 'Shani Orgad': 'Female', 'Jean-Christophe Plantin': 'Male', 'Alison Powell': 'Female', 'Terhi Rantanen': 'Female', 'Philipp Seuferling': 'Male', 'Damian Tambini': 'Male', 'jungmo youn': 'Male', 'Alessandro Castellini': 'Male', 'Dr Kate Gilchrist': 'Female', 'Dr Leslie Haddon': 'Male', 'Dr Jess Hannah': 'Female', 'Dr Maria Paola Pofi': 'Female', 'Professor Ahmet Atay': 'Male', 'Meghanne Barker': 'Female', 'Professor Sarah Banet-Weiser': 'Female', 'Professor Benedetta Brevini': 'Female', 'Dr Patricio Cabello': 'Male', 'Dr Helena Chmielewska-Szlajfer': 'Female', 'Dr Bei Deng': 'Male', 'Dr Torsten Geelan': 'Male', 'Dr Yael Gordon': 'Female', 'Alison Harcourt': 'Female', 'Velislava Hillman': 'Female', 'Dr Weizi Huang': 'Female', 'Yasemin İnceoğlu': 'Female', 'Professor Baharul Islam': 'Male', 'Reidar S Jessen': 'Male', 'Professor Christian Katzenbach': 'Male', 'Dr Anthony Kelly': 'Male', 'Baroness Beeban Kidron OBE': 'Female', 'Angelos Kissas': 'Male', 'Professor Jim Macnamara': 'Male', 'Dr Ewa Majczak': 'Female', 'Dr Jose-Carlos Mariategui': 'Male', 'Dr Terese Mendiguren': 'Female', 'Claire Milne': 'Female', 'Dr Rian Mulcahy': 'Male', 'Professor Hannu Nieminen': 'Male', 'Jedrzej Niklas': 'Male', 'Dr Gabriel Pereira': 'Male', 'Kruakae Pothong': 'Female', 'Miriam Rahali': 'Female', 'Dr Maggie Scammell': 'Female', 'Professor Philip Schlesinger': 'Male', 'Professor Hannele Seeck': 'Female', 'Professor Julian Sefton-Green': 'Male', 'Professor Katherine Sender': 'Female', 'Jonny Shipp': 'Male', 'Dr Johanna Sumiala': 'Female', 'Dr Jane Vincent': 'Female', 'Dr Nina Vindum Rasmussen': 'Female', 'Dr Grace Yuehan Wang': 'Female', 'Fiona Abades Barclay': 'Female', 'Dr Fabian Broeker': 'Male', 'Emma Goodman': 'Female', 'Diego Martinez': 'Male', 'Dr Pablo Morales': 'Male', 'Esra Ozkan': 'Female', 'Didem Ozkul': 'Female', 'Sanne Stevens': 'Female', 'Mariya Stoilova': 'Female', 'Tshepo Tshabalala': 'Male', 'Postdoctoral research fellowships': 'Male', 'Johan Ahlback': 'Male', 'Audrey Alejandro': 'Female', 'Eileen Alexander': 'Female', 'Professor Kenneth Benoit': 'Male', 'Dr. Flora Cornish': 'Female', 'Daniel De Kadt': 'Male', 'Zach Dickson': 'Male', 'Michael Ganslmeier': 'Male', 'Professor George Gaskell': 'Male', 'Friedrich Geiecke': 'Male', 'Patrick Gildersleve': 'Male', 'Isaac Haruna Ziaba': 'Male', 'Professor Jonathan Jackson': 'Male', 'Dr. Ellie Knott': 'Female', 'Professor Jouni Kuha': 'Male', 'Anne-Laure Mahe': 'Female', 'Blake Miller': 'Male', 'Seila Panizzolo': 'Female', 'Elly Power': 'Female', 'Aliya Rao': 'Female', 'Thomas Robinson': 'Male', 'Sally Stares': 'Female', 'Raphael Susewind': 'Male', 'Patrick Sturgis': 'Male', 'Dr. Chana Teeger': 'Female', 'Dr. Milena Tsvetkova': 'Female', 'Siân Brooke': 'Female', 'Islam al Khatib': 'Male', 'Amal Ali': 'Female', 'Akriti Mehta': 'Female', 'Nancy Breton': 'Female', 'Izzi Carter': 'Female', 'Christy Coulson': 'Male', 'Qi Cui': 'Male', 'Midanna de Almada': 'Female', 'Brandon Green': 'Male', 'Yuanmo He': 'Male', 'Katya Kostadintcheva': 'Female', 'Roni Küppers': 'Male', 'Rosanna Le Voir': 'Female', 'Christian Mueller': 'Male', 'Jasmine Oware': 'Female', 'Livio Miles Silva Muller profile page': 'Male', 'Chuyao Wang': 'Male', 'Wanting Wang': 'Female', 'Ian Brunton-Smith': 'Male', 'Aroop Chatterjee': 'Male', 'Patrick English': 'Male', 'Daniele Fanelli': 'Male', 'Monica Gerber': 'Female', 'Jennifer Hochschild': 'Female', 'Florian Kern': 'Male', 'Sonja Marzi': 'Female', 'Matthew Mulford': 'Male', 'Cohen Simpson': 'Male', 'Laura Pantzerhielm': 'Female', 'Chris Posch': 'Male', 'Martina Testori': 'Female', 'Joshua Townsley': 'Male', 'Dr Ben Wilson': 'Male', 'Prof J. McKenzie Alexander': 'Male', 'Catherine Audard': 'Female', 'Prof Nick Baigent': 'Male', 'Lukas Beck': 'Male', 'Prof Jonathan Birch': 'Male', 'Ali Boyle': 'Male', 'Prof Richard Bradley': 'Male', 'Liam Kofi Bright': 'Male', 'Harriet Fagerberg': 'Female', 'Prof Roman Frigg': 'Male', 'Prof Fernand Gobet': 'Male', 'Dr. Daniel Guillery': 'Male', 'Remco Heesen': 'Male', 'Prof Christian List': 'Male', 'Linnea Luuppala': 'Female', 'Prof Anna Mahtani': 'Female', 'Prof Miklós Rédei': 'Male', 'Bryan Roberts': 'Male', 'Lewis Ross': 'Male', 'Johanna Thoma': 'Female', 'Nikhil Venkatesh': 'Male', 'Prof Alex Voorhoeve': 'Male', 'Kate Vredenburgh': 'Female', 'Prof Charlotte Werndl': 'Male', 'Prof John Worrall': 'Male', 'Wesley Wrigley': 'Male', 'Jingyi Wu': 'Female', 'Dr Alex Gillespie': 'Male', 'Dr Frédéric Basso': 'Male', 'Professor Martin W Bauer': 'Male', 'Professor Paul Dolan': 'Male', 'Jeremy Ginges': 'Male', 'Dr Laura M Giurge': 'Female', 'Dr Ilka Gleibs ': 'Female', 'Dr Maxi Heitmayer': 'Male', 'Dr Ishan Jalan': 'Male', 'Professor Sandra Jovchelovitch': 'Female', 'Dr Kate Laffan': 'Female', 'Professor Saadi Lahlou': 'Male', 'Dr Grace Lordan': 'Female', 'Dr Tom Reader': 'Male', 'Dr Jet G.Sanders': 'Female', 'Professor Elizabeth Stokoe': 'Female', 'Alina Velias': 'Female', 'Dr Edoardo Zulato': 'Male', 'Gosia Brown': 'Female', 'Dr Bankole Falade': 'Male', 'Dr Ivan Kroupin': 'Male', 'Dr Georgios Melios': 'Male', 'Dr Maria Cecilia Dedios-Sanguineti': 'Female', 'Rachel Spicer': 'Female', 'Karin Ahlbäck': 'Female', 'Robin Bachmann': 'Male', 'Aishwarya Bellam': 'Female', 'Erika Brodnock': 'Female', 'Julia Buzan': 'Female', 'Ivan Cano-Gomez': 'Male', 'Jaesun Chae': 'Female', 'Hannah Chappell': 'Female', 'Alexandra Chesterfield': 'Female', 'Adam Davidson': 'Male', "Dallas O'Dell": 'Female', 'Ethan Dias': 'Male', 'Laura Fonseca-Durán': 'Female', 'Alex Goddard': 'Male', 'Benno Guenther': 'Male', 'Julie Hald': 'Female', 'Katharina Heby': 'Female', 'Juliet Hodges': 'Female', 'Pinar Jenkins': 'Female', 'Bessi Jonsdottir': 'Female', 'Alexandra Kirienko': 'Female', 'Fatima Koaik': 'Female', 'Veronika Luptáková': 'Female', 'Nils Mallock': 'Male', 'Nikita': 'Female', 'Atrina Oraee': 'Female', 'Alyssa Pandolfo': 'Female', 'Leah Jeongwon Park': 'Female', 'Daniele Pollicino': 'Male', 'Pia Marie Pose': 'Female', 'Sharon Raj': 'Female', 'Anandita Sabherwal': 'Female', 'Eric Schnell': 'Male', 'Joseph Sherlock': 'Male', 'Haoran Shi': 'Male', 'Anisha Singh': 'Female', 'Andy Stewart': 'Male', 'Chen-Ta Sung': 'Male', 'Audrey Van Hoecke': 'Female', 'Feiyang Wang': 'Female', 'Heidi Zamzow': 'Female', 'Katherine Ziegelbauer': 'Female', 'Professor Catherine Campbell': 'Female', 'Professor Patrick Humphreys': 'Male', 'Dr Janet Stockdale': 'Female', 'Dr Kavita Abraham-Dowsing': 'Female', 'Paul Adams': 'Male', 'Nihan Albayrak-Aydemir': 'Female', 'dr sanchayan banerjee': 'Male', 'Dr Denise Baron': 'Female', 'Dr Natalia Concha': 'Female', 'Elisabeth Costa': 'Female', 'Dr Fangming Cui': 'Female', 'Dr Marta Entradas': 'Female', 'Professor Natalie Gold': 'Female', 'Dr Caroline Howarth': 'Female', 'Dr Edward Hughes': 'Male', 'Dr Anni Kajanus': 'Female', 'Dr Meena Kotecha': 'Female', 'Dr Ploutarchos Kourtidis': 'Male', 'George Loewenstein': 'Male', 'Dr Lucia Macchia': 'Female', 'Professor Ivana Markova': 'Female', 'Dr Aleksandar Matic': 'Male', 'Dr Liora Moskovitz': 'Female', 'Dr Jeroen Nieboer': 'Male', 'Dr Cathy Nicholson': 'Female', 'Dr Sandra Obradović': 'Female', 'Dr Celestin Okoroji': 'Male', 'Dr Marcelo Ramella': 'Male', 'Dr Jessica Rea': 'Female', 'Dr Barry Rogers': 'Male', 'Dr Gordon Sammut': 'Male', 'Dr Lucia Sell-Trujillo': 'Female', 'Dr Ben Shenoy': 'Male', 'Professor Ahmet Suerdem': 'Male', 'Dr Umar Taj': 'Male', 'Dr Chris Tennant': 'Male', 'Dr Alessia Tosi': 'Female', 'Professor Ben G. Voyer': 'Male', 'Dr Paulius Yamin-Slotkus': 'Male', 'Scott Young': 'Male', 'Andrés Velasco': 'Male', 'Vanessa Rubio-Márquez': 'Female', 'Lloyd Gruber': 'Male', 'Dr Simon Bastow': 'Male', 'Professor Nava Ashraf': 'Female', 'Dr Gharad Bryan': 'Male', 'Professor Robin Burgess': 'Female', 'Professor Stephan Chambers': 'Male', 'Richard Davies': 'Male', 'Chenyang He': 'Female', 'Professor Sir Julian Le Grand': 'Male', 'Professor Rafael Hortala-Vallve': 'Male', 'Dr Ethan Ilzetzki': 'Male', 'Casey Kearney': 'Male', 'Professor Richard Layard': 'Male', 'Frank Muci': 'Male', 'Dr Berkay Ozcan': 'Male', 'Professor John Van Reenen': 'Male', 'Dr Jonathan Roberts': 'Male', 'Dr Philipp Rode': 'Male', 'Dr Lewis Ross': 'Male', 'Professor Mark Schankerman': 'Male', 'Dr Johannes Spinnewijn': 'Male', 'Professor Daniel Sturm': 'Male', 'Dr Savvas Verdis': 'Male', 'Professor Alex Voorhoeve': 'Male', 'Ignacio Banares-Sanchez': 'Male', 'Dr Daniel Brieba': 'Male', 'Dr Nuno F da Cruz': 'Male', 'Miguel Fajardo-Steinhauser': 'Male', 'Dr Omar Hammoud Gallego': 'Male', 'Marta Guasch-Rusinol': 'Female', 'Daniel Guillery': 'Male', 'Aung Hein': 'Male', 'Benedikt Margraf': 'Male', 'Dr Zhamilya Mukasheva': 'Female', 'Dr Eugenia Nazrullaeva': 'Female', 'Ekaterina Oparina': 'Female', 'Veronica Salazar-Restrepo': 'Female', 'Patrick Schneider ': 'Male', 'Jayatheerth Seethraman ': 'Male', 'Thomas Stephens': 'Male', 'Yuanhang': 'Male', 'Rt Hon Sir Vince Cable': 'Male', 'Hugh Cole': 'Male', 'Dimitri Demekas': 'Male', 'Ricardo Hausmann': 'Male', 'Laura Gilbert': 'Female', 'Valeria Gontareva': 'Female', 'Tim Leunig': 'Male', 'Ousmène Jacques Mandeng': 'Male', 'Dzhamilya Nigmatulina': 'Female', 'Charles Okehalam': 'Male', 'Lant Pritchett': 'Male', 'Valerio Riavez': 'Male', 'Michael Reid': 'Male', 'Dr Christopher Sabatini ': 'Male', 'Christopher Schildt': 'Male', 'Elod Takats': 'Male', 'Professor Ricky Burdett': 'Male', 'Professor Jo Beall': 'Female', 'Dr Binchung Meng': 'Female', 'Dr Timothy Hildebrandt': 'Male', 'Dr Tanvi Deshpande': 'Female', 'Yuezhou Yang': 'Female', 'Rahat Siddique': 'Female', 'Andrea Correa': 'Female', 'Camila Arroyo From': 'Female', 'Sheyla Enciso-Valdivia': 'Female', 'Jim Osborne': 'Male', 'pilar-frogley': 'Female', 'Caroline Boswell': 'Female', 'Jemima Warren': 'Female', 'Doreen Thompson-Addo': 'Female', 'Polly Liouta': 'Female', 'carolina-stern': 'Female', 'Andrew Brennen': 'Male', 'Georgina Whitham': 'Female', 'Sarah-Massey': 'Female', 'Irene Bucelli': 'Female', 'Dr Fabio Battaglia': 'Male', 'Dr Thomas Biegert': 'Male', 'Dr Shekhar Chandra': 'Male', 'Dr Leonidas Cheliotis': 'Male', 'Dr Shuang Chen': 'Female', 'Dr Sonia Exley': 'Female', 'Professor Timo Fleckenstein': 'Male', 'Fiona Gogescu': 'Female', 'Dr Mobarak Hossain': 'Male', 'Dr Johann Koehler': 'Male', 'Dr Sunil Kumar': 'Male', 'Dr Youngcho Lee': 'Female', 'Dr Zahid Mumtaz': 'Male', 'Professor Tim Newburn': 'Male', 'Professor Adam Oliver': 'Male', 'Professor Coretta Phillips': 'Female', 'Dr Virgi Sari': 'Female', 'Dr Hakan Seckinelgin': 'Male', 'Professor Almudena Sevilla': 'Female', 'Dr Michael Shiner': 'Male', 'Dr Kitty Stewart': 'Female', 'Dr Margaux Suteau': 'Female', 'Dr Iva Tasseva': 'Female', 'Professor Anne West': 'Female', 'Raegan Booth': 'Female', 'Pilar Cuevas-Ruiz': 'Female', 'Chiara Chiavaroli': 'Female', 'Joaquin Mayora Camus': 'Male', 'Dr Piera Bello': 'Female', 'Dr Mike Brewer': 'Male', 'Dr Tymofii Brik': 'Male', 'Dr Alessandra Casarico': 'Female', 'Dr Francesca Foliano': 'Female', 'Professor Paul Frijters': 'Male', 'Professor Cecilia Garcia Penalosa': 'Female', 'Dr Tony Hockley': 'Male', 'Dr Vanessa Hughes': 'Female', 'Dr Ching Leong': 'Female', 'Dr Greta Morando': 'Female', 'Dr Chiara Orsini': 'Female', 'Dr Deborah Outhwaite': 'Female', 'Dr Lynette Rawlings': 'Female', 'Professor Ismael Sanz': 'Male', 'Dr Amanda Sheely': 'Female', 'Dr John Carrier': 'Male', 'Professor Hartley Dean': 'Male', 'Professor David Downes': 'Male', 'Professor Howard Glennerster': 'Male', 'Professor Jane Lewis': 'Female', 'Professor Eileen Munro': 'Female', 'Professor Michael Murphy': 'Male', 'Professor Anne Power': 'Female', 'Mahvish Ahmad': 'Female', 'Robin Archer': 'Male', 'Chetan Bhatt': 'Male', 'Ayça Çubukçu': 'Female', 'Rebecca Elliott': 'Female', 'Janet Foster': 'Female', 'Carrie Friese': 'Female', 'Suzi Hall': 'Female', 'Ursula Henz': 'Female', 'Monika Krause': 'Female', 'David Madden': 'Male', 'Patrick McGovern': 'Male', 'Jana Melkumova-Reynolds': 'Female', 'Claire Moon': 'Female', 'David Pinzur': 'Male', 'Olivia Umurerwa Rutazibwa': 'Female', 'Mike Savage': 'Male', 'Don Slater': 'Male', 'Kristin Surak': 'Female', 'Fran Tonkiss': 'Female', 'Mai Taha': 'Female', 'Ioanna Gouseti': 'Female', 'baljit kaur': 'Female', 'Qilyu Hong': 'Female', 'Nils Peters': 'Male', 'Dena Qaddumi': 'Female', 'Shaheen, Faiza': 'Male', 'Sandal-Wilson, Hakan': 'Male', 'Leili Sreberny-Mohammadi': 'Female', 'Christopher Robert Badcock': 'Male', 'Eileen Barker': 'Female', 'Tony Giddens': 'Male', 'frances-heidensohn': 'Female', 'Christopher T Husbands': 'Male', 'Paul Rock': 'Male', 'Leslie Sklair': 'Male', 'Charis Thompson': 'Female', 'Judy Wajcman FBA': 'Female', 'Maya Adereth': 'Female', 'christiana-ajai-thomas': 'Male', 'Özgün Aksakal': 'Male', 'Nicolás Arenas': 'Male', 'Rajesh Bhattacherjee': 'Male', 'Amani Bhobe': 'Female', 'Dan Brown': 'Male', 'Asif Butt': 'Male', 'Donna Carmichael': 'Female', 'Iulia-Clara Cirdan': 'Female', 'Rodrigo Círigo-Jiménez': 'Male', 'Sigrid Corry': 'Female', 'Sherelle Davids': 'Male', 'Doğukan Dere': 'Male', 'Shanae Dyer': 'Female', 'Lucy Garbett': 'Female', 'Jan Gilles': 'Male', 'Adam Greenfield': 'Male', 'Victoria Gronwald': 'Female', 'Sacha Hilhorst': 'Female', 'Johannes Hollenhorst': 'Male', 'Will Kendall': 'Male', 'Lally, Amraj': 'Male', 'Helen Mackreath': 'Female', 'Hobeth I Martínez Carrillo': 'Female', 'Babette May': 'Female', 'Jack McGinn': 'Male', 'Anaïde Nahikian': 'Female', 'Noura Nasser': 'Female', 'Marta Pagnini': 'Female', 'Daniela Perez Aguilar': 'Female', 'Matt Reynolds': 'Male', 'Carla Rivera Blanco': 'Female', 'Rémy-Paulin Twahirwa': 'Male', 'Neil Warner': 'Male', 'Hannah Wilson': 'Female', 'Jawaher Al Sudairy': 'Male', 'Anthony Miro Born': 'Male', 'Tim White': 'Male', 'Gabriel Eidelman': 'Male', 'Miriam Glucksmann': 'Female', 'Ian Patel': 'Male', 'Thierry Rossier': 'Male', 'Svetlana Ruseishvili': 'Female', 'James Abdey': 'Male', 'Mona Azadkia': 'Female', 'Marcos Barreto': 'Male', 'Pauline Barrieu': 'Female', 'Erik Baurdoux': 'Male', 'Professor Wicher Bergsma': 'Male', 'Umut Cetin': 'Male', 'Yining Chen': 'Male', 'Yunxiao Chen': 'Male', 'Professor Angelos Dassios': 'Male', 'Piotr Fryzlewicz': 'Male', 'Sara Geneletti': 'Female', 'Kostas Kalogeropoulos': 'Male', 'Kostas Kardaras': 'Male', 'Jouni Kuha': 'Male', 'Clifford Lam': 'Male', 'Pik Liew': 'Male', 'Giulia Livieri': 'Female', 'Joshua Loftus': 'Male', 'Gelly Mitrodima': 'Female', 'Francesca Panero': 'Female', ' Xinghao Qiao': 'Male', 'Chengchun Shi': 'Male', 'Andreas Søjmark': 'Male', 'Fiona Steele': 'Female', 'Zoltan Szabo': 'Male', 'Milan Vojnović': 'Male', 'Tengyao Wang': 'Male', 'Qiwei Yao': 'Male', 'Christine Yuen': 'Female', 'Xiaolin Zhu': 'Female', 'Anthony Atkinson': 'Male', 'Dr Camilo Cárdenas Hurtado': 'Male', 'Phil Chan': 'Male', 'Yudong Chen': 'Male', 'Moez Draief': 'Male', 'Tomasz Dubiel-Teleszynski': 'Male', 'Shakeel Gavioli-Akilagun': 'Male', 'Anastasia Kakou': 'Female', 'Dr Anica Kostic': 'Female', 'Yirui Liu': 'Female', 'Martin Knott': 'Male', 'Milt Mavrakakis': 'Male', 'Karsten Shaw': 'Male', 'Howell Tong': 'Male', 'Gabriel Wallin': 'Male', 'Henry Wynn': 'Male', 'Kaifang Zhou': 'Female', 'George Zouros': 'Male', 'Jin Zhu': 'Male', 'Sakina Hansen': 'Female', 'Ziqing Ho': 'Female', 'Liyuan Hu': 'Female', 'Tao Ma': 'Male', 'Pingfan Su': 'Male', 'Xiaoyi Wen': 'Female', 'Xuzhi Yang': 'Female', 'Eduardo Ferioli-Gomes': 'Male', 'Mingwei Lin': 'Male', 'Alexandros Pavlis': 'Male', 'Pietro Maria Sparago': 'Male', 'Mohammad Fadil Sumun': 'Male', 'Zeng Zhang': 'Male', 'Zackary Allison': 'Male', 'Sahoko Ishida': 'Female', 'Sze Ming Lee': 'Male', 'Xinyi Liu': 'Female', 'Seyedpouya Mirrezaeiroudaki': 'Female', 'Motonori Oka': 'Male', 'August Shen': 'Male', 'Zetai Cen': 'Male', 'Weilin Chen': 'Male', 'Wenyu Cheng': 'Female', 'Sixing Hao': 'Male', 'Di Su': 'Female', 'Toby Wade': 'Male', 'Yutong Wang': 'Female', 'Shuhan Yang': 'Female', 'Xianghe Zhu': 'Male', 'Professor Larry Kramer': 'Male', 'Konstantinos Vamvourellis': 'Male', 'Chris Constantinou': 'Male',
 'David Trodden': 'Male',
 'Ebrahim Mohamed': 'Male',
 'Khamid Irgashev': 'Male',
 'Dr Farooq Mahmood': 'Male',
 'Imran Malik': 'Male',
 'Thomas McDermott': 'Male',
 'Philip McDonald': 'Male',
 'Yvonne Guthrie  ': 'Female',
 'Gary Crow': 'Male',
 'Jack Heaney': 'Male',
 'Sandra Ma': 'Female',
 'Rebecca Baker  ': 'Female',
 'Justin Adams ': 'Male',
 'Dorothy': 'Female',
 'Yuhua ChenHudson ': 'Female',
 'Ms Yanina Hinrichsen': 'Female',
 'Ms Chloe Davies': 'Female',
 'Ms Megan Garry-Evans': 'Female',
 'Mr Tomas Hinrichsen': 'Male',
 'Mr James Johnston': 'Male',
 'Ms Renata Todd': 'Female',
 'Dr Mayanka Mukherji': 'Female',
 'Professor Jean La Fontaine': 'Female',
 'Vanessa Galeano-Duque': 'Female',
 'Jill Beattie': 'Female',
 'Ellie Finlay': 'Female',
 'Lou Flandrin': 'Female',
 'Kevin Kittoe': 'Male',
 'Ginny Pavey': 'Female',
 'Johanna Tate': 'Female',
 'Ali Eggleton': 'Male',
 'Dr Stuart Bramwell': 'Male',
 'Tabtim Duenger': 'Female',
 'Riya Chhikara': 'Female',
 'Sara Luxmoore': 'Female',
 'Mustafa Can Ozkan': 'Male',
 'Alexander Soldatkin': 'Male',
 'Andreas': 'Male',
 'Wendy Wang': 'Female',
 'Dr Michael Muthukrishna': 'Male',
 'Dr Rachel Spicer': 'Female',
 'Philipp Darius': 'Male',
 'Fintan Oeri': 'Male',
 'Clint Claessen': 'Male',
 'Riccardo Ricciardi': 'Male',
 'Paride Carrara': 'Male',
 'Nader Hotait': 'Male',
 'Anastasiia Menshikova': 'Female',
 'Dr Yang Yang': 'Male',
 'Rachel Ngai': 'Female',
 'Chiara Sotis': 'Female',
 'Tatiana Komarova': 'Female',
 'Rachael Meager': 'Female',
 'Daniel Reck': 'Male',
 'Nallini Samuel': 'Female',
 'Andy Wilson': 'Male',
 'Sharon Peate': 'Female',
 'Dária': 'Female',
 'Deborah Adams': 'Female',
 'Irina Zaraisky': 'Female',
 'Oceana Taylor': 'Female',
 'Lakmini': 'Female',
 'Sarah Burton': 'Female',
 "Alice O'Donkor": 'Female',
 'John Canfield': 'Male',
 'Kelly Lewis': 'Female',
 'Rudi Clayton': 'Male',
 'Narmin Crorie': 'Female',
 'Ben Westall': 'Male',
 'Emma Taverner': 'Female',
 'Shareen Suleman': 'Female',
 'Shubhra Mitra': 'Female',
 'Lorna Severn': 'Female',
 'Mohsin Ahmed': 'Male',
 'Lakmini Staskus': 'Female',
 'Dave Donaldson': 'Male',
 'Anne Karing': 'Female',
 'Raul Sanchez de la Sierra': 'Male',
 'Michael Whinston': 'Male',
 'Lord Meghnad Desai': 'Male',
 'Alan Marin': 'Male',
 'Jim Thomas': 'Male',
 'Michele Fioretti': 'Female',
 'Michael McMahon': 'Male',
 'Clement Minaudier': 'Male',
 'David Seim': 'Male',
 'Tim Willems': 'Male',
 'Mr Rasif Alakbarov': 'Male',
 'Mr Daniel Albuquerque': 'Male',
 'Mr Philipp Barteska': 'Male',
 'Mr Marco Bellifemine': 'Male',
 'Mr Adam Brzezinski': 'Male',
 'Mr Arnaud Dyevre': 'Male',
 'Ms Tiernan Evans': 'Female',
 'Mr Miguel Fajardo-Steinhauser': 'Male',
 'Ms Shadi Farahzadi': 'Female',
 'Mr Jack Fisher': 'Male',
 'Ms Salome Fofana': 'Female',
 'Mr Maximilian Guennewig': 'Male',
 'Evie Jamieson': 'Female',
 'Ms Evgeniya Kudinova': 'Female',
 'Mr Peter Lambert': 'Male',
 'Mr Edoardo Leonardi': 'Male',
 'Mr Mars Leung': 'Male',
 'Mr Issac Zenon Martinez Centeno': 'Male',
 'Mr Roberto Maura-Rivero': 'Male',
 'Mr Thomas Monk': 'Male',
 'Ms Sidharth Moktan': 'Female',
 'Ms Kamila Nowakowicz': 'Female',
 "Mr Loughlan O'Doherty": 'Male',
 'Ms Lisa Pacheco': 'Female',
 'Mr Hugo Reichardt': 'Male',
 'Mr Soroush Sabet': 'Male',
 'Mr Jack Shannon': 'Male',
 'Ms Lachi Singh': 'Female',
 'Mr Aditya Soenarjo': 'Male',
 'Mr Rui Sun': 'Male',
 'Mr Kazunari Tanabe': 'Male',
 'Ms Maria Ventura': 'Female',
 'Dr Rui Miguel Vieira Marques da Costa': 'Male',
 'Mr Wei Wang': 'Male',
 'Ms Cecilia Wood': 'Female',
 'Mr Linchuan Xu': 'Male',
 'Mr Chia-Hung Yeh': 'Male',
 'Pinjas Albagli': 'Male',
 'Luzia Bruckamp': 'Female',
 'Gabriela Deschamps Ochoa': 'Female',
 'Sitong Ding': 'Female',
 'Yuxiao Hu': 'Male',
 'Azhar Hussain': 'Male',
 'Ananya Kotia': 'Female',
 'Evgeniya Kudinova': 'Female',
 'Runpu Li': 'Male',
 'Yuhao Qian': 'Male',
 'Emiliano Rinaldi': 'Male',
 'Rebecca Rose': 'Female',
 'Yasaman Saeidi': 'Female',
 'Jack Thiemel': 'Male',
 'Sarah Winton': 'Female',
 'Luiza Kunz-Aires': 'Female',
 'Giacomo Carlini': 'Male',
 'Fraser Clark': 'Male',
 'Adrian Couturier-Roguet': 'Male',
 'Sacha Dray': 'Female',
 'Andres Felipe Fajardo Ramirez': 'Male',
 'Johannes Anton Heil': 'Male',
 'Antonio Iavarone': 'Male',
 'Filip Klimenka': 'Male',
 'Antonio Lavarone': 'Male',
 'Gabriel Leite-Mariante': 'Male',
 'Manuel Francisco Libano Monteiro': 'Male',
 'Guido Spano': 'Male',
 'Hamza Mohammad Syed': 'Male',
 'Anshuman Tiwari': 'Male',
 'Hugo Vilares': 'Male',
 'Lukas Weidemann': 'Male',
 'Mengshan Xu': 'Female',
 'Yu Yi': 'Female',
 'Dr Mina Ishizu': 'Female',
 'Dr Rebecca Simson': 'Female',
 'Luisa Bicalho Ritzkat ': 'Female',
 'Nick Fitzhenry ': 'Male',
 'Andres Irarrazaval Garcia Huidobro ': 'Male',
 'Yangyang Liu ': 'Female',
 'Zane Jennings ': 'Male',
 'Viren Mahurkar ': 'Male',
 'Andy Park ': 'Male',
 'Nick Peyton ': 'Male',
 'Sheila Pugh ': 'Female',
 'Greg Salter ': 'Male',
 'Noah Sutter ': 'Male',
 'Tianning Zhu ': 'Female',
 'Dr Ahmet Akarli': 'Male',
 'Professor David Chambers': 'Male',
 'Jordi Domenich Feliu': 'Male',
 'Terry Gourvish': 'Male',
 'Professor Maarten Prak': 'Male',
 'Dr Niccolò Valmori': 'Male',
 'Oli Harrison': 'Female',
 'Kamilah Hassan': 'Female',
 'Helena Ivins': 'Female',
 'Tracy Keefe': 'Female',
 'Jennie Stayner': 'Female',
 'Darren Townsend': 'Male',
 'Ruoran Cheng': 'Female',
 'Marco Cokic': 'Male',
 'Eoin Dignam': 'Male',
 'Junqi': 'Male',
 'Florentine Friedrich': 'Female',
 'Hampton Gaddy': 'Male',
 'Mikhail Kolosov': 'Male',
 'Yangyang Liu': 'Female',
 'Viren Mahurkar': 'Male',
 'Victor Perez Sanchez': 'Male',
 'Sheila Pugh': 'Female',
 'Greg Salter': 'Male',
 "Ziyue 'Zoey' Shen": 'Female',
 'Charles Smith': 'Male',
 'Mek Ajay Sribhibhadh': 'Male',
 'David Teeters': 'Male',
 'Charles Udale': 'Male',
 'Hillary Vipond': 'Female',
 'Weizhe Zhang': 'Male',
 'Tianning Zhu': 'Female',
 'Jennie Stayner ': 'Female',
 'Mr. Mujtaba Rahman': 'Male',
 'Mr. Klaus Welle': 'Male',
 'Dr. Auke Willems': 'Male',
 'Tommaso Crescioli': 'Male',
 'Paul Kindermann': 'Male',
 'Dr Angie Adrikogiannapoulou': 'Female',
 'Professor Alex Edmans': 'Male',
 'Dr Yves Nosbusch': 'Male',
 'Dr Dimitris Papadimitriou': 'Male',
 'Dr Jojo Paul': 'Male',
 'Dr Alberto Pellicioli': 'Male',
 'Professor Pedro Saffi': 'Male',
 'Dr Konstantinos Baltas': 'Male',
 'Dr Guido Bodrato': 'Male',
 'Dr Gustavo Dias': 'Male',
 'Dr Min Park': 'Male',
 'Dr Marco Pelosi': 'Male',
 'Harriet Baylore': 'Female',
 'Rhys Cadman': 'Male',
 'Hei-Man Chan': 'Male',
 'Elena Christophorou': 'Female',
 'Mary Comben': 'Female',
 'Oliver Dowling': 'Male',
 'Eleanor Eiserman': 'Female',
 'Francesca Gil': 'Female',
 'Henrietta Griffith': 'Female',
 'Jasper Heeks': 'Male',
 'Miranda Imperial-Sanchez': 'Female',
 'Nicole Jashapara': 'Female',
 'Ella Lucas': 'Female',
 'Yana Marchuk': 'Female',
 'Joe Meegan': 'Male',
 'Isabella Peebles': 'Female',
 'Osmana Raie': 'Female',
 'Jamie Rishworth': 'Female',
 'Dr Domingos Romualdo': 'Male',
 'Paul Stoker': 'Male',
 'Simon Tuck': 'Male',
 'Grace Williams': 'Female',
 'Costanza Torre': 'Female',
 'Colette van der Ven': 'Female',
 'Geoffroy Guepie': 'Male',
 'Gedion Onyango ': 'Male',
 'Professor Alcinda Honwana ': 'Female',
 'Holly Porter ': 'Female',
 'Kasper Vrolijk ': 'Male',
 'Cristin Alexis Fergus ': 'Female',
 'Claire Elder ': 'Female',
 'Charlotte Brown ': 'Female',
 'Henrietta Burr': 'Female',
 'Rob Kirkland': 'Male',
 'Catherine Perry': 'Female',
 'Annie Robinson': 'Female',
 'Violet Fox': 'Female',
 'Oksana Potapova': 'Female',
 'Sam Colegate': 'Male',
 'Dr Matteo M Galizzi': 'Male',
 'Erica Borg': 'Female',
 'Álvaro Carbonell Rodríguez': 'Male',
 'Dr Magdalena Dominguez': 'Female',
 'Andrea Herrera': 'Female',
 'Merrill Hopper': 'Male',
 'Ali Kokbudak': 'Male',
 'Till Meisssner': 'Male',
 'Morgan Olive-Carmellini': 'Female',
 'Line Relisieux': 'Female',
 'Martina Rotolo': 'Female',
 'Tom Siddall': 'Male',
 'Anna Smith': 'Female',
 'Kapil Yadav': 'Male',
 'Ailin Zhang': 'Female',
 'Morgan Olive Carmellini': 'Female',
 'Alaa Iktash': 'Female',
 'Vivian Liu': 'Female',
 'Vishnu Prasad': 'Male',
 'Ivanilza Da Mata': 'Female',
 'Lee Mager': 'Male',
 'Maya King': 'Female',
 'Amy Cox': 'Female',
 'Ruby Bennett': 'Female',
 'Jodie Bispham': 'Female',
 'Hanna Wolodzko': 'Female',
 'Tom Jones': 'Male',
 'Catherine Mitchell': 'Female',
 'Isobel Jones': 'Female',
 'Emily Douglas': 'Female',
 'Caroline Cornish': 'Female',
 'Guy Morrell': 'Male',
 'Mark Nesbitt': 'Male',
 'Camilla Royle': 'Female',
 'Francis Salway': 'Male',
 'Callum Ward': 'Male',
 'Imran Iqbal': 'Male',
 'Natalie Paris': 'Female',
 'Tom Bailey': 'Male',
 'Charlie Tickle': 'Male',
 'Katerina Collins': 'Female',
 'BellaRose Lennon': 'Female',
 'Laure Van Rensburg': 'Female',
 'Emily Crook': 'Female',
 'Serena James': 'Female',
 'Karen Dickenson': 'Female',
 "Kay O'Neill": 'Female',
 'Clair Harrison': 'Male',
 'James Robins': 'Male',
 'Kevin Shields': 'Male',
 'Shelly Wilkley': 'Female',
 'Sarah Goff': 'Female',
 'Jill Stuart': 'Female',
 'Maximillian Afnan': 'Male',
 'Ville Aula': 'Male',
 'Morris Chan': 'Male',
 'Cristina Conesa Pla': 'Female',
 'Julia-Anne Costet': 'Female',
 'Andrea Delestrade': 'Female',
 'Tessa Devereaux-Evans': 'Female',
 'Melanie Erspamer': 'Female',
 'Ugo Gaudino': 'Male',
 'Kristof Horvath': 'Male',
 'Conor Kelly': 'Male',
 'Yusuf Khan': 'Male',
 'Matthias Kroenke': 'Male',
 'Xufan Ma': 'Male',
 'Alexander McKenna': 'Male',
 'Stefano Merlo': 'Male',
 'Lennard Metson': 'Male',
 'Sarah Tustin': 'Female',
 'Jan Wasserziehr': 'Male',
 'Felix Westeren': 'Male',
 'Laurence Antao': 'Male',
 'Reuben Finighan': 'Male',
 'Maximilian Fink': 'Male',
 'Hilke Gudel': 'Female',
 'Isolde Hegemann': 'Female',
 'Gaetano Inglese': 'Male',
 'Sarah Jewett': 'Female',
 'Miran Norderland': 'Male',
 'Oscar Nowlan': 'Male',
 'Ariel Perkins': 'Female',
 'Andreas-Johann': 'Male',
 'Carol Thanki': 'Female',
 'Khairulanwar Zaini': 'Male',
 'John Barnes': 'Male',
 'John Gray': 'Male',
 'John Madeley': 'Male',
 'Ernest Thorp': 'Male',
 'David Banisar': 'Male',
 'Sevinc Bermek': 'Female',
 'Andrea Capussela': 'Female',
 'Daniel de Kadt': 'Male',
 'Dominik Hangartner': 'Male',
 'Helios Herrera': 'Male',
 'Antoni Rodon-Casarramona': 'Male',
 'Lukas Slothuus': 'Male',
 'Amanda Gaddes': 'Female',
 'Tash Dalling': 'Female',
 'Rana James': 'Female',
 'Muheez Busari': 'Male',
 'Dr Dina Davaki': 'Female',
 'Christopher Roberts': 'Male',
 'Zaina Salama': 'Female',
 'Jacqueline McAndrew': 'Female',
 'Hayley Briars': 'Female',
 'Alex Rose': 'Male',
 'Janice Dickson': 'Female',
 'Alexia Honore': 'Female',
 'Angela Mehta': 'Female',
 'Sergio Rotundo': 'Male',
 'Susana Sidders': 'Female',
 'Rebekah Smith': 'Female',
 'Xiaoran Xu': 'Male',
 'Dr Paul Carter': 'Male',
 'Dr Joseph Kim': 'Male',
 'Professor Sir Mike Richards': 'Male',
 'Professor Lise Rochaix': 'Female',
 'Dr Victoria Serra-Sastre': 'Female',
 'Dr Matthew Skellern': 'Male',
 'Dr Harpreet Sood': 'Male',
 'Christos Stylianides': 'Male',
 'Avi Cherla': 'Male',
 'Sherry Merkur': 'Female',
 'Gemma Williams': 'Female',
 'Philipp Friemann': 'Male',
 'Dr Alan Glanz': 'Male',
 'Professor Ruth Hancock': 'Female',
 'Sarah Jasim': 'Female',
 'Mark Lowcock': 'Male',
 'Allison Benson Hernandez': 'Female',
 'Annie Kuse': 'Female',
 'Reece Sisto': 'Male',
 'Matthew Redfearn': 'Male',
 'Drucilla Nelson': 'Female',
 'Andrea Merino-Mayayo': 'Female',
 'Charlotte McKay': 'Female',
 'Maria Do-Prado': 'Female',
 'Ayman Hassan': 'Male',
 'Dipa Patel': 'Female',
 'Maya Bullen': 'Female',
 'Dr Monika Kruesmann': 'Female',
 'Faris Al-Sulayman': 'Male',
 'Lucas D. Arce ': 'Male',
 'Ritika Arora': 'Female',
 'Arbie Baguios': 'Male',
 'Parth Bhatia': 'Male',
 'Charlotte Brown': 'Female',
 'Javiera Caceres': 'Female',
 'Aisha Modibo': 'Female',
 'Evan Oliner': 'Male',
 'Ciara Silke': 'Female',
 'Marika Theros': 'Female',
 'Ms Demetra Frini': 'Female',
 'Mrs Jacquie Minter': 'Female',
 'Mrs Milada Fomina': 'Female',
 'Mrs Nayna Bhatti': 'Female',
 'Mr Mathew Betts': 'Male',
 'Dr Edlira Gjonca': 'Female',
 'Ms Laila Yousofi': 'Female',
 'Mr Maximilian': 'Male',
 'Maliha Ahmed': 'Female',
 'Dr Sara Al-Qaiwani': 'Female',
 'Amir Mehdi': 'Male',
 "Fionntan O'Hara": 'Male',
 'Michele Pajero': 'Female',
 'Emma Ceccato': 'Female',
 'Peter Kane': 'Male',
 'Miranda Saul': 'Female',
 'Sallyann Oates': 'Female',
 'Dan Nixon': 'Male',
 'Grace Farrell-Twiney': 'Female',
 'Saaga Leppänen': 'Male',
 'Adele Oliver': 'Female',
 'Julia Ryng': 'Female',
 'Mr Mark Hoffman': 'Male',
 'Dr Mia Certo': 'Female',
 'Dr John Kent': 'Male',
 'Mr Nicholas Sims': 'Male',
 'Professor William Wallace': 'Male',
 'Arthur Kilgore': 'Male',
 'Oksana Levkovych': 'Female',
 'David Rampton': 'Male',
 'James Strong': 'Male',
 'Paul Sullivan': 'Male',
 'David Bradley': 'Male',
 'Joe Jacob': 'Male',
 'Karla Barca Marrero': 'Female',
 'Megan Bennett': 'Female',
 'Ayana Brimacombe-Sakey': 'Female',
 'Ruairi Cahill-Fleury': 'Male',
 'Giu': 'Male',
 'Laura Carseldine': 'Female',
 'Denise De Castro': 'Female',
 'Lewina Coote': 'Female',
 'Jeffrey Franklin': 'Male',
 'Sophia Freckmann': 'Female',
 'Alison Grant': 'Female',
 'Zalika Henry': 'Female',
 'Lee Jackson': 'Male',
 'Alexandra Klegg': 'Female',
 'Megha Krishnakumar': 'Female',
 'Katrina Millett': 'Female',
 'Rebecca Newman': 'Female',
 'Niamh Sheehan': 'Female',
 'Saiful Siddeky': 'Male',
 'Fiona Thomas': 'Female',
 'Amanda Tinnams': 'Female',
 'Mike Twyman': 'Male',
 'Naomi Warren': 'Female',
 'Lucy Wright': 'Female',
 'Carl Baudenbacher': 'Male',
 'Conor Quigley': 'Male',
 'Dr Michael Kanu': 'Male',
 'Sharon Barnes': 'Female',
 'Nicola Foster': 'Female',
 'Jas Dhanda': 'Male',
 'Paulette Annon': 'Female',
 'Becci Ayling': 'Female',
 'Mallika Bandyopadhyay': 'Female',
 'Alex Banks': 'Male',
 'Kati Cooper': 'Female',
 'Daniella Duarte': 'Female',
 'Charlene Fairs': 'Female',
 'Sarah Greene': 'Female',
 'Freya Grisoni': 'Female',
 'Charlotte Harris': 'Female',
 'Olivia Keane': 'Female',
 'Dr Stephanie Lambert': 'Female',
 'Astrid Laurent': 'Female',
 'Michelle': 'Female',
 'Nicholas Martin': 'Male',
 'Sharan Purewal': 'Female',
 'Laura Rivers': 'Female',
 'Kareema Carrington': 'Female',
 'Annie Wang': 'Female',
 'Langyi Ye': 'Female',
 'Alessia Zaja': 'Female',
 'Becky Coggins': 'Female',
 'Sophie Gavriel': 'Female',
 'Barbara Miller Manzanares': 'Female',
 'Julie Gordon Shipley': 'Female',
 'Hrilina Lock': 'Female',
 'Cheryl Johnson': 'Female',
 'Linda Sclanders': 'Female',
 'Nayani Gunatilake': 'Female',
 'Alice Johns': 'Female',
 'Wasima Anwari': 'Female',
 'Hiba Dabis': 'Female',
 'Bethan Briggs': 'Female',
 'Ardy Cheung': 'Male',
 'Molly Healy': 'Female',
 'Emma Mullen': 'Female',
 'Raheela Shah': 'Female',
 'Laura Knowles': 'Female',
 'Sam Hunt': 'Male',
 'Paddy Hyde': 'Male',
 'Tevec Ibrahim': 'Male',
 'Daniel Marenghi': 'Male',
 'Désirée Meyerding': 'Female',
 'Nino Nizharadze': 'Male',
 'Leah Murray': 'Female',
 'Patience Ayalogu': 'Female',
 'Ellie Fleming': 'Female',
 'Camilla Kennedy Harper': 'Female',
 'Emma Ward': 'Female',
 'Emma Balaguette': 'Female',
 'Nenna Opara': 'Female',
 'Jessica Harrison': 'Female',
 'David Meech Mazumdar': 'Male',
 'Martha Langan': 'Female',
 'Kelly Benson': 'Female',
 'Dr Tal Harel': 'Male',
 'Maximilian Goehmann': 'Male',
 'Ozan Kirtac': 'Male',
 'Fabrizio Leone': 'Male',
 'Sir Thomas Hughes-Hallett': 'Male',
 'Jack Winterton': 'Male',
 'Emma Bayes': 'Female',
 'Amelia Bradley': 'Female',
 'Shanzeh Mahmood': 'Female',
 'Corrina Summers': 'Female',
 'Anna Townsend': 'Female',
 'Alexander Wright': 'Male',
 'Julia Ziemer': 'Female',
 'Kayleigh Cunoosamy': 'Female',
 'Fan Gu': 'Male',
 'Nell Lemaistre': 'Female',
 'Carl Moldestad': 'Male',
 'Aggrey Nyondwa': 'Male',
 'Keri Rowsell': 'Female',
 'Manuel Antunes': 'Male',
 'Swatee Deepak': 'Female',
 'Saul Estrin': 'Male',
 'Luke Fletcher': 'Male',
 'Sonal Sachdev Patel': 'Male',
 'Daniel Hulme': 'Male',
 'Matthew Bishop': 'Male',
 'Veronika Kapustina': 'Female',
 'Emily Kasriel': 'Female',
 'Jude Kelly': 'Male',
 'Katherine Lorenz': 'Female',
 'Sonia Medina': 'Female',
 'Jerome Misso': 'Male',
 'Eva Neitzert': 'Female',
 'Will Page': 'Male',
 'Aunnie Patton Power': 'Male',
 'William Perrin': 'Male',
 'Chelsea Phipps': 'Female',
 'Tom Rippin': 'Male',
 'David Robinson': 'Male',
 'Mabel Van Oranje': 'Female',
 'Nicole Garnier': 'Female',
 'Ethan Liu': 'Male',
 'Ben McDonald': 'Male',
 'Martina Zuccarella': 'Female',
 'Leo Beattie': 'Male',
 'Danielle Blasse': 'Female',
 'Brianna Crummy': 'Female',
 'Lwam Tesfay': 'Female',
 'Dr Jun Yu': 'Male',
 'Crystal Chia': 'Female',
 'John Curtis': 'Male',
 'Chau Le': 'Female',
 'Camilya Maleh': 'Female',
 'Serena Ngai': 'Female',
 'Lucia Pedrioli': 'Female',
 'Olivia Talland': 'Female',
 'Emeritus Professor Michael Laver': 'Male',
 'Professor Benjamin Lauderdale': 'Male',
 'Shira Ahissar': 'Female',
 'Anika Bloomfield': 'Female',
 'Dr Campbell Brown': 'Male',
 'Prof Nancy Cartwright': 'Male',
 'Talita Ferrantelli': 'Female',
 'Prof Wulf Gaertner': 'Male',
 'Isabel Lacurie': 'Female',
 'Becky Matthams': 'Female',
 'Dr Marie Milofsky': 'Female',
 'Dr Kieran Oberman': 'Male',
 'Dr Jonathan Parry': 'Male',
 'Dr. Paola Romero': 'Female',
 'Dr Charles Sherwood': 'Male',
 'Max Steuer': 'Male',
 'Mar Torró': 'Male',
 'Professor Liam Delaney': 'Male',
 'Professor Bradley Franks': 'Male',
 'Champa Heidbrink': 'Female',
 'Dr Deema Awad': 'Female',
 'Dr Thomas Curran': 'Male',
 'Dr Lucia Garcia-Lorenzo': 'Female',
 'Dr Christian Krekel': 'Male',
 'Dr Dario Krpan': 'Male',
 'Dr Miriam Tresh': 'Female',
 'Martha Clarke': 'Female',
 'Karine Gay': 'Female',
 'Carl Goodwin': 'Male',
 'Therese Holmqvist': 'Female',
 'Louise Millar': 'Female',
 'Salv Ridino': 'Male',
 'Sean Rooney': 'Male',
 'Will Stubbs': 'Male',
 'Dimitris Thomopoulos': 'Male',
 'Sharon Jackson': 'Female',
 'Martin Whitehead': 'Male',
 'Ms Mary McLennan': 'Female',
 'Lazaros': 'Male',
 'Wangjingyi Liao': 'Female',
 'Da’Quallon Smith': 'Male',
 'Sayeh Yousefi': 'Female',
 'Dr Frederic Basso': 'Male',
 'Dr Jens Madsen': 'Male',
 'Dr Jet Sanders': 'Male',
 'Professor Martin Bauer': 'Male',
 'Professor Alex Gillespie': 'Male',
 'Dr Laura M. Giurge': 'Female',
 'Dr Ilka Gleibs': 'Female',
 'Kate Laffan': 'Female',
 'Adam Brzezinski': 'Male',
 'Aidan Hughes': 'Male',
 'Kirstine La Cour': 'Female',
 'Dr Tamar Zeilberger': 'Female',
 'Philip Colins': 'Male',
 'Charlie Meyrick': 'Male',
 'Fernando Pino': 'Male',
 'Josh Hellings': 'Male',
 'Finn McEvoy': 'Male',
 'Michelle Batten': 'Female',
 'Victoria Bryant': 'Female',
 'Teresa Chung': 'Female',
 'Heather Gorrie': 'Female',
 'Zuzana Navratilova': 'Female',
 'Emma Nevell': 'Female',
 'Andrew Park': 'Male',
 'Verity Wilkes': 'Female',
 'Damian Roberts': 'Male',
 'Clare Gorman': 'Female',
 'Sevilay Erdogan': 'Female',
 'Adrian Knight': 'Male',
 'Dr Joe Macdonald': 'Male',
 'Nathaniel Ocquaye': 'Male',
 'Maria Schlegel': 'Female',
 'Dr Craig Stewart': 'Male',
 'Jake Watkins': 'Male',
 'Dr Hao Wu': 'Male',
 'Franco Fernandez Fleming': 'Male',
 'Manna Mostaghim': 'Female',
 'Professor Waqar Ahmad': 'Male',
 'Professor David Piachaud': 'Male',
 'Dr Gail Wilson': 'Female',
 'Lareau, Annette': 'Male',
 'Borowska, Emilia': 'Female',
 'Christofferson, Andrew': 'Male',
 'Clarke, Evie': 'Male',
 'Codjoe, Vivienne': 'Male',
 'Giles, Maddie': 'Male',
 'Lawrence, Louisa': 'Male',
 'Lyons, Clara': 'Male',
 'Parvez, Adeel': 'Male',
 'Rahman, Yadna': 'Male',
 'Tomlinson, Claire': 'Male',
 'Wain, Laura': 'Male',
 'Walker, Monika': 'Male',
 'Akdag, Muhammed': 'Male',
 'Amitay, Nitzan': 'Male',
 'Brundu-Gonzalez, Benjamin': 'Male',
 'Cutts, Danielle': 'Female',
 'Gray, Selena': 'Male',
 'Kherfi, Yasmine': 'Female',
 'Nichols, Georgia': 'Male',
 'Harris, Gabrielle': 'Male',
 'Kampmann, David': 'Male',
 'Bordonaro, Elettra': 'Male',
 'Dasgupta, Rohit K': 'Male',
 'Ghaziani, Amin': 'Male',
 'Hen-Smith, May': 'Male',
 'Kajta, Justyna': 'Female',
 'Kramer, Maria': 'Male',
 'Küçük, Bermal': 'Male',
 'Perolini, Marco': 'Male',
 'Taylor, Emma': 'Female',
 'Watt, Paul': 'Male',
 'Zherebkin, Sergey V.': 'Male',
 'De Keere, Kobe': 'Male',
 'Mallett, Victoria': 'Female',
 'McCurdy, Martha': 'Female',
 'Partyga, Dominika': 'Female',
 'White, Tim': 'Male',
 'Professor Colm O’Muircheartaigh': 'Male',
 'Dr Roel Oomen': 'Male',
 'Steve Ellis': 'Male',
 'Muhammed Sabih Iqbal': 'Male',
 'Yanli Ji': 'Female',
 'Sarah McManus': 'Female',
 'Penny Montague': 'Female',
 'Charlotte Morgan': 'Female',
 'Imelda Noble Andolfo': 'Female',
 'Penelope Smith': 'Female',
 'Montana Tracey': 'Female'}


This function counts the number of time an element occurs in a text.

In [15]:
def count_elements_occurrences(my_list, elements_to_count):

    list_string = ' '.join(map(lambda x: str(x).lower(), my_list))
    total_occurrences = 0
    for element in elements_to_count:
        pattern = re.compile(rf'\b{element}\b')
        total_occurrences += len(re.findall(pattern, list_string))

    return total_occurrences

The code we used to deduce the gender is below(please use the same API key as you did in the main analysis):

In [16]:
headers = {
    "X-API-KEY": api_key
}

In [17]:
for person, dict_ in gender_dict.items():
  if dict_['Link'] and person not in current_gender_dict.keys():
    try:
      response = requests.get(dict_['Link'])
      soup = BeautifulSoup(response.content, 'html.parser')
      if soup.find('div', class_="people__bio"):
        about = soup.find('div', class_="people__bio").text
      else:
        about = soup.text
      if count_elements_occurrences(about.split(), ['his', 'he']) > count_elements_occurrences(about.split(), ['she', 'her']):
        print(person, 'Male')
        current_gender_dict[person] = 'Male'
      elif count_elements_occurrences(about.split(), ['his', 'he']) < count_elements_occurrences(about.split(), ['she', 'her']):
        print(person, 'Female')
        current_gender_dict[person] = 'Female'
      else:
        name = person.lower().replace("dr ", "").replace("phd ", "").replace("professor ", "").replace("emeritus ", "").replace("dr.", "")
        url = f"https://v2.namsor.com/NamSorAPIv2/api2/json/genderFull/{name}"
        response = requests.get(url, headers=headers)
        data = response.json()
        print(dict_['Link'], name, data['likelyGender'].capitalize())
        current_gender_dict[person] = data['likelyGender'].capitalize()
    except Exception:
      print(dict_['Link'])
      name = person.lower().replace("dr ", "").replace("phd ", "").replace("professor ", "").replace("emeritus ", "")
      url = f"https://v2.namsor.com/NamSorAPIv2/api2/json/genderFull/{name}"
      response = requests.get(url, headers=headers)
      data = response.json()
      print(name, data['likelyGender'].capitalize())
      current_gender_dict[person] = data['likelyGender'].capitalize()
  elif person not in current_gender_dict.keys():
        name = person.lower().replace("dr ", "").replace("phd ", "").replace("professor ", "").replace("emeritus ", "").replace("dr.", "")
        url = f"https://v2.namsor.com/NamSorAPIv2/api2/json/genderFull/{name}"
        response = requests.get(url, headers=headers)
        data = response.json()
        print(dict_['Link'], name, data['likelyGender'].capitalize())
        current_gender_dict[person] = data['likelyGender'].capitalize()
current_gender_dict

Xitong Hui Female
https://www.lse.ac.uk/economics/people/research-students/Jingfeng-Zhang jingfeng zhang Male
William Davis Male
None zia simpson Female
Gabriel Ahlfeldt Male
https://jobs.lse.ac.uk/Vacancies/W/7328/0/425591/15539/lse-fellow-in-health-policy lse fellow in health policy Female
Kate Steward Female
Geoffrey Swenson Male
None abby howard Female
None katrina millet Female
None lucy ariyadasa Female
None dina rabie Female
https://www.lse.ac.uk/Methodology/People/Visiting-Fellows/Ulises-Moreno-Tabarez ulises moreno-tabarez Male
Caroline Roberts Female
Haziq Jamil Male
Mingcong Wu Male
Xuewen Yu Female


{'Peter Miller': 'Male',
 'Professor Rita Astuti': 'Female',
 'Dr Mukulika Banerjee': 'Female',
 'Professor Laura Bear': 'Female',
 'Dr Clara Devlieger': 'Female',
 'Dr Yazan Doughan': 'Male',
 'Professor Katy Gardner': 'Female',
 'Dr Nicholas Long': 'Male',
 'Dr Mathijs Pelkmans': 'Male',
 'Andrea Pia': 'Male',
 'Dr Alpa Shah': 'Female',
 'Professor Charles Stafford': 'Male',
 'Dr Hans Steinmüller': 'Male',
 'Dr Harry Walker': 'Male',
 'Dr Mareike Winchell': 'Female',
 'Dr Noah Walker-Crawford': 'Male',
 'Dr Anjana Bala': 'Female',
 'Dr Gareth Breen': 'Male',
 'Dr Agathe Faure': 'Female',
 'Dr Stephanie Postar': 'Female',
 'Dr Sofía Ugarte': 'Female',
 'Dr Frederick Wojnarowski': 'Male',
 'Dr Teodor Zidaru': 'Male',
 'Professor Karin Barber': 'Female',
 'Eona Bell': 'Female',
 'Hélène Bloch': 'Female',
 'Dr Rebecca Bowers': 'Female',
 'Sanda Caracentev': 'Female',
 'Dr Chris Chaplin': 'Male',
 'Professor Harriet Evans': 'Female',
 'Dr Luis Garcia Briceno': 'Male',
 'Dr Charlotte Hawki

___
<a id="gender"></a>
## Using SQL for our bonus Analysis

### 1. Setting up the tables:

In [18]:
import sqlite3

In [19]:
%load_ext sql
%config SqlMagic.displaylimit = None

displaylimit: Value None will be treated as 0 (no limit)

In [20]:
%%sql
sqlite://

DROP TABLE IF EXISTS Departments;
CREATE TABLE Departments (
    department_name TEXT NOT NULL PRIMARY KEY
);

DROP TABLE IF EXISTS Roles;
CREATE TABLE Roles (
    role_id INTEGER PRIMARY KEY,
    role_name TEXT NOT NULL,
    department_name TEXT,
    FOREIGN KEY (department_name) REFERENCES Departments(department_name)
);

DROP TABLE IF EXISTS Person;
CREATE TABLE Person (
    person_id INTEGER PRIMARY KEY,
    role_id INTEGER,
    role_name TEXT,
    person_name TEXT,
    department_name TEXT,
    FOREIGN KEY (role_id) REFERENCES Roles(role_id)
    FOREIGN KEY (department_name) REFERENCES Departments(department_name)
);
DROP TABLE IF EXISTS Gender;
CREATE TABLE Gender (
    gender TEXT,
    name TEXT PRIMARY KEY,
    FOREIGN KEY (name) REFERENCES Person(person_name)
);

Connecting to 'sqlite://'

++
||
++
++

In [21]:
%config SqlMagic.named_parameters="enabled"

In [22]:
for dept_name, roles_dict in correct_data.items():
    %sql INSERT INTO Departments (department_name) VALUES (:dept_name);
    for role, person_dict in roles_dict.items():
      %sql INSERT INTO Roles (role_name, department_name) VALUES (:role, :dept_name);
      role_id = %sql SELECT role_id FROM Roles WHERE role_name = :role AND department_name = :dept_name;
      role_id = role_id[0][0]
      for person in person_dict:
        %sql INSERT INTO Person (person_name, role_id, department_name, role_name) VALUES (:person, :role_id, :dept_name, :role);
for name, gender in current_gender_dict.items():
  %sql INSERT INTO Gender (name, gender) VALUES (:name, :gender);

Running query in 'sqlite://'

1 rows affected.

Running query in 'sqlite://'

1 rows affected.

Running query in 'sqlite://'

...

1 rows affected.

Running query in 'sqlite://'

1 rows affected.

Running query in 'sqlite://'

1 rows affected.

Checking the number of departments:

### 2. Running SQL Queries to Explore the Data:

In [23]:
%sql SELECT COUNT(*) FROM Departments;

Running query in 'sqlite://'

+----------+
| COUNT(*) |
+----------+
|    29    |
+----------+

Viewing the number of roles and distinct roles at LSE:

In [24]:
%sql SELECT COUNT(*) AS Total_Roles, COUNT(DISTINCT role_name) AS Distinct_Roles FROM Roles;

Running query in 'sqlite://'

+-------------+----------------+
| Total_Roles | Distinct_Roles |
+-------------+----------------+
|     214     |      145       |
+-------------+----------------+

Seeing how many people appear in different roles or departments:

In [25]:
%sql SELECT COUNT(*)-COUNT(DISTINCT person_name) AS People_Appearing_In_Different_Roles_And_Or_departments FROM Person;

Running query in 'sqlite://'

+--------------------------------------------------------+
| People_Appearing_In_Different_Roles_And_Or_departments |
+--------------------------------------------------------+
|                          423                           |
+--------------------------------------------------------+

Seeing the overall gender distribution of LSE staff:

In [26]:
%%sql SELECT
    SUM(gender='Male') AS Male_Staff,
    SUM(gender='Female') AS Female_Staff,
    ROUND(CAST(SUM(gender='Male') AS FLOAT) / SUM(gender='Female'), 2) AS Male_To_Female_Ratio ,
    ROUND((CAST(SUM(gender='Male') AS FLOAT) * 100.0) / COUNT(*), 1) AS Male_Percentage,
    ROUND((CAST(SUM(gender='Female') AS FLOAT) * 100.0) / COUNT(*), 1) AS Female_Percentage
FROM Gender;

Running query in 'sqlite://'

+------------+--------------+----------------------+-----------------+-------------------+
| Male_Staff | Female_Staff | Male_To_Female_Ratio | Male_Percentage | Female_Percentage |
+------------+--------------+----------------------+-----------------+-------------------+
|    1748    |     1428     |         1.22         |       55.0      |        45.0       |
+------------+--------------+----------------------+-----------------+-------------------+

Showing the male to female gender ratio per department from highest to lowest:

In [27]:
%%sql SELECT
    p.department_name,
    SUM(g.gender='Male') AS Male_Staff,
    SUM(g.gender='Female') AS Female_Staff,
    ROUND((CAST(SUM(g.gender='Male') AS FLOAT) * 100.0) / COUNT(*), 1) AS Male_Percentage,
    ROUND((CAST(SUM(g.gender='Female') AS FLOAT) * 100.0) / COUNT(*), 1) AS Female_Percentage
FROM Person p
JOIN Gender g ON p.person_name = g.name
GROUP BY p.department_name
ORDER BY Male_Percentage DESC;

Running query in 'sqlite://'

+-------------------------------------------------------+------------+--------------+-----------------+-------------------+
|                    department_name                    | Male_Staff | Female_Staff | Male_Percentage | Female_Percentage |
+-------------------------------------------------------+------------+--------------+-----------------+-------------------+
|               Department of Mathematics               |     82     |      26      |       75.9      |        24.1       |
|                Department of Economics                |    278     |     129      |       68.3      |        31.7       |
|                Department of Government               |    135     |      67      |       66.8      |        33.2       |
| Department of Philosophy, Logic and Scientific Method |     28     |      14      |       66.7      |        33.3       |
|                 Department of Finance                 |     90     |      49      |       64.7      |        35.3       |
|                Department of Accounting               |     33     |      18      |       64.7      |        35.3       |
|                Department of Statistics               |     62     |      34      |       64.6      |        35.4       |
|             Department of Economic History            |     68     |      38      |       64.2      |        35.8       |
|                School of Public Policy                |     91     |      51      |       64.1      |        35.9       |
|                 Data Science Institute                |     67     |      38      |       63.8      |        36.2       |
|          Department of International History          |     37     |      22      |       62.7      |        37.3       |
|                   European Institute                  |     75     |      52      |       59.1      |        40.9       |
|         Department of International Relations         |     64     |      47      |       57.7      |        42.3       |
|                Department of Sociology                |     71     |      53      |       57.3      |        42.7       |
|        Department of Geography and Environment        |    140     |     121      |       53.6      |        46.4       |
|              Department of Social Policy              |     46     |      41      |       52.9      |        47.1       |
|                     LSE Law School                    |     82     |      74      |       52.6      |        47.4       |
|                   Marshall Institute                  |     23     |      21      |       52.3      |        47.7       |
|               Department of Methodology               |     37     |      34      |       52.1      |        47.9       |
|              Department of Health Policy              |     71     |      75      |       48.6      |        51.4       |
|  Department of Psychological and Behavioural Science  |     86     |      97      |       47.0      |        53.0       |
|         Department of Media and Communications        |     45     |      55      |       45.0      |        55.0       |
|          International Inequalities Institute         |     70     |      88      |       44.3      |        55.7       |
|                Department of Management               |     85     |     109      |       43.8      |        56.2       |
|        Department of International Development        |     43     |      59      |       42.2      |        57.8       |
|               Department of Anthropology              |     29     |      40      |       42.0      |        58.0       |
|                    Language Centre                    |     23     |      33      |       41.1      |        58.9       |
|            Firoz Lalji Institute for Africa           |     26     |      52      |       33.3      |        66.7       |
|              Department of Gender Studies             |     7      |      51      |       12.1      |        87.9       |
+-------------------------------

Regrettably, prevailing gender stereotypes persist, as evidenced by the unequal gender ratios observed across scientific and literary departments. Scientific disciplines exhibit a pronounced male-to-female ratio, with mathematics representing the highest disparity at 76 males for every 24 females. Conversely, literary departments portray a contrasting picture, with women comprising a majority. For instance, the Language Centre reports 41 men for every 59 women. These findings underscore the persistence of gender stereotypes, which continue to influence the distribution of genders across academic disciplines. Addressing these disparities demands concerted efforts to challenge stereotypes and foster environments that promote gender diversity and equality across all fields of study.

Ordering the departments by the closeness of their gender percentage to 50%:

In [28]:
%%sql

SELECT
    p.department_name,
    SUM(g.gender='Male') AS Male_Staff,
    SUM(g.gender='Female') AS Female_Staff,
    ROUND((CAST(SUM(g.gender='Male') AS FLOAT) * 100.0) / COUNT(*), 1) AS Male_Percentage,
    ROUND((CAST(SUM(g.gender='Female') AS FLOAT) * 100.0) / COUNT(*), 1) AS Female_Percentage,
    ROUND(ABS(ROUND((CAST(SUM(g.gender='Male') AS FLOAT) * 100.0) / COUNT(*), 1) - 50), 1) AS Percentage_Difference
FROM Person p
JOIN Gender g ON p.person_name = g.name
GROUP BY p.department_name
ORDER BY Percentage_Difference ASC;

Running query in 'sqlite://'

+-------------------------------------------------------+------------+--------------+-----------------+-------------------+-----------------------+
|                    department_name                    | Male_Staff | Female_Staff | Male_Percentage | Female_Percentage | Percentage_Difference |
+-------------------------------------------------------+------------+--------------+-----------------+-------------------+-----------------------+
|              Department of Health Policy              |     71     |      75      |       48.6      |        51.4       |          1.4          |
|               Department of Methodology               |     37     |      34      |       52.1      |        47.9       |          2.1          |
|                   Marshall Institute                  |     23     |      21      |       52.3      |        47.7       |          2.3          |
|                     LSE Law School                    |     82     |      74      |       52.6      |        47.4       |          2.6          |
|              Department of Social Policy              |     46     |      41      |       52.9      |        47.1       |          2.9          |
|  Department of Psychological and Behavioural Science  |     86     |      97      |       47.0      |        53.0       |          3.0          |
|        Department of Geography and Environment        |    140     |     121      |       53.6      |        46.4       |          3.6          |
|         Department of Media and Communications        |     45     |      55      |       45.0      |        55.0       |          5.0          |
|          International Inequalities Institute         |     70     |      88      |       44.3      |        55.7       |          5.7          |
|                Department of Management               |     85     |     109      |       43.8      |        56.2       |          6.2          |
|                Department of Sociology                |     71     |      53      |       57.3      |        42.7       |          7.3          |
|         Department of International Relations         |     64     |      47      |       57.7      |        42.3       |          7.7          |
|        Department of International Development        |     43     |      59      |       42.2      |        57.8       |          7.8          |
|               Department of Anthropology              |     29     |      40      |       42.0      |        58.0       |          8.0          |
|                    Language Centre                    |     23     |      33      |       41.1      |        58.9       |          8.9          |
|                   European Institute                  |     75     |      52      |       59.1      |        40.9       |          9.1          |
|          Department of International History          |     37     |      22      |       62.7      |        37.3       |          12.7         |
|                 Data Science Institute                |     67     |      38      |       63.8      |        36.2       |          13.8         |
|                School of Public Policy                |     91     |      51      |       64.1      |        35.9       |          14.1         |
|             Department of Economic History            |     68     |      38      |       64.2      |        35.8       |          14.2         |
|                Department of Statistics               |     62     |      34      |       64.6      |        35.4       |          14.6         |
|                Department of Accounting               |     33     |      18      |       64.7      |        35.3       |          14.7         |
|                 Department of Finance                 |     90     |      49      |       64.7      |        35.3       |          14.7         |
| Department of Philosophy, Logic and Scientific Method |     28     |      14      |       66.7      |        33.3       |          16.7         |
|   

Quite surprisingly, the gender studies department is the least gender diverse!

Doing the same but for roles:

In [29]:
%%sql SELECT
    p.role_name,
    p.department_name,
    SUM(g.gender='Male') AS Male_Staff,
    SUM(g.gender='Female') AS Female_Staff,
    ROUND((CAST(SUM(g.gender='Male') AS FLOAT) * 100.0) / COUNT(*), 1) AS Male_Percentage,
    ROUND((CAST(SUM(g.gender='Female') AS FLOAT) * 100.0) / COUNT(*), 1) AS Female_Percentage
FROM Person p
JOIN Gender g ON p.person_name = g.name
GROUP BY p.role_name
ORDER BY Male_Percentage DESC;

Running query in 'sqlite://'

+-----------------------------------------------------------------------------------------+-----------------------------------------------------+------------+--------------+-----------------+-------------------+
|                                        role_name                                        |                   department_name                   | Male_Staff | Female_Staff | Male_Percentage | Female_Percentage |
+-----------------------------------------------------------------------------------------+-----------------------------------------------------+------------+--------------+-----------------+-------------------+
|                                    skills associates                                    |               School of Public Policy               |     1      |      0       |      100.0      |        0.0        |
|                               public engagement and impact                              |            Department of Economic History           |     1      |      0       |      100.0      |        0.0        |
|                        probability in finance and insurance group                       |               Department of Statistics              |     6      |      0       |      100.0      |        0.0        |
|                              lse law school leadership team                             |                    LSE Law School                   |     3      |      0       |      100.0      |        0.0        |
|                                literature and linguistics                               |                   Language Centre                   |     4      |      0       |      100.0      |        0.0        |
|                                    job opportunities                                    |        Department of Media and Communications       |     1      |      0       |      100.0      |        0.0        |
|                                    emeritus faculty                                     |               Department of Accounting              |     4      |      0       |      100.0      |        0.0        |
|                                        directors                                        |                  Marshall Institute                 |     2      |      0       |      100.0      |        0.0        |
|                                         director                                        |                   Language Centre                   |     1      |      0       |      100.0      |        0.0        |
|                               communications and marketing                              |             Department of Health Policy             |     1      |      0       |      100.0      |        0.0        |
|                                      centre manager                                     |                   Language Centre                   |     1      |      0       |      100.0      |        0.0        |
|                      bsc year 1 and 2 learning support and mentors                      |               Department of Accounting              |     3      |      0       |      100.0      |        0.0        |
|                                     affiliated staff                                    |             Department of Social Policy             |     1      |      0       |      100.0      |        0.0        |
|                               affiliated anthropologists                                |              Department of Anthropology             |     1      |      0       |      100.0      |        0.0        |
|                                emeritus and retired staff                               |        Department of International Relations        |     11     |      1       |       91.7      |        8.3        |
|              academic associates - emeritus professors and visiting staff               |              Department of Mathematics              |     11     |      1       |       91.7

As we get very small or big ratios due to the small length of roles, we will look at the ratios only for the roles that appear accross at least 2 or more departments:

In [30]:
%%sql

SELECT
    p.role_name,
    (SELECT COUNT(*) FROM Roles r WHERE r.role_name = p.role_name) AS Role_Count,
    SUM(g.gender='Male') AS Male_Staff,
    SUM(g.gender='Female') AS Female_Staff,
    ROUND((CAST(SUM(g.gender='Male') AS FLOAT) * 100.0) / COUNT(*), 1) AS Male_Percentage,
    ROUND((CAST(SUM(g.gender='Female') AS FLOAT) * 100.0) / COUNT(*), 1) AS Female_Percentage
FROM Person p
JOIN Gender g ON p.person_name = g.name
WHERE p.role_name IN (
    SELECT role_name
    FROM Roles
    GROUP BY role_name
    HAVING COUNT(*) > 1
)
GROUP BY p.role_name
ORDER BY Male_Percentage DESC;

Running query in 'sqlite://'

+-------------------------------------+------------+------------+--------------+-----------------+-------------------+
|              role_name              | Role_Count | Male_Staff | Female_Staff | Male_Percentage | Female_Percentage |
+-------------------------------------+------------+------------+--------------+-----------------+-------------------+
|            class teachers           |     2      |     51     |      13      |       79.7      |        20.3       |
|          academic visitors          |     2      |     22     |      6       |       78.6      |        21.4       |
|        visiting appointments        |     2      |     21     |      8       |       72.4      |        27.6       |
|           academic faculty          |     3      |     62     |      25      |       71.3      |        28.7       |
|            guest teachers           |     5      |     21     |      10      |       67.7      |        32.3       |
|               faculty               |     3      |     53     |      30      |       63.9      |        36.1       |
|        senior management team       |     2      |     7      |      4       |       63.6      |        36.4       |
|          research students          |     8      |    232     |     139      |       62.5      |        37.5       |
|                 head                |     5      |     39     |      24      |       61.9      |        38.1       |
|               emeritus              |     2      |     11     |      8       |       57.9      |        42.1       |
|            visiting staff           |     6      |     85     |      63      |       57.4      |        42.6       |
|            academic staff           |     10     |    189     |     157      |       54.6      |        45.4       |
|             lse fellows             |     2      |     4      |      4       |       50.0      |        50.0       |
|           visiting fellows          |     5      |     62     |      63      |       49.6      |        50.4       |
|            research staff           |     6      |     19     |      21      |       47.5      |        52.5       |
|             phd students            |     4      |     66     |      77      |       46.2      |        53.8       |
|           research staff            |     2      |     6      |      9       |       40.0      |        60.0       |
| graduate teaching assistants (gtas) |     2      |     11     |      18      |       37.9      |        62.1       |
|     professional services staff     |     17     |     82     |     191      |       30.0      |        70.0       |
+-------------------------------------+------------+------------+--------------+-----------------+-------------------+

In the analysis of gender distribution across departments at LSE, significant gender disparities become apparent. Notably, roles within the Professional Services Staff (PSS) category predominantly comprise women, while leadership positions exhibit a higher percentage of males. Despite the overall gender ratio at LSE hovering around 55% male and 45% female, this figure belies underlying inequalities, particularly in leadership representation. The skewed distribution of genders across various roles underscores the need for a more nuanced examination of gender dynamics within the institution. Specifically, addressing disparities in leadership roles is crucial for achieving gender equity and fostering an inclusive environment that promotes equal opportunities for all staff members.

Ordering the departments by number of people:

In [31]:
%%sql

SELECT
    department_name,
    COUNT(*) AS Number_of_People
FROM Person
GROUP BY department_name
ORDER BY Number_of_People DESC;

Running query in 'sqlite://'

+-------------------------------------------------------+------------------+
|                    department_name                    | Number_of_People |
+-------------------------------------------------------+------------------+
|                Department of Economics                |       407        |
|        Department of Geography and Environment        |       261        |
|                Department of Government               |       202        |
|                Department of Management               |       194        |
|  Department of Psychological and Behavioural Science  |       183        |
|          International Inequalities Institute         |       158        |
|                     LSE Law School                    |       156        |
|              Department of Health Policy              |       146        |
|                School of Public Policy                |       142        |
|                 Department of Finance                 |       139        |
|                   European Institute                  |       127        |
|                Department of Sociology                |       124        |
|         Department of International Relations         |       111        |
|               Department of Mathematics               |       108        |
|             Department of Economic History            |       106        |
|                 Data Science Institute                |       105        |
|        Department of International Development        |       102        |
|         Department of Media and Communications        |       100        |
|                Department of Statistics               |        96        |
|              Department of Social Policy              |        87        |
|            Firoz Lalji Institute for Africa           |        78        |
|               Department of Methodology               |        71        |
|               Department of Anthropology              |        69        |
|          Department of International History          |        59        |
|              Department of Gender Studies             |        58        |
|                    Language Centre                    |        56        |
|                Department of Accounting               |        51        |
|                   Marshall Institute                  |        44        |
| Department of Philosophy, Logic and Scientific Method |        42        |
+-------------------------------------------------------+------------------+

Seeing the people who appears the most:

In [32]:
%%sql

SELECT
    person_name,
    COUNT(*) AS Appearances
FROM Person
GROUP BY person_name
HAVING COUNT(*) = (
    SELECT MAX(Appearances)
    FROM (
        SELECT COUNT(*) AS Appearances
        FROM Person
        GROUP BY person_name
    ) AS max_counts
)
ORDER BY Appearances DESC;

Running query in 'sqlite://'

+---------------------------+-------------+
|        person_name        | Appearances |
+---------------------------+-------------+
| Professor Stephen Jenkins |      4      |
|        Nancy Holman       |      4      |
|    Dr Matteo M Galizzi    |      4      |
+---------------------------+-------------+